In [1]:
import sys
import os
sys.path.insert(0,os.path.abspath('..'))
from spatial_graphs.AmiraSpatialGraph import AmiraSpatialGraph,MatchBarrels
from spatial_graphs.Landmarks import Landmarks
from spatial_graphs.Surfaces import Surface
from spatial_graphs.Vectors import Vectors
from spatial_graphs.Alignment import Alignment
from spatial_graphs.Stats import Stats
from dask import compute,multiprocessing,delayed
import pathlib
import shutil
import glob
import pandas as pd
import vtk
from scipy.spatial import distance
import numpy as np
from scipy.interpolate import UnivariateSpline
from sklearn.cluster import k_means
import SimpleITK as sitk
from scipy.spatial import distance_matrix

# Create output folders

In [2]:
input_path = '/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Original_Data/'
ref_neun_depth_profile = '/nas1/Data_Mythreya/MotorCortexProject/V10/lit_review/densities_zprofiles_averages.xlsx'
surface_resolutions = [50,100,200,300,400,450,500]
exp_names = ['MG49_lhs','MG49_rhs','MG50_lhs','MG50_rhs','MG48_lhs','MG48_rhs']
exp_names_tangential = ['MG49_lhs','MG49_rhs','MG50_lhs','MG50_rhs']
#alignment_types = ['Using_vM1_PCA','Using_vM1','Using_vS1','Using_vS1_vM1_Surface_Points','Using_Pia_WM_Surfaces']
#alignment_types = ['Using_vS1_vM1_Surface_Points']
selected_surface_resolution = 50
selected_voxel_size=400
selected_cutoff = 0.1

selected_global_alignment_method = 'Using_Local_Rabies_Surfaces'
selected_vM1_alignment_method = 'Using_Rabies_Surface'
selected_vS1_alignment_method = 'Using_Rabies_Surface'
selected_barrel_alignment_method = 'Using_Barrel_and_Projections'

#lambda is 9000 from bragma

In [5]:
# create output folder structure
HOME = str(pathlib.Path(input_path).parent)
output_root = HOME + '/Outputs/'
pathlib.Path((output_root)).mkdir(exist_ok=True)

output_path_spatial_graphs = output_root + '/Spatial_Graphs/'
output_path_landmarks = output_root + '/Landmarks/'
output_path_surfaces = output_root + '/Surfaces/'
output_path_axis_fileds = output_root +'/Axis_Fields/'
output_path_vM1 = output_root + '/vM1_Ref_Surfaces/'
output_path_vS1 = output_root + '/vS1_Ref_Surfaces/'
#output_path_vM1_cleaned = output_root + '/vM1_Ref_Surfaces_Cleaned/'
#output_path_vS1_cleaned = output_root + '/vS1_Ref_Surfaces_Cleaned/'
output_path_vM1_axis_field = output_root + '/vM1_Ref_Surfaces_Axis_Fields/'
output_path_vS1_axis_field = output_root + '/vS1_Ref_Surfaces_Axis_Fields/'
#output_path_vM1_axis_field_cleaned = output_root + '/vM1_Ref_Surfaces_Axis_Fields_Cleaned/'
#output_path_vS1_axis_field_cleaned = output_root + '/vS1_Ref_Surfaces_Axis_Fields_Cleaned/'
output_path_stats = output_root + '/Stats/'
output_path_rabies_layers = output_root + '/Layers_Rabies/'
output_path_alignment = output_root + 'Alignment/'
output_path_txmats = output_root + 'Transformation_Matrices/'
output_path_avg = output_root + 'Average/'
output_path_stats = output_root + 'Stats/' 
output_path_neun_profiles = output_root + 'NeuN_Profiles/'
output_path_rabies_profiles = output_root + 'Rabies_Profiles/'
output_path_rabies_profiles_v2 = output_root + 'Rabies_Profiles_V2/'
output_path_neun_layer_borders = output_root+'Layer_Borders/'
output_path_neun_layer_borders_v2 = output_root+'Layer_Borders_V2/'
output_path_rabies_layer_borders_v2 = output_root+'Layers_Rabies_V2/'

pathlib.Path((output_path_spatial_graphs)).mkdir(exist_ok=True)
pathlib.Path((output_path_landmarks)).mkdir(exist_ok=True)
pathlib.Path((output_path_surfaces)).mkdir(exist_ok=True)
pathlib.Path((output_path_axis_fileds)).mkdir(exist_ok=True)
pathlib.Path((output_path_vM1)).mkdir(exist_ok=True)
pathlib.Path((output_path_vS1)).mkdir(exist_ok=True)
#pathlib.Path((output_path_vM1_cleaned)).mkdir(exist_ok=True)
#pathlib.Path((output_path_vS1_cleaned)).mkdir(exist_ok=True)
pathlib.Path(output_path_vM1_axis_field).mkdir(exist_ok=True)
pathlib.Path(output_path_vS1_axis_field).mkdir(exist_ok=True)
#pathlib.Path(output_path_vM1_axis_field_cleaned).mkdir(exist_ok=True)
#pathlib.Path(output_path_vS1_axis_field_cleaned).mkdir(exist_ok=True)
pathlib.Path((output_path_rabies_layers)).mkdir(exist_ok=True)
pathlib.Path(output_path_alignment).mkdir(exist_ok=True)
pathlib.Path(output_path_txmats).mkdir(exist_ok=True)
pathlib.Path(output_path_avg).mkdir(exist_ok=True)
pathlib.Path(output_path_stats).mkdir(exist_ok=True)
pathlib.Path(output_path_neun_profiles).mkdir(exist_ok=True)
pathlib.Path(output_path_rabies_profiles).mkdir(exist_ok=True)
pathlib.Path(output_path_neun_layer_borders).mkdir(exist_ok=True)
pathlib.Path(output_path_neun_layer_borders_v2).mkdir(exist_ok=True)
pathlib.Path(output_path_rabies_layer_borders_v2).mkdir(exist_ok=True)
pathlib.Path(output_path_rabies_profiles_v2).mkdir(exist_ok=True)

# Alignment Folders
output_path_alignment_analysis = output_root + 'Alignment_Analysis/'
pathlib.Path(output_path_alignment_analysis).mkdir(exist_ok=True)
output_path_alignment_surf_res = output_path_alignment_analysis + 'Surface_Resolution_Selection/'
pathlib.Path(output_path_alignment_surf_res).mkdir(exist_ok=True)

output_path_alignment_global_alignment = output_path_alignment_analysis + 'Global_Alignment/'
pathlib.Path(output_path_alignment_global_alignment).mkdir(exist_ok=True)
output_path_alignment_global_alignment_centers = output_path_alignment_global_alignment + 'Using_Center/'
pathlib.Path(output_path_alignment_global_alignment_centers).mkdir(exist_ok=True)
output_path_alignment_global_alignment_pcas = output_path_alignment_global_alignment + 'Using_PCA/'
pathlib.Path(output_path_alignment_global_alignment_pcas).mkdir(exist_ok=True)
output_path_alignment_global_alignment_surfaces = output_path_alignment_global_alignment + 'Using_Surface/'
pathlib.Path(output_path_alignment_global_alignment_surfaces).mkdir(exist_ok=True)

output_path_alignment_global_alignment_local_centers = output_path_alignment_global_alignment + 'Using_Local_Centers/'
pathlib.Path(output_path_alignment_global_alignment_local_centers).mkdir(exist_ok=True)
#output_path_alignment_global_alignment_local_pcas = output_path_alignment_global_alignment + 'Using_Local_PCAs/'
#pathlib.Path(output_path_alignment_global_alignment_local_pcas).mkdir(exist_ok=True)
output_path_alignment_global_alignment_local_surfaces = output_path_alignment_global_alignment + 'Using_Local_Surfaces/'
pathlib.Path(output_path_alignment_global_alignment_local_surfaces).mkdir(exist_ok=True)
output_path_alignment_global_alignment_local_rabies_surfaces = output_path_alignment_global_alignment + 'Using_Local_Rabies_Surfaces/'
pathlib.Path(output_path_alignment_global_alignment_local_rabies_surfaces).mkdir(exist_ok=True)
output_path_alignment_global_alignment_local_surfaces_m1_sbf = output_path_alignment_global_alignment + 'Using_M1_SBF/'
pathlib.Path(output_path_alignment_global_alignment_local_surfaces_m1_sbf).mkdir(exist_ok=True)

output_path_alignment_vs1 = output_path_alignment_analysis + 'vS1/'
pathlib.Path(output_path_alignment_vs1).mkdir(exist_ok=True)
output_path_alignment_vs1_center = output_path_alignment_vs1 + 'Using_Center/'
pathlib.Path(output_path_alignment_vs1_center).mkdir(exist_ok=True)
#output_path_alignment_vs1_pca = output_path_alignment_vs1 + 'Using_PCA/'
#pathlib.Path(output_path_alignment_vs1_pca).mkdir(exist_ok=True)
output_path_alignment_vs1_surface = output_path_alignment_vs1 + 'Using_Surface/'
pathlib.Path(output_path_alignment_vs1_surface).mkdir(exist_ok=True)
output_path_alignment_vs1_rabies_surface = output_path_alignment_vs1 + 'Using_Rabies_Surface/'
pathlib.Path(output_path_alignment_vs1_rabies_surface).mkdir(exist_ok=True)

output_path_alignment_vm1 = output_path_alignment_analysis + 'vM1/'
pathlib.Path(output_path_alignment_vm1).mkdir(exist_ok=True)
output_path_alignment_vm1_center = output_path_alignment_vm1 + 'Using_Center/'
pathlib.Path(output_path_alignment_vm1_center).mkdir(exist_ok=True)
#output_path_alignment_vm1_pca = output_path_alignment_vm1 + 'Using_PCA/'
#pathlib.Path(output_path_alignment_vm1_pca).mkdir(exist_ok=True)
output_path_alignment_vm1_surface = output_path_alignment_vm1 + 'Using_Surface/'
pathlib.Path(output_path_alignment_vm1_surface).mkdir(exist_ok=True)
output_path_alignment_vm1_rabies_surface = output_path_alignment_vm1 + 'Using_Rabies_Surface/'
pathlib.Path(output_path_alignment_vm1_rabies_surface).mkdir(exist_ok=True)

output_path_alignment_s1bf = output_path_alignment_analysis + 'S1BF/'
pathlib.Path(output_path_alignment_s1bf).mkdir(exist_ok=True)
output_path_alignment_s1bf_barrels_alone = output_path_alignment_s1bf + 'Use_Barrels_Alone/'
pathlib.Path(output_path_alignment_s1bf_barrels_alone).mkdir(exist_ok=True)
output_path_alignment_s1bf_projections_alone = output_path_alignment_s1bf + 'Use_Projections_Alone/'
pathlib.Path(output_path_alignment_s1bf_projections_alone).mkdir(exist_ok=True)
output_path_alignment_s1bf_barrels_and_projections = output_path_alignment_s1bf + 'Use_Both/'
pathlib.Path(output_path_alignment_s1bf_barrels_and_projections).mkdir(exist_ok=True)

output_path_alignment_rabies_clustering = output_path_alignment_analysis + 'Rabies_Clustering_Selection/'
pathlib.Path(output_path_alignment_rabies_clustering).mkdir(exist_ok=True)
output_path_alignment_rabies_clustering_vs1 = output_path_alignment_rabies_clustering + 'vS1/'
pathlib.Path(output_path_alignment_rabies_clustering_vs1).mkdir(exist_ok=True)
output_path_alignment_rabies_clustering_m1 = output_path_alignment_rabies_clustering + 'vM1/'
pathlib.Path(output_path_alignment_rabies_clustering_m1).mkdir(exist_ok=True)

output_path_atlas = output_root + 'Atlas/'
pathlib.Path(output_path_atlas).mkdir(exist_ok=True)


In [ ]:
for file in glob.glob(input_path+'*'):
    if os.path.isfile(file):
        if file.split('.')[-1]=='am':
            print(file)
            shutil.copy(file,output_path_spatial_graphs)
        if file.split('.')[-1]=='LandmarkAscii':
            print(file)
            shutil.copy(file,output_path_landmarks) 

# Create surfaces

In [ ]:
@delayed
def create_surface(file,coronal=False,rhs=False,nr_divisions=0,surf_resolution='200'):
    print(file)
    print(nr_divisions)
    os.system('/home/mythreya/project_src/BarrelField3D/DataAnalysis3D/BF3DRecon {} {} {}'.\
              format(file,output_path_surfaces+os.path.basename(file)[:-3],str(surf_resolution)))
    
    if coronal:
        tr_mat = [-1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1]
        sg = AmiraSpatialGraph(output_path_surfaces+os.path.basename(file)[:-3]+'_barrels.am')
        sg.apply_transformation(transformation_matrix=tr_mat)
        sg.write_spatial_graph(output_path_surfaces+os.path.basename(file)[:-3]+'_barrels.am')
        
         
        l = Landmarks(output_path_landmarks+os.path.basename(file)[:-3]+'_rabies_cortical_landmarks.LandmarkAscii')
        l.apply_transformation(transformation_matrix=tr_mat)
        l.write_landmarks(output_path_landmarks+os.path.basename(file)[:-3]+'_rabies_cortical_landmarks.LandmarkAscii')
        l = Landmarks(output_path_landmarks+os.path.basename(file)[:-3]+'_rabies_landmarks.LandmarkAscii')
        l.apply_transformation(transformation_matrix=tr_mat)
        l.write_landmarks(output_path_landmarks+os.path.basename(file)[:-3]+'_rabies_landmarks.LandmarkAscii')
        l = Landmarks(output_path_landmarks+os.path.basename(file)[:-3]+'_neun_landmarks_vM1_central_axis_2d.LandmarkAscii')
        l.apply_transformation(transformation_matrix=tr_mat)
        l.write_landmarks(output_path_landmarks+os.path.basename(file)[:-3]+'_neun_landmarks_vM1_central_axis_2d.LandmarkAscii')
        l = Landmarks(output_path_landmarks+os.path.basename(file)[:-7]+'_neun_landmarks.LandmarkAscii')
        l.apply_transformation(transformation_matrix=tr_mat)
        l.write_landmarks(output_path_landmarks+os.path.basename(file)[:-7]+'_neun_landmarks.LandmarkAscii')
        
        pia = Surface(output_path_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
        wm = Surface(output_path_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
        pia.apply_transformation(tr_mat)
        wm.apply_transformation(tr_mat)
        
        pia.write_surface_mesh(output_path_surfaces+os.path.basename(file)[:-3]+'_pia_voxel_size_{}.vtk'.format(surf_resolution))
        wm.write_surface_mesh(output_path_surfaces+os.path.basename(file)[:-3]+'_WM_voxel_size_{}.vtk'.format(surf_resolution))
        
        pia_bounds = Surface(pts=sg.pia.edge_pt_coords).surface.GetBounds()
        wm_bounds = Surface(pts=sg.wm.edge_pt_coords).surface.GetBounds()
        if rhs:
            pia.clip_surface_at_given_x(0,x_limit=pia_bounds[0]+400)
            wm.clip_surface_at_given_x(0,x_limit=wm_bounds[0]+400)
        else:
            pia.clip_surface_at_given_x(0,x_limit=pia_bounds[1]-400)
            wm.clip_surface_at_given_x(0,x_limit=wm_bounds[1]-400)
        
        pia.divide_surface(nr_divisions,pia.surface,\
            output_filename = output_path_surfaces+os.path.basename(file)[:-3]+'_pia_open_bottom_voxel_size_{}.vtk'.\
                           format(surf_resolution))
        wm.divide_surface(nr_divisions,wm.surface,\
                    output_filename = output_path_surfaces+os.path.basename(file)[:-3]+'_WM_open_bottom_voxel_size_{}.vtk'.\
                          format(surf_resolution))
        
        
    else:
        sg = AmiraSpatialGraph(output_path_surfaces+os.path.basename(file)[:-3]+'_barrels.am')
        sg.write_spatial_graph(output_path_surfaces+os.path.basename(file)[:-3]+'_barrels_{}.am'.format(surf_resolution))
    #    Surface(output_path_surfaces+os.path.basename(file)[:-3]+'_pia.vtk').clip_surface_at_given_z(-100,output_filename = output_path_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
    #    Surface(output_path_surfaces+os.path.basename(file)[:-3]+'_WM.vtk').clip_surface_at_given_z(-100,output_filename = output_path_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
        pia = Surface(output_path_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
        wm = Surface(output_path_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
        pia.write_surface_mesh(output_path_surfaces+os.path.basename(file)[:-3]+'_pia_voxel_size_{}.vtk'.format(surf_resolution))
        wm.write_surface_mesh(output_path_surfaces+os.path.basename(file)[:-3]+'_WM_voxel_size_{}.vtk'.format(surf_resolution)) 
        
        pia_bounds = Surface(pts=sg.pia.edge_pt_coords).surface.GetBounds()
        wm_bounds = Surface(pts=sg.wm.edge_pt_coords).surface.GetBounds()
        #print(pia_bounds)
        pia.clip_surface_at_given_z(0,z_limit=pia_bounds[5]-100)
        wm.clip_surface_at_given_z(0,z_limit=pia_bounds[5]-100)
        
        pia.divide_surface(nr_divisions,pia.surface,\
            output_filename = output_path_surfaces+os.path.basename(file)[:-3]+'_pia_open_bottom_voxel_size_{}.vtk'.\
                           format(surf_resolution))
        wm.divide_surface(nr_divisions,wm.surface,\
                          output_filename = output_path_surfaces+os.path.basename(file)[:-3]+'_WM_open_bottom_voxel_size_{}.vtk'.\
                          format(surf_resolution))
        
        

In [ ]:
fn_list_surfs = []
for i in range(len(surface_resolutions)): 
    for exp_name in exp_names:
        if exp_name.find('MG48') >= 0:
            if exp_name.find('rhs') >= 0:
                fn_list_surfs.append(create_surface(output_path_spatial_graphs + exp_name + '.am',\
                                                    coronal=True,rhs=True,nr_divisions=0,surf_resolution=surface_resolutions[i]))
            else:
                fn_list_surfs.append(create_surface(output_path_spatial_graphs + exp_name + '.am',\
                                                    coronal=True,nr_divisions=0,surf_resolution=surface_resolutions[i]))
        else:
            fn_list_surfs.append(create_surface(output_path_spatial_graphs + exp_name + '.am',\
                                                coronal=False,nr_divisions=0,surf_resolution=surface_resolutions[i]))
        
    
    compute(fn_list_surfs)  
    fn_list_surfs = []

In [ ]:
# write surface stats
cols = ['Exp_Name','Voxel_Size','Pia_Number_Of_Vertices','Pia_Number_Of_Triangles',\
                                      'WM_Number_Of_Vertices','WM_Number_Of_Triangles',\
                                      'Pia_Surface_Area','WM_Surface_Area','Pia_Volume','WM_Volume']
surf_stats_main = pd.DataFrame(columns=cols)
for i in range(len(surface_resolutions)): 
    surf_stats_df = pd.DataFrame(columns=cols)
    for exp_name in exp_names :
        pia = Surface(output_path_surfaces+exp_name+'_pia_open_bottom_voxel_size_{}.vtk'.format(surface_resolutions[i]))
        wm = Surface(output_path_surfaces+exp_name+'_WM_open_bottom_voxel_size_{}.vtk'.format(surface_resolutions[i]))
        pia_closed = Surface(output_path_surfaces+exp_name+'_pia_voxel_size_{}.vtk'.format(surface_resolutions[i]))
        wm_closed = Surface(output_path_surfaces+exp_name+'_WM_voxel_size_{}.vtk'.format(surface_resolutions[i]))
        
        surf_stats_df = surf_stats_df.append(pd.DataFrame(columns=surf_stats_df.columns,data = \
                                          [[exp_name,float(surface_resolutions[i]),\
                                           float(pia.surface.GetNumberOfPoints()),float(pia.surface.GetNumberOfCells()),\
              float(wm.surface.GetNumberOfPoints()),float(wm.surface.GetNumberOfCells()),\
              pia.get_surface_properties(prop='area'),wm.get_surface_properties(prop='area'),\
              pia_closed.get_surface_properties(prop='volume'),wm_closed.get_surface_properties(prop='volume')]]))
    stats = Stats(surf_stats_df)
    stats_returned = stats.add_mean_std_cov(tangential_exps=True, coronal_exps=True,)
    
    surf_stats_main = surf_stats_main.append(stats_returned)
    
surf_stats_main.to_csv(output_path_stats+'1_Pia_WM_Surface_Stats.csv')

# Generate Axis Field

In [ ]:
#TODO : Extend the axis progressively till it touches the WM, instead of having constant max vec length

In [ ]:
# How many axes shold be there ?
# A typical cortical column is 250 um in radius... avg surface area of our pia hemispheres is 200e6
200e6/(np.pi*250*250)
# surface resolutiuon of 450um gives the number of cells we need on wm to generate this aixs field

In [ ]:
# Generate axis fields for the smoothest surface from wm - pia
for exp_name in exp_names:
    pia = Surface(output_path_surfaces+exp_name+'_pia_open_bottom_voxel_size_450.vtk')
    wm = Surface(output_path_surfaces+exp_name+'_WM_open_bottom_voxel_size_450.vtk')
    #wm = wm.divide_surface(4,wm.surface)
    
    vectors_wm_pia_flipped = wm.create_axis_field(pia,op_sg_name=None,\
                         flip_normals=False, return_axes = True,max_allowed_axis_length=4000)
#     vectors_wm_pia = Surface(polydata=wm).create_axis_field(pia,op_sg_name=None,\
#                          flip_normals=False, return_axes = True,max_allowed_axis_length=4000)
    axis_field_sg = AmiraSpatialGraph(generic_graph=True)
    for edge in vectors_wm_pia_flipped:
        axis_field_sg.graph_data.add_edge(edge[0],edge[1])
#     for edge in vectors_wm_pia:
#         axis_field_sg.graph_data.add_edge(edge[0],edge[1])
    axis_field_sg.write_spatial_graph(output_path_axis_fileds+exp_name+'_wm_to_pia_axis_field.am')


In [ ]:
# Find the wrong axis by angle
#for exp_name in exp_names :
# exp_name = 'MG48_lhs'
# vectors_wm_pia_sg = AmiraSpatialGraph(output_path_axis_fileds+exp_name+'_wm_to_pia_axis_field.am',\
#                   generic_graph=True)
# outliers_sg = AmiraSpatialGraph()
# for edge in vectors_wm_pia_sg.graph_data.edge_list:
#     nearby_axes = Vectors().get_nearest_axes_to_ref_axis_within_radius(edge,vectors_wm_pia_sg.graph_data.edge_list,800)
#     # find the avg angle here
#     avg_uv = Vectors().get_avg_vectors(nearby_axes)
#     #print(avg_uv)
#     #print(Vectors().get_unit_vec(edge[0],edge[1]))
#     angle = Vectors().get_angle_between_unit_vectors(Vectors().get_unit_vec(edge[0],edge[1]),avg_uv)
#     if abs(angle) > 30:
#         outliers_sg.graph_data.add_edge(edge[0],edge[1])
# outliers_sg.write_spatial_graph(output_path_axis_fileds+exp_name+'_outliers_wm_to_pia_axis_field.am')


In [ ]:
#for i in range(len(surface_resolutions)): 
for exp_name in exp_names:
    if exp_name.find('MG48_lhs') >= 0:
        vec_extension_offset = 0.25
    elif exp_name.find('MG48_rhs') >= 0:
        vec_extension_offset = 0.25
    else:
        vec_extension_offset = 0.75
    axis_field_sg = AmiraSpatialGraph(output_path_axis_fileds+exp_name+'_wm_to_pia_axis_field.am',generic_graph=True)
    for i in range(len(surface_resolutions)):
        pia = Surface(output_path_surfaces+exp_name+'_pia_open_bottom_voxel_size_{}.vtk'.format(surface_resolutions[i]))
        wm = Surface(output_path_surfaces+exp_name+'_WM_open_bottom_voxel_size_{}.vtk'.format(surface_resolutions[i]))

        selected_axis_field_sg,wm_pts,pia_pts,missing_axis_field = pia.get_axis_field_within_instersecting_surface_with_trimming\
        (pia.surface,wm.surface,axis_field_sg.graph_data.edge_list,vec_extension_offset=vec_extension_offset)
        selected_axis_field_sg.write_spatial_graph(output_path_axis_fileds+exp_name+\
                                '_wm_to_pia_axis_field_voxel_size_{}.am'.format(surface_resolutions[i]))

        missing_axis_field.write_spatial_graph(output_path_axis_fileds+exp_name+\
                                  '_wm_to_pia_axis_field_voxel_size_{}_missing.am'.format(surface_resolutions[i]))
        
    
        
    

In [ ]:
# write axis field stats
cols =['Exp_Name','Voxel_Size','Vector_Lengths_Mean_WM_To_Pia','Vector_Lengths_Std_WM_To_Pia']
df_cortex_len_main = pd.DataFrame(columns=cols)
for surface_resolution in surface_resolutions: 
    df_cortex_len = pd.DataFrame(columns=cols)
    for exp_name in exp_names :
        vectors_wm_pia_sg = AmiraSpatialGraph(output_path_axis_fileds+exp_name+'_wm_to_pia_axis_field_voxel_size_{}.am'.\
                          format(surface_resolution),generic_graph=True)
        vec_wm_pia_lens, vec_wm_pia_lens_mean,vec_wm_pia_lens_std = Vectors().get_vec_lengths(vectors_wm_pia_sg.graph_data.edge_list)
    
        df_cortex_len = df_cortex_len.append(pd.DataFrame(columns=df_cortex_len.columns,data=[[exp_name,\
                                            float(surface_resolution),vec_wm_pia_lens_mean,vec_wm_pia_lens_std]]))
    stats = Stats(df_cortex_len)
    stats_returned = stats.add_mean_std_cov(tangential_exps=True, coronal_exps=True,)
    
    df_cortex_len_main = df_cortex_len_main.append(stats_returned)
    
df_cortex_len_main.to_csv(output_path_stats+'2_Cortical_Thickness.csv')

# K Means clustering

In [ ]:
# first do a k means cluster
ouput_path = output_path_landmarks+'K-Means/'
pathlib.Path(ouput_path).mkdir(exist_ok=True)
for exp in exp_names:
    landmarks = Landmarks(output_path_landmarks+exp+'_rabies_cortical_landmarks.LandmarkAscii').pts
    centroids,labels,bla = k_means(landmarks,n_clusters=2)
    
    # more rostral cluster is m1
    if centroids[0][1] < centroids[1][1]:
        Landmarks(pts = np.array(landmarks)[labels==0]).write_landmarks(ouput_path+exp+'_rabies_kmeans_cluster_vM1.landmarksAscii')
        Landmarks(pts = np.array(landmarks)[labels==1]).write_landmarks(ouput_path+exp+'_rabies_kmeans_cluster_vS1.landmarksAscii')
        Landmarks(pts=[centroids[0]]).write_landmarks(ouput_path+exp+'_rabies_kmeans_cluster_vM1_centroid.landmarksAscii')
        Landmarks(pts=[centroids[1]]).write_landmarks(ouput_path+exp+'_rabies_kmeans_cluster_vS1_centroid.landmarksAscii')
    else:
        Landmarks(pts = np.array(landmarks)[labels==0]).write_landmarks(ouput_path+exp+'_rabies_kmeans_cluster_vS1.landmarksAscii')
        Landmarks(pts = np.array(landmarks)[labels==1]).write_landmarks(ouput_path+exp+'_rabies_kmeans_cluster_vM1.landmarksAscii')
        Landmarks(pts=[centroids[0]]).write_landmarks(ouput_path+exp+'_rabies_kmeans_cluster_vS1_centroid.landmarksAscii')
        Landmarks(pts=[centroids[1]]).write_landmarks(ouput_path+exp+'_rabies_kmeans_cluster_vM1_centroid.landmarksAscii')

# Cluster vM1 Rabies

In [ ]:
# create vM1 clusters
df_vM1_rabies_cluster = pd.DataFrame(columns=['Exp_Name','Voxel_Size','Density_Lower_Threshold_Percentage','Max_Density',\
                                              'Actual_Lower_Density_Threshold'])
#voxel_size=int(500)
#dens_cutoff_threshold=0.15
ouput_path = output_path_landmarks+'Density_Clusters/'
pathlib.Path(ouput_path).mkdir(exist_ok=True)
for voxel_size in [50,100,200,300,400,450,500]:
    for dens_cutoff_threshold in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
        for exp_name in exp_names:
            print(voxel_size,dens_cutoff_threshold,exp_name)
            l = Landmarks(output_path_landmarks+'K-Means/'+'{}_rabies_kmeans_cluster_vM1.landmarksAscii'.format(exp_name))
            cluster_in_bounds,max_dens,dens_cutoff_threshold,threhold = \
            l.get_density_cluster(output_file=ouput_path+'{}_vM1_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
                                  format(exp_name,str(voxel_size),str(dens_cutoff_threshold)),\
                                  voxel_size=voxel_size,dens_cutoff_threshold=dens_cutoff_threshold,\
                                  write_density_images=True,min_threshold=0)

            df_vM1_rabies_cluster = df_vM1_rabies_cluster.append(pd.DataFrame(columns=df_vM1_rabies_cluster.columns,\
                                                                 data=[[exp_name,float(voxel_size),\
                                                                        dens_cutoff_threshold*100,max_dens,threhold]]))
            
    
df_vM1_rabies_cluster.to_csv(output_path_stats+'3_Rabies_Cluster_Stats_vM1.csv')
#stats = Stats(df_vM1_rabies_cluster)
#stats.add_mean_std_cov(output_filename=output_path_stats+'3_Rabies_Cluster_Stats.csv')

# Cluster vS1 Rabies

In [ ]:
# create vS1 clusters
df_vS1_rabies_cluster = pd.DataFrame(columns=['Exp_Name','Voxel_Size','Density_Lower_Threshold_Percentage','Max_Density',\
                                              'Actual_Lower_Density_Threshold'])
voxel_size=int(500)
dens_cutoff_threshold=0.15
ouput_path = output_path_landmarks+'Density_Clusters/'
pathlib.Path(ouput_path).mkdir(exist_ok=True)
for voxel_size in [50,100,200,300,400,450,500]:
    for dens_cutoff_threshold in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
        for exp_name in exp_names:
            print(voxel_size,dens_cutoff_threshold,exp_name)
            l = Landmarks(output_path_landmarks+'K-Means/'+'{}_rabies_kmeans_cluster_vS1.landmarksAscii'.format(exp_name))
            cluster_in_bounds,max_dens,dens_cutoff_threshold,threhold = \
            l.get_density_cluster(output_file=ouput_path+'{}_vS1_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
                                  format(exp_name,str(voxel_size),str(dens_cutoff_threshold)),\
                                  voxel_size=voxel_size,dens_cutoff_threshold=dens_cutoff_threshold,\
                                  write_density_images=True)

            df_vS1_rabies_cluster = df_vS1_rabies_cluster.append(pd.DataFrame(columns=df_vS1_rabies_cluster.columns,\
                                                                 data=[[exp_name,float(voxel_size),\
                                                                        dens_cutoff_threshold*100,max_dens,threhold]]))

df_vS1_rabies_cluster.to_csv(output_path_stats+'3_Rabies_Cluster_Stats_vS1.csv')
#stats = Stats(df_vS1_rabies_cluster)
#stats.add_mean_std_cov(output_filename=output_path_stats+'3_Rabies_Cluster_Stats_vS1.csv')

In [ ]:
# Find alignement best alignment method: Barrel, Projections or both

df_sbf_alignment_errors = pd.DataFrame(columns=['Exp_Name','Alignment_Type','Barrel_Names','Pia_Errors','Barrel_Center_Errors',\
                                                'WM_Errors','Column_Center_Errors'])
        
avg_sbf_file = input_path+'/sbf/average_barrel_field.am'
sg_sbf = AmiraSpatialGraph(avg_sbf_file,barrel_projections_present=True,reverse_barrel_direction=True)
sg_sbf.write_spatial_graph(input_path+'/sbf/average_barrel_field.am')

S1_convex_hull_name = input_path+'/sbf/average_barrel_field_S1_convex_hull.stl'
S1_hull = Surface(filename=S1_convex_hull_name)
S1_Layer_4_upper_filename = input_path+'/sbf/average_barrel_field_L4Upper.stl'
S1_Layer_4_lower_filename = input_path+'/sbf/average_barrel_field_L4Lower.stl'
S1_Layer_4_upper = Surface(filename=S1_Layer_4_upper_filename)
S1_Layer_4_lower = Surface(filename=S1_Layer_4_lower_filename)

for barrel_alignment_type in ['use_barrels_alone','use_projections_alone','use_both']:
    #print(barrel_alignment_type)
    mean_error_barels = []
    mean_error_column = []
    for exp_name in ['MG49_lhs','MG49_rhs','MG50_lhs','MG50_rhs']:
        #print(exp_name)
        sg = AmiraSpatialGraph(output_path_surfaces+exp_name+'_barrels.am',barrel_projections_present=True)
         
        # Register the standard sbf to these barrels
        if barrel_alignment_type == 'use_barrels_alone':
            matching_barrels = MatchBarrels(sg.barrels,sg_sbf.barrels,use_barrels_alone=True)
        elif barrel_alignment_type == 'use_projections_alone':
            matching_barrels = MatchBarrels(sg.barrels,sg_sbf.barrels,use_projections_alone=True)
        else:
            matching_barrels = MatchBarrels(sg.barrels,sg_sbf.barrels,)
        ref_pts,pts = matching_barrels.get_matching_barrel_centroids()
        mached_barrels_ref,matched_barrels = matching_barrels.get_matching_barrels()

        #Landmarks(pts=ref_pts).write_landmarks(output_path_spatial_graphs+'ref_pts')
        #Landmarks(pts=pts).write_landmarks(output_path_spatial_graphs+'pts')
        
        # align barrel centroid
        icp,txmat = Landmarks(pts=pts).align_landmarks(ref_pts)
        sg_sbf.apply_transformation(txmat)
        S1_hull.apply_icp_transform(icp)
        S1_Layer_4_upper.apply_icp_transform(icp)
        S1_Layer_4_lower.apply_icp_transform(icp)

        sg_sbf.write_spatial_graph(output_path_spatial_graphs+exp_name+'_'+barrel_alignment_type+'_sbf.am')
        sg_sbf = AmiraSpatialGraph(output_path_spatial_graphs+exp_name+'_'+barrel_alignment_type+'_sbf.am',\
                                   barrel_projections_present=True)

        matched_barrels = MatchBarrels(sg.barrels,sg_sbf.barrels,)
        error_df = (matched_barrels.get_errors(exp_name=exp_name,alignment_type=barrel_alignment_type))
        #print(error_df)
        df_sbf_alignment_errors = df_sbf_alignment_errors.append(error_df)
        
        #error_df.to_csv(output_path_stats+exp_name+'_'+barrel_alignment_type+'_barrel_alignment_errors.csv')
        #print(error_df.mean().mean())
        mean_error_barels.append(error_df['Barrel_Center_Errors'].mean())
        mean_error_column.append(error_df['Column_Center_Errors'].mean())
        
        #S1_hull.write_surface_mesh(output_path_surfaces+exp_name+'_using_both_reg_s1_hull.vtk')
        #S1_Layer_4_upper.write_surface_mesh(output_path_surfaces+exp_name+'_using_projections_both_s1_L4_upper.vtk')
        #S1_Layer_4_lower.write_surface_mesh(output_path_surfaces+exp_name+'_using_projections_both_s1_L4_lower.vtk')

        # find landmarks within the aligned sbf
        #l = Landmarks(output_path_landmarks+'{}_rabies_landmarks.LandmarkAscii'.format(exp_name))
        #l.get_landmarks_within_given_surface(S1_hull.surface,output_file=output_path_landmarks+'{}_vS1_rabies_landmarks.LandmarkAscii'.format(exp_name))
    
    print('Mean barrel center error across exps for alignment type {} is {}'.format(barrel_alignment_type,np.array(mean_error_barels).mean()))
    print('Mean column center error across exps for alignment type {} is {}'.format(barrel_alignment_type,np.array(mean_error_column).mean()))
df_sbf_alignment_errors.to_csv(output_path_stats+'4_SBF_Alignment_Errors.csv')

In [ ]:
# create vS1 clusters by finding the landmarks that fall within the aligned standard barrel field 

avg_sbf_file = input_path+'/sbf/average_barrel_field.am'
sg_sbf = AmiraSpatialGraph(avg_sbf_file,barrel_projections_present=True,reverse_barrel_direction=True)
sg_sbf.write_spatial_graph(input_path+'/sbf/average_barrel_field.am')

S1_convex_hull_name = input_path+'/sbf/average_barrel_field_S1_convex_hull.stl'
S1_hull = Surface(filename=S1_convex_hull_name)
S1_Layer_4_upper_filename = input_path+'/sbf/average_barrel_field_L4Upper.stl'
S1_Layer_4_lower_filename = input_path+'/sbf/average_barrel_field_L4Lower.stl'
S1_Layer_4_upper = Surface(filename=S1_Layer_4_upper_filename)
S1_Layer_4_lower = Surface(filename=S1_Layer_4_lower_filename)

barrel_alignment_type = 'use_barrels_alone '
for exp_name in ['MG49_lhs','MG49_rhs','MG50_lhs','MG50_rhs']:
    #print(exp_name)
    sg = AmiraSpatialGraph(output_path_surfaces+exp_name+'_barrels.am',barrel_projections_present=True)

    # Register the standard sbf to these barrels
    if barrel_alignment_type == 'use_barrels_alone':
        matching_barrels = MatchBarrels(sg.barrels,sg_sbf.barrels,use_barrels_alone=True)
    elif barrel_alignment_type == 'use_projections_alone':
        matching_barrels = MatchBarrels(sg.barrels,sg_sbf.barrels,use_projections_alone=True)
    else:
        matching_barrels = MatchBarrels(sg.barrels,sg_sbf.barrels,)
    ref_pts,pts = matching_barrels.get_matching_barrel_centroids()
    mached_barrels_ref,matched_barrels = matching_barrels.get_matching_barrels()

    # align barrel centroid
    icp,txmat = Landmarks(pts=pts).align_landmarks(ref_pts)
    sg_sbf.apply_transformation(txmat)
    sg_sbf.write_spatial_graph(output_path_spatial_graphs+exp_name+'_sbf.am')
    S1_hull.apply_icp_transform(icp)
    S1_Layer_4_upper.apply_icp_transform(icp)
    S1_Layer_4_lower.apply_icp_transform(icp)
    S1_hull.write_surface_mesh(output_path_surfaces+exp_name+'_reg_s1_hull.vtk')
    S1_Layer_4_upper.write_surface_mesh(output_path_layers+exp_name+'_s1_L4_upper.vtk')
    S1_Layer_4_lower.write_surface_mesh(output_path_layers+exp_name+'_s1_L4_lower.vtk')

    s1_hull_ind = Surface(pts=sg.barrels.all_rows_graphdata.edge_pt_coords).create_delunay_surface_3d(return_hull=True,\
                    output_filename = output_path_surfaces+exp_name+'_s1_hull.vtk')
    
    Surface(pts=sg.barrels.all_rows_graphdata.edge_pt_coords).create_delunay_surface_3d(make_cube=True,bounding_offset=10,\
                    output_filename = output_path_surfaces+exp_name+'_s1_hull_cube.vtk')
    
    # find landmarks within the aligned sbf
    for voxel_size in [50,100,200,300,400,500]:
        for cutoff in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5]:
            l = Landmarks(output_path_landmarks+'/Density_Clusters/'+'{}_vS1_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.format(exp_name,str(voxel_size),str(cutoff)))
            S1_hull.append(s1_hull_ind)
            l.get_landmarks_within_given_surface_using_vtk(s1_hull_ind,output_file=output_path_landmarks+'/Density_Clusters/'+'{}_vS1_rabies_landmarks_Within_Barrels_{}_{}.LandmarkAscii'.format(exp_name,str(voxel_size),str(cutoff)))

    l = Landmarks(output_path_landmarks+'{}_rabies_cortical_landmarks.LandmarkAscii'.format(exp_name))
    l.get_landmarks_within_given_surface_using_vtk(s1_hull_ind,output_file=output_path_landmarks+'/Density_Clusters/'+'{}_vS1_cortical_rabies_landmarks_Within_Barrels.LandmarkAscii'.format((exp_name)))
    

# Create vM1 Surfaces

In [ ]:
# 1) Get central axis and 2d pca
# 2) Based on the pca clip surface till it covers all the landmarks 


In [ ]:
def get_local_avg_axis(axis_field,pts,center_pt,pia,wm,radius):
    
    nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(axis_field,pts,radius,axis_validation_distance=3000)
    nearby_axes_sg = AmiraSpatialGraph(generic_graph=True)
    unit_vecs = []
    for axis in nearby_axes:
        nearby_axes_sg.graph_data.add_edge(axis[0],axis[1])
        unit_vecs.append(Vectors().get_unit_vec(axis[0],axis[1]))
    #print(len(nearby_axes_sg.graph_data.edge_list))
    #nearby_axes_sg.write_spatial_graph(output_root+'_axis_field_selected.am')
    wm_proj_pt=[]
    avg_uv = np.array(unit_vecs).mean(axis=0)
    end_pt = (avg_uv*-20000 + center_pt)
    wm_proj_pt,depth_wm = wm.get_vector_intersection_pt\
                    (end_pt,center_pt,extrapolation_len=0)
    if len(wm_proj_pt) == 0:
        end_pt = (avg_uv*20000 + center_pt)
        wm_proj_pt,depth_wm = wm.get_vector_intersection_pt\
                    (end_pt,center_pt,extrapolation_len=0)
        
    
    pia_proj_pt=[]
    end_pt = (avg_uv*20000 + center_pt)
    pia_proj_pt,depth_pia = pia.get_vector_intersection_pt\
                    (end_pt,center_pt,extrapolation_len=0)
    if len(pia_proj_pt)==0:
        end_pt = (avg_uv*-20000 + center_pt)
        pia_proj_pt,depth_pia = pia.get_vector_intersection_pt\
                    (end_pt,center_pt,extrapolation_len=0)
    
    if len(wm_proj_pt) > 0 and len(pia_proj_pt) > 0:
        return avg_uv,[wm_proj_pt,pia_proj_pt]
    else:
        return avg_uv,None

In [ ]:
def get_pca_from_flattened_pts(flattened_pts,output_path,l5_pt,central_axis,pia,wm,write_output=True):
    
    tr_mat,rot_mat,translation,pca_components,tr_pts = \
        Vectors().get_pca_transformation_matrix(flattened_pts,translate_to_center=True)
    print(pca_components)
    delunay_3d = Surface(pts=flattened_pts)
    rabies_hull_extended = delunay_3d.create_delunay_surface_3d(return_hull=True,make_cube=True)
    
    pcas = []
    neg_pcas = []
    for i in range(2):                                                        
        pca_component = pca_components[i]
        end_pt = Vectors().create_pt_along_vector_at_given_distance(-10000,[0,0,0],pca_component)

        vectors_sg_pca,wm_pts,pia_pts = delunay_3d.get_axis_field_within_instersecting_surface\
                                            (rabies_hull_extended,[[list(l5_pt),list(end_pt+l5_pt)]],\
                                             trim_edge=True)
        if write_output: 
            vectors_sg_pca.write_spatial_graph(output_path+'_vM1_PCA_{}.am'.format(i))
        pcas.append(vectors_sg_pca.graph_data.edge_list[0])
    
        end_pt = Vectors().create_pt_along_vector_at_given_distance(10000,[0,0,0],pca_component)
        vectors_sg_pca,wm_pts,pia_pts = delunay_3d.get_axis_field_within_instersecting_surface\
                                            (rabies_hull_extended,[[list(l5_pt),list(end_pt+l5_pt)]],\
                                             trim_edge=True)
        if write_output:
            vectors_sg_pca.write_spatial_graph(output_path+'_vM1_PCA_{}_Neg.am'.format(i))
        neg_pcas.append(vectors_sg_pca.graph_data.edge_list[0])
    
    sg = AmiraSpatialGraph(generic_graph=True)
    sg.graph_data.add_edge(l5_pt,central_axis[1])
    if write_output:
        sg.write_spatial_graph(output_path+'_vM1_PCA_2.am')
    pcas.append(sg.graph_data.edge_list[0])
    
    sg = AmiraSpatialGraph(generic_graph=True)
    sg.graph_data.add_edge(l5_pt,central_axis[0])
    if write_output:
        sg.write_spatial_graph(output_path+'_vM1_PCA_2_Neg.am')
    neg_pcas.append(sg.graph_data.edge_list[0])
    
    return pcas,neg_pcas

In [ ]:
def get_pca(landmarks,axis_field,pia,wm,output_path):
    
    # Find central axis
    centroid = np.array(landmarks).mean(axis=0)
    Landmarks(pts=[centroid]).write_landmarks(output_path+'_vM1_Centroid.landmarksAscii')
    
    # Find central axis
    uv,central_axis = get_local_avg_axis(axis_field,[centroid],centroid,pia,wm,800)
    write_edge_sg(central_axis,output_path+'_vM1_Central_Axis.am')
    
    # PCA 2d 
    # Find central plane and project pts onto it
    projected_pts = Vectors().get_projection_pts_to_plane(landmarks,central_axis,centroid)
    Landmarks(pts=projected_pts).write_landmarks(output_path+'_vM1_flattened_pts.landmarksAscii')
    
    pcas,neg_pcas = get_pca_from_flattened_pts(projected_pts,output_path,centroid,central_axis,pia,wm,write_output=True)
    # 
    
    return pcas,neg_pcas,central_axis

In [ ]:
def get_translated_axis(central_axis,pca_end_pt):
    up_pt = Vectors().create_pt_along_vector_at_given_distance(10000,central_axis[0],central_axis[1])
    down_pt = Vectors().create_pt_along_vector_at_given_distance(-10000,central_axis[0],central_axis[1])
    central_axis = np.array([up_pt,down_pt])
    pca_end_pt = np.array(pca_end_pt)
    central_pt = (up_pt+down_pt)/2
    translated_axis = [up_pt + pca_end_pt - central_pt , down_pt + pca_end_pt - central_pt]
    
    return translated_axis

In [ ]:
def get_trimmed_landmarks_for_given_half(central_axis,central_pt,pt1,pt2,pt3,landmarks,axis_field,pia,wm):
    '''Trim the landmarks bound by the required 2 quandrants so that we can find the extreme pt
       Pt 1 and 2 are colinear and 3 is perpendicular
    '''
    # 1) Find the corner z axes for the required half
    # extend axis first
    #central_pt = (np.array(central_axis[0]) + np.array(central_axis[1])) / 2
    
    axis1_pt = Vectors().create_pt_along_unit_vector_at_given_distance(10000,central_pt,Vectors().get_unit_vec(central_pt,pt2))
    axis2_pt = Vectors().create_pt_along_unit_vector_at_given_distance(10000,central_pt,Vectors().get_unit_vec(central_pt,pt3))
    
    axis1 = get_translated_axis(central_axis,axis1_pt)
    axis2 = get_translated_axis(central_axis,axis2_pt)
    axis3_pt = Vectors().create_pt_along_unit_vector_at_given_distance(10000,pt2,Vectors().get_unit_vec(central_pt,pt1))
    axis4_pt = Vectors().create_pt_along_unit_vector_at_given_distance(10000,pt3,Vectors().get_unit_vec(central_pt,pt1))
    axis3 = get_translated_axis(axis1,axis3_pt)
    axis4 = get_translated_axis(axis2,axis4_pt)
    
    # 2) Find the bounding box and the landmarks enclosed within it
    boundary_pts = [axis1[0],axis1[1],axis2[0],axis2[1],axis3[0],axis3[1],axis4[0],axis4[1]]
    boudning_srf = Surface(pts=boundary_pts).create_delunay_surface_3d(return_hull=True,make_cube=False,)
    #Surface(polydata=boudning_srf).write_surface_mesh(output_root+'bounding.vtk')
    landmarks_within_half = Landmarks(pts=landmarks).get_landmarks_within_given_surface_using_vtk(boudning_srf)
    Landmarks(pts=landmarks_within_half).write_landmarks(output_root+'landmarks_within_half')
    Landmarks(pts=[axis1_pt,axis2_pt,axis3_pt,axis4_pt]).write_landmarks(output_root+'central_plane_pts')
    # Create plane passing though the half in the middle
    surf = Surface(pts=[axis1_pt,axis2_pt,axis3_pt,axis4_pt])
    central_plane = surf.create_delunay_surface_2d(return_hull=True)
    Surface(polydata=central_plane).write_surface_mesh(output_root+'central_plane.vtk')
    return landmarks_within_half,central_plane

In [ ]:
def get_axis_intersection_pia_wm(vec,center_pt,pia,wm):
    
    avg_uv = Vectors().get_unit_vec(vec[0],vec[1])
    pia_proj_pt = []
    wm_proj_pt = []
    
    end_pt = (avg_uv*6000 + center_pt)
    pia_proj_pt,depth_pia = pia.get_vector_intersection_pt\
                    (end_pt,center_pt,extrapolation_len=0)
    if len(pia_proj_pt) == 0:
        end_pt = (avg_uv*-6000 + center_pt)
        pia_proj_pt,depth_pia = pia.get_vector_intersection_pt\
                    (end_pt,center_pt,extrapolation_len=0)
        end_pt = (avg_uv*6000 + center_pt)
        wm_proj_pt,depth_wm = wm.get_vector_intersection_pt\
                        (end_pt,center_pt,extrapolation_len=0)
    else:
        end_pt = (avg_uv*-6000 + center_pt)
        wm_proj_pt,depth_wm = wm.get_vector_intersection_pt\
                        (end_pt,center_pt,extrapolation_len=0)
    return [wm_proj_pt,pia_proj_pt]

In [ ]:
def write_edge_sg(edge,output_filename):
    if edge is not None and len(edge)>0:
        sg_axis = AmiraSpatialGraph(generic_graph=True)
        sg_axis.graph_data.add_edge(edge[0],edge[1])
        sg_axis.write_spatial_graph(output_filename)
    

In [ ]:
def get_boundaryline_axes(central_axis,ax1,ax2,ax3,axis_field,landmarks,pia,wm):
    # find pts parallel to ax1 and parpendicular to ax2 and 3
    # and find the avg of nearby axes to these pts 
    # find the farthest pt on this half and position the axis on it
    
    #center_pt = (np.array(central_axis[0]) + np.array(central_axis[0])) / 2
    # now find the farthest pt and position the avg axis here
    landmarks_within_half,central_plane = get_trimmed_landmarks_for_given_half(central_axis,ax1[0],ax1[1],ax2[1],ax3[1],landmarks,axis_field,\
                                                                 pia,wm)
    
    print('number pts in the half {}'.format(len(landmarks_within_half)))
    #print(central_plane.GetNumberOfCells())
    # find the avg axis 
    pts = []
    pts.append(ax1[1])
    pts.append(np.array(ax2[1]) + np.array(ax1[1])-np.array(ax1[0]))
    pts.append(np.array(ax3[1]) + np.array(ax1[1])-np.array(ax1[0]))
    
    #Landmarks(pts=pts).write_landmarks(output_root+'bounds.landmarksascii')
    uv,avg_axis = get_local_avg_axis(axis_field,pts,ax1[1],pia,wm,1000)
    #uv = Vectors().get_unit_vec(avg_axis[0],avg_axis[1])
    
    #print('avg_axis',avg_axis)
    # find the farthest axis that passes though the central plane and is at max dist from center
    max_dist = 0
    extreme_axis = []
    max_pt = []
    plane_intersecn_pts = []
    obb = vtk.vtkOBBTree()
    obb.SetDataSet(central_plane)
    obb.BuildLocator()
    for pt in landmarks_within_half:
        #axis = get_axis_intersection_pia_wm(avg_axis,pt,pia,wm)
        points = vtk.vtkPoints()
        code = obb.IntersectWithLine(uv*6000+pt, uv*-6000+pt, points, None)
        if points.GetNumberOfPoints() > 0:
            projected_2d_pts = Vectors().get_projection_pts_to_plane([points.GetPoint(0)],ax2,ax2[0])
            #print('projected_pt',projected_2d_pts[0])
            plane_intersecn_pts.append(points.GetPoint(0))
            dist = distance.euclidean(ax1[0],projected_2d_pts[0])
            if dist > max_dist:
                max_dist = dist
                extreme_axis = [uv*6000+pt, uv*-6000+pt]
                max_pt = pt
    #print(plane_intersecn_pts)
    return extreme_axis,landmarks_within_half,max_pt,central_plane

def get_boundary_axes(central_axis,axis_field,pcas,neg_pcas,landdmarks,pia,wm,offset=500):
    
    # from pca end pts get 1000um axes
    # right side
    avg_axis_right,landmarks_within_half_right,landmarks_within_half_right_max,central_plane_right = get_boundaryline_axes(central_axis,neg_pcas[1],pcas[0],neg_pcas[0],\
                                                                                                                           axis_field,landdmarks,pia,wm)
    write_edge_sg(avg_axis_right,output_root+'avg_axis_right')
    Landmarks(pts=landmarks_within_half_right).write_landmarks(output_root+'landmarks_within_half_right')
    Landmarks(pts=[landmarks_within_half_right_max]).write_landmarks(output_root+'landmarks_within_half_right_max')
    Surface(polydata=central_plane_right).write_surface_mesh(output_root+'central_plane_right.vtk')
    #Landmarks(pts=plane_intersecn_pts_right).write_landmarks(output_root+'plane_intersecn_pts_right')
    
    avg_axis_left,landmarks_within_half_left,landmarks_within_half_left_max,central_plane_left = get_boundaryline_axes(central_axis,pcas[1],pcas[0],neg_pcas[0],\
                                                                                                                       axis_field,landdmarks,pia,wm)
    write_edge_sg(avg_axis_left,output_root+'avg_axis_left')
    Landmarks(pts=landmarks_within_half_left).write_landmarks(output_root+'landmarks_within_half_left')
    Landmarks(pts=[landmarks_within_half_left_max]).write_landmarks(output_root+'landmarks_within_half_left_max')
    Surface(polydata=central_plane_left).write_surface_mesh(output_root+'central_plane_left.vtk')
    
    avg_axis_front,landmarks_within_half_front,landmarks_within_half_front_max,central_plane_front = get_boundaryline_axes(central_axis,pcas[0],pcas[1],neg_pcas[1],\
                                                                                                                           axis_field,landdmarks,pia,wm)
    write_edge_sg(avg_axis_front,output_root+'avg_axis_front')
    Landmarks(pts=landmarks_within_half_front).write_landmarks(output_root+'landmarks_within_half_front')
    Landmarks(pts=[landmarks_within_half_front_max]).write_landmarks(output_root+'landmarks_within_half_front_max')
    Surface(polydata=central_plane_front).write_surface_mesh(output_root+'central_plane_front.vtk')
    
    avg_axis_back,landmarks_within_half_back,landmarks_within_half_back_max,central_plane_back = get_boundaryline_axes(central_axis,neg_pcas[0],pcas[1],neg_pcas[1],\
                                                                                                                       axis_field,landdmarks,pia,wm)
    write_edge_sg(avg_axis_back,output_root+'avg_axis_back')
    Landmarks(pts=landmarks_within_half_back).write_landmarks(output_root+'landmarks_within_half_back')
    Landmarks(pts=[landmarks_within_half_back_max]).write_landmarks(output_root+'landmarks_within_half_back_max')
    Surface(polydata=central_plane_back).write_surface_mesh(output_root+'central_plane_back.vtk')
    
    print('extreme axes are: ')
    print(avg_axis_right,avg_axis_left,avg_axis_front,avg_axis_back)
    
    return avg_axis_right,avg_axis_left,avg_axis_front,avg_axis_back

In [ ]:
def create_vm1_surface(pia_surf,wm_surf,pcas,neg_pcas,axis_field,landmarks,central_axis,cutting_order):
    
    # find the edges on the border and find avg axis
    # position the edge on the extreme pt based on its 3d distance from center
    extreme_axis_x,extreme_axis_neg_x,extreme_axis_y,extreme_axis_neg_y = get_boundary_axes(central_axis,axis_field,\
                                                                                            pcas,neg_pcas,landmarks,pia_surf,pia_surf)
    
    # Clip surfaces usig the border planes
    a,y_delta = Vectors().get_center_and_unit_vec(pcas[0][0],pcas[0][1])
    b,x_delta = Vectors().get_center_and_unit_vec(pcas[1][0],pcas[1][1])
    
    plane_right,tri_right = Surface().get_cutting_plane(extreme_axis_x[0],extreme_axis_x[1],\
                                                        extreme_axis_x[0]+100*y_delta)
    plane_left,tri_left = Surface().get_cutting_plane(extreme_axis_neg_x[0],extreme_axis_neg_x[1],\
                                                        extreme_axis_neg_x[0]+100*y_delta)
    plane_front,tri_top = Surface().get_cutting_plane(extreme_axis_y[0],extreme_axis_y[1],\
                                                        extreme_axis_y[0]+100*x_delta)
    plane_back,tri_bottom = Surface().get_cutting_plane(extreme_axis_neg_y[0],extreme_axis_neg_y[1],\
                                                        extreme_axis_neg_y[0]+100*x_delta)
    Surface(polydata=tri_right).write_surface_mesh(output_root+'tri_right.vtk')
    Surface(polydata=tri_left).write_surface_mesh(output_root+'tri_left.vtk')
    Surface(polydata=tri_top).write_surface_mesh(output_root+'tri_top.vtk')
    Surface(polydata=tri_bottom).write_surface_mesh(output_root+'tri_bottom.vtk')
    
    #Surface(polydata=)
#     vm1_final,vm1_hull,vm1_pia,vm1_wm = Surface().clip_surfaces_using_given_planes(pia_surf,wm_surf,plane_right,plane_left,\
#                                                                            plane_front,plane_back,landmarks=[],\
#                                                                            central_axis=central_axis,\
#                                                                            order_of_cutting=cutting_order)
    return plane_right,plane_left,plane_front,plane_back
    #return vm1_final,vm1_hull,vm1_pia,vm1_wm

In [ ]:
df_vm1_surf_pca_stats = pd.DataFrame(columns=['Exp_Name','Surface_Resolution','Voxel_Size','Density_Cutoff',\
                                              'Surface_Area_Pia','Surface_Area_WM','Volume',\
                                              'PCA_0_Extent','PCA_1_Extent','PCA_2_Extent'])

    
#pia = Surface(filename=output_path_surfaces+exp+'_pia_voxel_size_{}.vtk'.format(surf_res))
#wm = Surface(filename=output_path_surfaces+exp+'_WM_voxel_size_{}.vtk'.format(surf_res))
#avg_L5_m1_pts = Landmarks(filename=output_path_layers+exp+'_vM1_L5_avg_pts.landmarksAscii').pts
for voxel_size in [50,100,200,300,400,450,500]:
    for cutoff in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
        for exp in exp_names:
#                 exp='MG50_rhs'
#                 voxel_size = 100
#                 cutoff = 0.3
            if exp == 'MG48_rhs' :
                cutting_order = [True,False,True,False]
            else:
                cutting_order = [True,False,False,True]

            print(cutting_order)
            pia = Surface(filename=output_path_surfaces+exp+'_pia_open_bottom_voxel_size_{}.vtk'.format(50))
            wm = Surface(filename=output_path_surfaces+exp+'_WM_open_bottom_voxel_size_{}.vtk'.format(50))
            print(exp,voxel_size,cutoff)
            vm1_rabies_pts = Landmarks(output_path_landmarks+'/Density_Clusters/'+exp+\
                                       '_vM1_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.format(voxel_size,cutoff)).pts
            vm1_axis_field = AmiraSpatialGraph(output_path_axis_fileds+exp+'_wm_to_pia_axis_field_voxel_size_{}.am'.format(50),generic_graph=True)

            print(len(vm1_rabies_pts),len(vm1_axis_field.graph_data.edge_list))

            if len(vm1_rabies_pts) > 20:
                pcas,neg_pcas,central_axis = get_pca(vm1_rabies_pts,vm1_axis_field.graph_data.edge_list,pia,wm,\
                                                     output_path_vM1+'{}_surf_res_{}_voxel_size_{}_cutoff_{}'.format\
                                                     (exp,50,voxel_size,cutoff))

                print(len(pcas),len(neg_pcas))
                plane_right,plane_left,plane_front,plane_back = create_vm1_surface(pia,wm,pcas,neg_pcas,\
                                                                vm1_axis_field.graph_data.edge_list,landmarks=vm1_rabies_pts,\
                                                                central_axis=central_axis,cutting_order=cutting_order)

                for surf_res in surface_resolutions:
                    print(surf_res)
                    pia = Surface(filename=output_path_surfaces+exp+'_pia_open_bottom_voxel_size_{}.vtk'.format(surf_res))
                    wm = Surface(filename=output_path_surfaces+exp+'_WM_open_bottom_voxel_size_{}.vtk'.format(surf_res))
                    
                    
                    vm1_final,vm1_hull,vm1_pia,vm1_wm = Surface().clip_surfaces_using_given_planes(pia.surface,wm.surface,\
                                                                           plane_back,plane_front,\
                                                                           plane_right,plane_left,landmarks=[],\
                                                                           central_axis=central_axis,\
                                                                           order_of_cutting=cutting_order)
                    
                    Surface(polydata=vm1_final).write_surface_mesh(output_path_vM1+exp+'_vM1_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.\
                                                                   format(surf_res,voxel_size,cutoff))
                    Surface(polydata=vm1_hull).write_surface_mesh(output_path_vM1+exp+'_vM1_hull_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.\
                                                                  format(surf_res,voxel_size,cutoff))
                    Surface(polydata=vm1_pia).write_surface_mesh(output_path_vM1+exp+'_vm1_pia_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.\
                                                                 format(surf_res,voxel_size,cutoff))
                    Surface(polydata=vm1_wm).write_surface_mesh(output_path_vM1+exp+'_vm1_wm_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.\
                                                                format(surf_res,voxel_size,cutoff))

                    df_vm1_surf_pca_stats_temp = pd.DataFrame(columns=['Exp_Name','Surface_Resolution','Voxel_Size','Density_Cutoff',\
                                              'Surface_Area_Pia','Surface_Area_WM','Volume',\
                                              'PCA_0_Extent','PCA_1_Extent','PCA_2_Extent'])
                    df_vm1_surf_pca_stats_temp['Exp_Name'] = [exp]
                    df_vm1_surf_pca_stats_temp['Surface_Resolution'] = [surf_res]
                    df_vm1_surf_pca_stats_temp['Voxel_Size'] = [voxel_size]
                    df_vm1_surf_pca_stats_temp['Density_Cutoff'] = [cutoff]
                    df_vm1_surf_pca_stats_temp['Surface_Area_Pia'] = [Surface(polydata=vm1_pia).get_surface_properties(prop='area')]
                    df_vm1_surf_pca_stats_temp['Surface_Area_WM'] = [Surface(polydata=vm1_wm).get_surface_properties(prop='area')]
                    df_vm1_surf_pca_stats_temp['Volume'] = [Surface(polydata=vm1_hull).get_surface_properties(prop='volume')]
                    df_vm1_surf_pca_stats_temp['PCA_0_Extent'] = [np.sqrt((pcas[0][1][0] - neg_pcas[0][1][0])**2 + (pcas[0][1][1] - neg_pcas[0][1][1])**2 + (pcas[0][1][2] - neg_pcas[0][1][2])**2 )]
                    df_vm1_surf_pca_stats_temp['PCA_1_Extent'] = [np.sqrt((pcas[1][1][0] - neg_pcas[1][1][0])**2 + (pcas[1][1][1] - neg_pcas[1][1][1])**2 + (pcas[1][1][2] - neg_pcas[1][1][2])**2 )]
                    df_vm1_surf_pca_stats_temp['PCA_2_Extent'] = [np.sqrt((pcas[2][1][0] - neg_pcas[2][1][0])**2 + (pcas[2][1][1] - neg_pcas[2][1][1])**2 + (pcas[2][1][2] - neg_pcas[2][1][2])**2 )]

                    df_vm1_surf_pca_stats = df_vm1_surf_pca_stats.append(df_vm1_surf_pca_stats_temp)
                   
df_vm1_surf_pca_stats.to_csv(output_path_stats+'4_vM1_Surface_PCA_Stats.csv')

In [ ]:
def trim_axis_to_surface(axis_filename,surface,write=True):
    #print(axis_filename)
    
    axis = AmiraSpatialGraph(axis_filename,generic_graph=True)
    #print(axis.graph_data.edge_list)
    deluny =Surface(polydata=surface).create_delunay_surface_3d(return_hull=True)
    vectors_sg_pca,wm_pts,pia_pts = Surface(polydata=deluny).get_axis_field_within_instersecting_surface\
                                                    (surface,axis.graph_data.edge_list,vec_extension_len=100000,trim_edge=True)
    #print(vectors_sg_pca.graph_data.edge_list)
    if write==True:
        vectors_sg_pca.write_spatial_graph(axis_filename)

In [ ]:
# Extend pca to touch the local surface
for voxel_size in [50,100,200,300,400,450,500]:
    for cutoff in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
        for surf_res in [50]:#surface_resolutions:
            for exp in exp_names:
                if os.path.exists(output_path_vM1+exp+'_vM1_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.\
                                                                       format(surf_res,voxel_size,cutoff)):
                    vm1_surf = Surface(output_path_vM1+exp+'_vM1_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.\
                                                                       format(surf_res,voxel_size,cutoff))
                                                                            
                    trim_axis_to_surface(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_0.am'.\
                                                                           format(surf_res,voxel_size,cutoff),\
                                                                             vm1_surf.surface,write=True)
                    trim_axis_to_surface(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_1.am'.\
                                                                           format(surf_res,voxel_size,cutoff),\
                                                                             vm1_surf.surface,write=True)
                    trim_axis_to_surface(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_2.am'.\
                                                                           format(surf_res,voxel_size,cutoff),\
                                                                             vm1_surf.surface,write=True)
                    trim_axis_to_surface(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_0_Neg.am'.\
                                                                           format(surf_res,voxel_size,cutoff),\
                                                                             vm1_surf.surface,write=True)
                    trim_axis_to_surface(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_1_Neg.am'.\
                                                                           format(surf_res,voxel_size,cutoff),\
                                                                             vm1_surf.surface,write=True)
                    trim_axis_to_surface(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_2_Neg.am'.\
                                                                           format(surf_res,voxel_size,cutoff),\
                                                                             vm1_surf.surface,write=True)
                                       
                    
                    

In [ ]:
# best voxel size and cutoff
# df_vm1_surf_pca_stats = pd.read_csv(output_path_stats+'4_vM1_Surface_PCA_Stats.csv')
# for voxel_size in [50,100,200,300,400,500]:
#     for cutoff in [0.05,0.1]:#[0,0.05,0.1,0.15,0.2,0.3,0.4,0.5]:
        
#         selected = df_vm1_surf_pca_stats[np.logical_and( np.array(df_vm1_surf_pca_stats['Voxel_Size']==voxel_size) ,\
#                                                         np.array( df_vm1_surf_pca_stats['Density_Cutoff']==cutoff),\
#                                                         np.array( df_vm1_surf_pca_stats['Surface_Resolution']==50))]
#         pia_cov = selected['Surface_Area_Pia'].std()/selected['Surface_Area_Pia'].mean()
#         volume_cov = selected['Volume'].std()/selected['Volume'].mean()
#         pca0_cov = selected['PCA_0_Extent'].std()/selected['PCA_0_Extent'].mean()
#         pca1_cov = selected['PCA_1_Extent'].std()/selected['PCA_1_Extent'].mean()
#         print(voxel_size,cutoff,pia_cov,volume_cov,pca1_cov,pca0_cov,(pca1_cov+pca0_cov)/2)

In [ ]:
# Trim neun landmarks for vM1
 
for exp in exp_names:
    print(exp)
    if exp.find('MG48')>=0:
        landmarks = Landmarks(output_path_landmarks+exp[:-4]+'_neun_landmarks.LandmarkAscii')
    else:
        landmarks = Landmarks(output_path_landmarks+exp+'_neun_landmarks.LandmarkAscii')
    
    for voxel_size in [500]:#[50,100,200,300,400,500]:
        for cutoff in [0.05]:#[0,0.05,0.1,0.15,0.2,0.3,0.4,0.5]:
            vm1_surf = Surface(output_path_vM1+exp+'_vM1_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.format(50,voxel_size,cutoff))

            cluster_in_bounds = landmarks.get_landmarks_within_given_surface(vm1_surf.surface)
            Landmarks(pts=cluster_in_bounds).write_landmarks(output_path_landmarks+exp+'_neun_landmarks_vM1_surf_res_{}_voxel_size_{}_cutoff_{}.LandmarkAscii'.format(50,voxel_size,cutoff))

## Create vM1 Axis Field

In [ ]:
# Create uniform vM1 axis field 
voxel_size = 500
cutoff=0.1
surf_res = 50
#for voxel_size in [50,100,200,300,400,450,500]:
#    for cutoff in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5]:
#for surf_res in [400,450,500]:#[50,100,200,300]:#surface_resolutions:
for exp in exp_names:

    if os.path.exists(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_0.am'.\
                                         format(50,voxel_size,cutoff)) and \
    len(AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_0.am'.\
                                         format(50,voxel_size,cutoff),generic_graph=True).\
       graph_data.edge_list) > 0:
        pia = Surface(output_path_vM1+exp+'_vm1_pia_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.format(surf_res,voxel_size,cutoff))
        wm = Surface(output_path_vM1+exp+'_vm1_wm_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.format(surf_res,voxel_size,cutoff))

        axis_field  = AmiraSpatialGraph(output_path_axis_fileds+exp+'_wm_to_pia_axis_field_voxel_size_{}.am'.\
                                        format(surf_res),generic_graph=True).graph_data.edge_list


        horizontal_axis = [AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_0.am'.\
                                             format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1],\
                    AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_0_Neg.am'.\
                                      format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1]]
        vertical_axis = [AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_2.am'.\
                                           format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1],\
                    AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_2_Neg.am'.\
                                      format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1]]
        side_axis = [AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_1.am'.\
                                       format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1],\
                    AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_1_Neg.am'.\
                                      format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1]]
        center_pt = Landmarks(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_Centroid.landmarksAscii'.\
                              format(50,voxel_size,cutoff)).pts[0]

        # make the grid
        grid_pts = []
        step_size = 500
        radius = 800
        uniform_axis_field = AmiraSpatialGraph(generic_graph=True)

        #uniform_axis_field.graph_data.add_edge(vertical_axis[0],vertical_axis[1])
        # project the central axis to wm 
        side_uv_left = Vectors().get_unit_vec(center_pt,side_axis[0])
        side_uv_right = Vectors().get_unit_vec(center_pt,side_axis[1])
        start_ind = 0
        step_direction = -1
        # extend the horzontal axis
        #     horizontal_axis_begin = Vectors().create_pt_along_vector_at_given_distance(10000,horizontal_axis[1],horizontal_axis[0])
        #     horizontal_axis_end = Vectors().create_pt_along_vector_at_given_distance(10000,horizontal_axis[0],horizontal_axis[1])

        for end_pt in [horizontal_axis[0],horizontal_axis[1]]:
            step_direction = -1*step_direction
            row_ind = step_direction
            for i in range(start_ind,20):

                uniform_axis_field_rowwise = AmiraSpatialGraph(generic_graph=True)
                uniform_axis_field_rowwise_left = AmiraSpatialGraph(generic_graph=True)
                uniform_axis_field_rowwise_right = AmiraSpatialGraph(generic_graph=True)
                pt = Vectors().create_pt_along_vector_at_given_distance(i*step_size,center_pt,end_pt)
                uv,axis = get_local_avg_axis(axis_field,[pt],pt,pia,wm,radius)
                #print(axis)
                if axis is not None:
                    uniform_axis_field.graph_data.add_edge(axis[0],axis[1])
                    uniform_axis_field_rowwise.graph_data.add_edge(axis[0],axis[1])
                    write_edge_sg(axis,output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}_{}.am'.\
                                  format(row_ind,0,surf_res,voxel_size,cutoff))
                    uniform_axis_field_rowwise_left.graph_data.add_edge(axis[0],axis[1])
                    uniform_axis_field_rowwise_right.graph_data.add_edge(axis[0],axis[1])
                # now move along the minor axis

                for j in range(1,10):
                    pt1 = (j*step_size * side_uv_left) + pt
                    uv,axis = get_local_avg_axis(axis_field,[pt1],pt1,pia,wm,radius)
                    if axis is not None:
                        uniform_axis_field.graph_data.add_edge(axis[0],axis[1])
                        uniform_axis_field_rowwise.graph_data.add_edge(axis[0],axis[1])
                        write_edge_sg(axis,output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}_{}.am'.\
                                      format(row_ind,-j,surf_res,voxel_size,cutoff))
                        uniform_axis_field_rowwise_left.graph_data.add_edge(axis[0],axis[1])
                for k in range(1,10):
                    pt2 = (k*step_size * side_uv_right) + pt
                    uv,axis = get_local_avg_axis(axis_field,[pt2],pt2,pia,wm,radius)
                    if axis is not None:
                        uniform_axis_field.graph_data.add_edge(axis[0],axis[1])
                        uniform_axis_field_rowwise.graph_data.add_edge(axis[0],axis[1])
                        write_edge_sg(axis,output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}_{}.am'.\
                                      format(row_ind,k,surf_res,voxel_size,cutoff))
                        uniform_axis_field_rowwise_right.graph_data.add_edge(axis[0],axis[1])
                if len(uniform_axis_field_rowwise.graph_data.edge_list) > 0:
                    uniform_axis_field_rowwise.write_spatial_graph(output_path_vM1_axis_field+exp+\
                                                                '_vM1_axis_field_row_{}_{}_{}_{}.am'.\
                                                                   format(row_ind,surf_res,voxel_size,cutoff)) 
                if len(uniform_axis_field_rowwise_left.graph_data.edge_list) > 0:
                    uniform_axis_field_rowwise_left.write_spatial_graph(output_path_vM1_axis_field+exp+\
                                                                    '_vM1_axis_field_row_{}_left_{}_{}_{}.am'.\
                                                                        format(row_ind,surf_res,voxel_size,cutoff)) 
                if len(uniform_axis_field_rowwise_right.graph_data.edge_list) > 0:
                    uniform_axis_field_rowwise_right.write_spatial_graph(output_path_vM1_axis_field+exp+\
                                                        '_vM1_axis_field_row_{}_right_{}_{}_{}.am'.\
                                                                         format(row_ind,surf_res,voxel_size,cutoff)) 
                row_ind = row_ind + step_direction
            start_ind = start_ind + 1

        uniform_axis_field.write_spatial_graph(output_path_vM1_axis_field+exp+'_vM1_axis_field_surf_res_{}_voxel_size_{}_cutoff_{}.am'.\
                                               format(surf_res,voxel_size,cutoff))  
        

In [ ]:
# write axis field stats
cols =['Exp_Name','Voxel_Size','Vector_Lengths_Mean_WM_To_Pia','Vector_Lengths_Std_WM_To_Pia']
df_cortex_len_main = pd.DataFrame(columns=cols)
for surface_resolution in [50]:#surface_resolutions: 
    df_cortex_len = pd.DataFrame(columns=cols)
    for exp_name in exp_names :
        vectors_wm_pia_sg = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_surf_res_{}_voxel_size_{}_cutoff_{}.am'.\
                                               format(surf_res,voxel_size,cutoff),generic_graph=True)
        vec_wm_pia_lens, vec_wm_pia_lens_mean,vec_wm_pia_lens_std = Vectors().get_vec_lengths(vectors_wm_pia_sg.graph_data.edge_list)
    
        df_cortex_len = df_cortex_len.append(pd.DataFrame(columns=df_cortex_len.columns,data=[[exp_name,\
                                            float(surface_resolution),vec_wm_pia_lens_mean,vec_wm_pia_lens_std]]))
    stats = Stats(df_cortex_len)
    stats_returned = stats.add_mean_std_cov(tangential_exps=True, coronal_exps=True,)
    
    df_cortex_len_main = df_cortex_len_main.append(stats_returned)
    
df_cortex_len_main.to_csv(output_path_stats+'4_vM1_Cortical_Thickness.csv')

# Create vS1 Surfaces

In [ ]:
def get_cylinder(radius,edge):
    center = (np.array(edge[0])+np.array(edge[1]))/2
    height = Vectors().get_vec_length(edge)
    cylinderSource = vtk.vtkCylinderSource()
    cylinderSource.SetCenter(center)
    cylinderSource.SetRadius(radius)
    cylinderSource.SetHeight(height)
    cylinderSource.SetResolution(100)
    cylinderSource.Update()

    pts = []
    ref_pts = []
    pts.append(center)
    end_pt = [center[0],cylinderSource.GetOutput().GetBounds()[2],center[2] ]
    pts.append(end_pt)

    ref_pts.append(center)
    end_pt = Vectors().get_unit_vec(center,edge[1])*height/2 + center
    ref_pts.append(end_pt)
    #Landmarks(pts=pts).write_landmarks(output_path_neun_profiles+'pts')
    #Landmarks(pts=ref_pts).write_landmarks(output_path_neun_profiles+'ref_pts')

    landmarks = Landmarks(pts=pts)
    icp,txmat = landmarks.align_landmarks(ref_pts)

    cyl_surf = Surface(polydata=cylinderSource.GetOutput())
    cyl_surf.apply_icp_transform(icp)
    #print(txmat)

    #Surface(polydata=cylinderSource.GetOutput()).write_surface_mesh(output_path_neun_profiles+'_cylinder.vtk')

    return cyl_surf.surface 

In [ ]:
df_vs1_surf_pca_stats = pd.DataFrame(columns=['Exp_Name','Surface_Resolution','Voxel_Size','Density_Cutoff',\
                                              'Surface_Area_Pia','Surface_Area_WM','Volume',\
                                              'PCA_0_Extent','PCA_1_Extent','PCA_2_Extent'])

#     if exp == 'MG48_rhs' :
#         cutting_order = [True,True,True,True]
#     else:
cutting_order = [True,True,False,True]

print(cutting_order)

    
#pia = Surface(filename=output_path_surfaces+exp+'_pia_voxel_size_{}.vtk'.format(surf_res))
#wm = Surface(filename=output_path_surfaces+exp+'_WM_voxel_size_{}.vtk'.format(surf_res))
#avg_L5_m1_pts = Landmarks(filename=output_path_layers+exp+'_vM1_L5_avg_pts.landmarksAscii').pts
for voxel_size in [50,100,200,300,400,450,500]:
    for cutoff in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
        for exp in exp_names:
#                 exp='MG50_rhs'
#                 voxel_size = 100
#                 cutoff = 0.3
            pia = Surface(filename=output_path_surfaces+exp+'_pia_open_bottom_voxel_size_{}.vtk'.format(50))
            wm = Surface(filename=output_path_surfaces+exp+'_WM_open_bottom_voxel_size_{}.vtk'.format(50))
            print(exp,voxel_size,cutoff)
            vs1_rabies_pts = Landmarks(output_path_landmarks+'/Density_Clusters/'+exp+\
                                       '_vS1_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.format(voxel_size,cutoff)).pts
            axis_field = AmiraSpatialGraph(output_path_axis_fileds+exp+'_wm_to_pia_axis_field_voxel_size_{}.am'.format(50),generic_graph=True)

            print(len(vs1_rabies_pts),len(axis_field.graph_data.edge_list))

            if len(vs1_rabies_pts) > 20:
                pcas,neg_pcas,central_axis = get_pca(vs1_rabies_pts,axis_field.graph_data.edge_list,pia,wm,\
                                                     output_path_vS1+'{}_surf_res_{}_voxel_size_{}_cutoff_{}'.format\
                                                     (exp,50,voxel_size,cutoff))

                print(len(pcas),len(neg_pcas))
                plane_right,plane_left,plane_front,plane_back = create_vm1_surface(pia,wm,pcas,neg_pcas,\
                                                                axis_field.graph_data.edge_list,landmarks=vs1_rabies_pts,\
                                                                central_axis=central_axis,cutting_order=cutting_order)

                for surf_res in surface_resolutions:
                    print(surf_res)
                    pia = Surface(filename=output_path_surfaces+exp+'_pia_open_bottom_voxel_size_{}.vtk'.format(surf_res))
                    wm = Surface(filename=output_path_surfaces+exp+'_WM_open_bottom_voxel_size_{}.vtk'.format(surf_res))
                    
                    
                    vs1_final,vs1_hull,vs1_pia,vs1_wm = Surface().clip_surfaces_using_given_planes(pia.surface,wm.surface,\
                                                                           plane_right,plane_left,\
                                                                           plane_front,plane_back,landmarks=[],\
                                                                           central_axis=central_axis,\
                                                                           order_of_cutting=cutting_order)
                    
                    Surface(polydata=vs1_final).write_surface_mesh(output_path_vS1+exp+'_vS1_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.\
                                                                   format(surf_res,voxel_size,cutoff))
                    Surface(polydata=vs1_hull).write_surface_mesh(output_path_vS1+exp+'_vS1_hull_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.\
                                                                  format(surf_res,voxel_size,cutoff))
                    Surface(polydata=vs1_pia).write_surface_mesh(output_path_vS1+exp+'_vs1_pia_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.\
                                                                 format(surf_res,voxel_size,cutoff))
                    Surface(polydata=vs1_wm).write_surface_mesh(output_path_vS1+exp+'_vs1_wm_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.\
                                                                format(surf_res,voxel_size,cutoff))

                    df_vs1_surf_pca_stats_temp = pd.DataFrame(columns=['Exp_Name','Surface_Resolution','Voxel_Size','Density_Cutoff',\
                                              'Surface_Area_Pia','Surface_Area_WM','Volume',\
                                              'PCA_0_Extent','PCA_1_Extent','PCA_2_Extent'])
                    df_vs1_surf_pca_stats_temp['Exp_Name'] = [exp]
                    df_vs1_surf_pca_stats_temp['Surface_Resolution'] = [surf_res]
                    df_vs1_surf_pca_stats_temp['Voxel_Size'] = [voxel_size]
                    df_vs1_surf_pca_stats_temp['Density_Cutoff'] = [cutoff]
                    df_vs1_surf_pca_stats_temp['Surface_Area_Pia'] = [Surface(polydata=vs1_pia).get_surface_properties(prop='area')]
                    df_vs1_surf_pca_stats_temp['Surface_Area_WM'] = [Surface(polydata=vs1_wm).get_surface_properties(prop='area')]
                    df_vs1_surf_pca_stats_temp['Volume'] = [Surface(polydata=vs1_hull).get_surface_properties(prop='volume')]
                    df_vs1_surf_pca_stats_temp['PCA_0_Extent'] = [np.sqrt((pcas[0][1][0] - neg_pcas[0][1][0])**2 + (pcas[0][1][1] - neg_pcas[0][1][1])**2 + (pcas[0][1][2] - neg_pcas[0][1][2])**2 )]
                    df_vs1_surf_pca_stats_temp['PCA_1_Extent'] = [np.sqrt((pcas[1][1][0] - neg_pcas[1][1][0])**2 + (pcas[1][1][1] - neg_pcas[1][1][1])**2 + (pcas[1][1][2] - neg_pcas[1][1][2])**2 )]
                    df_vs1_surf_pca_stats_temp['PCA_2_Extent'] = [np.sqrt((pcas[2][1][0] - neg_pcas[2][1][0])**2 + (pcas[2][1][1] - neg_pcas[2][1][1])**2 + (pcas[2][1][2] - neg_pcas[2][1][2])**2 )]

                    df_vs1_surf_pca_stats = df_vs1_surf_pca_stats.append(df_vs1_surf_pca_stats_temp)
                
    
                   
df_vs1_surf_pca_stats.to_csv(output_path_stats+'4_vS1_Surface_PCA_Stats.csv')

In [ ]:
# Find best voxel size and cutoff
# best voxel size and cutoff
for voxel_size in [50,100,200,300,400,500]:
    for cutoff in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5]:
        
        selected = df_vs1_surf_pca_stats[np.logical_and( np.array(df_vs1_surf_pca_stats['Voxel_Size']==voxel_size) ,\
                                                        np.array( df_vs1_surf_pca_stats['Density_Cutoff']==cutoff),\
                                                        np.array( df_vs1_surf_pca_stats['Surface_Resolution']==50))]
        pia_cov = selected['Surface_Area_Pia'].std()/selected['Surface_Area_Pia'].mean()
        volume_cov = selected['Volume'].std()/selected['Volume'].mean()
        pca0_cov = selected['PCA_0_Extent'].std()/selected['PCA_0_Extent'].mean()
        pca1_cov = selected['PCA_1_Extent'].std()/selected['PCA_1_Extent'].mean()
        print(voxel_size,cutoff,pia_cov,volume_cov,pca1_cov,pca0_cov,(pca1_cov+pca0_cov)/2)

In [ ]:
# Create columns for barrels and project them
for surf_res in surface_resolutions:
    for exp in exp_names_tangential:
        sg = AmiraSpatialGraph(output_path_surfaces+exp+'_barrels_{}.am'.format(surf_res),barrel_projections_present=True\
                               ,reverse_barrel_direction=True)
        all_barrel_columns_surfs = Surface()
        vs1_axis_field = AmiraSpatialGraph(generic_graph=True)
        for row in sg.barrels.rows_list:
            for barrel in row.single_barrels_list:
                # Extract barrel radius
                #print(barrel.contours.edge_pt_coords)
                if len(barrel.wm_projection_barrel_centroid) > 0 and len(barrel.pia_projection_barrel_centroid):
                    column_axis = [barrel.wm_projection_barrel_centroid,barrel.pia_projection_barrel_centroid]
                    vs1_axis_field.graph_data.add_edge(barrel.wm_projection_barrel_centroid,barrel.pia_projection_barrel_centroid)
                    cylinder = get_cylinder(250,column_axis)
                    surf = Surface(polydata=cylinder).create_delunay_surface_3d(return_hull=True,\
                                                                                output_filename=output_path_vS1+exp+'_'+barrel.barrel_name+'.vtk')
                    all_barrel_columns_surfs.append(surf)

        Surface(polydata=all_barrel_columns_surfs.surface).write_surface_mesh(output_path_vS1+exp+'_all_barrel_columns_{}.vtk'.format(surf_res))

        Surface(polydata=all_barrel_columns_surfs.surface).create_delunay_surface_3d(output_filename=output_path_vS1+exp+'_vS1_hull_{}.vtk'.format(surf_res))
        
        vs1_axis_field.write_spatial_graph(output_path_vS1_axis_field+exp+'_vS1_axis_field_{}.am'.format(surf_res))

## Create vS1 Axis Field

In [ ]:
## Create uniform vM1 axis field 
voxel_size = 500
cutoff=0.1
surf_res = 50
#for voxel_size in [50,100,200,300,400,450,500]:
#    for cutoff in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5]:
#for surf_res in [400,450,500]:#[50,100,200,300]:#surface_resolutions:
for exp in exp_names:

    if os.path.exists(output_path_vS1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_0.am'.\
                                         format(50,voxel_size,cutoff)) and \
    len(AmiraSpatialGraph(output_path_vS1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_0.am'.\
                                         format(50,voxel_size,cutoff),generic_graph=True).\
       graph_data.edge_list) > 0:
        pia = Surface(output_path_vS1+exp+'_vs1_pia_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.format(surf_res,voxel_size,cutoff))
        wm = Surface(output_path_vS1+exp+'_vs1_wm_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.format(surf_res,voxel_size,cutoff))

        axis_field  = AmiraSpatialGraph(output_path_axis_fileds+exp+'_wm_to_pia_axis_field_voxel_size_{}.am'.\
                                        format(surf_res),generic_graph=True).graph_data.edge_list


        horizontal_axis = [AmiraSpatialGraph(output_path_vS1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_0.am'.\
                                             format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1],\
                    AmiraSpatialGraph(output_path_vS1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_0_Neg.am'.\
                                      format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1]]
        vertical_axis = [AmiraSpatialGraph(output_path_vS1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_2.am'.\
                                           format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1],\
                    AmiraSpatialGraph(output_path_vS1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_2_Neg.am'.\
                                      format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1]]
        side_axis = [AmiraSpatialGraph(output_path_vS1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_1.am'.\
                                       format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1],\
                    AmiraSpatialGraph(output_path_vS1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_1_Neg.am'.\
                                      format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1]]
        center_pt = Landmarks(output_path_vS1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_Centroid.landmarksAscii'.\
                              format(50,voxel_size,cutoff)).pts[0]

        # make the grid
        grid_pts = []
        step_size = 500
        radius = 800
        uniform_axis_field = AmiraSpatialGraph(generic_graph=True)

        #uniform_axis_field.graph_data.add_edge(vertical_axis[0],vertical_axis[1])
        # project the central axis to wm 
        side_uv_left = Vectors().get_unit_vec(center_pt,side_axis[0])
        side_uv_right = Vectors().get_unit_vec(center_pt,side_axis[1])
        start_ind = 0
        step_direction = -1
        # extend the horzontal axis
        #     horizontal_axis_begin = Vectors().create_pt_along_vector_at_given_distance(10000,horizontal_axis[1],horizontal_axis[0])
        #     horizontal_axis_end = Vectors().create_pt_along_vector_at_given_distance(10000,horizontal_axis[0],horizontal_axis[1])

        for end_pt in [horizontal_axis[0],horizontal_axis[1]]:
            step_direction = -1*step_direction
            row_ind = step_direction
            for i in range(start_ind,20):

                uniform_axis_field_rowwise = AmiraSpatialGraph(generic_graph=True)
                uniform_axis_field_rowwise_left = AmiraSpatialGraph(generic_graph=True)
                uniform_axis_field_rowwise_right = AmiraSpatialGraph(generic_graph=True)
                pt = Vectors().create_pt_along_vector_at_given_distance(i*step_size,center_pt,end_pt)
                uv,axis = get_local_avg_axis(axis_field,[pt],pt,pia,wm,radius)
                #print(axis)
                if axis is not None:
                    uniform_axis_field.graph_data.add_edge(axis[0],axis[1])
                    uniform_axis_field_rowwise.graph_data.add_edge(axis[0],axis[1])
                    write_edge_sg(axis,output_path_vS1_axis_field+exp+'_vS1_axis_field_row_{}_{}_{}_{}_{}.am'.\
                                  format(row_ind,0,surf_res,voxel_size,cutoff))
                    uniform_axis_field_rowwise_left.graph_data.add_edge(axis[0],axis[1])
                    uniform_axis_field_rowwise_right.graph_data.add_edge(axis[0],axis[1])
                # now move along the minor axis

                for j in range(1,10):
                    pt1 = (j*step_size * side_uv_left) + pt
                    uv,axis = get_local_avg_axis(axis_field,[pt1],pt1,pia,wm,radius)
                    if axis is not None:
                        uniform_axis_field.graph_data.add_edge(axis[0],axis[1])
                        uniform_axis_field_rowwise.graph_data.add_edge(axis[0],axis[1])
                        write_edge_sg(axis,output_path_vS1_axis_field+exp+'_vS1_axis_field_row_{}_{}_{}_{}_{}.am'.\
                                      format(row_ind,-j,surf_res,voxel_size,cutoff))
                        uniform_axis_field_rowwise_left.graph_data.add_edge(axis[0],axis[1])
                for k in range(1,10):
                    pt2 = (k*step_size * side_uv_right) + pt
                    uv,axis = get_local_avg_axis(axis_field,[pt2],pt2,pia,wm,radius)
                    if axis is not None:
                        uniform_axis_field.graph_data.add_edge(axis[0],axis[1])
                        uniform_axis_field_rowwise.graph_data.add_edge(axis[0],axis[1])
                        write_edge_sg(axis,output_path_vS1_axis_field+exp+'_vS1_axis_field_row_{}_{}_{}_{}_{}.am'.\
                                      format(row_ind,k,surf_res,voxel_size,cutoff))
                        uniform_axis_field_rowwise_right.graph_data.add_edge(axis[0],axis[1])
                if len(uniform_axis_field_rowwise.graph_data.edge_list) > 0:
                    uniform_axis_field_rowwise.write_spatial_graph(output_path_vS1_axis_field+exp+\
                                                                '_vS1_axis_field_row_{}_{}_{}_{}.am'.\
                                                                   format(row_ind,surf_res,voxel_size,cutoff)) 
                if len(uniform_axis_field_rowwise_left.graph_data.edge_list) > 0:
                    uniform_axis_field_rowwise_left.write_spatial_graph(output_path_vS1_axis_field+exp+\
                                                                    '_vS1_axis_field_row_{}_left_{}_{}_{}.am'.\
                                                                        format(row_ind,surf_res,voxel_size,cutoff)) 
                if len(uniform_axis_field_rowwise_right.graph_data.edge_list) > 0:
                    uniform_axis_field_rowwise_right.write_spatial_graph(output_path_vS1_axis_field+exp+\
                                                        '_vS1_axis_field_row_{}_right_{}_{}_{}.am'.\
                                                                         format(row_ind,surf_res,voxel_size,cutoff)) 
                row_ind = row_ind + step_direction
            start_ind = start_ind + 1

        uniform_axis_field.write_spatial_graph(output_path_vS1_axis_field+exp+'_vS1_axis_field_surf_res_{}_voxel_size_{}_cutoff_{}.am'.\
                                               format(surf_res,voxel_size,cutoff))  

# Create Layer 5 plane based on Rabies vS1 and vM1 Axis Fields

In [ ]:
#TODO Fix MG48_lhs vM1 axis field
#TODO selecting axes near pts needs to check for selecting same axes again.. affects the avg vec length

In [ ]:
def get_intersecting_axes_and_layers(axis_field_sg,pts,surface=None,radius=500,make_cube=False,bounding_offset=0,\
                                     output_filename_axes=None,output_path_layers=None,df_cols=None,exp_name=''):
    
    
    # create a tight axis field selection
#     delunay_3d = Surface(pts=pts)
#     hull = vtk.vtkDelaunay3D()
#     hull.SetInputData(delunay_3d.surface)
#     hull.Update()
#     rabies_hull_tight = delunay_3d.create_delunay_surface_3d(return_hull=True,make_cube=True)
#     vectors_sg_tight,wm_pts,pia_pts = delunay_3d.get_axis_field_within_instersecting_surface\
#                                             (rabies_hull_tight,axis_field_sg.graph_data.edge_list)
    
    
#     # extend the axis field to include boudary landmarks
#     rabies_hull_extended = delunay_3d.create_delunay_surface_3d(return_hull=True,\
#                                                              make_cube=make_cube,bounding_offset=bounding_offset)
    
#     vectors_sg_extended,wm_pts,pia_pts = delunay_3d.get_axis_field_within_instersecting_surface\
#                                             (rabies_hull_extended,axis_field_sg.graph_data.edge_list,)
        
#     # select axis that are within the a given radius from the pts
#     selected_axes = Vectors().get_nearest_axes_to_pts_within_radius(vectors_sg_extended.graph_data.edge_list,pts,\
#                                                                  radius=radius)
#     final_vectors = Vectors().append(vectors_sg_tight.graph_data.edge_list,selected_axes,allow_duplicates=False)
      
    vectors_sg = AmiraSpatialGraph()
    if surface is not None:
        vectors_sg,wm_pts,pia_pts = surface.get_axis_field_within_instersecting_surface\
                                          (surface.surface,axis_field_sg.graph_data.edge_list,\
                                           trim_edge=False,vec_extension_len=3000)
    else:
        for pt in pts:
            min_edge,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = \
            Vectors().get_nearest_axis_to_pt(pt,axis_field_sg.graph_data.edge_list,axis_validation_distance=3000)
            vectors_sg.graph_data.add_edge(min_edge[0],min_edge[1])
    
    
    if output_filename_axes is not None:
        vectors_sg.write_spatial_graph(output_filename_axes)
    
    # create upper, lower and avg layer borders based on mean depth +- std
    axes_lens,l5_upper_depths,l5_lower_depths,l5_avg_depths = Vectors().get_landmarks_depth_along_axis_field\
                                                                (vectors_sg.graph_data.edge_list,pts,\
                                                                 radius=radius)
    
    avg_axes_len = np.array(axes_lens).mean(axis=0)
    std_axes_len = np.array(axes_lens).std(axis=0)
    avg_l5_upper_depths = np.array(l5_upper_depths).mean(axis=0)
    std_l5_upper_depths = np.array(l5_upper_depths).std(axis=0)
    avg_l5_lower_depths = np.array(l5_lower_depths).mean(axis=0)
    std_l5_lower_depths = np.array(l5_lower_depths).std(axis=0)
    avg_l5_avg_depths = np.array(l5_avg_depths).mean(axis=0)
    std_l5_avg_depths = np.array(l5_avg_depths).std(axis=0)
    
    np.array(l5_upper_depths).mean(axis=0)
    l5_upper_pts = Vectors().get_layer_pts_for_given_depth\
                    (vectors_sg.graph_data.edge_list,avg_l5_upper_depths-std_l5_upper_depths,\
                     realtive_depths=True,invert_axes=True)
    l5_lower_pts = Vectors().get_layer_pts_for_given_depth\
                    (vectors_sg.graph_data.edge_list,avg_l5_lower_depths+std_l5_lower_depths,\
                     realtive_depths=True,invert_axes=True)
    l5_avg_pts = Vectors().get_layer_pts_for_given_depth\
                    (vectors_sg.graph_data.edge_list,avg_l5_avg_depths,\
                     realtive_depths=True,invert_axes=True)
        
    if output_path_layers is not None:
        Landmarks(pts=l5_upper_pts).write_landmarks(output_path_layers+'_L5_upper_pts.landmarksAscii')
        Landmarks(pts=l5_lower_pts).write_landmarks(output_path_layers+'_L5_lower_pts.landmarksAscii')
        Landmarks(pts=l5_avg_pts).write_landmarks(output_path_layers+'_L5_avg_pts.landmarksAscii')

        Surface(pts=l5_upper_pts).create_delunay_surface_2d(output_filename=output_path_layers+'_L5_upper_surf.vtk')
        Surface(pts=l5_lower_pts).create_delunay_surface_2d(output_filename=output_path_layers+'_L5_lower_surf.vtk')
        Surface(pts=l5_avg_pts).create_delunay_surface_2d(output_filename=output_path_layers+'_L5_avg_surf.vtk')
    
    df = pd.DataFrame()
    df['Exp_Name'] = [exp_name]
    df['Vectors_Length_Mean'] = [float(avg_axes_len)]
    df['Vectors_Length_Std'] = [float(std_axes_len)]
    df['L5_Depth_Upper_Mean'] = [float(avg_l5_upper_depths)]
    df['L5_Depth_Upper_Std'] = [float(std_l5_upper_depths)]
    df['L5_Depth_Lower_Mean'] = [float(avg_l5_lower_depths)]
    df['L5_Depth_Lower_Std'] = [float(std_l5_lower_depths)]
    df['L5_Depth_Avg_Mean'] = [float(avg_l5_avg_depths)]
    df['L5_Depth_Avg_Std'] = [float(std_l5_avg_depths)]
    
    return vectors_sg,df

In [ ]:
df_cols = ['Exp_Name','Vectors_Length_Mean','Vectors_Length_Std',\
            'L5_Depth_Upper_Mean','L5_Depth_Upper_Std',\
            'L5_Depth_Lower_Mean','L5_Depth_Lower_Std',\
            'L5_Depth_Avg_Mean','L5_Depth_Avg_Std']
df_vm1_axis_field_stats = pd.DataFrame(columns=df_cols)
df_vs1_axis_field_stats = pd.DataFrame(columns=df_cols)

for exp in exp_names:
    print(exp)
    # get vm1 axis field
    vectors_sg = AmiraSpatialGraph(filename=output_path_axis_fileds+exp+'_wm_to_pia_axis_field_voxel_size_50.am',generic_graph=True)
    pts = Landmarks(filename=output_path_landmarks+'/Density_Clusters/'+exp+'_vM1_rabies_landmarks_cutoff_0.1.LandmarkAscii').pts
    surface = Surface(output_path_vM1+exp+'_vM1_hull.vtk')
    vm1_edge_list_sg,df_vm1 = get_intersecting_axes_and_layers(vectors_sg,pts,surface=surface,radius=500,\
                                                        make_cube=True,bounding_offset=0,\
                                             output_filename_axes=output_path_axis_fileds+exp+'_vM1_axis_field.am',\
                                             output_path_layers=output_path_layers+exp+'_vM1',df_cols=df_cols,exp_name=exp)
    
    #print(df_vm1)
    df_vm1_axis_field_stats = df_vm1_axis_field_stats.append(df_vm1)
    
    # get vS1 axis field using near vs1 rabies cluster
    #if os.path.basename(exp).find('MG48') < 0:
    pts = Landmarks(filename=output_path_landmarks+'/Density_Clusters/'+exp+'_vS1_rabies_landmarks_cutoff_0.1.LandmarkAscii').pts
    #if len(pts)>0:
    vs1_edge_list_sg,df_vs1_rabies = get_intersecting_axes_and_layers(vectors_sg,pts,radius=500,\
                                                        make_cube=True,bounding_offset=0,\
                                         output_filename_axes=output_path_axis_fileds+exp+'_vS1_rabies_axis_field.am',\
                                         output_path_layers=output_path_layers+exp+'_vS1_rabies',df_cols=df_cols,exp_name=exp)
    df_vs1_axis_field_stats_rabies = df_vs1_axis_field_stats.append(df_vs1_rabies)
    
    # get vS1 axis field bounded by vS1 barrels
    if exp.find('MG48')<0:
        surface = Surface(output_path_vS1+exp+'_vS1_hull.vtk')
        vs1_edge_list_sg,df_vs1 = get_intersecting_axes_and_layers(vectors_sg,pts,surface=surface,radius=500,\
                                                        make_cube=True,bounding_offset=0,\
                                         output_filename_axes=output_path_axis_fileds+exp+'_vS1_axis_field.am',\
                                         output_path_layers=output_path_layers+exp+'_vS1',df_cols=df_cols,exp_name=exp)
        df_vs1_axis_field_stats = df_vs1_axis_field_stats.append(df_vs1)
    
Stats(df_vm1_axis_field_stats).add_mean_std_cov(tangential_exps=True, coronal_exps=True,\
                                                output_filename=output_path_stats+'5_vM1_L5_Stats.csv')
Stats(df_vs1_axis_field_stats_rabies).add_mean_std_cov(tangential_exps=True, coronal_exps=True,\
                                                output_filename=output_path_stats+'5_vS1_L5_Stats_rabies.csv') 
Stats(df_vs1_axis_field_stats).add_mean_std_cov(tangential_exps=True, coronal_exps=False,\
                                                output_filename=output_path_stats+'5_vS1_L5_Stats.csv')

In [ ]:
# Temp

# vectors_sg = AmiraSpatialGraph(filename=output_path_axis_fileds+exp+'_wm_to_pia_axis_field_voxel_size_50.am',generic_graph=True)
# pts = Landmarks(filename=output_path_landmarks+exp+'_vM1_rabies_landmarks.LandmarkAscii').pts
# vm1_edge_list_sg,df_vm1 = get_intersecting_axes_and_layers(pts,vectors_sg,radius=250,\
#                                                     make_cube=True,bounding_offset=0,\
#                                          output_filename_axes=output_path_axis_fileds+'_vM1_axis_field.am',\
#                                          output_path_layers=output_path_layers+exp+'_vM1',df_cols=df_cols,exp_name=exp)




# NeuN based Layers for vS1

In [13]:
def get_cylinder(radius,edge,get_original=False):
    center = (np.array(edge[0])+np.array(edge[1]))/2
    height = Vectors().get_vec_length(edge)
    cylinderSource = vtk.vtkCylinderSource()
    cylinderSource.SetCenter(center)
    cylinderSource.SetRadius(radius)
    cylinderSource.SetHeight(height)
    cylinderSource.SetResolution(100)
    cylinderSource.Update()

    pts = []
    ref_pts = []
    pts.append(center)
    end_pt = [center[0],cylinderSource.GetOutput().GetBounds()[2],center[2] ]
    pts.append(end_pt)

    ref_pts.append(center)
    end_pt = Vectors().get_unit_vec(center,edge[1])*height/2 + center
    ref_pts.append(end_pt)
    #Landmarks(pts=pts).write_landmarks(output_path_neun_profiles+'pts')
    #Landmarks(pts=ref_pts).write_landmarks(output_path_neun_profiles+'ref_pts')

    landmarks = Landmarks(pts=pts)
    icp,txmat = landmarks.align_landmarks(ref_pts)

    cyl_surf = Surface(polydata=cylinderSource.GetOutput())
    cyl_surf.apply_icp_transform(icp)
    #print(txmat)

    #Surface(polydata=cylinderSource.GetOutput()).write_surface_mesh(output_path_neun_profiles+'_cylinder.vtk')
    if get_original:
        return cyl_surf.surface,cylinderSource.GetOutput(),icp,txmat
    else:
        return cyl_surf.surface

In [14]:
def inverse_landmarks(edge,given_landmarks):
    center = (np.array(edge[0])+np.array(edge[1]))/2
    height = Vectors().get_vec_length(edge)
    cylinderSource = vtk.vtkCylinderSource()
    cylinderSource.SetCenter(center)
    cylinderSource.SetRadius(radius)
    cylinderSource.SetHeight(height)
    cylinderSource.SetResolution(100)
    cylinderSource.Update()

    pts = []
    ref_pts = []
    pts.append(center)
    end_pt = [center[0],cylinderSource.GetOutput().GetBounds()[2],center[2] ]
    pts.append(end_pt)

    ref_pts.append(center)
    end_pt = Vectors().get_unit_vec(center,edge[1])*height/2 + center
    ref_pts.append(end_pt)
    #Landmarks(pts=pts).write_landmarks(output_path_neun_profiles+'pts')
    #Landmarks(pts=ref_pts).write_landmarks(output_path_neun_profiles+'ref_pts')

    landmarks = Landmarks(pts=pts)
    icp,txmat = landmarks.align_landmarks(ref_pts)

    givenl = Landmarks(pts=given_landmarks)
    givenl.apply_transformation(transformation_matrix=txmat,inverse=True)
    #cyl_surf = Surface(polydata=cylinderSource.GetOutput())
    #cyl_surf.apply_icp_transform(icp)
    #print(txmat)

    #Surface(polydata=cylinderSource.GetOutput()).write_surface_mesh(output_path_neun_profiles+'_cylinder.vtk')

    return givenl.pts

In [15]:
def get_line_pts(edge,res):
    total_len = distance.euclidean(edge[0],edge[1])
    number_of_pts_required = int((total_len+0.5*res) /res)
    #print(number_of_pts_required)
    line_pts = []
    unit_vec = Vectors().get_unit_vec(edge[0],edge[1])
    unit_vec_neg = Vectors().get_unit_vec(edge[1],edge[0])
    first_pt = (unit_vec_neg * 0.5 * res) + edge[0]
    line_pts.append(first_pt)
    for i in range(number_of_pts_required):
        #print(i)
        pt = (unit_vec * (i+1) * res) + first_pt
        #print(pt)
        line_pts.append(pt)
    #line_pts.append(edge[1])
    return line_pts

In [16]:
def get_cell_count_profile(edge,res,barrel_radius,cell_count_landmarks):
    line_pts = get_line_pts(edge,res)
    profile = []
    # go through pts of the line and get density value for each pt at given z res and barrel radius
    for i in range(len(line_pts)-2):
        # create a bounding box oriented in the direction of the line
        cylinder = get_cylinder(line_pts[i+1],barrel_radius,res/2)
        landmarks_within_cylinder = Landmarks(pts=cell_count_landmarks).get_landmarks_within_given_surface(cylinder)
        profile.append(len(landmarks_within_cylinder))
    return profile

In [17]:
def get_cell_density_profile(edge,res,barrel_radius,cell_count_landmarks,exp,barrel):
    line_pts = get_line_pts(edge,res)
    profile = []
    total_nr = 0
    total_vol = 0
    column_center = (np.array(edge[0]) + np.array(edge[1])) / 2
    column_cylinder = get_cylinder(barrel_radius,edge)
    column_cylinder_srf = Surface(polydata=column_cylinder).create_delunay_surface_3d(return_hull=True,make_cube=False)
    Surface(polydata=column_cylinder_srf).write_surface_mesh(output_path_neun_profiles+'cylinder_{}_{}.vtk'.format(exp,barrel))
    landmarks_within_column = Landmarks(pts=cell_count_landmarks).get_landmarks_within_given_surface_using_vtk(column_cylinder_srf,inside_out=False)
    #print(len(landmarks_within_column))
    #print(output_path_neun_profiles+'landmarks_within_column_{}_{}.landmarksAscii'.format(exp,barrel))
    Landmarks(pts=landmarks_within_column).write_landmarks(output_path_neun_profiles+\
                                        'landmarks_within_column_{}_{}.landmarksAscii'.format(exp,barrel))
#     sg = AmiraSpatialGraph(generic_graph=True)
#     sg.graph_data.add_edge(edge[0],edge[1])
#     sg.write_spatial_graph(output_path_neun_profiles+'landmarks_within_column.am')
    
    # go through pts of the line and get density value for each pt at given z res and barrel radius
    for i in range(len(line_pts)-1):
        # create a bounding box oriented in the direction of the line
        cylinder = get_cylinder(barrel_radius,[line_pts[i],line_pts[i+1]])
        cylinder_srf = Surface(polydata=cylinder).create_delunay_surface_3d(return_hull=True,make_cube=False)
#         Surface(polydata=cylinder_srf).write_surface_mesh(output_path_neun_profiles+'Step_{}_cylinder_{}_{}.vtk'.format(i,exp,barrel))
        landmarks_within_cylinder = Landmarks(pts=landmarks_within_column).get_landmarks_within_given_surface_using_vtk(cylinder_srf)
#         Landmarks(pts=landmarks_within_cylinder).write_landmarks(output_path_neun_profiles+\
#                                         'Step_{}_landmarks_within_column_{}_{}.landmarksAscii'.format(i,exp,barrel))
        volume = np.pi*(barrel_radius*barrel_radius)*res#Surface(polydata=cylinder).get_surface_properties(prop='volume')
        total_nr = total_nr + len(landmarks_within_cylinder)
        total_vol = total_vol+volume
        density = len(landmarks_within_cylinder)/volume # in nr / cubic_um
        profile.append(density*1000000)# in 1000 / cubic mm 
    print('column volume in {} and num of neurons {}'.format(total_vol,total_nr))
    return profile,column_cylinder_srf,landmarks_within_column

In [18]:
def get_cell_density_profile2(edge,res,barrel_radius,cell_count_landmarks,exp,barrel):
    line_pts = get_line_pts(edge,res)
    profile = []
    total_nr = 0
    total_vol = 0
    column_center = (np.array(edge[0]) + np.array(edge[1])) / 2
    column_cylinder,tilted_cylinder,icp,txmat = get_cylinder(barrel_radius,edge,get_original=True)
    column_cylinder_srf = Surface(polydata=column_cylinder).create_delunay_surface_3d(return_hull=True,make_cube=False)
    column_cylinder_srf_tilted = Surface(polydata=tilted_cylinder).create_delunay_surface_3d(return_hull=True,make_cube=False)
    Surface(polydata=column_cylinder_srf).write_surface_mesh(output_path_neun_profiles+'cylinder_{}_{}.vtk'.format(exp,barrel))
    tilted_landmarks = inverse_landmarks(edge,cell_count_landmarks)
    landmarks_within_column = Landmarks(pts=cell_count_landmarks).get_landmarks_within_given_surface_using_vtk(column_cylinder_srf,inside_out=False)
    landmarks_within_column_tilted = Landmarks(pts=tilted_landmarks).get_landmarks_within_given_surface_using_vtk(column_cylinder_srf_tilted,inside_out=False)
    landmarks_within_column_untilted = Landmarks(pts=landmarks_within_column_tilted)
    landmarks_within_column_untilted.apply_transformation(transformation_matrix=txmat,inverse=False)
    #print(len(landmarks_within_column))
    #print(output_path_neun_profiles+'landmarks_within_column_{}_{}.landmarksAscii'.format(exp,barrel))
    Landmarks(pts=landmarks_within_column).write_landmarks(output_path_neun_profiles+\
                                        'landmarks_within_column_{}_{}.landmarksAscii'.format(exp,barrel))
    Landmarks(pts=landmarks_within_column_tilted).write_landmarks(output_path_neun_profiles+\
                                        'landmarks_within_column_{}_{}_tilted.landmarksAscii'.format(exp,barrel))
#     sg = AmiraSpatialGraph(generic_graph=True)
#     sg.graph_data.add_edge(edge[0],edge[1])
#     sg.write_spatial_graph(output_path_neun_profiles+'landmarks_within_column.am')
    
    # go through pts of the line and get density value for each pt at given z res and barrel radius
    for i in range(len(line_pts)-1):
        # create a bounding box oriented in the direction of the line
        cylinder = get_cylinder(barrel_radius,[line_pts[i],line_pts[i+1]])
        cylinder_srf = Surface(polydata=cylinder).create_delunay_surface_3d(return_hull=True,make_cube=False)
#         Surface(polydata=cylinder_srf).write_surface_mesh(output_path_neun_profiles+'Step_{}_cylinder_{}_{}.vtk'.format(i,exp,barrel))
        landmarks_within_cylinder = Landmarks(pts=landmarks_within_column).get_landmarks_within_given_surface_using_vtk(cylinder_srf)
#         Landmarks(pts=landmarks_within_cylinder).write_landmarks(output_path_neun_profiles+\
#                                         'Step_{}_landmarks_within_column_{}_{}.landmarksAscii'.format(i,exp,barrel))
        volume = np.pi*(barrel_radius*barrel_radius)*res#Surface(polydata=cylinder).get_surface_properties(prop='volume')
        total_nr = total_nr + len(landmarks_within_cylinder)
        total_vol = total_vol+volume
        density = len(landmarks_within_cylinder)/volume # in nr / cubic_um
        profile.append(density*1000000)# in 1000 / cubic mm 
    print('column volume in {} and num of neurons {}'.format(total_vol,total_nr))
    return profile,column_cylinder_srf,landmarks_within_column_untilted.pts

In [19]:
def interpolate_array(old,new_length=43):
    #print(len(old))
    old_indices = np.arange(0,len(old))
    #new_length = 43
    new_indices = np.linspace(0,len(old)-1,new_length)
    
    spl = UnivariateSpline(old_indices,old,k=3,s=0,check_finite=True)
    new_array = spl(new_indices)

    return new_array

##  vS1 Profiles

In [ ]:
# reference column depth is 2150 um 
ref_col_depth = 2150
density_peak_3d = 121.237 # (in 1000 / cubic mm)
res = 50
barrel_radius = 250

df_2d_3d_scales = pd.DataFrame()
df_unstretched = pd.DataFrame()
df_unstretched['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
df_unstretched_scaled = pd.DataFrame()
df_unstretched_scaled['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
df_stretched = pd.DataFrame()
df_stretched['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
df_scaled = pd.DataFrame()
df_scaled['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
for exp in ['MG50_lhs']:#exp_names_tangential:
    print(exp)
    df_exp_2d = pd.DataFrame() 
    profile = []
    neun_landmarks_vS1 = Landmarks(output_path_landmarks+exp+'_neun_landmarks_vS1.LandmarkAscii')
   
    # speed up by considering only neun within the barres
#     S1_hull = Surface(output_path_surfaces+exp+'_s1_hull_50.vtk').surface
#     s1_neun_landmarks = neun_landmarks.get_landmarks_within_given_surface(S1_hull)
#     Landmarks(pts=s1_neun_landmarks).write_landmarks(output_path_neun_profiles+'vS1_NeuN_Landmarks_{}.landmarksAscii'.format(exp))
    sg = AmiraSpatialGraph(output_path_surfaces+exp+'_barrels_50.am',barrel_projections_present=True,reverse_barrel_direction=True)
    
    # get profiles for C2,C3,D2,D3
    for barrel_row in sg.barrels.rows_list:
        for barrel in barrel_row.single_barrels_list:
            #row = sg.get_barrel_row(row_label=row_label)
            #barrel = row.single_barrels_list[barrel_ind+1]
            
            barrel_name = barrel.barrel_name
            #print(barrel_name)
            
            if barrel_name == 'D2':#barrel_name == 'C2' or barrel_name == 'C3' or barrel_name == 'D2' or barrel_name == 'D3':
                print(barrel_name)
                if len(barrel.pia_projection_barrel_centroid) > 0 and len(barrel.wm_projection_barrel_centroid)>0:
                    edge = [barrel.pia_projection_barrel_centroid,barrel.wm_projection_barrel_centroid]
                    
#                     sg = AmiraSpatialGraph(generic_graph=True)
#                     sg.graph_data.add_edge([edge[0],edge[1]])
#                     sg.write_spatial_graph(output_path_neun_profiles+'dummy.am')
                    
#                     break
                    
                    #profile = get_cell_count_profile(edge,res,barrel_radius,s1_neun_landmarks)
                    profile,profile_cylinder,profile_landmarks = get_cell_density_profile(edge,res,barrel_radius,neun_landmarks_vS1.pts,exp,barrel.barrel_name)
                    #print(len(profile))
                    
                    df_profile_unstretched = pd.DataFrame()
                    df_profile_unstretched['NeuN_Density_{}_{}'.format(exp,barrel_name)] = profile
                    df_profile_unstretched.to_csv(output_path_neun_profiles+'vS1_NeuN_Depth_Profile_{}_{}.csv'.format(exp,barrel.barrel_name))
                    # append nan to the rest of the profile
                    if len(profile) < round(ref_col_depth / res):
                        new_prof = []
                        for i in range(len(profile)):
                            new_prof.append(profile)
                        for i in range(round(ref_col_depth / res)-len(profile)):
                            new_prof.append(np.nan)
                        df_unstretched['NeuN_Density_{}_{}'.format(exp,barrel_name)] = new_prof
                    else:
                        df_unstretched['NeuN_Density_{}_{}'.format(exp,barrel_name)] = profile[0:43]


                    #print(len(profile))
                    profile_streched = interpolate_array(profile,round(ref_col_depth/res))
                    #print((profile_streched))
                    df_exp_2d = pd.DataFrame()
                    df_exp_2d['NeuN_Density_{}_{}'.format(exp,barrel_name)] = profile_streched
                    df_exp_2d.to_csv(output_path_neun_profiles+'vS1_NeuN_Depth_Profile_{}_{}_stretched.csv'.format(exp,barrel.barrel_name))

                    df_stretched = pd.concat([df_stretched,df_exp_2d],ignore_index=False,axis=1)
                    # stretch or clip it to match the reference column depth if needed
                    #if abs(barrel.column_height) < ref_col_depth:

                    #else:
                        # need to shrink the column
                    #    profile_3d = profile[0:round(ref_col_depth/res)]

                    # scale from 2d to 3D
                    max_density = max(profile_streched)
                    scale = density_peak_3d / max_density
                    profile_3d = np.array(profile_streched)*scale 

                    df_scales = pd.DataFrame()
                    df_scales['NeuN_Scale_{}_{}'.format(exp,barrel_name)] = [scale]
                    df_2d_3d_scales = pd.concat([df_2d_3d_scales,df_scales],ignore_index=False,axis=1)

                    df_profile = pd.DataFrame()
                    df_profile['NeuN_Density_{}_{}'.format(exp,barrel_name)] = profile_3d # in 1000 / cubic mm
                    df_profile.to_csv(output_path_neun_profiles+'vS1_NeuN_Depth_Profile_{}_{}_stretched_scaled.csv'.format(exp,barrel.barrel_name))

                    df_scaled = pd.concat([df_scaled,df_profile],ignore_index=False,axis=1)
                    #print(df)

    df_exp_2d.to_csv(output_root+'vS1_NeuN_Depth_Profile_{}.csv'.format(exp))
    
    
    
df_unstretched.to_csv(output_root+'vS1_NeuN_Depth_Profile_Unstretched.csv')
df_stretched.to_csv(output_root+'vS1_NeuN_Depth_Profile_Stretched.csv')
df_scaled.to_csv(output_root+'vS1_NeuN_Depth_Profile_Scaled.csv')

df_2d_3d_scales.to_csv(output_root+'vS1_NeuN_2D_To_3D_Scaling_Factors.csv')
# df.to_csv(output_path_neun_profiles+'_vS1_NeuN_Depth_Profile_Without_stretch.csv')
# df_2d_3d_scales.to_csv(output_path_neun_profiles+'_vS1_NeuN_2D_To_3D_Scaling_Factors_Without_stretch.csv')
#df.to_csv(output_path_neun_profiles+'_vS1_NeuN_Depth_Profile_stretch_only.csv')
#df_2d_3d_scales.to_csv(output_path_neun_profiles+'_vS1_NeuN_2D_To_3D_Scaling_Factors_stretch_only.csv')

## Derive Layer Borders

In [ ]:
# Select profiles that are suitable: mean profile density > 25

# Find layer borders based on peaks and valleys within a window

# Layer border windows:
# L1: 150
# L2/3: 250-1000
L1_lower = int(50/50) - 1
L1_upper = int(250/50) - 1

L2_3_lower = int(400/50) - 1
L2_3_upper = int(600/50) - 1
# L5A: 750-1250
L4_lower = int(500/50)-1
L4_upper = int(1500/50)-1
# L5B: 1500-2500
L5_lower = int(1250/50)-1
L5_upper = int(2150/50)-1

depth_max = 2150

In [ ]:
layer_borders_df = pd.DataFrame(columns=['Exp_Name','Barrel_Name','L1_Border','L2_3_Border','L4_Border','L5_Border',\
                                        'L1_Border_Per','L2_3_Border_Per','L4_Border_Per','L5_Border_Per'])
# rowwise profiles going from caudal to rostral
exp = 'MG50_lhs'
#%matplotlib notebook
#figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')
#p = plt.subplot(psize=[10,10])

lhs_exp_means = []
lhs_exp_stds = []
rhs_exp_means = []
rhs_exp_stds = []
global_exp_means = []
global_exp_stds = []

df = pd.read_csv(output_path_neun_profiles+'vS1_NeuN_Depth_Profile_Stretched.csv')

for exp in ['MG49_lhs','MG50_lhs','MG50_rhs']:
    layer_borders_exp_df = pd.DataFrame(columns=['Exp_Name','Barrel_Name','L1_Border','L2_3_Border','L4_Border','L5_Border',\
                                        'L1_Border_Per','L2_3_Border_Per','L4_Border_Per','L5_Border_Per'])
    exp_rows = []
    exp_profiles = []
    for barrel in ['C2','C3','D2','D3']:

        rows = []
        #for j in range(2,len(df.columns)):
        profile = df['NeuN_Density_{}_{}'.format(exp,barrel)]
        #if np.array(profile).mean(axis=0) > 1:
        L1_ind = np.where(profile>0)[0][0]#np.array(np.array(profile)[L1_lower:L1_upper]).argmax()
        L1_border = (df[df.columns[1]][L1_lower+L1_ind])

        L2_3_ind = np.array(np.array(profile)[L2_3_lower:L2_3_upper]).argmin()
        L2_3_border = (df[df.columns[1]][L2_3_lower+L2_3_ind])

        L4_ind = np.array(np.array(profile)[L4_lower:L4_upper]).argmin()
        L4_border = (df[df.columns[1]][L4_lower+L4_ind])

        L5_ind = np.array(np.array(profile)[L5_lower:L5_upper]).argmax()
        L5_border = (df[df.columns[1]][L5_lower+L5_ind])

        df_tmp = pd.DataFrame(columns=layer_borders_df.columns,data=[[exp,barrel,L1_border,L2_3_border,L4_border,\
                                                                      L5_border,\
                                                        abs(L1_border/depth_max),abs(L2_3_border/depth_max),\
                                                         abs(L4_border/depth_max),abs(L5_border/depth_max)] ])
        layer_borders_exp_df = layer_borders_exp_df.append(df_tmp)

        layer_borders_df = layer_borders_df.append(df_tmp)
        #print(L2_3_border,L5A_border,L5B_border)
        #plt.plot(df[df.columns[j]],df['Depth_in_um'])
        rows.append(profile)
        exp_profiles.append(profile)
                    
    global_exp_means.append(np.array(exp_profiles).mean(axis=0))
    global_exp_stds.append(np.array(exp_profiles).std(axis=0))
    if exp.find('_lhs')>0:
        lhs_exp_means.append(np.array(exp_profiles).mean(axis=0))
        lhs_exp_stds.append(np.array(exp_profiles).std(axis=0))
    else:
        rhs_exp_means.append(np.array(exp_profiles).mean(axis=0))
        rhs_exp_stds.append(np.array(exp_profiles).std(axis=0))
            #print(rows)
            
            #if len(rows)>0:
            #    x = np.array(rows).mean(axis=0)
            #    exp_rows.append(x)
                #print(x)
                #plt.plot(x,df['Depth_in_um'])
    
#     y = df['Depth_in_um']
#     x = np.array(exp_profiles).mean(axis=0)
#     error = np.array(exp_profiles).std(axis=0)
#     plt.plot(x,y)
#     plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
#     plt.savefig(layers_path+exp+'_vS1_Avg_Depth_Profile.eps',format='eps')
#     plt.cla()
    
    # store it to approp cat for avg plot
    
    #plt.plot(mean,df['Depth_in_um'])
    #plt.plot(mean+std,df['Depth_in_um'])
    #plt.plot(mean-std,df['Depth_in_um'])
    
    layer_borders_exp_df.to_csv(output_path_neun_layer_borders+exp+'_vS1_Borders.csv')
layer_borders_df.to_csv(output_path_neun_layer_borders+'vS1_Borders.csv')

# Plot and save the average plots
# y = df['Depth_in_um']
# x = np.array(lhs_exp_means).mean(axis=0)
# error = np.array(lhs_exp_stds).mean(axis=0)
# plt.plot(x,y)
# plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
# plt.savefig(layers_path+'vS1_Avg_Depth_Profile_LHS.eps',format='eps')
# plt.cla()

# x = np.array(rhs_exp_means).mean(axis=0)
# error = np.array(rhs_exp_stds).mean(axis=0)
# plt.plot(x,y)
# plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
# plt.savefig(layers_path+'vS1_Avg_Depth_Profile_RHS.eps',format='eps')
# plt.cla()

# x = np.array(global_exp_means).mean(axis=0)
# error = np.array(global_exp_stds).mean(axis=0)
# plt.plot(x,y)
# plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
# plt.savefig(layers_path+'vS1_Avg_Depth_Profile.eps',format='eps')
# plt.cla()
  


## Create Layer Border Surfaces

In [ ]:
lb = pd.read_csv(output_path_neun_layer_borders+'vS1_Borders.csv')
for exp in ['MG49_lhs','MG50_lhs','MG50_rhs',]:
    lb = pd.read_csv(output_path_neun_layer_borders+'vS1_Borders.csv')
    for surf_res in surface_resolutions:
        
        vectors_sg = AmiraSpatialGraph(output_path_vS1_axis_field+exp+'_vS1_axis_field_{}.am'.format(surf_res),generic_graph=True)
        
        for layer in ['L1_Border_Per','L2_3_Border_Per','L4_Border_Per','L5_Border_Per',]:
            layer_depth_mean = lb[lb['Exp_Name']==exp][layer].mean() * 100
            layer_depth_upper = layer_depth_mean - (lb[lb['Exp_Name']==exp][layer].std() * 100)
            layer_depth_lower = layer_depth_mean + (lb[lb['Exp_Name']==exp][layer].std() * 100)
            #print(layer_depth)
            for layer_depth in [layer_depth_mean,layer_depth_upper,layer_depth_lower]:
                layer_border_pts = Vectors().get_layer_pts_for_given_depth(vectors_sg.graph_data.edge_list,layer_depth,\
                                                                                              realtive_depths=True,invert_axes=True,rel_depth_given=True)
                layer_surf = Surface(pts=layer_border_pts).create_delunay_surface_2d(return_hull=True)
                # write mean, upper or lower layer border surface
                if layer_depth == layer_depth_mean:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_path_neun_layer_borders+ exp +'_vS1_'+layer[:-4]+'_{}_mean.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_path_neun_layer_borders+exp +'_vS1_'+layer[:-4]+'_{}_mean.vtk'.format(surf_res))
                elif layer_depth == layer_depth_upper:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_path_neun_layer_borders+ exp +'_vS1_'+layer[:-4]+'_{}_upper.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_path_neun_layer_borders+exp +'_vS1_'+layer[:-4]+'_{}_upper.vtk'.format(surf_res))
                else:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_path_neun_layer_borders+ exp +'_vS1_'+layer[:-4]+'_{}_lower.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_path_neun_layer_borders+exp +'_vS1_'+layer[:-4]+'_{}_lower.vtk'.format(surf_res))


In [ ]:
# # read layer borders
# lb = pd.read_csv(output_path_neun_layer_borders+'vM1_Borders.csv')
# relative_depths = []
# # Create layer borders
# for exp in exp_names:
#     for surf_res in surface_resolutions:
#         # Read axis field vectors
#         #vectors_sg = AmiraSpatialGraph(output_path_axis_fileds+exp+'_vM1_axis_field.am',generic_graph=True)
#         i = 0
#         for layer in ['L1_Border_Per','L2_3_Border_Per','L5A_Border_Per','L5B_Border_Per',]:
#             #layer_depth = lb[layer][i]*100
#             # mean layer border
#             layer_depth_mean = lb[lb['Exp_Name']==exp][layer].mean() * 100
#             layer_depth_upper = layer_depth_mean - (lb[lb['Exp_Name']==exp][layer].std() * 100)
#             layer_depth_lower = layer_depth_mean + (lb[lb['Exp_Name']==exp][layer].std() * 100)
#             #print(layer_depth)
#             for layer_depth in [layer_depth_mean,layer_depth_upper,layer_depth_lower]:
#                 i = i +1
#                 layer_surf = Surface()
#                 arr = [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
#                 for j in range(len(arr)):
#                     #j = arr[k]

#                     vectors_sg_left = None
#                     vectors_sg_right = None
#                     vectors_sg_next_left = None
#                     vectors_sg_next_right = None
#                     layer_border_pts_left = []
#                     layer_border_pts_right = []
#                     if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
#                                       .format(arr[j],surf_res,500,0.05)) and \
#                            os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
#                                           .format(arr[j+1],surf_res,500,0.05)):
#                         vectors_sg = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
#                                                        .format(arr[j],surf_res,500,0.05),\
#                                                        generic_graph=True)
#                         if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
#                                           .format(arr[j],surf_res,500,0.05)):
#                             vectors_sg_left = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
#                                                                 .format(arr[j],surf_res,500,0.05),\
#                                                        generic_graph=True)
#                         if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
#                                           .format(arr[j],surf_res,500,0.05)):
#                             vectors_sg_right = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
#                                                                  .format(arr[j],surf_res,500,0.05),\
#                                                        generic_graph=True)

#                         vectors_sg_next = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
#                                                             .format(arr[j+1],surf_res,500,0.05),\
#                                                        generic_graph=True)

#                         if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
#                                           .format(arr[j+1],surf_res,500,0.05)):
#                             vectors_sg_next_left = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
#                                                                      .format(arr[j+1],surf_res,500,0.05),\
#                                                        generic_graph=True)
#                         if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
#                                           .format(arr[j+1],surf_res,500,0.05)):
#                             vectors_sg_next_right = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
#                                                                       .format(arr[j+1],surf_res,500,0.05),\
#                                                        generic_graph=True)

#                         # now fetch layer specific pt
#                         if len(vectors_sg.graph_data.edge_list) > 0 and len(vectors_sg_next.graph_data.edge_list) > 0:
#                             pts = []
#                             left_pts = []
#                             right_pts = []
#                             layer_border_pts = Vectors().get_layer_pts_for_given_depth(vectors_sg.graph_data.edge_list,layer_depth,\
#                                                                                          realtive_depths=True,invert_axes=True,rel_depth_given=True)
#                             if vectors_sg_left is not None and len(vectors_sg_left.graph_data.edge_list) > 0:
#                                 layer_border_pts_left = Vectors().get_layer_pts_for_given_depth(vectors_sg_left.graph_data.edge_list,layer_depth,\
#                                                                                          realtive_depths=True,invert_axes=True,rel_depth_given=True)
#                             if vectors_sg_right is not None and len(vectors_sg_right.graph_data.edge_list) > 0:
#                                 layer_border_pts_right = Vectors().get_layer_pts_for_given_depth(vectors_sg_right.graph_data.edge_list,layer_depth,\
#                                                                                          realtive_depths=True,invert_axes=True,rel_depth_given=True)

#                             layer_border_pts_next = Vectors().get_layer_pts_for_given_depth(vectors_sg_next.graph_data.edge_list,layer_depth,\
#                                                                                          realtive_depths=True,invert_axes=True,rel_depth_given=True)
#                             if vectors_sg_next_left is not None and len(vectors_sg_next_left.graph_data.edge_list) > 0: 
#                                 layer_border_pts_next_left = Vectors().get_layer_pts_for_given_depth(vectors_sg_next_left.graph_data.edge_list,layer_depth,\
#                                                                                          realtive_depths=True,invert_axes=True,rel_depth_given=True)
#                             if vectors_sg_next_right is not None and len(vectors_sg_next_right.graph_data.edge_list) > 0:
#                                 layer_border_pts_next_right = Vectors().get_layer_pts_for_given_depth(vectors_sg_next_right.graph_data.edge_list,layer_depth,\
#                                                                                          realtive_depths=True,invert_axes=True,rel_depth_given=True)

#                             for pt in layer_border_pts:
#                                 pts.append(pt)
#                             for pt in layer_border_pts_next:
#                                 pts.append(pt)

#                             for pt in layer_border_pts_left:
#                                 left_pts.append(pt)
#                             for pt in layer_border_pts_next_left:
#                                 left_pts.append(pt)
#                             for pt in layer_border_pts_right:
#                                 right_pts.append(pt)
#                             for pt in layer_border_pts_next_right:
#                                 right_pts.append(pt)

#                             # left side surface
#                             #max(len(left_pts))
#                             #print(len(layer_border_pts_left),len(layer_border_pts_next_left))
#                             for pt_ind in range(max(len(layer_border_pts_left),len(layer_border_pts_next_left))-1):
#                                 #print(pt_ind)
#                                 pts = []
#                                 #pts_next = []
#                                 if len(layer_border_pts_left)>0:
#                                     if pt_ind>len(layer_border_pts_left)-2:
#                                         pts.append(layer_border_pts_left[len(layer_border_pts_left)-1])
#                                     else:
#                                         pts.append(layer_border_pts_left[pt_ind])
#                                         pts.append(layer_border_pts_left[pt_ind+1])
#                                 if len(layer_border_pts_next_left)>0:
#                                     if pt_ind>len(layer_border_pts_next_left)-2:
#                                         pts.append(layer_border_pts_next_left[len(layer_border_pts_next_left)-1])
#                                     else:
#                                         pts.append(layer_border_pts_next_left[pt_ind])
#                                         pts.append(layer_border_pts_next_left[pt_ind+1])
#                                 #print(pts)
#                                 #pts_next = [layer_border_pts_left[pt_ind+1],layer_border_pts_left[pt_ind+1]]

#                                 surf = Surface(pts=pts).create_delunay_surface_2d(return_hull=True,alpha=0,offset=100)

#                                 layer_surf.append(surf)
#                             for pt_ind in range(max(len(layer_border_pts_right),len(layer_border_pts_next_right))-1):
#                                 #print(pt_ind)
#                                 pts = []
#                                 #pts_next = []
#                                 if len(layer_border_pts_right)>0:
#                                     if pt_ind>len(layer_border_pts_right)-2:
#                                         pts.append(layer_border_pts_right[len(layer_border_pts_right)-1])
#                                     else:
#                                         pts.append(layer_border_pts_right[pt_ind])
#                                         pts.append(layer_border_pts_right[pt_ind+1])
#                                 if len(layer_border_pts_next_right)>0:
#                                     if pt_ind>len(layer_border_pts_next_right)-2:
#                                         pts.append(layer_border_pts_next_right[len(layer_border_pts_next_right)-1])
#                                     else:
#                                         pts.append(layer_border_pts_next_right[pt_ind])
#                                         pts.append(layer_border_pts_next_right[pt_ind+1])
#                                 #print(pts)
#                                 #pts_next = [layer_border_pts_left[pt_ind+1],layer_border_pts_left[pt_ind+1]]
#                                 surf = Surface(pts=pts).create_delunay_surface_2d(return_hull=True,alpha=0,offset=100)
#                                 layer_surf.append(surf)

#                 vertices = []
#                 for vert_ind in range(layer_surf.surface.GetNumberOfPoints()):
#                     vertices.append(layer_surf.surface.GetPoints().GetPoint(vert_ind))
                
#                 # write mean, upper or lower layer border surface
#                 if layer_depth == layer_depth_mean:
#                     Landmarks(pts = vertices).write_landmarks( output_path_neun_layer_borders+ exp +'_vM1_'+layer[:-4]+'_{}_mean.landmarksAscii'.format(surf_res))
#                     layer_surf.write_surface_mesh(output_path_neun_layer_borders+exp +'_vM1_'+layer[:-4]+'_{}_mean.vtk'.format(surf_res))
#                 elif layer_depth == layer_depth_upper:
#                     Landmarks(pts = vertices).write_landmarks( output_path_neun_layer_borders+ exp +'_vM1_'+layer[:-4]+'_{}_upper.landmarksAscii'.format(surf_res))
#                     layer_surf.write_surface_mesh(output_path_neun_layer_borders+exp +'_vM1_'+layer[:-4]+'_{}_upper.vtk'.format(surf_res))
#                 else:
#                     Landmarks(pts = vertices).write_landmarks( output_path_neun_layer_borders+ exp +'_vM1_'+layer[:-4]+'_{}_lower.landmarksAscii'.format(surf_res))
#                     layer_surf.write_surface_mesh(output_path_neun_layer_borders+exp +'_vM1_'+layer[:-4]+'_{}_lower.vtk'.format(surf_res))

                

# NeuN based Layers for vM1

In [ ]:
# def get_number_of_axes_from_field(axis_field,number_of_axes_needed):
#     axis_filed_len = len(axis_field)
#     offset = int(axis_filed_len/number_of_axes_needed)
#     return_list = []
#     for i in range(len(axis_field)):
#         return_list.append(axis_field[i])
#         i = i + offset
#         if len(return_list) == number_of_axes_needed:
#             return return_list

In [ ]:
# find avg central axis length to use as ref length
central_axis_lens = []
for exp in exp_names:
    
    
    sg_pos = AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_50_voxel_size_500_cutoff_0.05_vM1_PCA_2.am',generic_graph=True)
    sg_neg = AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_50_voxel_size_500_cutoff_0.05_vM1_PCA_2_Neg.am',generic_graph=True)
    
    length = distance.euclidean(sg_pos.graph_data.edge_list[0][1],sg_neg.graph_data.edge_list[0][1])
    
    central_axis_lens.append(length)
    
    print(exp,length)
print(np.array(central_axis_lens).mean(axis=0))

## Whole vM1 Profiles

In [ ]:
# Read the vm1 axis field from caudal to rostral
ref_col_depth = 2600
#density_peak_3d = 121.237 # (in 1000 / cubic mm)
res = 50
radius = 250
offset = 10

for exp in ['MG48_lhs']:#exp_names:
    
    neun_landmarks = Landmarks(output_path_landmarks+exp+'_neun_landmarks_vM1_surf_res_50_voxel_size_500_cutoff_0.05.LandmarkAscii')
    
#     if exp.find('MG49_rhs') >=0  and exp.find('MG50_rhs') >=0:
#         rowwise_axis_sg_list = sorted(glob.glob(output_path_vM1_axis_field+exp+'_vM1_axis_field_row*_50_500_0.05.am'),reverse=True)
#     else:
#         rowwise_axis_sg_list = sorted(glob.glob(output_path_vM1_axis_field+exp+'_vM1_axis_field_row*_50_500_0.05.am'))
#     #print(rowwise_axis_sg_list)
    
    # move from caudal to rostral vm1
    col_landmarks = Landmarks()
    col_cylinders = Surface()
    for row in range(-10,10):
        df_unstreched_main = pd.DataFrame()
        df_streched_main = pd.DataFrame()
        df_unstreched_main['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
        df_streched_main['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
        row_present = False
        
        for col in range(-10,10):
            print(row,col)
            if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_50_500_0.05.am'.format(row,col)):
                print(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_50_500_0.05.am'.format(row,col))
            
                sg = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_50_500_0.05.am'.\
                                       format(row,col),generic_graph=True)
                
                edge = sg.graph_data.edge_list[0]
                profile,profile_cylinder,profile_landmarks = get_cell_density_profile\
                    (edge,res,radius,neun_landmarks.pts,exp,'row_'+str(row)+'_'+str(col))
                profile_interpolated = interpolate_array(profile,round(ref_col_depth/res))

                df_profile_unstreched = pd.DataFrame()
                df_profile_unstreched['NeuN_Density_{}_{}'.format(exp,col)] = profile # in 1000 / cubic mm

                df_profile_streched = pd.DataFrame()
                df_profile_streched['NeuN_Density_{}_{}'.format(exp,col)] = profile_interpolated # in 1000 / cubic mm

                df_unstreched_main = pd.concat([df_unstreched_main,df_profile_unstreched],ignore_index=False,axis=1)
                df_streched_main = pd.concat([df_streched_main,df_profile_streched],ignore_index=False,axis=1)
                row_present = True
                col_landmarks.append_landmarks(profile_landmarks)
                col_cylinders.append(profile_cylinder)
                
        if row_present:
            df_unstreched_main.to_csv(output_path_neun_profiles+'_vM1_NeuN_Depth_Profile_Unstreched_using_{}_row_{}.csv'.format(exp,row))
            df_streched_main.to_csv(output_path_neun_profiles+'_vM1_NeuN_Depth_Profile_Streched_using_{}_row_{}.csv'.format(exp,row))
        
    col_landmarks.write_landmarks(output_path_neun_profiles+exp+'_vM1_NeuN_Column_Landmarks.landmarksAscii')
    col_cylinders.write_surface_mesh(output_path_neun_profiles+exp+'_vM1_NeuN_Column_Cylinders.vtk')

## Central Axis Profiles: 2D and 3D

In [ ]:
ref_col_depth = 2600
#density_peak_3d = 121.237 # (in 1000 / cubic mm)
res = 50
radius = 250
#offset = 10
for mode in ['central_axis_2d','central_axis_3d','whole_vm1']:
    df_unstreched_main = pd.DataFrame()
    df_streched_main = pd.DataFrame()
    df_unstreched_main['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
    df_streched_main['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
    for exp in exp_names:
        print(exp)
        sg = AmiraSpatialGraph(output_path_vM1+exp+'_vM1_PCA_2_Full_old.am',generic_graph=True)
        if mode == 'whole_vm1':
            neun_landmarks = Landmarks(output_path_landmarks+exp+'_neun_landmarks_vM1.LandmarkAscii')
            edge_list = AmiraSpatialGraph(output_path_axis_fileds+exp+'_vM1_axis_field.am',generic_graph=True).graph_data.edge_list
            print(len(edge_list))
        elif mode == 'central_axis_2d':
            neun_landmarks = Landmarks(output_path_landmarks+exp+'_neun_landmarks_vM1_central_axis_2d.LandmarkAscii')
            edge_list = sg.graph_data.edge_list
        elif mode == 'central_axis_3d' and exp == 'MG50_lhs':
            #neun_landmarks = Landmarks(output_path_landmarks+exp+'_neun_landmarks_vM1_central_axis_3d.LandmarkAscii')
            neun_landmarks = Landmarks('/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts_3D/MG50_lhs/Transformed_To_Section_Image/'\
                                       +exp+'_neun_landmarks_central_axis_3d.LandmarkAscii')
            edge_list = sg.graph_data.edge_list
        
        for i in range(len(edge_list)):
            if (mode == 'central_axis_3d' and exp == 'MG50_lhs') or mode == 'central_axis_2d' or mode == 'whole_vm1':
            #profile = get_cell_count_profile(edge,res,barrel_radius,s1_neun_landmarks)
                edge = edge_list[i]
                profile = get_cell_density_profile(edge,res,radius,neun_landmarks.pts,exp,mode)
                profile_interpolated = interpolate_array(profile,round(ref_col_depth/res))

                df_profile_unstreched = pd.DataFrame()
                df_profile_unstreched['NeuN_Density_{}_{}'.format(exp,i)] = profile # in 1000 / cubic mm

                df_profile_streched = pd.DataFrame()
                df_profile_streched['NeuN_Density_{}_{}'.format(exp,i)] = profile_interpolated # in 1000 / cubic mm
                
                df_unstreched_main = pd.concat([df_unstreched_main,df_profile_unstreched],ignore_index=False,axis=1)
                df_streched_main = pd.concat([df_streched_main,df_profile_streched],ignore_index=False,axis=1)
               
#     df_unstreched_main.to_csv(output_path_neun_profiles+'_vM1_NeuN_Depth_Profile_using_{}.csv'.format(mode))
#     df_streched_main.to_csv(output_path_neun_profiles+'_vM1_NeuN_Depth_Profile_using_{}.csv'.format(mode))
    df_unstreched_main.to_csv(output_path_neun_profiles+'_vM1_NeuN_Depth_Profile_using_{}.csv'.format(mode))
    df_streched_main.to_csv(output_path_neun_profiles+'_vM1_NeuN_Depth_Profile_using_{}.csv'.format(mode))

In [ ]:
sg = AmiraSpatialGraph(generic_graph=True)
for edge in selected_edges:
    sg.graph_data.add_edge(edge[0],edge[1])
sg.write_spatial_graph(output_path_neun_profiles+exp+'_selected_vM1_axes.am')

## Derive Layer Borders from Depth Profiles

In [ ]:
# Select profiles that are suitable: mean profile density > 25

# Find layer borders based on peaks and valleys within a window

# Layer border windows:
# L1: 150
# L2/3: 250-1000
L1_lower = int(50/50) - 1
L1_upper = int(300/50) - 1

L2_3_lower = int(250/50) - 1
L2_3_upper = int(1000/50) - 1
# L5A: 750-1250
L5A_lower = int(750/50)-1
L5A_upper = int(1250/50)-1
# L5B: 1500-2500
L5B_lower = int(1500/50)-1
L5B_upper = int(2500/50)-1

depth_max = 2600

In [ ]:
layer_borders_df = pd.DataFrame(columns=['Exp_Name','L1_Border','L2_3_Border','L5A_Border','L5B_Border',\
                                        'L1_Border_Per','L2_3_Border_Per','L5A_Border_Per','L5B_Border_Per'])
# rowwise profiles going from caudal to rostral
exp = 'MG50_lhs'
#%matplotlib notebook
#figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')
#p = plt.subplot(psize=[10,10])

lhs_exp_means = []
lhs_exp_stds = []
rhs_exp_means = []
rhs_exp_stds = []
global_exp_means = []
global_exp_stds = []

for exp in exp_names:
    layer_borders_exp_df = pd.DataFrame(columns=['Exp_Name','L1_Border','L2_3_Border','L5A_Border','L5B_Border',\
                                        'L1_Border_Per','L2_3_Border_Per','L5A_Border_Per','L5B_Border_Per'])
    exp_rows = []
    exp_profiles = []
    for i in range(-10,10):

        file = None
        if i <0:
            if len(glob.glob(output_path_neun_profiles+'_vM1_NeuN_Depth_Profile_Streched_using_'+exp+'_row_{}.csv'.format(i)))>0:
                file = glob.glob(output_path_neun_profiles+'_vM1_NeuN_Depth_Profile_Streched_using_'+exp+'_row_{}.csv'.format(i))[0]
        else:
            if len(glob.glob(output_path_neun_profiles+'_vM1_NeuN_Depth_Profile_Streched_using_'+exp+'_row_{}.csv'.format(i)))>0:
                file = glob.glob(output_path_neun_profiles+'_vM1_NeuN_Depth_Profile_Streched_using_'+exp+'_row_{}.csv'.format(i))[0]
        if file is not None:
            #print(file)
            df = pd.read_csv(file)
            #print(df)
            rows = []
            for j in range(2,len(df.columns)):
                profile_correct_direction_list = list(df[df.columns[j]])
                profile_correct_direction_list.reverse()
                if np.array(profile_correct_direction_list).mean(axis=0) > 25:
                    L1_ind = np.array(np.array(profile_correct_direction_list)[L1_lower:L1_upper]).argmax()
                    L1_border = (df[df.columns[1]][L1_lower+L1_ind])
                    
                    L2_3_ind = np.array(np.array(profile_correct_direction_list)[L2_3_lower:L2_3_upper]).argmax()
                    L2_3_border = (df[df.columns[1]][L2_3_lower+L2_3_ind])

                    L5A_ind = np.array(np.array(profile_correct_direction_list)[L5A_lower:L5A_upper]).argmin()
                    L5A_border = (df[df.columns[1]][L5A_lower+L5A_ind])

                    L5B_ind = np.array(np.array(profile_correct_direction_list)[L5B_lower:L5B_upper]).argmax()
                    L5B_border = (df[df.columns[1]][L5B_lower+L5B_ind])
                    
                    df_tmp = pd.DataFrame(columns=layer_borders_df.columns,data=[[exp,L1_border,L2_3_border,L5A_border,\
                                                                                  L5B_border,\
                                                                    abs(L1_border/depth_max),abs(L2_3_border/depth_max),\
                                                                     abs(L5A_border/depth_max),abs(L5B_border/depth_max)] ])
                    layer_borders_exp_df = layer_borders_exp_df.append(df_tmp)
                    
                    layer_borders_df = layer_borders_df.append(df_tmp)
                    #print(L2_3_border,L5A_border,L5B_border)
                    #plt.plot(df[df.columns[j]],df['Depth_in_um'])
                    rows.append(profile_correct_direction_list)
                    exp_profiles.append(profile_correct_direction_list)
                    
    global_exp_means.append(np.array(exp_profiles).mean(axis=0))
    global_exp_stds.append(np.array(exp_profiles).std(axis=0))
    if exp.find('_lhs')>0:
        lhs_exp_means.append(np.array(exp_profiles).mean(axis=0))
        lhs_exp_stds.append(np.array(exp_profiles).std(axis=0))
    else:
        rhs_exp_means.append(np.array(exp_profiles).mean(axis=0))
        rhs_exp_stds.append(np.array(exp_profiles).std(axis=0))
            #print(rows)
            
            #if len(rows)>0:
            #    x = np.array(rows).mean(axis=0)
            #    exp_rows.append(x)
                #print(x)
                #plt.plot(x,df['Depth_in_um'])
    
    y = df['Depth_in_um']
    x = np.array(exp_profiles).mean(axis=0)
    error = np.array(exp_profiles).std(axis=0)
#     plt.plot(x,y)
#     plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
#     plt.savefig(layers_path+exp+'_vM1_Avg_Depth_Profile.eps',format='eps')
#     plt.cla()
    
    # store it to approp cat for avg plot
    
    
    #plt.plot(mean,df['Depth_in_um'])
    #plt.plot(mean+std,df['Depth_in_um'])
    #plt.plot(mean-std,df['Depth_in_um'])
    
    layer_borders_exp_df.to_csv(output_path_neun_layer_borders+exp+'_vM1_Borders.csv')
layer_borders_df.to_csv(output_path_neun_layer_borders+'vM1_Borders.csv')

# # Plot and save the average plots
# y = df['Depth_in_um']
# x = np.array(lhs_exp_means).mean(axis=0)
# error = np.array(lhs_exp_stds).mean(axis=0)
# plt.plot(x,y)
# plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
# plt.savefig(layers_path+'vM1_Avg_Depth_Profile_LHS.eps',format='eps')
# plt.cla()

# x = np.array(rhs_exp_means).mean(axis=0)
# error = np.array(rhs_exp_stds).mean(axis=0)
# plt.plot(x,y)
# plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
# plt.savefig(layers_path+'vM1_Avg_Depth_Profile_RHS.eps',format='eps')
# plt.cla()

# x = np.array(global_exp_means).mean(axis=0)
# error = np.array(global_exp_stds).mean(axis=0)
# plt.plot(x,y)
# plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
# plt.savefig(layers_path+'vM1_Avg_Depth_Profile.eps',format='eps')
# plt.cla()
  

  

## Create Layer Border Surfaces

In [ ]:
# # read layer borders
lb = pd.read_csv(output_path_neun_layer_borders_v2+'vM1_Borders.csv')
relative_depths = []
# Create layer borders
for exp in exp_names:
    for surf_res in surface_resolutions:
        # Read axis field vectors
        #vectors_sg = AmiraSpatialGraph(output_path_axis_fileds+exp+'_vM1_axis_field.am',generic_graph=True)
        i = 0
        for layer in ['L1_Border_Per','L2_3_Border_Per','L5A_Border_Per','L5B_Border_Per',]:
            #layer_depth = lb[layer][i]*100
            # mean layer border
            layer_depth_mean = lb[lb['Exp_Name']==exp][layer].mean() * 100
            layer_depth_upper = layer_depth_mean - (lb[lb['Exp_Name']==exp][layer].std() * 100)
            layer_depth_lower = layer_depth_mean + (lb[lb['Exp_Name']==exp][layer].std() * 100)
            #print(layer_depth)
            for layer_depth in [layer_depth_mean,layer_depth_upper,layer_depth_lower]:
                i = i +1
                layer_surf = Surface()
                arr = [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
                for j in range(len(arr)):
                    #j = arr[k]

                    vectors_sg_left = None
                    vectors_sg_right = None
                    vectors_sg_next_left = None
                    vectors_sg_next_right = None
                    layer_border_pts_left = []
                    layer_border_pts_right = []
                    if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                      .format(arr[j],surf_res,500,0.05)) and \
                           os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                          .format(arr[j+1],surf_res,500,0.05)):
                        vectors_sg = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                                       .format(arr[j],surf_res,500,0.05),\
                                                       generic_graph=True)
                        if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                          .format(arr[j],surf_res,500,0.05)):
                            vectors_sg_left = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                                                .format(arr[j],surf_res,500,0.05),\
                                                       generic_graph=True)
                        if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                          .format(arr[j],surf_res,500,0.05)):
                            vectors_sg_right = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                                                 .format(arr[j],surf_res,500,0.05),\
                                                       generic_graph=True)

                        vectors_sg_next = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                                            .format(arr[j+1],surf_res,500,0.05),\
                                                       generic_graph=True)

                        if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                          .format(arr[j+1],surf_res,500,0.05)):
                            vectors_sg_next_left = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                                                     .format(arr[j+1],surf_res,500,0.05),\
                                                       generic_graph=True)
                        if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                          .format(arr[j+1],surf_res,500,0.05)):
                            vectors_sg_next_right = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                                                      .format(arr[j+1],surf_res,500,0.05),\
                                                       generic_graph=True)

                        # now fetch layer specific pt
                        if len(vectors_sg.graph_data.edge_list) > 0 and len(vectors_sg_next.graph_data.edge_list) > 0:
                            pts = []
                            left_pts = []
                            right_pts = []
                            layer_border_pts = Vectors().get_layer_pts_for_given_depth(vectors_sg.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)
                            if vectors_sg_left is not None and len(vectors_sg_left.graph_data.edge_list) > 0:
                                layer_border_pts_left = Vectors().get_layer_pts_for_given_depth(vectors_sg_left.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)
                            if vectors_sg_right is not None and len(vectors_sg_right.graph_data.edge_list) > 0:
                                layer_border_pts_right = Vectors().get_layer_pts_for_given_depth(vectors_sg_right.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)

                            layer_border_pts_next = Vectors().get_layer_pts_for_given_depth(vectors_sg_next.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)
                            if vectors_sg_next_left is not None and len(vectors_sg_next_left.graph_data.edge_list) > 0: 
                                layer_border_pts_next_left = Vectors().get_layer_pts_for_given_depth(vectors_sg_next_left.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)
                            if vectors_sg_next_right is not None and len(vectors_sg_next_right.graph_data.edge_list) > 0:
                                layer_border_pts_next_right = Vectors().get_layer_pts_for_given_depth(vectors_sg_next_right.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)

                            for pt in layer_border_pts:
                                pts.append(pt)
                            for pt in layer_border_pts_next:
                                pts.append(pt)

                            for pt in layer_border_pts_left:
                                left_pts.append(pt)
                            for pt in layer_border_pts_next_left:
                                left_pts.append(pt)
                            for pt in layer_border_pts_right:
                                right_pts.append(pt)
                            for pt in layer_border_pts_next_right:
                                right_pts.append(pt)

                            # left side surface
                            #max(len(left_pts))
                            #print(len(layer_border_pts_left),len(layer_border_pts_next_left))
                            for pt_ind in range(max(len(layer_border_pts_left),len(layer_border_pts_next_left))-1):
                                #print(pt_ind)
                                pts = []
                                #pts_next = []
                                if len(layer_border_pts_left)>0:
                                    if pt_ind>len(layer_border_pts_left)-2:
                                        pts.append(layer_border_pts_left[len(layer_border_pts_left)-1])
                                    else:
                                        pts.append(layer_border_pts_left[pt_ind])
                                        pts.append(layer_border_pts_left[pt_ind+1])
                                if len(layer_border_pts_next_left)>0:
                                    if pt_ind>len(layer_border_pts_next_left)-2:
                                        pts.append(layer_border_pts_next_left[len(layer_border_pts_next_left)-1])
                                    else:
                                        pts.append(layer_border_pts_next_left[pt_ind])
                                        pts.append(layer_border_pts_next_left[pt_ind+1])
                                #print(pts)
                                #pts_next = [layer_border_pts_left[pt_ind+1],layer_border_pts_left[pt_ind+1]]

                                surf = Surface(pts=pts).create_delunay_surface_2d(return_hull=True,alpha=0,offset=100)

                                layer_surf.append(surf)
                            for pt_ind in range(max(len(layer_border_pts_right),len(layer_border_pts_next_right))-1):
                                #print(pt_ind)
                                pts = []
                                #pts_next = []
                                if len(layer_border_pts_right)>0:
                                    if pt_ind>len(layer_border_pts_right)-2:
                                        pts.append(layer_border_pts_right[len(layer_border_pts_right)-1])
                                    else:
                                        pts.append(layer_border_pts_right[pt_ind])
                                        pts.append(layer_border_pts_right[pt_ind+1])
                                if len(layer_border_pts_next_right)>0:
                                    if pt_ind>len(layer_border_pts_next_right)-2:
                                        pts.append(layer_border_pts_next_right[len(layer_border_pts_next_right)-1])
                                    else:
                                        pts.append(layer_border_pts_next_right[pt_ind])
                                        pts.append(layer_border_pts_next_right[pt_ind+1])
                                #print(pts)
                                #pts_next = [layer_border_pts_left[pt_ind+1],layer_border_pts_left[pt_ind+1]]
                                surf = Surface(pts=pts).create_delunay_surface_2d(return_hull=True,alpha=0,offset=100)
                                layer_surf.append(surf)

                vertices = []
                for vert_ind in range(layer_surf.surface.GetNumberOfPoints()):
                    vertices.append(layer_surf.surface.GetPoints().GetPoint(vert_ind))
                
                # write mean, upper or lower layer border surface
                if layer_depth == layer_depth_mean:
                    Landmarks(pts = vertices).write_landmarks( output_path_neun_layer_borders_v2+ exp +'_vM1_'+layer[:-4]+'_{}_mean.landmarksAscii'.format(surf_res))
                    layer_surf.write_surface_mesh(output_path_neun_layer_borders_v2+exp +'_vM1_'+layer[:-4]+'_{}_mean.vtk'.format(surf_res))
                elif layer_depth == layer_depth_upper:
                    Landmarks(pts = vertices).write_landmarks( output_path_neun_layer_borders_v2+ exp +'_vM1_'+layer[:-4]+'_{}_upper.landmarksAscii'.format(surf_res))
                    layer_surf.write_surface_mesh(output_path_neun_layer_borders_v2+exp +'_vM1_'+layer[:-4]+'_{}_upper.vtk'.format(surf_res))
                else:
                    Landmarks(pts = vertices).write_landmarks( output_path_neun_layer_borders_v2+ exp +'_vM1_'+layer[:-4]+'_{}_lower.landmarksAscii'.format(surf_res))
                    layer_surf.write_surface_mesh(output_path_neun_layer_borders_v2+exp +'_vM1_'+layer[:-4]+'_{}_lower.vtk'.format(surf_res))

                

## Find 2D to 3D Scaling and Cell counts

# Create Rabies Borders vM1

## Approach without using cylinders


In [24]:
def is_between_pts(a,b,c):
    x1,y1,z1 = a
    x2,y2,z2 = b
    x,y,z = c
    AB = np.sqrt((x2-x1)*(x2-x1)+(y2-y1)*(y2-y1)+(z2-z1)*(z2-z1))
    AP = np.sqrt((x-x1)*(x-x1)+(y-y1)*(y-y1)+(z-z1)*(z-z1))
    PB = np.sqrt((x2-x)*(x2-x)+(y2-y)*(y2-y)+(z2-z)*(z2-z))
    if(AB == AP + PB):
        return True
    return False

In [25]:
def get_points_on_line_segment(a,b,pts):
    pts_found = []
    for pt in pts:
        if is_between_pts(a,b,pt):
            pts_found.append(pt)
    return pts_found

In [28]:
def get_cell_density_profile_from_axis_projected_landmarks(edge,landmarks,res=50,barrel_radius=250):
    
    line_pts = get_line_pts(edge,res)
    profile = []
    total_nr = 0
    total_vol = 0
    #column_center = (np.array(edge[0]) + np.array(edge[1])) / 2
    
    # go through pts of the line and get density value for each pt at given z res and barrel radius
    for i in range(len(line_pts)-1):
        # create a bounding box oriented in the direction of the line
        landmarks_within_cylinder = get_points_on_line_segment(line_pts[i],line_pts[i+1],landmarks)
        
        volume = np.pi*(barrel_radius*barrel_radius)*res#Surface(polydata=cylinder).get_surface_properties(prop='volume')
        total_nr = total_nr + len(landmarks_within_cylinder)
        total_vol = total_vol+volume
        density = len(landmarks_within_cylinder)/volume # in nr / cubic_um
        profile.append(density*1000000)# in 1000 / cubic mm 
    print('column volume in {} and num of neurons {}'.format(total_vol,total_nr))
    return profile
    

In [33]:
# Read the vm1 axis field from caudal to rostral
ref_col_depth = 2600
#density_peak_3d = 121.237 # (in 1000 / cubic mm)
res = 50
radius = 250
offset = 10

for exp in exp_names:
    
    rabies_landmarks = Landmarks(output_path_landmarks+'Density_Clusters/'+exp+'_vM1_rabies_landmarks_voxel_500_cutoff_0.1.LandmarkAscii')
    axis_field = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_surf_res_50_voxel_size_500_cutoff_0.1.am',generic_graph=True)
    axes_lens,l5_upper_depths,l5_lower_depths,l5_avg_depths,pts_on_axes = Vectors().get_landmarks_depth_along_axis_field\
                                                                (axis_field.graph_data.edge_list,rabies_landmarks.pts,\
                                                                 radius=250)
    Landmarks(pts=pts_on_axes).write_landmarks(output_path_rabies_profiles_v2+exp+'_axes_mapped_rabies_landmarks.landamrksAscii')
    
    # move from caudal to rostral vm1
    col_landmarks = Landmarks()
    col_cylinders = Surface()
    for row in range(-10,10):
        df_unstreched_main = pd.DataFrame()
        df_streched_main = pd.DataFrame()
        df_unstreched_main['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
        df_streched_main['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
        row_present = False
        
        for col in range(-10,10):
            print(row,col)
            if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_50_500_0.1.am'.format(row,col)):
                print(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_50_500_0.1.am'.format(row,col))
            
                sg = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_50_500_0.1.am'.\
                                       format(row,col),generic_graph=True)
                
                edge = sg.graph_data.edge_list[0]
                profile = get_cell_density_profile_from_axis_projected_landmarks(edge,pts_on_axes)
                profile_interpolated = interpolate_array(profile,round(ref_col_depth/res))

                df_profile_unstreched = pd.DataFrame()
                df_profile_unstreched['NeuN_Density_{}_{}'.format(exp,col)] = profile # in 1000 / cubic mm

                df_profile_streched = pd.DataFrame()
                df_profile_streched['NeuN_Density_{}_{}'.format(exp,col)] = profile_interpolated # in 1000 / cubic mm

                df_unstreched_main = pd.concat([df_unstreched_main,df_profile_unstreched],ignore_index=False,axis=1)
                df_streched_main = pd.concat([df_streched_main,df_profile_streched],ignore_index=False,axis=1)
                row_present = True
                #col_landmarks.append_landmarks(profile_landmarks)
                #col_cylinders.append(profile_cylinder)
                
        if row_present:
            df_unstreched_main.to_csv(output_path_rabies_profiles_v2+'_vM1_NeuN_Depth_Profile_Unstreched_using_{}_row_{}.csv'.format(exp,row))
            df_streched_main.to_csv(output_path_rabies_profiles_v2+'_vM1_NeuN_Depth_Profile_Streched_using_{}_row_{}.csv'.format(exp,row))
        
    #col_landmarks.write_landmarks(output_path_rabies_profiles+exp+'_vM1_NeuN_Column_Landmarks.landmarksAscii')
    #col_cylinders.write_surface_mesh(output_path_rabies_profiles+exp+'_vM1_NeuN_Column_Cylinders.vtk')

-10 -10
-10 -9
-10 -8
-10 -7
-10 -6
-10 -5
-10 -4
-10 -3
-10 -2
-10 -1
-10 0
-10 1
-10 2
-10 3
-10 4
-10 5
-10 6
-10 7
-10 8
-10 9
-9 -10
-9 -9
-9 -8
-9 -7
-9 -6
-9 -5
-9 -4
-9 -3
-9 -2
-9 -1
-9 0
-9 1
-9 2
-9 3
-9 4
-9 5
-9 6
-9 7
-9 8
-9 9
-8 -10
-8 -9
-8 -8
-8 -7
-8 -6
-8 -5
-8 -4
-8 -3
-8 -2
-8 -1
-8 0
-8 1
-8 2
-8 3
-8 4
-8 5
-8 6
-8 7
-8 8
-8 9
-7 -10
-7 -9
-7 -8
-7 -7
-7 -6
-7 -5
-7 -4
-7 -3
-7 -2
-7 -1
-7 0
-7 1
-7 2
-7 3
-7 4
-7 5
-7 6
-7 7
-7 8
-7 9
-6 -10
-6 -9
-6 -8
-6 -7
-6 -6
-6 -5
-6 -4
-6 -3
-6 -2
-6 -1
-6 0
-6 1
-6 2
-6 3
-6 4
-6 5
-6 6
-6 7
-6 8
-6 9
-5 -10
-5 -9
-5 -8
-5 -7
-5 -6
-5 -5
-5 -4
-5 -3
-5 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_lhs_vM1_axis_field_row_-5_-2_50_500_0.1.am
column volume in 431968989.868597 and num of neurons 1
-5 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_lhs_vM1_axis_field_row_-5_-1_50_500_0.1.am
column volume in 441786

column volume in 510508806.208342 and num of neurons 3
1 4
1 5
1 6
1 7
1 8
1 9
2 -10
2 -9
2 -8
2 -7
2 -6
2 -5
2 -4
2 -3
2 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_lhs_vM1_axis_field_row_2_-2_50_500_0.1.am
column volume in 461421420.99600136 and num of neurons 7
2 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_lhs_vM1_axis_field_row_2_-1_50_500_0.1.am
column volume in 461421420.99600136 and num of neurons 4
2 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_lhs_vM1_axis_field_row_2_0_50_500_0.1.am
column volume in 481056375.0809376 and num of neurons 7
2 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_lhs_vM1_axis_field_row_2_1_50_500_0.1.am
column volume in 471238898.0384695 and num of neurons 2
2 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//

column volume in 314159265.3589794 and num of neurons 1
-6 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_-6_1_50_500_0.1.am
column volume in 304341788.3165113 and num of neurons 2
-6 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_-6_2_50_500_0.1.am
column volume in 294524311.27404314 and num of neurons 1
-6 3
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_-6_3_50_500_0.1.am
column volume in 314159265.3589794 and num of neurons 1
-6 4
-6 5
-6 6
-6 7
-6 8
-6 9
-5 -10
-5 -9
-5 -8
-5 -7
-5 -6
-5 -5
-5 -4
-5 -3
-5 -2
-5 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_-5_-1_50_500_0.1.am
column volume in 333794219.44391567 and num of neurons 1
-5 0
/nas1/Data_Mythreya/MotorCortexProject

column volume in 471238898.0384695 and num of neurons 2
3 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_3_0_50_500_0.1.am
column volume in 461421420.99600136 and num of neurons 0
3 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_3_1_50_500_0.1.am
column volume in 500691329.1658739 and num of neurons 1
3 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_3_2_50_500_0.1.am
column volume in 441786466.9110651 and num of neurons 0
3 3
3 4
3 5
3 6
3 7
3 8
3 9
4 -10
4 -9
4 -8
4 -7
4 -6
4 -5
4 -4
4 -3
4 -2
4 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_4_-1_50_500_0.1.am
column volume in 431968989.868597 and num of neurons 2
4 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Out

column volume in 431968989.868597 and num of neurons 4
-2 3
-2 4
-2 5
-2 6
-2 7
-2 8
-2 9
-1 -10
-1 -9
-1 -8
-1 -7
-1 -6
-1 -5
-1 -4
-1 -3
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_lhs_vM1_axis_field_row_-1_-3_50_500_0.1.am
column volume in 392699081.69872445 and num of neurons 9
-1 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_lhs_vM1_axis_field_row_-1_-2_50_500_0.1.am
column volume in 431968989.868597 and num of neurons 24
-1 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_lhs_vM1_axis_field_row_-1_-1_50_500_0.1.am
column volume in 481056375.0809376 and num of neurons 35
-1 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_lhs_vM1_axis_field_row_-1_0_50_500_0.1.am
column volume in 490873852.12340575 and num of neurons 39
-1 1
/nas1/Data_Mythreya/MotorCortexProject/V

...

column volume in 323976742.40144753 and num of neurons 2
7 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_rhs_vM1_axis_field_row_7_1_50_500_0.1.am
column volume in 323976742.40144753 and num of neurons 5
7 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_rhs_vM1_axis_field_row_7_2_50_500_0.1.am
column volume in 304341788.3165113 and num of neurons 4
7 3
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_rhs_vM1_axis_field_row_7_3_50_500_0.1.am
column volume in 235619449.01923445 and num of neurons 0
7 4
7 5
7 6
7 7
7 8
7 9
8 -10
8 -9
8 -8
8 -7
8 -6
8 -5
8 -4
8 -3
8 -2
8 -1
8 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_rhs_vM1_axis_field_row_8_0_50_500_0.1.am
column volume in 314159265.3589794 and num of neurons 4
8 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/O

column volume in 373064127.6137882 and num of neurons 0
-4 6
-4 7
-4 8
-4 9
-3 -10
-3 -9
-3 -8
-3 -7
-3 -6
-3 -5
-3 -4
-3 -3
-3 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_-3_-2_50_500_0.1.am
column volume in 402516558.7411926 and num of neurons 0
-3 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_-3_-1_50_500_0.1.am
column volume in 373064127.6137882 and num of neurons 1
-3 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_-3_0_50_500_0.1.am
column volume in 343611696.4863838 and num of neurons 1
-3 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_-3_1_50_500_0.1.am
column volume in 382881604.6562563 and num of neurons 0
-3 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Fra

column volume in 373064127.6137882 and num of neurons 0
2 5
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_2_5_50_500_0.1.am
column volume in 294524311.27404314 and num of neurons 0
2 6
2 7
2 8
2 9
3 -10
3 -9
3 -8
3 -7
3 -6
3 -5
3 -4
3 -3
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_3_-3_50_500_0.1.am
column volume in 451603943.95353323 and num of neurons 1
3 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_3_-2_50_500_0.1.am
column volume in 422151512.82612884 and num of neurons 0
3 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_3_-1_50_500_0.1.am
column volume in 422151512.82612884 and num of neurons 3
3 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_

column volume in 382881604.6562563 and num of neurons 2
-5 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_-5_0_50_500_0.1.am
column volume in 392699081.69872445 and num of neurons 1
-5 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_-5_1_50_500_0.1.am
column volume in 402516558.7411926 and num of neurons 0
-5 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_-5_2_50_500_0.1.am
column volume in 402516558.7411926 and num of neurons 0
-5 3
-5 4
-5 5
-5 6
-5 7
-5 8
-5 9
-4 -10
-4 -9
-4 -8
-4 -7
-4 -6
-4 -5
-4 -4
-4 -3
-4 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_-4_-2_50_500_0.1.am
column volume in 382881604.6562563 and num of neurons 0
-4 -1
/nas1/Data_Mythreya/MotorCortexProject/

column volume in 412334035.7836607 and num of neurons 4
4 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_4_1_50_500_0.1.am
column volume in 363246650.57132006 and num of neurons 2
4 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_4_2_50_500_0.1.am
column volume in 392699081.69872445 and num of neurons 0
4 3
4 4
4 5
4 6
4 7
4 8
4 9
5 -10
5 -9
5 -8
5 -7
5 -6
5 -5
5 -4
5 -3
5 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_5_-2_50_500_0.1.am
column volume in 353429173.5288519 and num of neurons 1
5 -1
5 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_5_0_50_500_0.1.am
column volume in 402516558.7411926 and num of neurons 0
5 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/O

## vM1 Rabies Depth Profiles

In [32]:
# Read the vm1 axis field from caudal to rostral
ref_col_depth = 2600
#density_peak_3d = 121.237 # (in 1000 / cubic mm)
res = 50
radius = 250
offset = 10

for exp in exp_names:
    
    rabies_landmarks = Landmarks(output_path_landmarks+'Density_Clusters/'+exp+'_vM1_rabies_landmarks_voxel_500_cutoff_0.1.LandmarkAscii')
    
#     if exp.find('MG49_rhs') >=0  and exp.find('MG50_rhs') >=0:
#         rowwise_axis_sg_list = sorted(glob.glob(output_path_vM1_axis_field+exp+'_vM1_axis_field_row*_50_500_0.05.am'),reverse=True)
#     else:
#         rowwise_axis_sg_list = sorted(glob.glob(output_path_vM1_axis_field+exp+'_vM1_axis_field_row*_50_500_0.05.am'))
#     #print(rowwise_axis_sg_list)
    
    # move from caudal to rostral vm1
    col_landmarks = Landmarks()
    col_cylinders = Surface()
    for row in range(-10,10):
        df_unstreched_main = pd.DataFrame()
        df_streched_main = pd.DataFrame()
        df_unstreched_main['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
        df_streched_main['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
        row_present = False
        
        for col in range(-10,10):
            print(row,col)
            if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_50_500_0.1.am'.format(row,col)):
                print(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_50_500_0.1.am'.format(row,col))
            
                sg = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_50_500_0.1.am'.\
                                       format(row,col),generic_graph=True)
                
                edge = sg.graph_data.edge_list[0]
                profile,profile_cylinder,profile_landmarks = get_cell_density_profile\
                    (edge,res,radius,rabies_landmarks.pts,exp,'row_'+str(row)+'_'+str(col))
                profile_interpolated = interpolate_array(profile,round(ref_col_depth/res))

                df_profile_unstreched = pd.DataFrame()
                df_profile_unstreched['NeuN_Density_{}_{}'.format(exp,col)] = profile # in 1000 / cubic mm

                df_profile_streched = pd.DataFrame()
                df_profile_streched['NeuN_Density_{}_{}'.format(exp,col)] = profile_interpolated # in 1000 / cubic mm

                df_unstreched_main = pd.concat([df_unstreched_main,df_profile_unstreched],ignore_index=False,axis=1)
                df_streched_main = pd.concat([df_streched_main,df_profile_streched],ignore_index=False,axis=1)
                row_present = True
                col_landmarks.append_landmarks(profile_landmarks)
                col_cylinders.append(profile_cylinder)
                
        if row_present:
            df_unstreched_main.to_csv(output_path_rabies_profiles+'_vM1_NeuN_Depth_Profile_Unstreched_using_{}_row_{}.csv'.format(exp,row))
            df_streched_main.to_csv(output_path_rabies_profiles+'_vM1_NeuN_Depth_Profile_Streched_using_{}_row_{}.csv'.format(exp,row))
        
    col_landmarks.write_landmarks(output_path_rabies_profiles+exp+'_vM1_NeuN_Column_Landmarks.landmarksAscii')
    col_cylinders.write_surface_mesh(output_path_rabies_profiles+exp+'_vM1_NeuN_Column_Cylinders.vtk')

-10 -10
-10 -9
-10 -8
-10 -7
-10 -6
-10 -5
-10 -4
-10 -3
-10 -2
-10 -1
-10 0
-10 1
-10 2
-10 3
-10 4
-10 5
-10 6
-10 7
-10 8
-10 9
-9 -10
-9 -9
-9 -8
-9 -7
-9 -6
-9 -5
-9 -4
-9 -3
-9 -2
-9 -1
-9 0
-9 1
-9 2
-9 3
-9 4
-9 5
-9 6
-9 7
-9 8
-9 9
-8 -10
-8 -9
-8 -8
-8 -7
-8 -6
-8 -5
-8 -4
-8 -3
-8 -2
-8 -1
-8 0
-8 1
-8 2
-8 3
-8 4
-8 5
-8 6
-8 7
-8 8
-8 9
-7 -10
-7 -9
-7 -8
-7 -7
-7 -6
-7 -5
-7 -4
-7 -3
-7 -2
-7 -1
-7 0
-7 1
-7 2
-7 3
-7 4
-7 5
-7 6
-7 7
-7 8
-7 9
-6 -10
-6 -9
-6 -8
-6 -7
-6 -6
-6 -5
-6 -4
-6 -3
-6 -2
-6 -1
-6 0
-6 1
-6 2
-6 3
-6 4
-6 5
-6 6
-6 7
-6 8
-6 9
-5 -10
-5 -9
-5 -8
-5 -7
-5 -6
-5 -5
-5 -4
-5 -3
-5 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_lhs_vM1_axis_field_row_-5_-2_50_500_0.1.am
column volume in 431968989.868597 and num of neurons 43
-5 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_lhs_vM1_axis_field_row_-5_-1_50_500_0.1.am
column volume in 44178

column volume in 471238898.0384695 and num of neurons 94
1 3
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_lhs_vM1_axis_field_row_1_3_50_500_0.1.am
column volume in 510508806.208342 and num of neurons 0
1 4
1 5
1 6
1 7
1 8
1 9
2 -10
2 -9
2 -8
2 -7
2 -6
2 -5
2 -4
2 -3
2 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_lhs_vM1_axis_field_row_2_-2_50_500_0.1.am
column volume in 461421420.99600136 and num of neurons 46
2 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_lhs_vM1_axis_field_row_2_-1_50_500_0.1.am
column volume in 461421420.99600136 and num of neurons 138
2 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_lhs_vM1_axis_field_row_2_0_50_500_0.1.am
column volume in 481056375.0809376 and num of neurons 288
2 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Out

column volume in 323976742.40144753 and num of neurons 0
-6 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_-6_0_50_500_0.1.am
column volume in 314159265.3589794 and num of neurons 0
-6 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_-6_1_50_500_0.1.am
column volume in 304341788.3165113 and num of neurons 60
-6 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_-6_2_50_500_0.1.am
column volume in 294524311.27404314 and num of neurons 29
-6 3
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_-6_3_50_500_0.1.am
column volume in 314159265.3589794 and num of neurons 0
-6 4
-6 5
-6 6
-6 7
-6 8
-6 9
-5 -10
-5 -9
-5 -8
-5 -7
-5 -6
-5 -5
-5 -4
-5 -3
-5 -2
-5 -1
/nas1/Data_Mythreya/MotorCortexProjec

column volume in 382881604.6562563 and num of neurons 0
2 3
2 4
2 5
2 6
2 7
2 8
2 9
3 -10
3 -9
3 -8
3 -7
3 -6
3 -5
3 -4
3 -3
3 -2
3 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_3_-1_50_500_0.1.am
column volume in 471238898.0384695 and num of neurons 0
3 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_3_0_50_500_0.1.am
column volume in 461421420.99600136 and num of neurons 0
3 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_3_1_50_500_0.1.am
column volume in 500691329.1658739 and num of neurons 50
3 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG49_rhs_vM1_axis_field_row_3_2_50_500_0.1.am
column volume in 441786466.9110651 and num of neurons 0
3 3
3 4
3 5
3 6
3 7
3 8
3 9
4 -10
4 -9
4 -8
4 -7
4 -6
4 -5
4 -

column volume in 481056375.0809376 and num of neurons 1200
-2 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_lhs_vM1_axis_field_row_-2_1_50_500_0.1.am
column volume in 412334035.7836607 and num of neurons 328
-2 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_lhs_vM1_axis_field_row_-2_2_50_500_0.1.am
column volume in 431968989.868597 and num of neurons 129
-2 3
-2 4
-2 5
-2 6
-2 7
-2 8
-2 9
-1 -10
-1 -9
-1 -8
-1 -7
-1 -6
-1 -5
-1 -4
-1 -3
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_lhs_vM1_axis_field_row_-1_-3_50_500_0.1.am
column volume in 392699081.69872445 and num of neurons 156
-1 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_lhs_vM1_axis_field_row_-1_-2_50_500_0.1.am
column volume in 431968989.868597 and num of neurons 302
-1 -1
/nas1/Data_Mythreya/MotorCortexPro

...

column volume in 304341788.3165113 and num of neurons 0
6 3
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_rhs_vM1_axis_field_row_6_3_50_500_0.1.am
column volume in 235619449.01923445 and num of neurons 0
6 4
6 5
6 6
6 7
6 8
6 9
7 -10
7 -9
7 -8
7 -7
7 -6
7 -5
7 -4
7 -3
7 -2
7 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_rhs_vM1_axis_field_row_7_-1_50_500_0.1.am
column volume in 333794219.44391567 and num of neurons 0
7 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_rhs_vM1_axis_field_row_7_0_50_500_0.1.am
column volume in 323976742.40144753 and num of neurons 0
7 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG50_rhs_vM1_axis_field_row_7_1_50_500_0.1.am
column volume in 323976742.40144753 and num of neurons 32
7 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Ou

column volume in 382881604.6562563 and num of neurons 0
-4 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_-4_0_50_500_0.1.am
column volume in 382881604.6562563 and num of neurons 0
-4 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_-4_1_50_500_0.1.am
column volume in 353429173.5288519 and num of neurons 35
-4 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_-4_2_50_500_0.1.am
column volume in 392699081.69872445 and num of neurons 0
-4 3
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_-4_3_50_500_0.1.am
column volume in 402516558.7411926 and num of neurons 0
-4 4
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_-4

column volume in 520326283.25081015 and num of neurons 0
2 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_2_-2_50_500_0.1.am
column volume in 520326283.25081015 and num of neurons 0
2 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_2_-1_50_500_0.1.am
column volume in 481056375.0809376 and num of neurons 48
2 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_2_0_50_500_0.1.am
column volume in 500691329.1658739 and num of neurons 0
2 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_2_1_50_500_0.1.am
column volume in 471238898.0384695 and num of neurons 0
2 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_lhs_vM1_axis_field_row_2_2_50

column volume in 402516558.7411926 and num of neurons 0
-7 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_-7_1_50_500_0.1.am
column volume in 402516558.7411926 and num of neurons 0
-7 2
-7 3
-7 4
-7 5
-7 6
-7 7
-7 8
-7 9
-6 -10
-6 -9
-6 -8
-6 -7
-6 -6
-6 -5
-6 -4
-6 -3
-6 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_-6_-2_50_500_0.1.am
column volume in 402516558.7411926 and num of neurons 40
-6 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_-6_-1_50_500_0.1.am
column volume in 392699081.69872445 and num of neurons 0
-6 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_-6_0_50_500_0.1.am
column volume in 392699081.69872445 and num of neurons 0
-6 1
/nas1/Data_Mythreya/MotorCortex

column volume in 461421420.99600136 and num of neurons 0
2 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_2_0_50_500_0.1.am
column volume in 471238898.0384695 and num of neurons 0
2 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_2_1_50_500_0.1.am
column volume in 481056375.0809376 and num of neurons 0
2 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_2_2_50_500_0.1.am
column volume in 500691329.1658739 and num of neurons 0
2 3
2 4
2 5
2 6
2 7
2 8
2 9
3 -10
3 -9
3 -8
3 -7
3 -6
3 -5
3 -4
3 -3
3 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vM1_Ref_Surfaces_Axis_Fields/MG48_rhs_vM1_axis_field_row_3_-2_50_500_0.1.am
column volume in 392699081.69872445 and num of neurons 0
3 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outpu

In [31]:
1/(0.3*0.25)

13.333333333333334

## Derive Rabies Border

In [ ]:
depth_max = 2600
layer_borders_df = pd.DataFrame(columns=['Exp_Name','L5_Upper_Border','L5_Mean_Border','L5_Lower_Border',\
                                        'L5_Upper_Border_Per','L5_Mean_Border_Per','L5_Lower_Border_Per'])
# rowwise profiles going from caudal to rostral
#exp = 'MG50_lhs'
#%matplotlib notebook
#figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')
#p = plt.subplot(psize=[10,10])

lhs_exp_means = []
lhs_exp_stds = []
rhs_exp_means = []
rhs_exp_stds = []
global_exp_means = []
global_exp_stds = []

thresh = 0.1# 90% of peak density

for exp in exp_names:
    layer_borders_exp_df = pd.DataFrame(columns=['Exp_Name','L5_Upper_Border','L5_Mean_Border','L5_Lower_Border',\
                                        'L5_Upper_Border_Per','L5_Mean_Border_Per','L5_Lower_Border_Per'])
    exp_rows = []
    exp_profiles = []
    for i in range(-10,10):

        file = None
        if i <0:
            if len(glob.glob(output_path_rabies_profiles+'_vM1_NeuN_Depth_Profile_Streched_using_'+exp+'_row_{}.csv'.format(i)))>0:
                file = glob.glob(output_path_rabies_profiles+'_vM1_NeuN_Depth_Profile_Streched_using_'+exp+'_row_{}.csv'.format(i))[0]
        else:
            if len(glob.glob(output_path_rabies_profiles+'_vM1_NeuN_Depth_Profile_Streched_using_'+exp+'_row_{}.csv'.format(i)))>0:
                file = glob.glob(output_path_rabies_profiles+'_vM1_NeuN_Depth_Profile_Streched_using_'+exp+'_row_{}.csv'.format(i))[0]
        if file is not None:
            #print(file)
            df = pd.read_csv(file)
            #print(df)
            rows = []
            for j in range(2,len(df.columns)):
                profile_correct_direction_list = list(df[df.columns[j]])
                profile_correct_direction_list.reverse()
                # scale profile to be neurons per cubic mm instead of 
                profile_correct_direction = abs(np.array(profile_correct_direction_list))*1000
                if np.array(profile_correct_direction).mean(axis=0) > 1:
                    
                    
                    
                    #print(L2_3_border,L5A_border,L5B_border)
                    #plt.plot(df[df.columns[j]],df['Depth_in_um'])
                    rows.append(profile_correct_direction)
                    exp_profiles.append(profile_correct_direction)
                    
    global_exp_means.append(np.array(exp_profiles).mean(axis=0))
    global_exp_stds.append(np.array(exp_profiles).std(axis=0))
    if exp.find('_lhs')>0:
        lhs_exp_means.append(np.array(exp_profiles).mean(axis=0))
        lhs_exp_stds.append(np.array(exp_profiles).std(axis=0))
    else:
        rhs_exp_means.append(np.array(exp_profiles).mean(axis=0))
        rhs_exp_stds.append(np.array(exp_profiles).std(axis=0))
            #print(rows)
            
            #if len(rows)>0:
            #    x = np.array(rows).mean(axis=0)
            #    exp_rows.append(x)
                #print(x)
                #plt.plot(x,df['Depth_in_um'])
    
    y = df['Depth_in_um']
    x = np.array(exp_profiles).mean(axis=0)
    error = np.array(exp_profiles).std(axis=0)
#     plt.plot(x,y)
#     #plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
#     plt.savefig(layers_rabies_path+exp+'_vM1_Avg_Depth_Profile.eps',format='eps')
#     plt.cla()
    
    
    # Create layers for only mean exp profile
    thresh = x.max()*0.1
    max_ind = x.argmax()
    L5_Mean = x.argmax()*-50
    L5_Upper = np.where(x[0:max_ind]>thresh)[0][0]*-50
    L5_Lower = np.where(x[max_ind:len(x)]<thresh)[0][0]*-50 + \
                    (max_ind*-50)
    df_tmp = pd.DataFrame(columns=layer_borders_df.columns,data=[[exp,L5_Upper,L5_Mean,L5_Lower,\
                                                                    abs(L5_Upper/depth_max),abs(L5_Mean/depth_max),\
                                                                     abs(L5_Lower/depth_max),] ])
    layer_borders_exp_df = layer_borders_exp_df.append(df_tmp)

    layer_borders_df = layer_borders_df.append(df_tmp)
    
    # store it to approp cat for avg plot
    
    #plt.plot(mean,df['Depth_in_um'])
    #plt.plot(mean+std,df['Depth_in_um'])
    #plt.plot(mean-std,df['Depth_in_um'])
    
    layer_borders_exp_df.to_csv(output_path_rabies_layers+exp+'_vM1_Borders.csv')
layer_borders_df.to_csv(output_path_rabies_layers+'vM1_Borders.csv')

# # Plot and save the average plots
# y = df['Depth_in_um']
# x = np.array(lhs_exp_means).mean(axis=0)
# error = np.array(lhs_exp_means).std(axis=0)
# plt.plot(x,y)
# plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
# plt.savefig(layers_rabies_path+'vM1_Avg_Depth_Profile_LHS.eps',format='eps')
# plt.cla()

# x = np.array(rhs_exp_means).mean(axis=0)
# error = np.array(rhs_exp_means).std(axis=0)
# plt.plot(x,y)
# plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
# plt.savefig(layers_rabies_path+'vM1_Avg_Depth_Profile_RHS.eps',format='eps')
# plt.cla()

# x = np.array(global_exp_means).mean(axis=0)
# error = np.array(global_exp_means).std(axis=0)
# plt.plot(x,y)
# plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
# plt.savefig(layers_rabies_path+'vM1_Avg_Depth_Profile.eps',format='eps')
# plt.cla()
  

## Create Rabies Border Surfaces

In [ ]:
# # read layer borders
lb = pd.read_csv(output_path_rabies_layers+'vM1_Borders.csv')
relative_depths = []
# Create layer borders
for exp in exp_names:
    for surf_res in surface_resolutions:
        # Read axis field vectors
        #vectors_sg = AmiraSpatialGraph(output_path_axis_fileds+exp+'_vM1_axis_field.am',generic_graph=True)
        i = 0
        for layer in ['L5_Upper_Border_Per','L5_Mean_Border_Per','L5_Lower_Border_Per',]:
            #layer_depth = lb[layer][i]*100
            # mean layer border
            layer_depth_mean = lb[lb['Exp_Name']==exp][layer].mean() * 100
            layer_depth_upper = layer_depth_mean - (lb[lb['Exp_Name']==exp][layer].std() * 100)
            layer_depth_lower = layer_depth_mean + (lb[lb['Exp_Name']==exp][layer].std() * 100)
            print(layer_depth_mean,layer_depth_upper,layer_depth_lower)
            for layer_depth in [layer_depth_mean,layer_depth_upper,layer_depth_lower]:
                i = i +1
                print(layer_depth)
                layer_surf = Surface()
                arr = [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
                for j in range(len(arr)):
                    #j = arr[k]

                    vectors_sg_left = None
                    vectors_sg_right = None
                    vectors_sg_next_left = None
                    vectors_sg_next_right = None
                    layer_border_pts_left = []
                    layer_border_pts_right = []
                    if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                      .format(arr[j],surf_res,500,0.05)) and \
                           os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                          .format(arr[j+1],surf_res,500,0.05)):
                        vectors_sg = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                                       .format(arr[j],surf_res,500,0.05),\
                                                       generic_graph=True)
                        if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                          .format(arr[j],surf_res,500,0.05)):
                            vectors_sg_left = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                                                .format(arr[j],surf_res,500,0.05),\
                                                       generic_graph=True)
                        if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                          .format(arr[j],surf_res,500,0.05)):
                            vectors_sg_right = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                                                 .format(arr[j],surf_res,500,0.05),\
                                                       generic_graph=True)

                        vectors_sg_next = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                                            .format(arr[j+1],surf_res,500,0.05),\
                                                       generic_graph=True)

                        if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                          .format(arr[j+1],surf_res,500,0.05)):
                            vectors_sg_next_left = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                                                     .format(arr[j+1],surf_res,500,0.05),\
                                                       generic_graph=True)
                        if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                          .format(arr[j+1],surf_res,500,0.05)):
                            vectors_sg_next_right = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                                                      .format(arr[j+1],surf_res,500,0.05),\
                                                       generic_graph=True)

                        # now fetch layer specific pt
                        if len(vectors_sg.graph_data.edge_list) > 0 and len(vectors_sg_next.graph_data.edge_list) > 0:
                            pts = []
                            left_pts = []
                            right_pts = []
                            layer_border_pts = Vectors().get_layer_pts_for_given_depth(vectors_sg.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)
                            if vectors_sg_left is not None and len(vectors_sg_left.graph_data.edge_list) > 0:
                                layer_border_pts_left = Vectors().get_layer_pts_for_given_depth(vectors_sg_left.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)
                            if vectors_sg_right is not None and len(vectors_sg_right.graph_data.edge_list) > 0:
                                layer_border_pts_right = Vectors().get_layer_pts_for_given_depth(vectors_sg_right.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)

                            layer_border_pts_next = Vectors().get_layer_pts_for_given_depth(vectors_sg_next.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)
                            if vectors_sg_next_left is not None and len(vectors_sg_next_left.graph_data.edge_list) > 0: 
                                layer_border_pts_next_left = Vectors().get_layer_pts_for_given_depth(vectors_sg_next_left.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)
                            if vectors_sg_next_right is not None and len(vectors_sg_next_right.graph_data.edge_list) > 0:
                                layer_border_pts_next_right = Vectors().get_layer_pts_for_given_depth(vectors_sg_next_right.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)

                            for pt in layer_border_pts:
                                pts.append(pt)
                            for pt in layer_border_pts_next:
                                pts.append(pt)

                            for pt in layer_border_pts_left:
                                left_pts.append(pt)
                            for pt in layer_border_pts_next_left:
                                left_pts.append(pt)
                            for pt in layer_border_pts_right:
                                right_pts.append(pt)
                            for pt in layer_border_pts_next_right:
                                right_pts.append(pt)

                            # left side surface
                            #max(len(left_pts))
                            #print(len(layer_border_pts_left),len(layer_border_pts_next_left))
                            for pt_ind in range(max(len(layer_border_pts_left),len(layer_border_pts_next_left))-1):
                                #print(pt_ind)
                                pts = []
                                #pts_next = []
                                if len(layer_border_pts_left)>0:
                                    if pt_ind>len(layer_border_pts_left)-2:
                                        pts.append(layer_border_pts_left[len(layer_border_pts_left)-1])
                                    else:
                                        pts.append(layer_border_pts_left[pt_ind])
                                        pts.append(layer_border_pts_left[pt_ind+1])
                                if len(layer_border_pts_next_left)>0:
                                    if pt_ind>len(layer_border_pts_next_left)-2:
                                        pts.append(layer_border_pts_next_left[len(layer_border_pts_next_left)-1])
                                    else:
                                        pts.append(layer_border_pts_next_left[pt_ind])
                                        pts.append(layer_border_pts_next_left[pt_ind+1])
                                #print(pts)
                                #pts_next = [layer_border_pts_left[pt_ind+1],layer_border_pts_left[pt_ind+1]]

                                surf = Surface(pts=pts).create_delunay_surface_2d(return_hull=True,alpha=0,offset=100)

                                layer_surf.append(surf)
                            for pt_ind in range(max(len(layer_border_pts_right),len(layer_border_pts_next_right))-1):
                                #print(pt_ind)
                                pts = []
                                #pts_next = []
                                if len(layer_border_pts_right)>0:
                                    if pt_ind>len(layer_border_pts_right)-2:
                                        pts.append(layer_border_pts_right[len(layer_border_pts_right)-1])
                                    else:
                                        pts.append(layer_border_pts_right[pt_ind])
                                        pts.append(layer_border_pts_right[pt_ind+1])
                                if len(layer_border_pts_next_right)>0:
                                    if pt_ind>len(layer_border_pts_next_right)-2:
                                        pts.append(layer_border_pts_next_right[len(layer_border_pts_next_right)-1])
                                    else:
                                        pts.append(layer_border_pts_next_right[pt_ind])
                                        pts.append(layer_border_pts_next_right[pt_ind+1])
                                #print(pts)
                                #pts_next = [layer_border_pts_left[pt_ind+1],layer_border_pts_left[pt_ind+1]]
                                surf = Surface(pts=pts).create_delunay_surface_2d(return_hull=True,alpha=0,offset=100)
                                layer_surf.append(surf)

                vertices = []
                for vert_ind in range(layer_surf.surface.GetNumberOfPoints()):
                    vertices.append(layer_surf.surface.GetPoints().GetPoint(vert_ind))
                
                # write mean, upper or lower layer border surface
                if layer_depth == layer_depth_mean:
                    Landmarks(pts = vertices).write_landmarks( output_path_rabies_layers+ exp +'_vM1_'+layer[:-4]+'_{}_mean.landmarksAscii'.format(surf_res))
                    layer_surf.write_surface_mesh(output_path_rabies_layers+exp +'_vM1_'+layer[:-4]+'_{}_mean.vtk'.format(surf_res))
                elif layer_depth == layer_depth_upper:
                    Landmarks(pts = vertices).write_landmarks( output_path_rabies_layers+ exp +'_vM1_'+layer[:-4]+'_{}_upper.landmarksAscii'.format(surf_res))
                    layer_surf.write_surface_mesh(output_path_rabies_layers+exp +'_vM1_'+layer[:-4]+'_{}_upper.vtk'.format(surf_res))
                else:
                    Landmarks(pts = vertices).write_landmarks( output_path_rabies_layers+ exp +'_vM1_'+layer[:-4]+'_{}_lower.landmarksAscii'.format(surf_res))
                    layer_surf.write_surface_mesh(output_path_rabies_layers+exp +'_vM1_'+layer[:-4]+'_{}_lower.vtk'.format(surf_res))
            
           

# Create Rabies Borders vS1

## Approach without using cylinders

In [37]:
ref_col_depth = 2150
density_peak_3d = 121.237 # (in 1000 / cubic mm)
res = 50
barrel_radius = 250

df_2d_3d_scales = pd.DataFrame()
df_unstretched = pd.DataFrame()
df_unstretched['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
df_unstretched_scaled = pd.DataFrame()
df_unstretched_scaled['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
df_stretched = pd.DataFrame()
df_stretched['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
df_scaled = pd.DataFrame()
df_scaled['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
for exp in ['MG49_lhs','MG50_lhs','MG50_rhs']:#exp_names_tangential:
    print(exp)
    df_exp_2d = pd.DataFrame() 
    profile = []
    
    rabies_landmarks = Landmarks(output_path_landmarks+'Density_Clusters/'+exp+'_vS1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii')
    axis_field = AmiraSpatialGraph(output_path_vS1_axis_field+exp+'_vS1_axis_field_surf_res_50_voxel_size_500_cutoff_0.1.am',generic_graph=True)
    axes_lens,l5_upper_depths,l5_lower_depths,l5_avg_depths,pts_on_axes = Vectors().get_landmarks_depth_along_axis_field\
                                                                (axis_field.graph_data.edge_list,rabies_landmarks.pts,\
                                                                 radius=250)
    Landmarks(pts=pts_on_axes).write_landmarks(output_path_rabies_profiles_v2+exp+'_axes_mapped_rabies_landmarks_vS1.landamrksAscii')
    
    # move from caudal to rostral vm1
    col_landmarks = Landmarks()
    col_cylinders = Surface()
    for row in range(-10,10):
        df_unstreched_main = pd.DataFrame()
        df_streched_main = pd.DataFrame()
        df_unstreched_main['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
        df_streched_main['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
        row_present = False
        
        for col in range(-10,10):
            print(row,col)
            if os.path.exists(output_path_vS1_axis_field+exp+'_vS1_axis_field_row_{}_{}_50_500_0.1.am'.format(row,col)):
                print(output_path_vS1_axis_field+exp+'_vS1_axis_field_row_{}_{}_50_500_0.1.am'.format(row,col))
            
                sg = AmiraSpatialGraph(output_path_vS1_axis_field+exp+'_vS1_axis_field_row_{}_{}_50_500_0.1.am'.\
                                       format(row,col),generic_graph=True)
                
                edge = sg.graph_data.edge_list[0]
                profile = get_cell_density_profile_from_axis_projected_landmarks(edge,pts_on_axes)
                profile_interpolated = interpolate_array(profile,round(ref_col_depth/res))

                df_profile_unstreched = pd.DataFrame()
                df_profile_unstreched['NeuN_Density_{}_{}'.format(exp,col)] = profile # in 1000 / cubic mm

                df_profile_streched = pd.DataFrame()
                df_profile_streched['NeuN_Density_{}_{}'.format(exp,col)] = profile_interpolated # in 1000 / cubic mm

                df_unstreched_main = pd.concat([df_unstreched_main,df_profile_unstreched],ignore_index=False,axis=1)
                df_streched_main = pd.concat([df_streched_main,df_profile_streched],ignore_index=False,axis=1)
                row_present = True
                #col_landmarks.append_landmarks(profile_landmarks)
                #col_cylinders.append(profile_cylinder)
                
        if row_present:
            df_unstreched_main.to_csv(output_path_rabies_profiles_v2+'_vS1_NeuN_Depth_Profile_Unstreched_using_{}_row_{}.csv'.format(exp,row))
            df_streched_main.to_csv(output_path_rabies_profiles_v2+'_vS1_NeuN_Depth_Profile_Streched_using_{}_row_{}.csv'.format(exp,row))
        
    #col_landmarks.write_landmarks(output_path_rabies_profiles+exp+'_vM1_NeuN_Column_Landmarks.landmarksAscii')
    #col_cylinders.write_surface_mesh(output_path_rabies_profiles+exp+'_vM1_NeuN_Column_Cylinders.vtk')

MG49_lhs
-10 -10
-10 -9
-10 -8
-10 -7
-10 -6
-10 -5
-10 -4
-10 -3
-10 -2
-10 -1
-10 0
-10 1
-10 2
-10 3
-10 4
-10 5
-10 6
-10 7
-10 8
-10 9
-9 -10
-9 -9
-9 -8
-9 -7
-9 -6
-9 -5
-9 -4
-9 -3
-9 -2
-9 -1
-9 0
-9 1
-9 2
-9 3
-9 4
-9 5
-9 6
-9 7
-9 8
-9 9
-8 -10
-8 -9
-8 -8
-8 -7
-8 -6
-8 -5
-8 -4
-8 -3
-8 -2
-8 -1
-8 0
-8 1
-8 2
-8 3
-8 4
-8 5
-8 6
-8 7
-8 8
-8 9
-7 -10
-7 -9
-7 -8
-7 -7
-7 -6
-7 -5
-7 -4
-7 -3
-7 -2
-7 -1
-7 0
-7 1
-7 2
-7 3
-7 4
-7 5
-7 6
-7 7
-7 8
-7 9
-6 -10
-6 -9
-6 -8
-6 -7
-6 -6
-6 -5
-6 -4
-6 -3
-6 -2
-6 -1
-6 0
-6 1
-6 2
-6 3
-6 4
-6 5
-6 6
-6 7
-6 8
-6 9
-5 -10
-5 -9
-5 -8
-5 -7
-5 -6
-5 -5
-5 -4
-5 -3
-5 -2
-5 -1
-5 0
-5 1
-5 2
-5 3
-5 4
-5 5
-5 6
-5 7
-5 8
-5 9
-4 -10
-4 -9
-4 -8
-4 -7
-4 -6
-4 -5
-4 -4
-4 -3
-4 -2
-4 -1
-4 0
-4 1
-4 2
-4 3
-4 4
-4 5
-4 6
-4 7
-4 8
-4 9
-3 -10
-3 -9
-3 -8
-3 -7
-3 -6
-3 -5
-3 -4
-3 -3
-3 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG49_lhs_vS1_axis_field_row_-3_-2_50_500_0.1

column volume in 314159265.3589794 and num of neurons 3
3 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG49_lhs_vS1_axis_field_row_3_0_50_500_0.1.am
column volume in 323976742.40144753 and num of neurons 0
3 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG49_lhs_vS1_axis_field_row_3_1_50_500_0.1.am
column volume in 343611696.4863838 and num of neurons 1
3 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG49_lhs_vS1_axis_field_row_3_2_50_500_0.1.am
column volume in 343611696.4863838 and num of neurons 0
3 3
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG49_lhs_vS1_axis_field_row_3_3_50_500_0.1.am
column volume in 333794219.44391567 and num of neurons 2
3 4
3 5
3 6
3 7
3 8
3 9
4 -10
4 -9
4 -8
4 -7
4 -6
4 -5
4 -4
4 -3
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref

column volume in 294524311.27404314 and num of neurons 10
-2 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG50_lhs_vS1_axis_field_row_-2_-2_50_500_0.1.am
column volume in 314159265.3589794 and num of neurons 9
-2 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG50_lhs_vS1_axis_field_row_-2_-1_50_500_0.1.am
column volume in 314159265.3589794 and num of neurons 11
-2 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG50_lhs_vS1_axis_field_row_-2_0_50_500_0.1.am
column volume in 314159265.3589794 and num of neurons 4
-2 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG50_lhs_vS1_axis_field_row_-2_1_50_500_0.1.am
column volume in 333794219.44391567 and num of neurons 2
-2 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG50_lhs_vS1_axis_field_

column volume in 363246650.57132006 and num of neurons 6
4 2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG50_lhs_vS1_axis_field_row_4_2_50_500_0.1.am
column volume in 363246650.57132006 and num of neurons 2
4 3
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG50_lhs_vS1_axis_field_row_4_3_50_500_0.1.am
column volume in 373064127.6137882 and num of neurons 0
4 4
4 5
4 6
4 7
4 8
4 9
5 -10
5 -9
5 -8
5 -7
5 -6
5 -5
5 -4
5 -3
5 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG50_lhs_vS1_axis_field_row_5_-2_50_500_0.1.am
column volume in 265071880.14663875 and num of neurons 1
5 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG50_lhs_vS1_axis_field_row_5_-1_50_500_0.1.am
column volume in 284706834.231575 and num of neurons 1
5 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs/

column volume in 363246650.57132006 and num of neurons 0
-2 4
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG50_rhs_vS1_axis_field_row_-2_4_50_500_0.1.am
column volume in 353429173.5288519 and num of neurons 0
-2 5
-2 6
-2 7
-2 8
-2 9
-1 -10
-1 -9
-1 -8
-1 -7
-1 -6
-1 -5
-1 -4
-1 -3
-1 -2
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG50_rhs_vS1_axis_field_row_-1_-2_50_500_0.1.am
column volume in 284706834.231575 and num of neurons 0
-1 -1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG50_rhs_vS1_axis_field_row_-1_-1_50_500_0.1.am
column volume in 304341788.3165113 and num of neurons 1
-1 0
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs//vS1_Ref_Surfaces_Axis_Fields/MG50_rhs_vS1_axis_field_row_-1_0_50_500_0.1.am
column volume in 323976742.40144753 and num of neurons 3
-1 1
/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_R

## vS1 Depth Profiles

In [ ]:
# reference column depth is 2150 um 
ref_col_depth = 2150
density_peak_3d = 121.237 # (in 1000 / cubic mm)
res = 50
barrel_radius = 250

df_2d_3d_scales = pd.DataFrame()
df_unstretched = pd.DataFrame()
df_unstretched['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
df_unstretched_scaled = pd.DataFrame()
df_unstretched_scaled['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
df_stretched = pd.DataFrame()
df_stretched['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
df_scaled = pd.DataFrame()
df_scaled['Depth_in_um'] = np.array(range(1,round(ref_col_depth / res)+1))*-res
for exp in ['MG49_lhs','MG50_lhs']:#exp_names_tangential:
    print(exp)
    df_exp_2d = pd.DataFrame() 
    profile = []
    rabies_landmarks_vS1 = Landmarks(output_path_landmarks+'Density_Clusters/'+exp+'_vS1_rabies_landmarks_voxel_500_cutoff_0.05.LandmarkAscii')
    
    # speed up by considering only neun within the barres
#     S1_hull = Surface(output_path_surfaces+exp+'_s1_hull_50.vtk').surface
#     s1_neun_landmarks = neun_landmarks.get_landmarks_within_given_surface(S1_hull)
#     Landmarks(pts=s1_neun_landmarks).write_landmarks(output_path_neun_profiles+'vS1_NeuN_Landmarks_{}.landmarksAscii'.format(exp))
    sg = AmiraSpatialGraph(output_path_surfaces+exp+'_barrels_50.am',barrel_projections_present=True,reverse_barrel_direction=True)
    
    # get profiles for C2,C3,D2,D3
    for barrel_row in sg.barrels.rows_list:
        for barrel in barrel_row.single_barrels_list:
            #row = sg.get_barrel_row(row_label=row_label)
            #barrel = row.single_barrels_list[barrel_ind+1]
            
            barrel_name = barrel.barrel_name
            #print(barrel_name)
            
            #if barrel_name == 'C2' or barrel_name == 'C3' or barrel_name == 'D2' or barrel_name == 'D3':
            print(barrel_name)
            if len(barrel.pia_projection_barrel_centroid) > 0 and len(barrel.wm_projection_barrel_centroid)>0:
                edge = [barrel.pia_projection_barrel_centroid,barrel.wm_projection_barrel_centroid]

#                     sg = AmiraSpatialGraph(generic_graph=True)
#                     sg.graph_data.add_edge([edge[0],edge[1]])
#                     sg.write_spatial_graph(output_path_neun_profiles+'dummy.am')

#                     break

                #profile = get_cell_count_profile(edge,res,barrel_radius,s1_neun_landmarks)
                profile,profile_cylinder,profile_landmarks = get_cell_density_profile(edge,res,barrel_radius,rabies_landmarks_vS1.pts,exp,barrel.barrel_name)
                #print(len(profile))

                df_profile_unstretched = pd.DataFrame()
                df_profile_unstretched['NeuN_Density_{}_{}'.format(exp,barrel_name)] = profile
                df_profile_unstretched.to_csv(output_path_rabies_profiles+'vS1_NeuN_Depth_Profile_{}_{}.csv'.format(exp,barrel.barrel_name))
                # append nan to the rest of the profile
                if len(profile) < round(ref_col_depth / res):
                    new_prof = []
                    for i in range(len(profile)):
                        new_prof.append(profile)
                    for i in range(round(ref_col_depth / res)-len(profile)):
                        new_prof.append(np.nan)
                    df_unstretched['NeuN_Density_{}_{}'.format(exp,barrel_name)] = new_prof
                else:
                    df_unstretched['NeuN_Density_{}_{}'.format(exp,barrel_name)] = profile[0:43]


                #print(len(profile))
                profile_streched = interpolate_array(profile,round(ref_col_depth/res))
                #print((profile_streched))
                df_exp_2d = pd.DataFrame()
                df_exp_2d['NeuN_Density_{}_{}'.format(exp,barrel_name)] = profile_streched
                df_exp_2d.to_csv(output_path_rabies_profiles+'vS1_NeuN_Depth_Profile_{}_{}_stretched.csv'.format(exp,barrel.barrel_name))

                df_stretched = pd.concat([df_stretched,df_exp_2d],ignore_index=False,axis=1)
                # stretch or clip it to match the reference column depth if needed
                #if abs(barrel.column_height) < ref_col_depth:

                #else:
                    # need to shrink the column
                #    profile_3d = profile[0:round(ref_col_depth/res)]

                # scale from 2d to 3D
                max_density = max(profile_streched)
                scale = density_peak_3d / max_density
                profile_3d = np.array(profile_streched)*scale 

                df_scales = pd.DataFrame()
                df_scales['NeuN_Scale_{}_{}'.format(exp,barrel_name)] = [scale]
                df_2d_3d_scales = pd.concat([df_2d_3d_scales,df_scales],ignore_index=False,axis=1)

                df_profile = pd.DataFrame()
                df_profile['NeuN_Density_{}_{}'.format(exp,barrel_name)] = profile_3d # in 1000 / cubic mm
                df_profile.to_csv(output_path_rabies_profiles+'vS1_NeuN_Depth_Profile_{}_{}_stretched_scaled.csv'.format(exp,barrel.barrel_name))

                df_scaled = pd.concat([df_scaled,df_profile],ignore_index=False,axis=1)
                    #print(df)

    df_exp_2d.to_csv(output_path_rabies_profiles+'vS1_NeuN_Depth_Profile_{}.csv'.format(exp))
    
    
    
df_unstretched.to_csv(output_path_rabies_profiles+'vS1_NeuN_Depth_Profile_Unstretched.csv')
df_stretched.to_csv(output_path_rabies_profiles+'vS1_NeuN_Depth_Profile_Stretched.csv')
df_scaled.to_csv(output_path_rabies_profiles+'vS1_NeuN_Depth_Profile_Scaled.csv')

df_2d_3d_scales.to_csv(output_path_rabies_profiles+'vS1_NeuN_2D_To_3D_Scaling_Factors.csv')

## Derive Rabies Border

In [ ]:
depth_max = 2150
layer_borders_df = pd.DataFrame(columns=['Exp_Name','L5_Upper_Border','L5_Mean_Border','L5_Lower_Border',\
                                        'L5_Upper_Border_Per','L5_Mean_Border_Per','L5_Lower_Border_Per'])
# rowwise profiles going from caudal to rostral
exp = 'MG50_lhs'
#%matplotlib notebook
#figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')
#p = plt.subplot(psize=[10,10])

lhs_exp_means = []
lhs_exp_stds = []
rhs_exp_means = []
rhs_exp_stds = []
global_exp_means = []
global_exp_stds = []

thresh = 0.1# 90% of peak density

df = pd.read_csv(output_path_rabies_profiles+'vS1_NeuN_Depth_Profile_Stretched.csv')

for exp in ['MG49_lhs','MG50_lhs']:#exp_names:
    layer_borders_exp_df = pd.DataFrame(columns=['Exp_Name','L5_Upper_Border','L5_Mean_Border','L5_Lower_Border',\
                                        'L5_Upper_Border_Per','L5_Mean_Border_Per','L5_Lower_Border_Per'])
    exp_rows = []
    exp_profiles = []
    for barrel in ['C2','C3','D2','D3']:

        rows = []
        #for j in range(2,len(df.columns)):
        profile_correct_direction = abs(np.array(df['NeuN_Density_{}_{}'.format(exp,barrel)])) * 100
        #print(profile_correct_direction)
        if np.array(profile_correct_direction).mean(axis=0) > 0:
            rows.append(profile_correct_direction)
            exp_profiles.append(profile_correct_direction)
            
        
    global_exp_means.append(np.array(exp_profiles).mean(axis=0))
    global_exp_stds.append(np.array(exp_profiles).std(axis=0))
        
    if exp.find('_lhs')>0:
        lhs_exp_means.append(np.array(exp_profiles).mean(axis=0))
        lhs_exp_stds.append(np.array(exp_profiles).std(axis=0))
    else:
        rhs_exp_means.append(np.array(exp_profiles).mean(axis=0))
        rhs_exp_stds.append(np.array(exp_profiles).std(axis=0))
            #print(rows)
            
            #if len(rows)>0:
            #    x = np.array(rows).mean(axis=0)
            #    exp_rows.append(x)
                #print(x)
                #plt.plot(x,df['Depth_in_um'])
    
    y = df['Depth_in_um']
    x = np.array(exp_profiles).mean(axis=0)
    error = np.array(exp_profiles).std(axis=0)
#     # plot individual exps
#     plt.plot(x,y)
#     #plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
#     plt.savefig(layers_rabies_path+exp+'_vS1_Avg_Depth_Profile.eps',format='eps')
#     plt.cla()
    
    
    # Create layers for only mean exp profile
    thresh = x.max()*0.1
    max_ind = x.argmax()
    L5_Mean = x.argmax()*-50
    L5_Upper = np.where(x[0:max_ind]>thresh)[0][0]*-50
    L5_Lower = np.where(x[max_ind:len(x)]<thresh)[0][0]*-50 + \
                    (max_ind*-50)
    df_tmp = pd.DataFrame(columns=layer_borders_df.columns,data=[[exp,L5_Upper,L5_Mean,L5_Lower,\
                                                                    abs(L5_Upper/depth_max),abs(L5_Mean/depth_max),\
                                                                     abs(L5_Lower/depth_max),] ])
    layer_borders_exp_df = layer_borders_exp_df.append(df_tmp)
    layer_borders_df = layer_borders_df.append(df_tmp)
    
    layer_borders_exp_df.to_csv(output_path_rabies_layers+exp+'_vS1_Borders.csv')
layer_borders_df.to_csv(output_path_rabies_layers+'vS1_Borders.csv')

# # Plot and save the average plots
# y = df['Depth_in_um']
# x = np.array(lhs_exp_means).mean(axis=0)
# error = np.array(lhs_exp_means).std(axis=0)
# plt.plot(x,y)
# plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
# plt.savefig(layers_rabies_path+'vS1_Avg_Depth_Profile_LHS.eps',format='eps')
# plt.cla()

# # Plot and save the average plots
# y = df['Depth_in_um']
# x = np.array(global_exp_means).mean(axis=0)
# error = np.array(global_exp_means).std(axis=0)
# plt.plot(x,y)
# plt.fill_betweenx(y, x-error, x+error,alpha=0.25)
# plt.savefig(layers_rabies_path+'vS1_Avg_Depth_Profile.eps',format='eps')
# plt.cla()

## Create vS1 Rabies Layer Surfaces

In [ ]:
lb = pd.read_csv(output_path_rabies_layers+'vS1_Borders.csv')
for exp in ['MG49_lhs','MG50_lhs',]:
    lb = pd.read_csv(output_path_rabies_layers+'vS1_Borders.csv')
    for surf_res in surface_resolutions:
        
        vectors_sg = AmiraSpatialGraph(output_path_vS1_axis_field+exp+'_vS1_axis_field_{}.am'.format(surf_res),generic_graph=True)
        
        for layer in ['L5_Upper_Border_Per','L5_Mean_Border_Per','L5_Lower_Border_Per',]:
            layer_depth_mean = lb[lb['Exp_Name']==exp][layer].mean() * 100
            layer_depth_upper = layer_depth_mean - (lb[lb['Exp_Name']==exp][layer].std() * 100)
            layer_depth_lower = layer_depth_mean + (lb[lb['Exp_Name']==exp][layer].std() * 100)
            #print(layer_depth)
            for layer_depth in [layer_depth_mean,layer_depth_upper,layer_depth_lower]:
                layer_border_pts = Vectors().get_layer_pts_for_given_depth(vectors_sg.graph_data.edge_list,layer_depth,\
                                                                                              realtive_depths=True,invert_axes=True,rel_depth_given=True)
                layer_surf = Surface(pts=layer_border_pts).create_delunay_surface_2d(return_hull=True)
                # write mean, upper or lower layer border surface
                if layer_depth == layer_depth_mean:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_path_rabies_layers+ exp +'_vS1_'+layer[:-4]+'_{}_mean.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_path_rabies_layers+exp +'_vS1_'+layer[:-4]+'_{}_mean.vtk'.format(surf_res))
                elif layer_depth == layer_depth_upper:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_path_rabies_layers+ exp +'_vS1_'+layer[:-4]+'_{}_upper.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_path_rabies_layers+exp +'_vS1_'+layer[:-4]+'_{}_upper.vtk'.format(surf_res))
                else:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_path_rabies_layers+ exp +'_vS1_'+layer[:-4]+'_{}_lower.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_path_rabies_layers+exp +'_vS1_'+layer[:-4]+'_{}_lower.vtk'.format(surf_res))


# Create psuedo coronal 

In [ ]:
output_path = output_path_alignment + 'Semi_Coronal' + '/'
pathlib.Path(output_path).mkdir(exist_ok=True)
#for exp in exp_names_tangential:
aligner = Alignment()    
aligner.transform_folders_into_semicoronal(output_root,output_path)
aligner.transform_folders_into_semicoronal(output_root+'/Landmarks/',output_path+'/Landmarks/')

In [ ]:
output_path = output_path_alignment + 'RHS_Mirrored' + '/'
pathlib.Path(output_path).mkdir(exist_ok=True)
#for exp in exp_names_tangential:
aligner = Alignment()    
aligner.transform_rhs_into_lhs(output_path_alignment + 'Semi_Coronal' + '/',output_path)
aligner.transform_rhs_into_lhs(output_path_alignment + 'Semi_Coronal' + '/'+'/Landmarks/',output_path+'/Landmarks/')

# Selection of Alignment methods

In [ ]:
# Methods: Center, PCA, Surface
# 1) Surface resolution
# 2) Global: Rabies Centers, vS1 and vM1 PCAs, Surfaces, Hemisphere Alignment
# 3) vS1, vM1: Rabies Centers, vS1 and vM1 PCAs, Surfaces, Hemisphere Alignment 

## Surface Res Selection

In [ ]:
# Align surfaces using different surf 
# Centered and pca aligned
for surf_res in [50]:#surface_resolutions:
    
    output_path_root = output_path_alignment_surf_res+'/Surface_Res_{}/'.format(surf_res)
    pathlib.Path(output_path_root).mkdir(exist_ok=True)
    
    
    for hem in ['rhs']:
        ref_surf_pia = Surface(output_path_surfaces+'MG48_{}_pia_open_bottom_voxel_size_{}.vtk'.format(hem,surf_res))
        ref_surf_wm = Surface(output_path_surfaces+'MG48_{}_WM_open_bottom_voxel_size_{}.vtk'.format(hem,surf_res))
        ref_surf = Surface()
        ref_surf.append(ref_surf_pia.surface)
        ref_surf.append(ref_surf_wm.surface)
        
        for alignment_type in ['PCA_Aligned',]:
            output_path = output_path_root + alignment_type+'/'
            pathlib.Path(output_path).mkdir(exist_ok=True)
            for exp in ['MG48',]:
                pia = Surface(output_path_surfaces + '{}_{}_pia_open_bottom_voxel_size_{}.vtk'.format(exp,hem,surf_res))
                wm = Surface(output_path_surfaces + '{}_{}_WM_open_bottom_voxel_size_{}.vtk'.format(exp,hem,surf_res))
                surf = Surface()
                surf.append(pia.surface)
                surf.append(wm.surface)
                align_by_pca = False
                nr_iterations = 10000
                if hem=='lhs':
                    lhs=True
                else:
                    lhs=False
                if alignment_type == 'Centered':
                    nr_iterations = 0
                elif alignment_type == 'PCA_Aligned':
                    align_by_pca = True
                
                    
                if exp == 'MG48':
                    aligner = Alignment(ref_surf.surface,ref_surf.surface,align_by_pca=False,nr_iterations=0,lhs=lhs)
                    txmat = aligner.get_transformation_matrix()
                    icp = aligner.get_icp_transform()
                else:
                    aligner = Alignment(ref_surf.surface,surf.surface,align_by_pca=align_by_pca,nr_iterations=nr_iterations,lhs=lhs)
                    txmat = aligner.get_transformation_matrix()
                    icp = aligner.get_icp_transform()
                #print(txmat)
                aligner.transform_folders(output_root,output_path,exp,icp=icp,txmat = txmat,hem_specific_alignment=True,)
                aligner.transform_folders(output_root + '/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True)

                
            
        

In [ ]:
for surf_res in [50]:#surface_resolutions:
    input_path_root = output_path_alignment_surf_res+'/Surface_Res_{}/PCA_Aligned/'.format(surf_res)
    for hem in ['rhs']:
        input_surf_path = input_path_root + '/Surfaces/'
        ref_surf_pia = Surface(input_surf_path+'MG48_{}_pia_open_bottom_voxel_size_{}.vtk'.format(hem,surf_res))
        ref_surf_wm = Surface(input_surf_path+'MG48_{}_WM_open_bottom_voxel_size_{}.vtk'.format(hem,surf_res))
        ref_surf = Surface()
        ref_surf.append(ref_surf_pia.surface)
        ref_surf.append(ref_surf_wm.surface)
        
        for alignment_type in ['Surface_Aligned',]:
            output_path = output_path_alignment_surf_res+'/Surface_Res_{}/{}/'.format(surf_res,alignment_type)
            pathlib.Path(output_path).mkdir(exist_ok=True)
            for exp in ['MG49','MG50',]:
                pia = Surface(input_surf_path + '{}_{}_pia_open_bottom_voxel_size_{}.vtk'.format(exp,hem,surf_res))
                wm = Surface(input_surf_path + '{}_{}_WM_open_bottom_voxel_size_{}.vtk'.format(exp,hem,surf_res))
                surf = Surface()
                surf.append(pia.surface)
                surf.append(wm.surface)
                align_by_pca = False
                nr_iterations = 10000
                if hem=='lhs':
                    lhs=True
                else:
                    lhs=False
                if alignment_type == 'Centered':
                    nr_iterations = 0
                elif alignment_type == 'PCA_Aligned':
                    align_by_pca = True
                
                if exp == 'MG48':
                    aligner = Alignment(ref_surf.surface,ref_surf.surface,align_by_pca=align_by_pca,nr_iterations=nr_iterations,lhs=lhs)
                    txmat = aligner.get_transformation_matrix()
                    icp = aligner.get_icp_transform()
                else:
                    aligner = Alignment(ref_surf.surface,surf.surface,align_by_pca=align_by_pca,nr_iterations=nr_iterations,lhs=lhs)
                    txmat = aligner.get_transformation_matrix()
                    icp = aligner.get_icp_transform()
                #print(txmat)
                  
                aligner.transform_folders(input_path_root,output_path,exp,icp=icp,txmat = txmat,hem_specific_alignment=True,)
                aligner.transform_folders(input_path_root + '/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True)

                
            

In [ ]:
# pia only
for surf_res in [50]:#surface_resolutions:
    input_path_root = output_path_alignment_surf_res+'/Surface_Res_{}/PCA_Aligned/'.format(surf_res)
    for hem in ['lhs','rhs']:
        input_surf_path = input_path_root + '/Surfaces/'
        ref_surf_pia = Surface(input_surf_path+'MG48_{}_pia_open_bottom_voxel_size_{}.vtk'.format(hem,surf_res))
        #ref_surf_wm = Surface(input_surf_path+'MG48_{}_WM_open_bottom_voxel_size_{}.vtk'.format(hem,surf_res))
        ref_surf = Surface()
        ref_surf.append(ref_surf_pia.surface)
        #ref_surf.append(ref_surf_wm.surface)
        
        for alignment_type in ['Surface_Aligned_Pia_Only',]:
            output_path = output_path_alignment_surf_res+'/Surface_Res_{}/{}/'.format(surf_res,alignment_type)
            pathlib.Path(output_path).mkdir(exist_ok=True)
            for exp in ['MG48','MG49','MG50',]:
                pia = Surface(input_surf_path + '{}_{}_pia_open_bottom_voxel_size_{}.vtk'.format(exp,hem,surf_res))
                #wm = Surface(input_surf_path + '{}_{}_WM_open_bottom_voxel_size_{}.vtk'.format(exp,hem,surf_res))
                surf = Surface()
                surf.append(pia.surface)
                #surf.append(wm.surface)
                align_by_pca = False
                nr_iterations = 10000
                if hem=='lhs':
                    lhs=True
                else:
                    lhs=False
                if alignment_type == 'Centered':
                    nr_iterations = 0
                elif alignment_type == 'PCA_Aligned':
                    align_by_pca = True
                
                if exp == 'MG48':
                    aligner = Alignment(ref_surf.surface,ref_surf.surface,align_by_pca=align_by_pca,nr_iterations=nr_iterations,lhs=lhs)
                    txmat = aligner.get_transformation_matrix()
                    icp = aligner.get_icp_transform()
                else:
                    aligner = Alignment(ref_surf.surface,surf.surface,align_by_pca=align_by_pca,nr_iterations=nr_iterations,lhs=lhs)
                    txmat = aligner.get_transformation_matrix()
                    icp = aligner.get_icp_transform()
                #print(txmat)
                  
                aligner.transform_folders(input_path_root,output_path,exp,icp=icp,txmat = txmat,hem_specific_alignment=True,)
                aligner.transform_folders(input_path_root + '/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True)

                
            

## Evaluate Best Surface Resolution for Surface Alignment

In [ ]:
# Per ray that is cast, we can compute the avg and std with intersection pts at pia, wm and center
df_main = pd.DataFrame()
for aligned_using_surf_res in surface_resolutions:
    for alignment_type in ['Surface_Aligned_Pia_Only']:#[]
        input_path_root = output_path_alignment_surf_res+'/Surface_Res_{}/{}/'.format(aligned_using_surf_res,alignment_type)
        for surf_res in surface_resolutions:
            # get aligned surfaces list
            centroid,centroid_lhs,centroid_rhs,aligned_pias_list_lhs,aligned_wms_list_lhs,aligned_pias_list_rhs,aligned_wms_list_rhs = \
                Surface().get_aligned_surfaces_list(input_path_root,surf_res)
            
            # get obb tree for surf lists
            obb_pias_lhs = Surface().get_obb_tree_list(aligned_pias_list_lhs)
            obb_pias_rhs = Surface().get_obb_tree_list(aligned_pias_list_rhs)
            obb_wms_lhs = Surface().get_obb_tree_list(aligned_wms_list_lhs)
            obb_wms_rhs = Surface().get_obb_tree_list(aligned_wms_list_rhs)
            
            # get rays from surfaces center
            #rays_sg = Surface().get_rays(centroid,theta_res=33,phi_res=32)
            rays_sg,sphere = Surface().get_rays(centroid,theta_res=33,phi_res=32,ray_core_radius=1,tesselation=True,return_sphere=True)
            rays_sg_lhs,sphere = Surface().get_rays(centroid_lhs,theta_res=33,phi_res=32,ray_core_radius=1,tesselation=True,return_sphere=True)
            rays_sg_rhs,sphere = Surface().get_rays(centroid_rhs,theta_res=33,phi_res=32,ray_core_radius=1,tesselation=True,return_sphere=True)
            
            rays_sg.write_spatial_graph(input_path_root+'/Average/Rays_{}.am'.format(surf_res))
            rays_sg_lhs.write_spatial_graph(input_path_root+'/Average/Rays_{}_lhs.am'.format(surf_res))
            rays_sg_rhs.write_spatial_graph(input_path_root+'/Average/Rays_{}_rhs.am'.format(surf_res))
            Surface(polydata=sphere).write_surface_mesh(input_path_root+'/Average/Sphere_{}.vtk'.format(surf_res))
            
            # get avg intersection pts for rays cast
            #for hem in ['lhs','rhs']:
            avg_pts_pia_lhs,avg_pts_wm_lhs,wm_pia_rays_sg_lhs,ellisoids_lhs,df_lhs = Surface().get_avg_pts\
                (rays_sg_lhs,obb_pias_lhs,obb_wms_lhs,alignment_type=alignment_type,hem='LHS',\
                 aligned_surf_res=aligned_using_surf_res,surf_res=surf_res,create_error_ellisoids=False)
            #ellisoids_lhs.write_surface_mesh(input_path_root+'/Average/Avg_Error_Ellipsoids_LHS_{}.vtk'.format(surf_res))
            
            df_main = df_main.append(df_lhs)
            avg_pts_pia_rhs,avg_pts_wm_rhs,wm_pia_rays_sg_rhs,ellisoids_rhs,df_rhs = Surface().get_avg_pts\
                (rays_sg_rhs,obb_pias_rhs,obb_wms_rhs,alignment_type=alignment_type,hem='RHS'\
                 ,aligned_surf_res=aligned_using_surf_res,surf_res=surf_res,create_error_ellisoids=False)
            #ellisoids_rhs.write_surface_mesh(input_path_root+'/Average/Avg_Error_Ellipsoids_RHS_{}.vtk'.format(surf_res))
            df_main = df_main.append(df_rhs)
            # write output avg rays
            wm_pia_rays_sg_lhs.write_spatial_graph(input_path_root+'/Average/Avg_Rays_LHS_{}.am'.format(surf_res))
            wm_pia_rays_sg_rhs.write_spatial_graph(input_path_root+'/Average/Avg_Rays_RHS_{}.am'.format(surf_res))
            

df_main.to_csv(input_path_root+'/Average/Stats_{}.csv'.format(aligned_using_surf_res))            

In [ ]:
# combine the surf res eval results
output_path = output_path_alignment_surf_res
df_final = pd.DataFrame()
for aligned_using_surf_res in surface_resolutions:
    for alignment_type in ['Surface_Aligned_Pia_Only']:#[]
        input_path_root = output_path_alignment_surf_res+'/Surface_Res_{}/{}/'.format(aligned_using_surf_res,alignment_type)
        df = pd.read_csv(input_path_root+'/Average/Stats_{}.csv'.format(aligned_using_surf_res))
        df_final = df_final.append(df)
df_final.to_csv(output_path+'Alignmen_Stats.csv')

In [ ]:
# which resolution should be used for alignment
df_selection_summary = pd.DataFrame(columns=['Resolution_Used','Pia_Mean_Dist_3D','Pia_STD_Dist_3D','WM_Mean_Dist_3D','WM_STD_Dist_3D',\
                                             'Pia_WM_Mean_Dist_3D','Pia_WM_STD_Dist_3D'])
for res in surface_resolutions:
    pia_mean = df_final[df_final['Aligned_Using_Surf_Resolution']==res]['Pia_Mean_Dist_3D'].mean()
    pia_std = df_final[df_final['Aligned_Using_Surf_Resolution']==res]['Pia_STD_Dist_3D'].mean()
    wm_mean = df_final[df_final['Aligned_Using_Surf_Resolution']==res]['WM_Mean_Dist_3D'].mean()
    wm_std = df_final[df_final['Aligned_Using_Surf_Resolution']==res]['WM_STD_Dist_3D'].mean()
    pia_wm_mean = (pia_mean+wm_mean)/2
    pia_wm_std = (pia_std+wm_std)/2
    df_selection_summary = df_selection_summary.append(pd.DataFrame(columns=df_selection_summary.columns,\
                                                                    data = [[res,pia_mean,pia_std,wm_mean,wm_std,\
                                                                            pia_wm_mean,pia_wm_std]]))
df_selection_summary.to_csv(output_path_alignment_surf_res+'Resolution_Selection_Stats.csv')    

In [ ]:
df_selection_summary# 200 is best res to use for alignment

In [ ]:
# which resolution surf gets aligned the best
df_selection_summary_2 = pd.DataFrame(columns=['Resolution_Used','Pia_Mean_Dist_3D','Pia_STD_Dist_3D','WM_Mean_Dist_3D','WM_STD_Dist_3D',\
                                             'Pia_WM_Mean_Dist_3D','Pia_WM_STD_Dist_3D'])
for res in surface_resolutions:
    pia_mean = df_final[df_final['Present_Surf_Resolution']==res]['Pia_Mean_Dist_3D'].mean()
    pia_std = df_final[df_final['Present_Surf_Resolution']==res]['Pia_STD_Dist_3D'].mean()
    wm_mean = df_final[df_final['Present_Surf_Resolution']==res]['WM_Mean_Dist_3D'].mean()
    wm_std = df_final[df_final['Present_Surf_Resolution']==res]['WM_STD_Dist_3D'].mean()
    pia_wm_mean = (pia_mean+wm_mean)/2
    pia_wm_std = (pia_std+wm_std)/2
    df_selection_summary_2 = df_selection_summary_2.append(pd.DataFrame(columns=df_selection_summary.columns,\
                                                                    data = [[res,pia_mean,pia_std,wm_mean,wm_std,\
                                                                            pia_wm_mean,pia_wm_std]]))
df_selection_summary_2.to_csv(output_path_alignment_surf_res+'Resolution_Selection_Stats_2.csv')    

In [ ]:
df_selection_summary_2 # error decreases continously with smoothness

## Global Alignment

In [ ]:
# Align based on : whole brain: center , pca, surface
# Align based on : Local vs1,vm1: center , pca, surface

In [ ]:
# Align based on : whole brain: center , pca, surface

for alignment_type in ['Using_Center','Using_PCA','Using_Surface']:
    for hem in ['lhs','rhs']:
        for exp in ['MG48','MG49','MG50']:
            print(alignment_type,hem,exp)
            
            if alignment_type is 'Using_Center':
                input_path = output_root
                output_path = output_path_alignment_global_alignment_centers
                ref_surf = Surface()
                surf = Surface()

                ref_surf.append(Surface(input_path+\
                                    '/Surfaces/{}_{}_pia_open_bottom_voxel_size_50.vtk'.format('MG48',hem)).surface)
                ref_surf.append(Surface(input_path+\
                                    '/Surfaces/{}_{}_WM_open_bottom_voxel_size_50.vtk'.format('MG48',hem)).surface)
                
                surf.append(Surface(input_path+\
                                    '/Surfaces/{}_{}_pia_open_bottom_voxel_size_50.vtk'.format(exp,hem)).surface)
                surf.append(Surface(input_path+\
                                    '/Surfaces/{}_{}_WM_open_bottom_voxel_size_50.vtk'.format(exp,hem)).surface)
                
                
                if exp == 'MG48' : 
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_pts = []
                    pts = []
                    ref_pts.append(ref_surf.get_center_of_mass())
                    
                    pts.append(surf.get_center_of_mass())
                    
                    icp,txmat = Landmarks(pts=pts).align_landmarks(ref_pts)
                
            elif alignment_type is 'Using_PCA':
                input_path = output_path_alignment_global_alignment_centers
                output_path = output_path_alignment_global_alignment_pcas
                if exp == 'MG48':
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_pca_pts = []
                    pca_pts = []
                    ref_surf = Surface()
                    surf = Surface()

                    ref_surf.append(Surface(input_path+\
                                        '/Surfaces/{}_{}_pia_open_bottom_voxel_size_50.vtk'.format('MG48',hem)).surface)
                    ref_surf.append(Surface(input_path+\
                                        '/Surfaces/{}_{}_WM_open_bottom_voxel_size_50.vtk'.format('MG48',hem)).surface)

                    surf.append(Surface(input_path+\
                                        '/Surfaces/{}_{}_pia_open_bottom_voxel_size_50.vtk'.format(exp,hem)).surface)
                    surf.append(Surface(input_path+\
                                        '/Surfaces/{}_{}_WM_open_bottom_voxel_size_50.vtk'.format(exp,hem)).surface)
            
                    ref_pca_pts = Alignment().get_pca_for_surf(ref_surf.surface,output_path=None,ref_coronal=True,lhs=(hem=='lhs'))
                     
                    pca_pts = Alignment().get_pca_for_surf(surf.surface,output_path=None,ref_coronal=False,lhs=(hem=='lhs'))
                    
                        
                    icp,txmat = Landmarks(pts=pca_pts).align_landmarks(ref_pca_pts)

            else:
                input_path = output_path_alignment_global_alignment_pcas
                output_path = output_path_alignment_global_alignment_surfaces
                print(input_path,output_path)
                if exp == 'MG48':
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_surf = Surface()
                    surf = Surface()
                    
                    ref_surf.append(Surface(input_path+\
                                        '/Surfaces/{}_{}_pia_open_bottom_voxel_size_50.vtk'.format('MG48',hem)).surface)
                    #ref_surf.append(Surface(input_path+\
                    #                    '/Surfaces/{}_{}_WM_open_bottom_voxel_size_200.vtk'.format('MG48',hem)).surface)

                    surf.append(Surface(input_path+\
                                        '/Surfaces/{}_{}_pia_open_bottom_voxel_size_50.vtk'.format(exp,hem)).surface)
                    #surf.append(Surface(input_path+\
                    #                    '/Surfaces/{}_{}_WM_open_bottom_voxel_size_200.vtk'.format(exp,hem)).surface)
                    
                        
                    aligner = Alignment(ref_surf.surface,surf.surface,align_by_pca=False,nr_iterations=10000,)
                    txmat = aligner.get_transformation_matrix()
                    icp = aligner.get_icp_transform()
                
                
            if icp is not None:
                aligner = Alignment(lhs=(hem=='lhs'))
                aligner.transform_folders(input_path,output_path,exp,icp=icp,txmat = txmat,hem_specific_alignment=True,)
                aligner.transform_folders(input_path + '/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True)
                
            else:
                aligner = Alignment(lhs=(hem=='lhs'))
                aligner.transform_folders(input_path,output_path,exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                aligner.transform_folders(input_path + '/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)

#             if icp is not None:
               
#                 surf_pia = Surface(input_path+'Surfaces/{}_{}_pia_open_bottom_voxel_size_200.vtk'.format(exp,hem))
#                 surf_pia.apply_icp_transform(icp)
#                 surf_pia.write_surface_mesh(output_path+'{}_{}_pia_open_bottom_voxel_size_200.vtk'.format(exp,hem))
#             else:
#                 surf_pia = Surface(input_path+'Surfaces/{}_{}_pia_open_bottom_voxel_size_200.vtk'.format(exp,hem))
#                 txed_pia = Alignment().apply_polydata_transformation(surf_pia.surface,txmat)
#                 #surf_pia.apply_icp_transform(icp)
#                 Surface(polydata=txed_pia).write_surface_mesh(output_path+'{}_{}_pia_open_bottom_voxel_size_200.vtk'.format(exp,hem))

In [ ]:
# Align based on : Local vs1,vm1: center , pca, surface

#input_path_root = output_path_alignment_+'Surface_Res_200/Surface_Aligned_Pia_Only/'
for alignment_type in ['Using_Local_Rabies_Surfaces']:#['Using_Local_Centers','Using_Local_Surfaces','Using_Local_Rabies_Surfaces','Using_M1_SBF']:
    for hem in ['lhs','rhs']:
        for exp in ['MG48','MG49','MG50']:
            print(alignment_type,hem,exp)
            if alignment_type is 'Using_Local_Centers':
                input_path = output_path_alignment_global_alignment_surfaces
                output_path = output_path_alignment_global_alignment_local_centers
                ref_rabies_center_vm1 = Landmarks(input_path+\
                                        '/vM1_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_Centroid.landmarksAscii'.format('MG48',hem))
                ref_rabies_center_vs1 = Landmarks(input_path+\
                                        '/vS1_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_Centroid.landmarksAscii'.format('MG48',hem))
                
                rabies_center_vm1 = Landmarks(input_path+\
                                        '/vM1_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_Centroid.landmarksAscii'.format(exp,hem))
                rabies_center_vs1 = Landmarks(input_path+\
                                        '/vS1_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_Centroid.landmarksAscii'.format(exp,hem))
               
                if exp == 'MG48' : 
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_pts = []
                    pts = []
                    ref_pts.append(ref_rabies_center_vm1.pts[0])
                    ref_pts.append(ref_rabies_center_vs1.pts[0])
                    pts.append(rabies_center_vm1.pts[0])
                    pts.append(rabies_center_vs1.pts[0])
                    icp,txmat = Landmarks(pts=pts).align_landmarks(ref_pts)
                
#             elif alignment_type is 'Using_Local_PCA':
#                 input_path = output_path_alignment_global_alignment_surfaces
#                 output_path = output_path_alignment_global_alignment_local_pcas
#                 if exp == 'MG48':
#                     txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
#                     icp=None
#                 else:
#                     ref_pca_pts = []
#                     pca_pts = []
#                     ref_vm1_surf = Surface(input_path+\
#                                         '/vM1_Ref_Surfaces/{}_{}_vM1_surf_res_200_voxel_size_500_cutoff_0.05.vtk'.format('MG48',hem))
#                     ref_vs1_surf = Surface(input_path+\
#                                         '/vS1_Ref_Surfaces/{}_{}_vS1_surf_res_200_voxel_size_500_cutoff_0.05.vtk'.format('MG48',hem))
            
#                     vm1_surf = Surface(input_path+\
#                                         '/vM1_Ref_Surfaces/{}_{}_vM1_surf_res_200_voxel_size_500_cutoff_0.05.vtk'.format(exp,hem))
#                     vs1_surf = Surface(input_path+\
#                                         '/vS1_Ref_Surfaces/{}_{}_vS1_surf_res_200_voxel_size_500_cutoff_0.05.vtk'.format(exp,hem))
            
            
#                     ref_vm1_pts = Alignment().get_pca_for_surf(ref_vm1_surf.surface,output_path=None,ref_coronal=True,lhs=(hem=='lhs'))
#                     ref_vs1_pts = Alignment().get_pca_for_surf(ref_vs1_surf.surface,output_path=None,ref_coronal=True,lhs=(hem=='lhs'))
#                     vm1_pts = Alignment().get_pca_for_surf(vm1_surf.surface,output_path=None,ref_coronal=False,lhs=(hem=='lhs'))
#                     vs1_pts = Alignment().get_pca_for_surf(vs1_surf.surface,output_path=None,ref_coronal=False,lhs=(hem=='lhs'))
                    
#                     for pt in ref_vm1_pts:
#                         ref_pca_pts.append(pt)
#                     for pt in ref_vs1_pts:
#                         ref_pca_pts.append(pt)
#                     for pt in vm1_pts:
#                         pca_pts.append(pt)
#                     for pt in vs1_pts:
#                         pca_pts.append(pt)
                        
#                     icp,txmat = Landmarks(pts=pca_pts).align_landmarks(ref_pca_pts)

            elif alignment_type is 'Using_M1_SBF':
                input_path = output_path_alignment_global_alignment_surfaces
                ref_path_sbf = output_path_atlas+'/Aligned_To_MG48_bottom_open_final_New/SBF_{}_sg.vtk'.format(hem)
                output_path = output_path_alignment_global_alignment_local_surfaces_m1_sbf
                if exp == 'MG48':
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_surf = Surface()
                    surf = Surface()
                    
                    ref_vm1_surf = Surface(input_path+\
                                        '/vM1_Ref_Surfaces/{}_{}_vM1_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format('MG48',hem))
                    ref_vs1_surf = Surface(ref_path_sbf)
            
                    vm1_surf = Surface(input_path+\
                                        '/vM1_Ref_Surfaces/{}_{}_vM1_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(exp,hem))
                    vs1_surf = Surface(input_path+\
                                        '/Surfaces/{}_{}_s1_hull.vtk'.format(exp,hem))
            
                    ref_surf.append(ref_vm1_surf.surface)
                    #if ref_vm1_surf.surface.GetNumberOfCells() > ref_vs1_surf.surface.GetNumberOfCells():
                    iteration = int(ref_vm1_surf.surface.GetNumberOfCells() / ref_vs1_surf.surface.GetNumberOfCells())
                    for i in range(iteration):
                        ref_surf.append(ref_vs1_surf.surface)
                    
                    surf.append(vm1_surf.surface)
                    iteration = int(vm1_surf.surface.GetNumberOfCells() / vs1_surf.surface.GetNumberOfCells())
                    for i in range(iteration):
                        surf.append(vs1_surf.surface)
                    
                    aligner = Alignment(ref_surf.surface,surf.surface,align_by_pca=False,nr_iterations=10000,)
                    txmat = aligner.get_transformation_matrix()
                    icp = aligner.get_icp_transform()
                    
                    #surf.apply_icp_transform(icp)
                    #surf.write_surface_mesh(output_root+'bla.vtk')
                
            elif alignment_type is 'Using_Local_Surfaces':
                input_path = output_path_alignment_global_alignment_surfaces
                output_path = output_path_alignment_global_alignment_local_surfaces
                if exp == 'MG48':
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_surf = Surface()
                    surf = Surface()
                    
                    ref_vm1_surf = Surface(input_path+\
                                        '/vM1_Ref_Surfaces/{}_{}_vM1_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format('MG48',hem))
                    ref_vs1_surf = Surface(input_path+\
                                        '/vS1_Ref_Surfaces/{}_{}_vS1_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format('MG48',hem))
            
                    vm1_surf = Surface(input_path+\
                                        '/vM1_Ref_Surfaces/{}_{}_vM1_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(exp,hem))
                    vs1_surf = Surface(input_path+\
                                        '/vS1_Ref_Surfaces/{}_{}_vS1_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(exp,hem))
            
                    ref_surf.append(ref_vm1_surf.surface)
                    ref_surf.append(ref_vs1_surf.surface)
                    surf.append(vm1_surf.surface)
                    surf.append(vs1_surf.surface)
                    
                        
                    aligner = Alignment(ref_surf.surface,surf.surface,align_by_pca=False,nr_iterations=10000,)
                    txmat = aligner.get_transformation_matrix()
                    icp = aligner.get_icp_transform()
                
            elif alignment_type is 'Using_Local_Rabies_Surfaces':
                input_path = output_path_alignment_global_alignment_surfaces
                output_path = output_path_alignment_global_alignment_local_rabies_surfaces
                if exp == 'MG48':
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_surf = Surface()
                    surf = Surface()
                    
                    ref_vm1_surf = Surface(input_path+\
                                        '/Landmarks/Density_Clusters/{}_{}_vM1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format('MG48',hem))
                    ref_vs1_surf = Surface(input_path+\
                                        '/Landmarks/Density_Clusters/{}_{}_vS1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format('MG48',hem))
            
                    vm1_surf = Surface(input_path+\
                                        '/Landmarks/Density_Clusters/{}_{}_vM1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format(exp,hem))
                    vs1_surf = Surface(input_path+\
                                        '/Landmarks/Density_Clusters/{}_{}_vS1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format(exp,hem))
            
                    ref_surf.append(ref_vm1_surf.surface)
                    ref_surf.append(ref_vs1_surf.surface)
                    surf.append(vm1_surf.surface)
                    surf.append(vs1_surf.surface)
                    
                        
                    aligner = Alignment(ref_surf.surface,surf.surface,align_by_pca=False,nr_iterations=10000,)
                    txmat = aligner.get_transformation_matrix()
                    icp = aligner.get_icp_transform()
                
                
                
            if icp is not None:
                aligner = Alignment(lhs=(hem=='lhs'))
                aligner.transform_folders(input_path,output_path,exp,icp=icp,txmat = txmat,hem_specific_alignment=True,)
                aligner.transform_folders(input_path + '/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True)
                
                #aligner.transform_folder(input_path+'/Surfaces/',output_path+'/Surfaces/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                #aligner.transform_folder(input_path+'/vM1_Ref_Surfaces/',output_path+'/vM1_Ref_Surfaces/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                
            else:
                aligner = Alignment(lhs=(hem=='lhs'))
                aligner.transform_folders(input_path,output_path,exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                aligner.transform_folders(input_path + '/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                #aligner.transform_folder(input_path+'/Surfaces/',output_path+'/Surfaces/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                
#             if icp is not None:
               
#                 surf_pia = Surface(input_path+'Surfaces/{}_{}_pia_open_bottom_voxel_size_200.vtk'.format(exp,hem))
#                 surf_pia.apply_icp_transform(icp)
#                 surf_pia.write_surface_mesh(output_path+'{}_{}_pia_open_bottom_voxel_size_200.vtk'.format(exp,hem))
#             else:
#                 surf_pia = Surface(input_path+'Surfaces/{}_{}_pia_open_bottom_voxel_size_200.vtk'.format(exp,hem))
#                 txed_pia = Alignment().apply_polydata_transformation(surf_pia.surface,txmat)
#                 #surf_pia.apply_icp_transform(icp)
#                 Surface(polydata=txed_pia).write_surface_mesh(output_path+'{}_{}_pia_open_bottom_voxel_size_200.vtk'.format(exp,hem))

## vS1 Alignment

In [ ]:
# Align based on : barrel centers or column centers 

In [ ]:
# Align based on : Local vs1,vm1: center , pca, surface

#input_path_root = output_path_alignment_+'Surface_Res_200/Surface_Aligned_Pia_Only/'
for alignment_type in ['Using_Local_Centers','Using_Local_Surfaces','Using_Local_Rabies_Surfaces']:
    for hem in ['lhs','rhs']:
        for exp in ['MG48','MG49','MG50']:
            print(alignment_type,hem,exp)
            if alignment_type is 'Using_Local_Centers':
                input_path = output_path_alignment_global_alignment_surfaces
                output_path = output_path_alignment_vs1_center
#                 ref_rabies_center_vm1 = Landmarks(input_path+\
#                                         '/vM1_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_Centroid.landmarksAscii'.format('MG48',hem))
                ref_rabies_center_vs1 = Landmarks(input_path+\
                                        '/vS1_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_Centroid.landmarksAscii'.format('MG48',hem))
                
#                 rabies_center_vm1 = Landmarks(input_path+\
#                                         '/vM1_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_Centroid.landmarksAscii'.format(exp,hem))
                rabies_center_vs1 = Landmarks(input_path+\
                                        '/vS1_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_Centroid.landmarksAscii'.format(exp,hem))
               
                if exp == 'MG48' : 
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_pts = []
                    pts = []
                    #ref_pts.append(ref_rabies_center_vm1.pts[0])
                    ref_pts.append(ref_rabies_center_vs1.pts[0])
                    #pts.append(rabies_center_vm1.pts[0])
                    pts.append(rabies_center_vs1.pts[0])
                    icp,txmat = Landmarks(pts=pts).align_landmarks(ref_pts)
                
                
            elif alignment_type is 'Using_Local_Surfaces':
                input_path = output_path_alignment_global_alignment_surfaces
                output_path = output_path_alignment_vs1_surface
                if exp == 'MG48':
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_surf = Surface()
                    surf = Surface()
                    
                    #ref_vm1_surf = Surface(input_path+\
                     #                   '/vM1_Ref_Surfaces/{}_{}_vM1_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format('MG48',hem))
                    ref_vs1_surf = Surface(input_path+\
                                        '/vS1_Ref_Surfaces/{}_{}_vS1_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format('MG48',hem))
            
                    #vm1_surf = Surface(input_path+\
                    #                    '/vM1_Ref_Surfaces/{}_{}_vM1_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(exp,hem))
                    vs1_surf = Surface(input_path+\
                                        '/vS1_Ref_Surfaces/{}_{}_vS1_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(exp,hem))
            
                    #ref_surf.append(ref_vm1_surf.surface)
                    ref_surf.append(ref_vs1_surf.surface)
                    #surf.append(vm1_surf.surface)
                    surf.append(vs1_surf.surface)
                    
                        
                    aligner = Alignment(ref_surf.surface,surf.surface,align_by_pca=False,nr_iterations=10000,)
                    txmat = aligner.get_transformation_matrix()
                    icp = aligner.get_icp_transform()
                
            elif alignment_type is 'Using_Local_Rabies_Surfaces':
                input_path = output_path_alignment_global_alignment_surfaces
                output_path = output_path_alignment_vs1_rabies_surface
                if exp == 'MG48':
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_surf = Surface()
                    surf = Surface()
                    
                    #ref_vm1_surf = Surface(input_path+\
                    #                    '/vM1_Ref_Surfaces/{}_{}_vM1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format('MG48',hem))
                    ref_vs1_surf = Surface(input_path+\
                                        '/Landmarks/Density_Clusters/{}_{}_vS1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format('MG48',hem))
            
                    #vm1_surf = Surface(input_path+\
                    #                    '/vM1_Ref_Surfaces/{}_{}_vM1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format(exp,hem))
                    vs1_surf = Surface(input_path+\
                                        '/Landmarks/Density_Clusters/{}_{}_vS1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format(exp,hem))
            
                    #ref_surf.append(ref_vm1_surf.surface)
                    ref_surf.append(ref_vs1_surf.surface)
                    #surf.append(vm1_surf.surface)
                    surf.append(vs1_surf.surface)
                    
                        
                    aligner = Alignment(ref_surf.surface,surf.surface,align_by_pca=False,nr_iterations=10000,)
                    txmat = aligner.get_transformation_matrix()
                    icp = aligner.get_icp_transform()
                
                
                
            if icp is not None:
                aligner = Alignment(lhs=(hem=='lhs'))
                aligner.transform_folders(input_path,output_path,exp,icp=icp,txmat = txmat,hem_specific_alignment=True,)
                aligner.transform_folders(input_path + '/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True)
                
                #aligner.transform_folder(input_path+'/Surfaces/',output_path+'/Surfaces/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                #aligner.transform_folder(input_path+'/vM1_Ref_Surfaces/',output_path+'/vM1_Ref_Surfaces/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                
            else:
                aligner = Alignment(lhs=(hem=='lhs'))
                aligner.transform_folders(input_path,output_path,exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                aligner.transform_folders(input_path + '/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                #aligner.transform_folder(input_path+'/Surfaces/',output_path+'/Surfaces/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
               

## vM1 Alignment

In [ ]:
# Align based on : Local vs1,vm1: center , pca, surface

#input_path_root = output_path_alignment_+'Surface_Res_200/Surface_Aligned_Pia_Only/'
for alignment_type in ['Using_Local_Centers','Using_Local_Surfaces','Using_Local_Rabies_Surfaces',]:
    for hem in ['lhs','rhs']:
        for exp in ['MG49','MG50']:
            print(alignment_type,hem,exp)
            if alignment_type is 'Using_Local_Centers':
                input_path = output_path_alignment_global_alignment_surfaces
                output_path = output_path_alignment_vm1_center
                ref_rabies_center_vm1 = Landmarks(input_path+\
                                        '/vM1_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_Centroid.landmarksAscii'.format('MG48',hem))
#                 ref_rabies_center_vs1 = Landmarks(input_path+\
#                                         '/vS1_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_Centroid.landmarksAscii'.format('MG48',hem))
                
                rabies_center_vm1 = Landmarks(input_path+\
                                        '/vM1_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_Centroid.landmarksAscii'.format(exp,hem))
#                 rabies_center_vs1 = Landmarks(input_path+\
#                                         '/vS1_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_Centroid.landmarksAscii'.format(exp,hem))
               
                if exp == 'MG48' : 
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_pts = []
                    pts = []
                    ref_pts.append(ref_rabies_center_vm1.pts[0])
                    #ref_pts.append(ref_rabies_center_vs1.pts[0])
                    pts.append(rabies_center_vm1.pts[0])
                    #pts.append(rabies_center_vs1.pts[0])
                    icp,txmat = Landmarks(pts=pts).align_landmarks(ref_pts)
                
                
            elif alignment_type is 'Using_Local_Surfaces':
                input_path = output_path_alignment_global_alignment_surfaces
                output_path = output_path_alignment_vm1_surface
                if exp == 'MG48':
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_surf = Surface()
                    surf = Surface()
                    
                    ref_vm1_surf = Surface(input_path+\
                                       '/vM1_Ref_Surfaces/{}_{}_vM1_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format('MG48',hem))
#                     ref_vs1_surf = Surface(input_path+\
#                                         '/vS1_Ref_Surfaces/{}_{}_vS1_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format('MG48',hem))
            
                    vm1_surf = Surface(input_path+\
                                       '/vM1_Ref_Surfaces/{}_{}_vM1_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(exp,hem))
#                     vs1_surf = Surface(input_path+\
#                                         '/vS1_Ref_Surfaces/{}_{}_vS1_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(exp,hem))
            
                    ref_surf.append(ref_vm1_surf.surface)
                    #ref_surf.append(ref_vs1_surf.surface)
                    surf.append(vm1_surf.surface)
                    #surf.append(vs1_surf.surface)
                    
                        
                    aligner = Alignment(ref_surf.surface,surf.surface,align_by_pca=False,nr_iterations=10000,)
                    txmat = aligner.get_transformation_matrix()
                    icp = aligner.get_icp_transform()
                
            elif alignment_type is 'Using_Local_Rabies_Surfaces':
                input_path = output_path_alignment_global_alignment_surfaces
                output_path = output_path_alignment_vm1_rabies_surface
                if exp == 'MG48':
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_surf = Surface()
                    surf = Surface()
                    
                    ref_vm1_surf = Surface(input_path+\
                                        '/Landmarks/Density_Clusters/{}_{}_vM1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format('MG48',hem))
                    #ref_vs1_surf = Surface(input_path+\
                    #                    '/Landmarks/Density_Clusters/{}_{}_vS1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format('MG48',hem))
            
                    vm1_surf = Surface(input_path+\
                                        '/Landmarks/Density_Clusters/{}_{}_vM1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format(exp,hem))
                    #vs1_surf = Surface(input_path+\
                    #                    '/Landmarks/Density_Clusters/{}_{}_vS1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format(exp,hem))
            
                    ref_surf.append(ref_vm1_surf.surface)
                    #ref_surf.append(ref_vs1_surf.surface)
                    surf.append(vm1_surf.surface)
                    #surf.append(vs1_surf.surface)
                    
                        
                    aligner = Alignment(ref_surf.surface,surf.surface,align_by_pca=False,nr_iterations=10000,)
                    txmat = aligner.get_transformation_matrix()
                    icp = aligner.get_icp_transform()
                
                    print(txmat)
                    
            if icp is not None:
                aligner = Alignment(lhs=(hem=='lhs'))
                aligner.transform_folders(input_path,output_path,exp,icp=icp,txmat = txmat,hem_specific_alignment=True,)
                aligner.transform_folders(input_path + '/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True)
                
                #aligner.transform_folder(input_path+'/Surfaces/',output_path+'/Surfaces/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                #aligner.transform_folder(input_path+'/vM1_Ref_Surfaces/',output_path+'/vM1_Ref_Surfaces/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                
            else:
                aligner = Alignment(lhs=(hem=='lhs'))
                aligner.transform_folders(input_path,output_path,exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                aligner.transform_folders(input_path + '/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                #aligner.transform_folder(input_path+'/Surfaces/',output_path+'/Surfaces/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
               

## SBF Alignment

In [ ]:
# Align based on : Local vs1,vm1: center , pca, surface

#input_path_root = output_path_alignment_+'Surface_Res_200/Surface_Aligned_Pia_Only/'
for alignment_type in ['Use_Both']:#['Use_Barrels_Alone','Use_Projections_Alone','Use_Both']:#['Using_Local_Centers','Using_Local_PCA','Using_Local_Surfaces']:
    for hem in ['lhs','rhs']:
        
        ref_sg = AmiraSpatialGraph(output_path_alignment_global_alignment_local_rabies_surfaces+\
                                   'Surfaces/'+'MG50_'+hem+'_barrels_50.am',barrel_projections_present=True)
        
        for exp in ['MG48','MG49','MG50']:
            print(alignment_type,hem,exp)
            if alignment_type is 'Use_Barrels_Alone':
                input_path = output_path_alignment_global_alignment_local_surfaces
                output_path = output_path_alignment_s1bf_barrels_alone
                
                if exp == 'MG48' : 
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_pts = []
                    pts = []
                    sg = AmiraSpatialGraph(output_path_alignment_global_alignment_local_surfaces+\
                                   'Surfaces/'+exp+'_'+hem+'_barrels_50.am',barrel_projections_present=True)
                    
                    matching_barrels = MatchBarrels(ref_sg.barrels,sg.barrels,use_barrels_alone=True)
                    ref_pts,pts = matching_barrels.get_matching_barrel_centroids()
                    icp,txmat = Landmarks(pts=pts).align_landmarks(ref_pts)
                
                
            elif alignment_type is 'Use_Projections_Alone':
                input_path = output_path_alignment_global_alignment_local_surfaces
                output_path = output_path_alignment_s1bf_projections_alone
                if exp == 'MG48':
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_pts = []
                    pts = []
                    sg = AmiraSpatialGraph(output_path_alignment_global_alignment_local_surfaces+\
                                   'Surfaces/'+exp+'_'+hem+'_barrels_50.am',barrel_projections_present=True)
                    
                    matching_barrels = MatchBarrels(ref_sg.barrels,sg.barrels,use_projections_alone=True)
                    ref_pts,pts = matching_barrels.get_matching_barrel_centroids()
                    icp,txmat = Landmarks(pts=pts).align_landmarks(ref_pts)
                
            elif alignment_type is 'Use_Both':
                input_path = output_path_alignment_global_alignment_local_rabies_surfaces
                output_path = output_path_alignment_s1bf_barrels_and_projections
                if exp == 'MG48':
                    txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
                    icp=None
                else:
                    ref_pts = []
                    pts = []
                    sg = AmiraSpatialGraph(output_path_alignment_global_alignment_local_surfaces+\
                                   'Surfaces/'+exp+'_'+hem+'_barrels_50.am',barrel_projections_present=True)
                    
                    matching_barrels = MatchBarrels(ref_sg.barrels,sg.barrels)
                    ref_pts,pts = matching_barrels.get_matching_barrel_centroids()
                    icp,txmat = Landmarks(pts=pts).align_landmarks(ref_pts)
                
                
            if icp is not None:
                aligner = Alignment(lhs=(hem=='lhs'))
                aligner.transform_folders(input_path,output_path,exp,icp=icp,txmat = txmat,hem_specific_alignment=True,)
                aligner.transform_folders(input_path + '/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True)
                
                #aligner.transform_folder(input_path+'/Surfaces/',output_path+'/Surfaces/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                #aligner.transform_folder(input_path+'/vM1_Ref_Surfaces/',output_path+'/vM1_Ref_Surfaces/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                
            else:
                aligner = Alignment(lhs=(hem=='lhs'))
                aligner.transform_folders(input_path,output_path,exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                aligner.transform_folders(input_path + '/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
                #aligner.transform_folder(input_path+'/Surfaces/',output_path+'/Surfaces/',exp,icp=icp,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
               

## Rabies Clustering Selection

In [ ]:
def align_s1_m1_surfs(input_path_landmarks,input_path_surfaces,output_path,br,voxel_size,cutoff,surf_type):
                       
    for hem in ['lhs',]:
        aligned_surfs = Surface()
        aligned_surfs_cortical = Surface()
        aligned_surf_list = []
        aligned_landmarks = Landmarks()
        #aligned_landmarks_cleaned = Landmarks()
        ref_surf = Surface(input_path_landmarks+'MG48_'+hem+'_'+br+'_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii_surface.vtk'\
                               .format(voxel_size,cutoff))
        ref_landmarks = Landmarks(input_path_landmarks+'MG48_'+hem+'_'+br+'_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'\
                               .format(voxel_size,cutoff))
        aligned_landmarks.append_landmarks(ref_landmarks.pts)
        ref_landmarks.write_landmarks(output_path+'/aligned_MG48_{}_{}_{}_{}.LandmarkAscii'.format(hem,br,voxel_size,cutoff))
        aligned_surfs.append(ref_surf.surface)
        ref_surf.write_surface_mesh(output_path+'/aligned_MG48_{}_{}_{}_{}.vtk'.format(hem,br,voxel_size,cutoff))
        ref_surf.create_delunay_surface_3d(output_filename=output_path+'/aligned_MG48_{}_{}_{}_{}_delunay.vtk'.format(hem,br,voxel_size,cutoff))
        
        ref_surf_cortical = Surface(input_path_surfaces+'MG48_'+hem+'_'+br+'_surf_res_50_voxel_size_{}_cutoff_{}.vtk'\
                               .format(voxel_size,cutoff))
#         print(input_path_surfaces+'MG48_'+hem+'_'+br+'surf_res_50_voxel_size_{}_cutoff_{}.vtk'\
#                                .format(voxel_size,cutoff))
        ref_surf_cortical.write_surface_mesh(output_path+'/aligned_{}_{}_{}_{}_{}_cortical.vtk'.format('MG48',hem,br,voxel_size,cutoff))
        aligned_surfs_cortical.append(ref_surf_cortical.surface)
        
#         if surf_type.find('raw')>0:
#             ref_landmarks = Landmarks(input_path_landmarks+'MG48_'+hem+'_'+br+'_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii_raw.LandmarkAscii'\
#                                .format(voxel_size,cutoff,surf_type))
#         elif surf_type.find('cleaned')>0:
#             ref_landmarks = Landmarks(input_path_landmarks+'MG48_'+hem+'_'+br+'_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii_cleaned.LandmarkAscii'\
#                                .format(voxel_size,cutoff))
#         else:
    #         ref_landmarks = Landmarks(input_path_landmarks+'MG48_'+hem+'_'+br+'_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'\
    #                            .format(voxel_size,cutoff))
        
        
        
#         landmarks_cleaned = Landmarks(input_path+'MG48_'+hem+'_'+br+'_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii_cleaned.LandmarkAscii'\
#                                .format(voxel_size,cutoff))
#         aligned_landmarks_cleaned.append_landmarks(landmarks_cleaned.pts)
#         landmarks_cleaned.write_landmarks(output_path+'/aligned_MG48_{}_{}_{}_{}_cleaned.LandmarkAscii'.format(hem,br,voxel_size,cutoff))
        
        for exp in ['MG49','MG50']:
            
            surf = Surface(input_path_landmarks+exp+'_'+hem+'_'+br+'_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii_surface.vtk'\
                           .format(voxel_size,cutoff))
            surf_cortical = Surface(input_path_surfaces+exp+'_'+hem+'_'+br+'_surf_res_50_voxel_size_{}_cutoff_{}.vtk'\
                               .format(voxel_size,cutoff))
            if surf_type.find('raw')>0:
                landmarks = Landmarks(input_path_landmarks+exp+'_'+hem+'_'+br+'_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii_raw.LandmarkAscii'\
                                   .format(voxel_size,cutoff,surf_type))
            elif surf_type.find('cleaned')>0:
                landmarks = Landmarks(input_path_landmarks+exp+'_'+hem+'_'+br+'_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii_cleaned.LandmarkAscii'\
                               .format(voxel_size,cutoff))
            else:
                landmarks = Landmarks(input_path_landmarks+exp+'_'+hem+'_'+br+'_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'\
                                   .format(voxel_size,cutoff,surf_type))
        
            #landmarks_cleaned = Landmarks(input_path+exp+'_'+hem+'_'+br+'_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii_cleaned.LandmarkAscii'\
            #                       .format(voxel_size,cutoff))
            
            #aligner = Alignment(ref_surf.surface,surf.surface,align_by_pca=False,nr_iterations=10000,)
            if surf_type == 'rabies_cortical_surface':
                aligner = Alignment(ref_surf_cortical.surface,surf_cortical.surface,align_by_pca=False,nr_iterations=10000,)
            else:
                aligner = Alignment(ref_surf.surface,surf.surface,align_by_pca=False,nr_iterations=10000,)
            
            txmat = aligner.get_transformation_matrix()
            icp = aligner.get_icp_transform()
            
            surf.apply_icp_transform(icp)
            aligned_surfs.append(surf.surface)
            aligned_surf_list.append(surf.surface)
            surf.write_surface_mesh(output_path+'/aligned_{}_{}_{}_{}_{}.vtk'.format(exp,hem,br,voxel_size,cutoff))
            #surf_del = Surface(output_path+'/aligned_{}_{}_{}_{}_{}.vtk'.format(exp,hem,br,voxel_size,cutoff))
            #surf_del.create_delunay_surface_3d(output_filename=output_path+'/aligned_{}_{}_{}_{}_{}_delunay.vtk'.format(exp,hem,br,voxel_size,cutoff))
            #Surface(polydata=surf.surface).create_delunay_surface_3d(output_filename=output_path+'/aligned_{}_{}_{}_{}_{}_delunay.vtk'.format(exp,hem,br,voxel_size,cutoff))
            landmarks.apply_transformation(txmat)
            landmarks.write_landmarks(output_path+'/aligned_{}_{}_{}_{}_{}.LandmarkAscii'.format(exp,hem,br,voxel_size,cutoff))
            aligned_landmarks.append_landmarks(landmarks.pts)
        
            Surface(polydata=surf.surface).create_delunay_surface_3d(output_filename=output_path+'/aligned_{}_{}_{}_{}_{}_delunay.vtk'.format(exp,hem,br,voxel_size,cutoff))
        
            
            surf_cortical.apply_icp_transform(icp)
            surf_cortical.write_surface_mesh(output_path+'/aligned_{}_{}_{}_{}_{}_cortical.vtk'.format(exp,hem,br,voxel_size,cutoff))
            aligned_surfs_cortical.append(surf_cortical.surface)
#             landmarks_cleaned.apply_transformation(txmat)
#             landmarks_cleaned.write_landmarks(output_path+'/aligned_{}_{}_{}_{}_{}_cleaned.LandmarkAscii'.format(exp,hem,br,voxel_size,cutoff))
#             aligned_landmarks_cleaned.append_landmarks(landmarks_cleaned.pts)
            
        aligned_surfs.write_surface_mesh(output_path+'/aligned_{}_{}_{}_{}.vtk'.format(hem,br,voxel_size,cutoff))
        aligned_landmarks.write_landmarks(output_path+'/aligned_{}_{}_{}_{}.LandmarkAscii'.format(hem,br,voxel_size,cutoff))
        aligned_surfs_cortical.write_surface_mesh(output_path+'/aligned_{}_{}_{}_{}_cortical.vtk'.format(hem,br,voxel_size,cutoff))
        
        #aligned_landmarks_cleaned.write_landmarks(output_path+'/aligned_{}_{}_{}_{}_cleaned.LandmarkAscii'.format(hem,br,voxel_size,cutoff))
        
        #         r1 = vtk.vtkPolyDataReader()
#         r1.SetFileName(output_path+'/aligned_MG48_{}_{}_{}_{}.vtk'.format(hem,br,voxel_size,cutoff))
#         r2 = vtk.vtkPolyDataReader()
#         r2.SetFileName(output_path+'/aligned_MG49_{}_{}_{}_{}.vtk'.format(hem,br,voxel_size,cutoff))
#         r3 = vtk.vtkPolyDataReader()
#         r3.SetFileName(output_path+'/aligned_MG50_{}_{}_{}_{}.vtk'.format(hem,br,voxel_size,cutoff))


#         inter = vtk.vtkIntersectionPolyDataFilter()
        
#         inter.SetInputConnection(0,r1.GetOutputPort())
#         inter.SetInputConnection(1,r2.GetOutputPort())
#         inter.SetInputConnection(2,r3.GetOutputPort())
#         inter.Update()
#         Surface(polydata=inter.GetOutput()).write_surface_mesh(output_path+'/intersection_{}_{}_{}_{}.vtk'.format(hem,br,voxel_size,cutoff))

In [ ]:
input_path_root = output_path_alignment_global_alignment_surfaces

#for outer_alignment_type in ['Using_Surface']:
for inner_alignment_type in ['Rabies_Surface','Rabies_Cortical_Surface']:#['Rabies_Surface','Largest_Rabies_Surface','Rabies_Delunay_Surface',\
                             #'Largest_Rabies_Delunay_Surface','Rabies_Delunay_Surface_Cleaned']:


    for br in ['vM1','vS1']:
        if br == 'vM1':
            output_path_alignment = output_path_alignment_rabies_clustering_m1
        else:
            output_path_alignment = output_path_alignment_rabies_clustering_vs1

        output_path = output_path_alignment#output_path_alignment+'/'+outer_alignment_type+'/'+inner_alignment_type
        pathlib.Path(output_path).mkdir(exist_ok=True)

        input_path_landmarks = input_path_root+'/Landmarks/Density_Clusters/'
        input_path_surfaces = input_path_root+br+'_Ref_Surfaces/'

        if inner_alignment_type == 'Rabies_Surface':
            surf_type = 'surface'
            output_path = output_path + '/Using_Rabies_Surface'
            pathlib.Path(output_path).mkdir(exist_ok=True)
        elif inner_alignment_type == 'Rabies_Cortical_Surface':
            surf_type = 'rabies_cortical_surface'
            output_path = output_path + '/Using_Rabies_Cortical_Surface'
            pathlib.Path(output_path).mkdir(exist_ok=True)
#         elif inner_alignment_type == 'Rabies_Delunay_Surface':
#             surf_type = 'delunay'
#         elif inner_alignment_type == 'Rabies_Delunay_Surface_Cleaned':
#             surf_type = 'delunay_cleaned'
#         else:
#             surf_type = 'delunay_raw'

        for voxel_size in [50,100,200,300,400,450,500]:
            for cutoff in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
                # align surfaces
                print(br,voxel_size,cutoff,surf_type)
                align_s1_m1_surfs(input_path_landmarks,input_path_surfaces,output_path,br,voxel_size,cutoff,surf_type)
                    
                   
    

In [ ]:
# Find overlap between things
cols = ['Surf_Type','BR','Hem','Voxel_Size','Cutoff',\
        'Vol_Mean_Rabies_Surface','Vol_Std_Rabies_Surface','Vol_CoV_Rabies_Surface',\
        'Vol_Mean_Rabies_Surface_Cortical','Vol_Std_Rabies_Surface_Cortical','Vol_CoV_Rabies_Surface_Cortical',\
        'Vol_Overlap_Rabies_Surface','Landmarks_Overlap_Rabies_Surface',\
        'Vol_Overlap_Rabies_Surface_Cortical','Landmarks_Overlap_Rabies_Surface_Cortical',\
        'Centriod_Dist_Mean_Rabies_Surface','Centriod_Dist_Std_Rabies_Surface','Centriod_Dist_CoV_Rabies_Surface',\
        'Centriod_Dist_Mean_Rabies_Surface_Cortical','Centriod_Dist_Std_Rabies_Surface_Cortical','Centriod_Dist_CoV_Rabies_Surface_Cortical',\
        'Centriod_Rabies_Dist_Mean','Centriod_Rabies_Dist_Std','Centriod_Rabies_Dist_CoV',\
        'Rabies_Landmarks_Ratio','Aligned_Volume_Ratio_Rabies','Aligned_Volume_Ratio_Rabies_Cortical']
#df_main = pd.DataFrame(columns=cols)

#for outer_alignment_type in ['Using_Surface']:
for inner_alignment_type in ['Rabies_Surface','Rabies_Cortical_Surface']:#['Rabies_Surface','Largest_Rabies_Surface','Rabies_Delunay_Surface',\
                             #'Largest_Rabies_Delunay_Surface','Rabies_Delunay_Surface_Cleaned']:
    df_main = pd.DataFrame(columns=cols)

    for br in ['vM1','vS1']:
        if br == 'vM1':
            output_path_alignment = output_path_alignment_rabies_clustering_m1
        else:
            output_path_alignment = output_path_alignment_rabies_clustering_vs1

        output_path = output_path_alignment#output_path_alignment+'/'+outer_alignment_type+'/'+inner_alignment_type
        #output_path='/nas1/Data_Mythreya/MotorCortexProject/V11/backups/vM1/Using_Surface/Rabies_Surface/'
        if inner_alignment_type == 'Rabies_Surface':
            surf_type = 'surface'
            output_path = output_path_alignment + '/Using_Rabies_Surface'
            #pathlib.Path(output_path).mkdir(exist_ok=True)
        elif inner_alignment_type == 'Rabies_Cortical_Surface':
            surf_type = 'rabies_cortical_surface'
            output_path = output_path_alignment + '/Using_Rabies_Cortical_Surface'
            #pathlib.Path(output_path).mkdir(exist_ok=True)
        for hem in ['lhs','rhs']:
            for voxel_size in [50,100,200,300,400,450,500]:
                for cutoff in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,]:
                    # align surfaces
                    print(surf_type,hem,br,voxel_size,cutoff)

                    input_path_landmarks = output_path_alignment_global_alignment_surfaces+'/Landmarks/Density_Clusters/'
                    input_path_landmarks_all  = output_path_alignment_global_alignment_surfaces+'/Landmarks/K-Means/'

                    file1 = output_path+'/aligned_MG48_{}_{}_{}_{}.vtk'.format(hem,br,voxel_size,cutoff)
                    file2 = output_path+'/aligned_MG49_{}_{}_{}_{}.vtk'.format(hem,br,voxel_size,cutoff)
                    file3 = output_path+'/aligned_MG50_{}_{}_{}_{}.vtk'.format(hem,br,voxel_size,cutoff)
                    
                    file1_cortical = output_path+'/aligned_MG48_{}_{}_{}_{}_cortical.vtk'.format(hem,br,voxel_size,cutoff)
                    file2_cortical = output_path+'/aligned_MG49_{}_{}_{}_{}_cortical.vtk'.format(hem,br,voxel_size,cutoff)
                    file3_cortical = output_path+'/aligned_MG50_{}_{}_{}_{}_cortical.vtk'.format(hem,br,voxel_size,cutoff)
                    
                    ref_surf = Surface(file1)
                    surf1 = Surface(file2)
                    surf2 = Surface(file3)
                    
                    ref_surf_cortical = Surface(file1_cortical)
                    surf1_cortical = Surface(file2_cortical)
                    surf2_cortical = Surface(file3_cortical)

                    l_48 = Landmarks(output_path+'/aligned_MG48_{}_{}_{}_{}.LandmarkAscii'.format(hem,br,voxel_size,cutoff))
                    l_49 = Landmarks(output_path+'/aligned_MG49_{}_{}_{}_{}.LandmarkAscii'.format(hem,br,voxel_size,cutoff))
                    l_50 = Landmarks(output_path+'/aligned_MG50_{}_{}_{}_{}.LandmarkAscii'.format(hem,br,voxel_size,cutoff))

                    l_48_all = Landmarks(input_path_landmarks_all+'MG48_{}_rabies_kmeans_cluster_{}.landmarksAscii'.format(hem,br))
                    l_49_all = Landmarks(input_path_landmarks_all+'MG49_{}_rabies_kmeans_cluster_{}.landmarksAscii'.format(hem,br))
                    l_50_all = Landmarks(input_path_landmarks_all+'MG50_{}_rabies_kmeans_cluster_{}.landmarksAscii'.format(hem,br))

                    if (surf1.surface.GetNumberOfCells()) > 0 and (surf2.surface.GetNumberOfCells()) > 0 and (ref_surf.surface.GetNumberOfCells())>0:

                        df = pd.DataFrame(columns=cols)
                        df['Surf_Type'] = [inner_alignment_type]
                        df['BR'] = [br]
                        df['Hem'] = [hem]
                        df['Voxel_Size'] = [voxel_size]
                        df['Cutoff'] = [cutoff]  
                        
                        volumes_rabies_surf = Surface().get_surface_volumes([ref_surf,surf1,surf2])
                        df['Vol_Mean_Rabies_Surface']= np.array(volumes_rabies_surf).mean(axis=0)
                        df['Vol_Std_Rabies_Surface']= np.array(volumes_rabies_surf).std(axis=0)
                        df['Vol_CoV_Rabies_Surface']= np.array(volumes_rabies_surf).std(axis=0)/np.array(volumes_rabies_surf).mean(axis=0)
                        
                        
                        vol_ratio,landmarks_ratio = Landmarks().get_volume_overlap_from_bounding_box([ref_surf,surf1,surf2],[l_48,l_49,l_50])
                        df['Vol_Overlap_Rabies_Surface']=vol_ratio
                        df['Landmarks_Overlap_Rabies_Surface']=landmarks_ratio
                        
                        

                        dists_rabies_surf = Surface().get_distance_between_centroids([ref_surf,surf1,surf2],)
                        df['Centriod_Dist_Mean_Rabies_Surface']=np.array(dists_rabies_surf).mean(axis=0)
                        df['Centriod_Dist_Std_Rabies_Surface']=np.array(dists_rabies_surf).std(axis=0)
                        df['Centriod_Dist_CoV_Rabies_Surface']=np.array(dists_rabies_surf).std(axis=0)/np.array(dists_rabies_surf).mean(axis=0)
                        
                        
                        dists_rabies_landmarks = Landmarks().get_distance_between_centroids([l_48,l_49,l_50])
                        df['Centriod_Rabies_Dist_Mean']=np.array(dists_rabies_landmarks).mean(axis=0)
                        df['Centriod_Rabies_Dist_Std']=np.array(dists_rabies_landmarks).std(axis=0)
                        df['Centriod_Rabies_Dist_CoV']=np.array(dists_rabies_landmarks).std(axis=0)/np.array(dists_rabies_landmarks).mean(axis=0)
                        
                        df['Rabies_Landmarks_Ratio']= ( len(l_48.pts) + len(l_49.pts) + len(l_50.pts) )/ \
                                                    (len(l_48_all.pts) + len(l_49_all.pts) + len(l_50_all.pts))
                        if (surf1_cortical.surface.GetNumberOfCells()) > 0 and (surf2_cortical.surface.GetNumberOfCells()) > 0 and (ref_surf_cortical.surface.GetNumberOfCells())>0:
                        
                            df['Aligned_Volume_Ratio_Rabies']= Surface().get_delunay_volume([ref_surf,surf1,surf2])/(volumes_rabies_surf[0]+volumes_rabies_surf[1]+volumes_rabies_surf[2])

                            volumes_rabies_surf_cortical = Surface().get_surface_volumes([ref_surf_cortical,surf1_cortical,surf2_cortical])
                            df['Vol_Mean_Rabies_Surface_Cortical']=np.array(volumes_rabies_surf_cortical).mean(axis=0)
                            df['Vol_Std_Rabies_Surface_Cortical']=np.array(volumes_rabies_surf_cortical).std(axis=0)
                            df['Vol_CoV_Rabies_Surface_Cortical']=np.array(volumes_rabies_surf_cortical).std(axis=0)/np.array(volumes_rabies_surf_cortical).mean(axis=0)

                            vol_ratio_cortical,landmarks_ratio_cortical = Landmarks().get_volume_overlap_from_bounding_box([ref_surf_cortical,surf1_cortical,surf2_cortical],[l_48,l_49,l_50])
                            df['Vol_Overlap_Rabies_Surface_Cortical']=vol_ratio_cortical
                            df['Landmarks_Overlap_Rabies_Surface_Cortical']=landmarks_ratio_cortical

                            dists_rabies_surf_cortical = Surface().get_distance_between_centroids([ref_surf_cortical,surf1_cortical,surf2_cortical],)
                            df['Centriod_Dist_Mean_Rabies_Surface_Cortical']=np.array(dists_rabies_surf_cortical).mean(axis=0)
                            df['Centriod_Dist_Std_Rabies_Surface_Cortical']=np.array(dists_rabies_surf_cortical).std(axis=0)
                            df['Centriod_Dist_CoV_Rabies_Surface_Cortical']=np.array(dists_rabies_surf_cortical).std(axis=0)/np.array(dists_rabies_surf_cortical).mean(axis=0)

                            df['Aligned_Volume_Ratio_Rabies_Cortical']= Surface().get_delunay_volume([ref_surf_cortical,surf1_cortical,surf2_cortical]) /\
                                                                (volumes_rabies_surf_cortical[0]+volumes_rabies_surf_cortical[1]+volumes_rabies_surf_cortical[2])

                        df_main = df_main.append(df)

                                
    #df_main.to_csv(output_path_alignment_analysis+'Rabies_Density_Clustering_Evaluation.csv') 
    df_main.to_csv(output_path_alignment_analysis+'Rabies_Density_Clustering_{}.csv'.format(surf_type)) 
                        
                        
#                         

In [ ]:
l_48.pts,l_49.pts,l_50.pts

## Evaluation of Alignment Methods

In [ ]:
# visualize alignment methods by putting the aligned landmarks together
output_path = output_path_alignment_global_alignment + 'Aligned_data/'
pathlib.Path(output_path).mkdir(exist_ok=True)
for folder in glob.glob(output_path_alignment_global_alignment+'/*'):
    if os.path.isdir(folder) and os.path.basename(folder)!= 'Aligned_data':
        l = Landmarks()
        pia = Surface()
        m1_s1 = Surface()
        barrels = Surface()
        for exp in ['MG48','MG49','MG50']:
            for hem in ['lhs','rhs']:
                for br in ['M1','S1']:
                    l.append_landmarks(Landmarks(folder+'/Landmarks/Density_Clusters/{}_{}_v{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(exp,hem,br)).pts)
                    m1_s1.append(Surface(folder+'/v{}_Ref_Surfaces/{}_{}_v{}_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(br,exp,hem,br)).surface)
                pia.append(Surface(folder+'/Surfaces/{}_{}_pia_open_bottom_voxel_size_50.vtk'.format(exp,hem)).surface)
                barrels.append(Surface(folder+'/Surfaces/{}_{}_s1_hull.vtk'.format(exp,hem)).surface)
                    
        l.write_landmarks(output_path+os.path.basename(folder)+'_rabies.landmarksAscii')
        pia.write_surface_mesh(output_path+os.path.basename(folder)+'_pia.vtk')
        m1_s1.write_surface_mesh(output_path+os.path.basename(folder)+'_m1_s1.vtk')
        barrels.write_surface_mesh(output_path+os.path.basename(folder)+'_s1_hull.vtk')

In [ ]:
# visualize alignment methods by putting the aligned landmarks together
output_path = output_path_alignment_vs1 + 'Aligned_data/'
pathlib.Path(output_path).mkdir(exist_ok=True)
for folder in glob.glob(output_path_alignment_vs1+'/*'):
    if os.path.isdir(folder) and os.path.basename(folder)!= 'Aligned_data':
        l = Landmarks()
        pia = Surface()
        m1_s1 = Surface()
        barrels = Surface()
        for exp in ['MG48','MG49','MG50']:
            for hem in ['lhs','rhs']:
                for br in ['M1','S1']:
                    l.append_landmarks(Landmarks(folder+'/Landmarks/Density_Clusters/{}_{}_v{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(exp,hem,br)).pts)
                    m1_s1.append(Surface(folder+'/v{}_Ref_Surfaces/{}_{}_v{}_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(br,exp,hem,br)).surface)
                pia.append(Surface(folder+'/Surfaces/{}_{}_pia_open_bottom_voxel_size_50.vtk'.format(exp,hem)).surface)
                barrels.append(Surface(folder+'/Surfaces/{}_{}_s1_hull.vtk'.format(exp,hem)).surface)
                    
        l.write_landmarks(output_path+os.path.basename(folder)+'_rabies.landmarksAscii')
        pia.write_surface_mesh(output_path+os.path.basename(folder)+'_pia.vtk')
        m1_s1.write_surface_mesh(output_path+os.path.basename(folder)+'_m1_s1.vtk')
        barrels.write_surface_mesh(output_path+os.path.basename(folder)+'_s1_hull.vtk')

In [ ]:
# visualize alignment methods by putting the aligned landmarks together
output_path = output_path_alignment_vm1 + 'Aligned_data/'
pathlib.Path(output_path).mkdir(exist_ok=True)
for folder in glob.glob(output_path_alignment_vm1+'/*'):
    if os.path.isdir(folder) and os.path.basename(folder)!= 'Aligned_data':
        l = Landmarks()
        pia = Surface()
        m1_s1 = Surface()
        barrels = Surface()
        for exp in ['MG48','MG49','MG50']:
            for hem in ['lhs','rhs']:
                for br in ['M1','S1']:
                    l.append_landmarks(Landmarks(folder+'/Landmarks/Density_Clusters/{}_{}_v{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(exp,hem,br)).pts)
                    m1_s1.append(Surface(folder+'/v{}_Ref_Surfaces/{}_{}_v{}_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(br,exp,hem,br)).surface)
                pia.append(Surface(folder+'/Surfaces/{}_{}_pia_open_bottom_voxel_size_50.vtk'.format(exp,hem)).surface)
                barrels.append(Surface(folder+'/Surfaces/{}_{}_s1_hull.vtk'.format(exp,hem)).surface)
                    
        l.write_landmarks(output_path+os.path.basename(folder)+'_rabies.landmarksAscii')
        pia.write_surface_mesh(output_path+os.path.basename(folder)+'_pia.vtk')
        m1_s1.write_surface_mesh(output_path+os.path.basename(folder)+'_m1_s1.vtk')
        barrels.write_surface_mesh(output_path+os.path.basename(folder)+'_s1_hull.vtk')

In [ ]:
def get_local_avg_axis_from_obbtree(axis_field,pts,center_pt,pia,wm,pia_obb,wm_obb,radius):
    
    nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(axis_field,pts,radius,axis_validation_distance=3000)
    nearby_axes_sg = AmiraSpatialGraph(generic_graph=True)
    unit_vecs = []
    for axis in nearby_axes:
        nearby_axes_sg.graph_data.add_edge(axis[0],axis[1])
        unit_vecs.append(Vectors().get_unit_vec(axis[0],axis[1]))
    #print(len(nearby_axes_sg.graph_data.edge_list))
    #nearby_axes_sg.write_spatial_graph(output_root+'_axis_field_selected.am')
    wm_proj_pt=[]
    avg_uv = np.array(unit_vecs).mean(axis=0)
    end_pt = (avg_uv*-20000 + center_pt)
    wm_proj_pt,depth_wm = wm.get_vector_intersection_pt_from_obb_tree\
                    (wm_obb,end_pt,center_pt,extrapolation_len=0)
    if len(wm_proj_pt) == 0:
        end_pt = (avg_uv*20000 + center_pt)
        wm_proj_pt,depth_wm = wm.get_vector_intersection_pt_from_obb_tree\
                    (wm_obb,end_pt,center_pt,extrapolation_len=0)
        
    
    pia_proj_pt=[]
    end_pt = (avg_uv*20000 + center_pt)
    pia_proj_pt,depth_pia = pia.get_vector_intersection_pt_from_obb_tree\
                    (pia_obb,end_pt,center_pt,extrapolation_len=0)
    if len(pia_proj_pt)==0:
        end_pt = (avg_uv*-20000 + center_pt)
        pia_proj_pt,depth_pia = pia.get_vector_intersection_pt_from_obb_tree\
                    (pia_obb,end_pt,center_pt,extrapolation_len=0)
    
    if len(wm_proj_pt) > 0 and len(pia_proj_pt) > 0:
        return avg_uv,[wm_proj_pt,pia_proj_pt]
    else:
        return avg_uv,None

In [ ]:
def get_local_avg_axis(axis_field,pts,center_pt,pia,wm,radius):
    #print(pts,center_pt,pia.surface.GetNumberOfCells(),wm.surface.GetNumberOfCells())
    #return
    nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(axis_field,pts,radius,axis_validation_distance=3000)
    nearby_axes_sg = AmiraSpatialGraph(generic_graph=True)
    unit_vecs = []
    for axis in nearby_axes:
        nearby_axes_sg.graph_data.add_edge(axis[0],axis[1])
        unit_vecs.append(Vectors().get_unit_vec(axis[0],axis[1]))
    #print(len(nearby_axes_sg.graph_data.edge_list))
    #nearby_axes_sg.write_spatial_graph(output_root+'_axis_field_selected.am')
    wm_proj_pt=[]
    avg_uv = np.array(unit_vecs).mean(axis=0)
    end_pt = (avg_uv*-20000 + center_pt)
    wm_proj_pt,depth_wm = wm.get_vector_intersection_pt\
                    (end_pt,center_pt,extrapolation_len=0)
    if len(wm_proj_pt) == 0:
        end_pt = (avg_uv*20000 + center_pt)
        wm_proj_pt,depth_wm = wm.get_vector_intersection_pt\
                    (end_pt,center_pt,extrapolation_len=0)
        
    
    pia_proj_pt=[]
    end_pt = (avg_uv*20000 + center_pt)
    pia_proj_pt,depth_pia = pia.get_vector_intersection_pt\
                    (end_pt,center_pt,extrapolation_len=0)
    if len(pia_proj_pt)==0:
        end_pt = (avg_uv*-20000 + center_pt)
        pia_proj_pt,depth_pia = pia.get_vector_intersection_pt\
                    (end_pt,center_pt,extrapolation_len=0)
    
    if len(wm_proj_pt) > 0 and len(pia_proj_pt) > 0:
        return avg_uv,[wm_proj_pt,pia_proj_pt]
    else:
        return avg_uv,None

In [ ]:
def get_br_centers(input_path,br='vS1',hem='lhs'):
    #for hem in ['lhs','rhs']:
    centers = []
    for exp in ['MG48','MG49','MG50']:
        center = Landmarks(input_path+ \
                                 '/{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_500_cutoff_0.05_{}_Centroid.landmarksAscii'.format(br,exp,hem,'vM1')).pts
        
        #print(center[0])
        centers.append(center[0])
    centroid = np.array(centers).mean(axis=0)
    dists = []
    for i in range(len(centers)):
        dists.append(distance.euclidean(centers[i],centroid))
    return np.array(dists).mean(axis=0),np.array(dists).std(axis=0)

In [ ]:
def get_centroid_and_dists(centers):
    centroid = np.array(centers).mean(axis=0)
    dists = []
    for i in range(len(centers)):
        dists.append(distance.euclidean(centers[i],centroid))
    return dists,np.array(dists).mean(axis=0),np.array(dists).std(axis=0)

In [ ]:
def get_mean_angle_between_vectors(uvs,unit_vectors_given=True):
    mean_uv = np.array(uvs).mean(axis=0)
    
#     print(mean_uv)
#     print(uvs[0])
#     print(uvs[1])
#     print(uvs[2])
    angles = []
    if unit_vectors_given:
        for i in range(len(uvs)):
            angles.append(Vectors().get_angle_between_unit_vectors(mean_uv,uvs[i]))
    else:
        for i in range(len(uvs)):
            angles.append(Vectors().get_angle_between_vectors(mean_uv,uvs[i]))
        #dists.append(distance.euclidean(centers[i],centroid))
    return angles,np.array(angles).mean(axis=0),np.array(angles).std(axis=0),mean_uv

In [ ]:
def get_surf_centers(pia_list,wm_list):
    centers = []
    for i in range(len(pia_list)): 
        pia_wm = Surface()
        pia_wm.append(pia_list[i].surface)
        pia_wm.append(wm_list[i].surface)
        centers.append(pia_wm.get_center_of_mass())
    
    return centers

In [ ]:
def get_rabies_surf_centers(surf_list):
    centers = []
    for surf in surf_list:
        centers.append(surf.get_center_of_mass())
    return centers
    

In [ ]:
def get_rabies_centers(landmarks_list):
    centers = []
    for l in landmarks_list:
        centers.append(np.array(l.pts).mean(axis=0))
    return centers

In [ ]:
def get_closest_vector(ref_pca_comp,pca_coms_list):
    closest_axis = []
    min_angle = 10000
    closest_ind = 0
    for i in range(1,4):
        angle = Vectors().get_angle_between_vectors(ref_pca_comp,[pca_coms_list[0],pca_coms_list[i]],)
        #print('angle ',angle)
        if angle < min_angle:
            min_angle = angle
            closest_axis = [pca_coms_list[0],pca_coms_list[i]]
            closest_ind = i
    #print('closest_ind is {}'.format(closest_ind))
    return closest_axis

In [ ]:
def get_angles_between_pcas(pcas_animal1,pcas_animal2,pcas_animal3):
    pca0_angles = []
    pca1_angles= []
    pca2_angles= []
    
    # use the nearest pca from other animals to determine the angle
    # since pcas across animasl are not in the same order
    pca0 = [pcas_animal1[0],pcas_animal1[1]]
    pca0_list = []
    pca0_list.append([pcas_animal1[0],pcas_animal1[1]])
    #print('for pca 1')
    #print('looking at animal 1')
    pca0_list.append(get_closest_vector(pca0,pcas_animal2))
    #print('looking at animal 2')
    pca0_list.append(get_closest_vector(pca0,pcas_animal3))
    
    pca0_angles,bla,ble,blu=get_mean_angle_between_vectors(pca0_list,unit_vectors_given=False)
    
    #print('for pca 2')
    pca1 = [pcas_animal1[0],pcas_animal1[2]]
    pca1_list = []
    pca1_list.append([pcas_animal1[0],pcas_animal1[2]])
    #print('looking at animal 1')
    pca1_list.append(get_closest_vector(pca1,pcas_animal2))
    #print('looking at animal 2')
    pca1_list.append(get_closest_vector(pca1,pcas_animal3))
    
    pca1_angles,bla,ble,blu=get_mean_angle_between_vectors(pca1_list,unit_vectors_given=False)
    
    #print('for pca 3')
    pca2 = [pcas_animal1[0],pcas_animal1[3]]
    pca2_list = []
    #print('looking at animal 2')
    pca2_list.append([pcas_animal1[0],pcas_animal1[3]])
    #print('looking at animal 2')
    pca2_list.append(get_closest_vector(pca2,pcas_animal2))
    pca2_list.append(get_closest_vector(pca2,pcas_animal3))
    
    pca2_angles,bla,ble,blu=get_mean_angle_between_vectors(pca2_list,unit_vectors_given=False)
    
    
    
    return pca0_angles,pca1_angles,pca2_angles
                                

In [ ]:
def get_alignment_stats(input_path_root,outer_alignment_type,inner_alignment_type,surf_type,hem,debug=True):
    df = pd.DataFrame()
    #for hem in ['lhs',]:
    #surf_type = 'vM1'
    #hem = 'lhs'
    #debug=True
    df['Outer_Alignment_Type'] = [outer_alignment_type]
    df['Inner_Alignment_Type'] = [inner_alignment_type]
    df['Hem'] = [hem]
    df['BR'] = [surf_type]
    
    
    output_tmp = input_path_root + '/tmp/'
    pathlib.Path(output_tmp).mkdir(exist_ok=True)
    output_tmp = output_tmp + surf_type+'/'
    pathlib.Path(output_tmp).mkdir(exist_ok=True)
    output_tmp = output_tmp + hem+'/'
    pathlib.Path(output_tmp).mkdir(exist_ok=True)
    
    if surf_type is 'vM1' or surf_type is 'vS1':
        ref_axis_field_untrimmed = AmiraSpatialGraph(input_path_root+'{}_Ref_Surfaces_Axis_Fields'.format(surf_type)+\
                            '/MG48_{}_{}_axis_field_surf_res_50_voxel_size_500_cutoff_0.1.am'.format(hem,surf_type),generic_graph=True)
        axis_field1_untrimmed = AmiraSpatialGraph(input_path_root+'{}_Ref_Surfaces_Axis_Fields'.format(surf_type)+\
                            '/MG49_{}_{}_axis_field_surf_res_50_voxel_size_500_cutoff_0.1.am'.format(hem,surf_type),generic_graph=True)
        axis_field2_untrimmed = AmiraSpatialGraph(input_path_root+'{}_Ref_Surfaces_Axis_Fields'.format(surf_type)+\
                            '/MG50_{}_{}_axis_field_surf_res_50_voxel_size_500_cutoff_0.1.am'.format(hem,surf_type),generic_graph=True)
        if surf_type is 'vM1':
            br = 'vm1'
        else:
            br='vs1'
        ref_pia = Surface(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG48_{}_{}_pia_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(hem,br))
        pia1 = Surface(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG49_{}_{}_pia_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(hem,br))
        pia2 = Surface(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG50_{}_{}_pia_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(hem,br),)
        ref_wm = Surface(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG48_{}_{}_wm_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(hem,br),)
        wm1 = Surface(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG49_{}_{}_wm_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(hem,br),)
        wm2 = Surface(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG50_{}_{}_wm_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(hem,br),)

        ref_hull = input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG48_{}_{}_hull_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(hem,br)
        hull1 = input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG49_{}_{}_hull_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(hem,br)
        hull2 = input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG50_{}_{}_hull_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(hem,br)

        ref_rabies_landmarks = Landmarks(input_path_root+'Landmarks/Density_Clusters/'+\
                                'MG48_{}_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(hem,surf_type))
        rabies_landmarks1 = Landmarks(input_path_root+'Landmarks/Density_Clusters/'+\
                                'MG49_{}_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(hem,surf_type))
        rabies_landmarks2 = Landmarks(input_path_root+'Landmarks/Density_Clusters/'+\
                                'MG50_{}_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(hem,surf_type))

        ref_rabies_surf = Surface(input_path_root+'Landmarks/Density_Clusters/'+\
                                'MG48_{}_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format(hem,surf_type))
        rabies_surf1 = Surface(input_path_root+'Landmarks/Density_Clusters/'+\
                                'MG49_{}_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format(hem,surf_type))
        rabies_surf2 = Surface(input_path_root+'Landmarks/Density_Clusters/'+\
                                'MG50_{}_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format(hem,surf_type))

        ref_axis_field,wm_ptsb,pia_ptsb,missing_axis_fieldb = ref_pia.get_axis_field_within_instersecting_surface_with_trimming\
            (ref_pia.surface,ref_wm.surface,ref_axis_field_untrimmed.graph_data.edge_list,vec_extension_offset=1)
        axis_field1,wm_ptsb,pia_ptsb,missing_axis_fieldb = pia1.get_axis_field_within_instersecting_surface_with_trimming\
            (pia1.surface,wm2.surface,axis_field1_untrimmed.graph_data.edge_list,vec_extension_offset=1)
        axis_field2,wm_ptsb,pia_ptsb,missing_axis_fieldb = pia2.get_axis_field_within_instersecting_surface_with_trimming\
            (pia2.surface,wm2.surface,axis_field2_untrimmed.graph_data.edge_list,vec_extension_offset=1)

        ref_pca_0  = AmiraSpatialGraph(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG48_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0.am'.format(hem,),\
                          generic_graph=True).graph_data.edge_list[0]
        ref_pca_1  = AmiraSpatialGraph(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG48_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1.am'.format(hem,),\
                          generic_graph=True).graph_data.edge_list[0]
        ref_pca_2  = AmiraSpatialGraph(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG48_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2.am'.format(hem,),\
                          generic_graph=True).graph_data.edge_list[0]

        a1_pca_0  = AmiraSpatialGraph(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG49_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0.am'.format(hem,),\
                          generic_graph=True).graph_data.edge_list[0]
        a1_pca_1  = AmiraSpatialGraph(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG49_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1.am'.format(hem,),\
                          generic_graph=True).graph_data.edge_list[0]
        a1_pca_2  = AmiraSpatialGraph(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG49_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2.am'.format(hem,),\
                          generic_graph=True).graph_data.edge_list[0]

        a2_pca_0  = AmiraSpatialGraph(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG50_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0.am'.format(hem,),\
                          generic_graph=True).graph_data.edge_list[0]
        a2_pca_1  = AmiraSpatialGraph(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG50_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1.am'.format(hem,),\
                          generic_graph=True).graph_data.edge_list[0]
        a2_pca_2  = AmiraSpatialGraph(input_path_root+'{}_Ref_Surfaces'.format(surf_type)+\
                            '/MG50_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2.am'.format(hem,),\
                          generic_graph=True).graph_data.edge_list[0]

        pca0_angles,pca1_angles,pca2_angles = get_angles_between_pcas(\
                                [ref_pca_0[0],ref_pca_0[1],ref_pca_1[1],ref_pca_2[1]],\
                                [a1_pca_0[0],a1_pca_0[1],a1_pca_1[1],a1_pca_2[1]],\
                                [a2_pca_0[0],a2_pca_0[1],a2_pca_1[1],a2_pca_2[1]])
        #print(pca0_andgles_mean)
        #ref_axis_field.write_spatial_graph(output_root+'bla.am')
    
    elif surf_type is 'WB':
        # scale down spatial graphs for whole brain , otherwise it will take forever
        ref_axis_field = AmiraSpatialGraph(input_path_root+'Axis_Fields'+\
                            '/MG48_{}_wm_to_pia_axis_field_voxel_size_50.am'.format(hem),generic_graph=True)
        axis_field1 = AmiraSpatialGraph(input_path_root+'Axis_Fields'+\
                            '/MG49_{}_wm_to_pia_axis_field_voxel_size_50.am'.format(hem),generic_graph=True)
        axis_field2 = AmiraSpatialGraph(input_path_root+'Axis_Fields'+\
                            '/MG50_{}_wm_to_pia_axis_field_voxel_size_50.am'.format(hem),generic_graph=True)

#         step_size = 20
#         ref_axis_field = AmiraSpatialGraph(generic_graph=True)
#         for i in range(0,len(ref_axis_field_original.graph_data.edge_list),step_size):
#             edge = ref_axis_field_original.graph_data.edge_list[i]
#             ref_axis_field.graph_data.add_edge(edge[0],edge[1])
        
#         axis_field1 = AmiraSpatialGraph(generic_graph=True)
#         for i in range(0,len(axis_field1_original.graph_data.edge_list),step_size):
#             edge = axis_field1_original.graph_data.edge_list[i]
#             axis_field1.graph_data.add_edge(edge[0],edge[1])
        
#         axis_field2 = AmiraSpatialGraph(generic_graph=True)
#         for i in range(0,len(axis_field2_original.graph_data.edge_list),step_size):
#             edge = axis_field2_original.graph_data.edge_list[i]
#             axis_field2.graph_data.add_edge(edge[0],edge[1])

#         ref_axis_field.write_spatial_graph(output_tmp+'scaled_down_48.am')
#         axis_field1.write_spatial_graph(output_tmp+'scaled_down_49.am')
#         axis_field2.write_spatial_graph(output_tmp+'scaled_down_50.am')
#         print(len(ref_axis_field.graph_data.edge_list))
        ref_pia = Surface(input_path_root+'Surfaces'+\
                            '/MG48_{}_pia_open_bottom_voxel_size_50.vtk'.format(hem))
        pia1 = Surface(input_path_root+'Surfaces'+\
                            '/MG49_{}_pia_open_bottom_voxel_size_50.vtk'.format(hem))
        pia2 = Surface(input_path_root+'Surfaces'+\
                            '/MG50_{}_pia_open_bottom_voxel_size_50.vtk'.format(hem),)

        ref_wm = Surface(input_path_root+'Surfaces'+\
                            '/MG48_{}_WM_open_bottom_voxel_size_50.vtk'.format(hem),)
        wm1 = Surface(input_path_root+'Surfaces'+\
                            '/MG49_{}_WM_open_bottom_voxel_size_50.vtk'.format(hem),)
        wm2 = Surface(input_path_root+'Surfaces'+\
                            '/MG50_{}_WM_open_bottom_voxel_size_50.vtk'.format(hem),)

        ref_pca_pts = Alignment().get_pca_for_surf(ref_pia.surface,output_path=None,ref_coronal=True,lhs=(hem=='lhs'))
        pca_pts1 = Alignment().get_pca_for_surf(pia1.surface,output_path=None,ref_coronal=False,lhs=(hem=='lhs'))
        pca_pts2 = Alignment().get_pca_for_surf(pia2.surface,output_path=None,ref_coronal=False,lhs=(hem=='lhs'))

        if debug:
            ref_pca_sg = AmiraSpatialGraph(generic_graph=True)
            ref_pca_sg.graph_data.add_edge(ref_pca_pts[0],ref_pca_pts[1])
            ref_pca_sg.graph_data.add_edge(ref_pca_pts[0],ref_pca_pts[2])
            ref_pca_sg.graph_data.add_edge(ref_pca_pts[0],ref_pca_pts[3])
            ref_pca_sg.write_spatial_graph(output_tmp+'ref_pca_sg.am')
            pca1_sg = AmiraSpatialGraph(generic_graph=True)
            pca1_sg.graph_data.add_edge(pca_pts1[0],pca_pts1[1])
            pca1_sg.graph_data.add_edge(pca_pts1[0],pca_pts1[2])
            pca1_sg.graph_data.add_edge(pca_pts1[0],pca_pts1[3])
            pca1_sg.write_spatial_graph(output_tmp+'pca_sg1.am')
            pca2_sg = AmiraSpatialGraph(generic_graph=True)
            pca2_sg.graph_data.add_edge(pca_pts2[0],pca_pts2[1])
            pca2_sg.graph_data.add_edge(pca_pts2[0],pca_pts2[2])
            pca2_sg.graph_data.add_edge(pca_pts2[0],pca_pts2[3])
            pca2_sg.write_spatial_graph(output_tmp+'pca_sg2.am')
        pca0_angles,pca1_angles,pca2_angles = get_angles_between_pcas(\
                                [ref_pca_pts[0],ref_pca_pts[1],ref_pca_pts[2],ref_pca_pts[3]],\
                                [pca_pts1[0],pca_pts1[1],pca_pts1[2],pca_pts1[3]],\
                                [pca_pts2[0],pca_pts2[1],pca_pts2[2],pca_pts2[3]])
        #print(pca0_andgles_mean)

    #return df
    print('got input')
    
    #########################################################################################
    ##################### Pia WM precision ##################################################
    pia_errors_mean = []
    pia_errors_std = []
    pia_errors_dists = []
    pia_errors_dists_means = []
    pia_errors_dists_stds = []

    wm_errors_mean = []
    wm_errors_std = []
    wm_errors_dists = []
    wm_errors_dists_means = []
    wm_errors_dists_stds = []

    pia_landmarks = []
    wm_landmarks = []
    #avg_angles = []
    angles_means = []
    angles_stds = []
    overlapping_axes = []
    
    axis_count = 0
    pia1_obb = vtk.vtkOBBTree()
    pia1_obb.SetDataSet(pia1.surface)
    pia1_obb.BuildLocator()
    pia2_obb = vtk.vtkOBBTree()
    pia2_obb.SetDataSet(pia2.surface)
    pia2_obb.BuildLocator()
    
    wm1_obb = vtk.vtkOBBTree()
    wm1_obb.SetDataSet(wm1.surface)
    wm1_obb.BuildLocator()
    wm2_obb = vtk.vtkOBBTree()
    wm2_obb.SetDataSet(wm2.surface)
    wm2_obb.BuildLocator()
    
    for axis in ref_axis_field.graph_data.edge_list:
        #print(axis_count)
        
        # find the avg axis field at this point by finding intersection with wm and axis at that wm pt
#         axis1,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = Vectors().get_nearest_axis_to_pt(axis[0],\
#                                                                     axis_field1.graph_data.edge_list,axis_validation_distance=3000)
#         axis2,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = Vectors().get_nearest_axis_to_pt(axis[0],
#                                                                     axis_field2.graph_data.edge_list,axis_validation_distance=3000)
        center_pt = np.array(axis).mean(axis=0)
        uv1,axis1 = get_local_avg_axis_from_obbtree(axis_field1.graph_data.edge_list,[center_pt],center_pt,pia1,wm1,pia1_obb,wm1_obb,800)
        uv2,axis2 = get_local_avg_axis_from_obbtree(axis_field2.graph_data.edge_list,[center_pt],center_pt,pia2,wm2,pia2_obb,wm2_obb,800)
        uv = Vectors().get_unit_vec(axis[0],axis[1])
        #print(axis1,axis2)
#         if axis1[0][0] != 0 and axis1[0][1] != 0 and axis1[0][2]!= 0 \
#              and axis2[0][0] != 0 and axis2[0][1] != 0 and axis2[0][2] != 0:
#             uv1 = Vectors().get_unit_vec(axis1[0],axis1[1])
#             uv2 = Vectors().get_unit_vec(axis2[0],axis2[1])
        if axis1 is not None and axis2 is not None:
            # found corresponding axes
            axis_count = axis_count+1
            overlapping_axes.append(axis)
            pia_pts = []
            wm_pts = []
            wm_pts.append(axis[0])
            wm_pts.append(axis1[0])
            wm_pts.append(axis2[0])
            pia_pts.append(axis[1])
            pia_pts.append(axis1[1])
            pia_pts.append(axis2[1])
            #print(pia_pts,wm_pts)
            pia_errors_mean.append(np.array(pia_pts).mean(axis=0))
            pia_landmarks.append(np.array(pia_pts).mean(axis=0))
            pia_errors_std.append(np.array(pia_pts).std(axis=0))

            wm_errors_mean.append(np.array(wm_pts).mean(axis=0))
            wm_landmarks.append(np.array(wm_pts).mean(axis=0))
            wm_errors_std.append(np.array(wm_pts).std(axis=0))

            pia_dists,pia_dist_mean,pia_dist_std = get_centroid_and_dists(pia_pts)
            wm_dists,wm_dist_mean,wm_dist_std = get_centroid_and_dists(wm_pts)
            #print(pia_dists,wm_dists)
            pia_errors_dists_means.append(pia_dist_mean)
            pia_errors_dists_stds.append(pia_dist_std)
            wm_errors_dists_means.append(wm_dist_mean)
            wm_errors_dists_stds.append(wm_dist_std)

            angles,angles_mean,angles_std,mean_uv = get_mean_angle_between_vectors([uv,uv1,uv2])
            angles_means.append(angles_mean)
            angles_stds.append(angles_std)

            if debug:# for checking angle stuff
                sg1 = AmiraSpatialGraph(generic_graph=True)
                sg1.graph_data.add_edge(axis[0],axis[1])
                sg1.write_spatial_graph(output_tmp+'edge1.am')

                sg2 = AmiraSpatialGraph(generic_graph=True)
                sg2.graph_data.add_edge(axis1[0],axis1[1])
                sg2.write_spatial_graph(output_tmp+'edge2.am')

                sg2 = AmiraSpatialGraph(generic_graph=True)
                sg2.graph_data.add_edge(axis2[0],axis2[1])
                sg2.write_spatial_graph(output_tmp+'edge3.am')

                sgm = AmiraSpatialGraph(generic_graph=True)
                sgm.graph_data.add_edge(center_pt,mean_uv*10000+center_pt)
                sgm.write_spatial_graph(output_root+'edgemean.am')
                #print(angles[0],angles[1],angles[2])
                Landmarks(pts=[center_pt]).write_landmarks(output_tmp+'axis_center.landmarksAscii')

                Landmarks(pts=pia_landmarks).write_landmarks(output_tmp+'pia_avg_pts.landmarksAscii')
                Landmarks(pts=wm_landmarks).write_landmarks(output_tmp+'wm_avg_pts.landmarksAscii')

    df['Pia_Precision_Mean'] = [np.array(pia_errors_dists_means).mean(axis=0)]
    df['Pia_Precision_Std'] = [np.array(pia_errors_dists_stds).mean(axis=0)]
    df['Pia_Error_Dist_SEM'] = [np.array(pia_errors_dists_means).std(axis=0)]
    #df['Pia_Error_Dist_CoV'] = [np.array(pia_errors_dists).std(axis=0) / np.array(pia_errors_dists).mean(axis=0)]
    #df['Pia_Error_Dist_SEM'] = [np.array(pia_errors_dists).std(axis=0) / np.sqrt(len(pia_errors_dists))]

    df['WM_Precision_Mean'] = [np.array(wm_errors_dists_means).mean(axis=0)]
    df['WM_Precision_Std'] = [np.array(wm_errors_dists_stds).mean(axis=0)]
    df['WM_Error_Dist_SEM'] = [np.array(wm_errors_dists_means).std(axis=0)]
    #df['WM_Error_Dist_CoV'] = [np.array(wm_errors_dists).std(axis=0) / np.array(wm_errors_dists).mean(axis=0)]
    #df['WM_Error_Dist_SEM'] = [np.array(wm_errors_dists).std(axis=0) / np.sqrt(len(wm_errors_dists))]

    df['Pia_Error_Ellipsoid_Mean'] = [np.array(pia_errors_std).mean(axis=0)]
    df['Pia_Error_Ellipsoid_Mean_Norm'] = [np.linalg.norm(np.array(pia_errors_std).mean(axis=0))]
    df['WM_Error_Ellipsoid_Mean'] = [np.array(wm_errors_std).mean(axis=0)]
    df['WM_Error_Ellipsoid_Mean_Norm'] = [np.linalg.norm(np.array(wm_errors_std).mean(axis=0))]
    
    df['Pia_Error_Ellipsoid_Std'] = [np.array(pia_errors_std).std(axis=0)]
    df['Pia_Error_Ellipsoid_Std_Norm'] = [np.linalg.norm(np.array(pia_errors_std).std(axis=0))]
    df['Pia_Error_Ellipsoid_Std'] = [np.array(pia_errors_std).std(axis=0)]
    df['Pia_Error_Ellipsoid_Std_Norm'] = [np.linalg.norm(np.array(pia_errors_std).std(axis=0))]
    
    df['Angular_Precision_Mean'] = [np.array(angles_means).mean(axis=0)]
    df['Angular_Precision_Std'] = [np.array(angles_stds).mean(axis=0)]
    df['Angular_Error_SEM'] = [np.array(angles_means).std(axis=0)]
    
    print('pia/wm done')
    
    #########################################################################################
    ##################### Overlap measures ##################################################
    #Landmarks(pts= pia_landmarks).write_landmarks(output_root+'pia.landmarksAscii')  
    #Landmarks(pts = wm_landmarks).write_landmarks(output_root+'wm.landmarksAscii')
    overlap_boundary_pts = []
    for pt in pia_landmarks:
        overlap_boundary_pts.append(pt)
    for pt in wm_landmarks:
        overlap_boundary_pts.append(pt)

    if debug:
        overlap_hull = Surface(pts=overlap_boundary_pts).create_delunay_surface_3d(output_filename=output_tmp+'overlap_delu.vtk',return_hull=True)
    else:
        overlap_hull = Surface(pts=overlap_boundary_pts).create_delunay_surface_3d(return_hull=True)
    overlap_vol = Surface(polydata=overlap_hull).get_surface_properties(prop='volume')

    # find overall volume
#     overall_surf = Surface()
#     overall_surf.append(ref_pia.surface)
#     overall_surf.append(pia1.surface)
#     overall_surf.append(pia2.surface)
#     overall_surf.append(ref_wm.surface)
#     overall_surf.append(wm1.surface)
#     overall_surf.append(wm2.surface)
    overall_surf_pts = []
    
    for edge in ref_axis_field.graph_data.edge_list:
        overall_surf_pts.append(edge[0])
        overall_surf_pts.append(edge[1])
    for edge in axis_field1.graph_data.edge_list:
        overall_surf_pts.append(edge[0])
        overall_surf_pts.append(edge[1])
    for edge in axis_field2.graph_data.edge_list:
        overall_surf_pts.append(edge[0])
        overall_surf_pts.append(edge[1])
        
    if debug:
        Landmarks(pts=overall_surf_pts).write_landmarks(output_tmp+'overall_pts.landmarksAscii')
        overall_hull = Surface(pts=overall_surf_pts).create_delunay_surface_3d(output_filename=output_tmp+'overall_delu.vtk',return_hull=True)
    else:
        overall_hull = Surface(pts=overall_surf_pts).create_delunay_surface_3d(return_hull=True)
    overall_vol = Surface(polydata=overall_hull).get_surface_properties(prop='volume')

    df['Overlap_Volume'] = [overlap_vol]
    df['Overall_Volume'] = [overall_vol]
    df['Volume_Overlap_Percentage'] = [overlap_vol / overall_vol]
    print('found overlap volume')
    # Get Landmarks overlap if needed
    if surf_type is 'vM1' or surf_type is 'vS1':
        rabies_landmarks_all = Landmarks()
        rabies_landmarks_all.append_landmarks(ref_rabies_landmarks.pts)
        rabies_landmarks_all.append_landmarks(rabies_landmarks1.pts)
        rabies_landmarks_all.append_landmarks(rabies_landmarks2.pts)

        overlap_landmarks = rabies_landmarks_all.get_landmarks_within_given_surface_using_vtk(overlap_hull)
        Landmarks(pts=overlap_landmarks).write_landmarks(output_root+'rabies_overlap.landmarksAscii')
        df['Overlap_Rabies_landmarks'] = [len(overlap_landmarks)]
        df['Overall_Rabies_landmarks'] = [len(rabies_landmarks_all.pts)]
        df['Rabies_Landmarks_Overlap_Percentage'] = [len(overlap_landmarks) / len(rabies_landmarks_all.pts)]

    print('overlap done')
    ######################################################################################################
    ############################## Center and PCA precision ##############################################
    centers = get_surf_centers([ref_pia,pia1,pia2],[ref_wm,wm1,wm2])
    dists,dists_mean,dists_std = get_centroid_and_dists(centers)

    df['Surf_Center_Ellipsoid'] = [np.array(centers).std(axis=0)]
    df['Surf_Center_Ellipsoid_Norm'] = [np.linalg.norm(np.array(centers).std(axis=0))]
    df['Surf_Center_Precision_Mean'] = [dists_mean]
    df['Surf_Center_Precision_Std'] = [dists_std]
    df['Surf_Center_Error_Dists_SEM'] = [dists_std/np.sqrt(3)]

    if surf_type is 'vM1' or surf_type is 'vS1':
        centers = get_rabies_surf_centers([ref_rabies_surf,rabies_surf1,rabies_surf2])
        dists,dists_mean,dists_std = get_centroid_and_dists(centers)

        df['Rabies_Surf_Center_Ellipsoid'] = [np.array(centers).std(axis=0)]
        df['Rabies_Surf_Center_Ellipsoid_Norm'] = [np.linalg.norm(np.array(centers).std(axis=0))]
        df['Rabies_Surf_Center_Dists_Mean'] = [dists_mean]
        df['Rabies_Surf_Center_Dists_Std'] = [dists_std]
        df['Rabies_Surf_Center_Error_Dists_SEM'] = [dists_std/np.sqrt(3)]

        centers = get_rabies_centers([ref_rabies_landmarks,rabies_landmarks1,rabies_landmarks2])
        dists,dists_mean,dists_std = get_centroid_and_dists(centers)

        df['Rabies_Center_Ellipsoid'] = [np.array(centers).std(axis=0)]
        df['Rabies_Center_Ellipsoid_Norm'] = [np.linalg.norm(np.array(centers).std(axis=0))]
        df['Rabies_Center_Precision_Mean'] = [dists_mean]
        df['Rabies_Center_Precision_Std'] = [dists_std]
        df['Rabies_Center_Error_Dists_SEM'] = [dists_std/np.sqrt(3)]

    # pca orientation
    df['PCA_0_Angle_Between_Axes_Mean'] = [np.array(pca0_angles).mean(axis=0)]
    df['PCA_0_Angle_Between_Axes_Std'] = [np.array(pca0_angles).std(axis=0)]
    #df['Angle_Between_Axes_CoV'] = [np.array(avg_angles).std(axis=0) / np.array(avg_angles).mean(axis=0)]
    #df['PCA_0_Angle_Between_Axes_SEM'] = [np.array(pca0_andgles_mean).std(axis=0)]
    
    df['PCA_1_Angle_Between_Axes_Mean'] = [np.array(pca1_angles).mean(axis=0)]
    df['PCA_1_Angle_Between_Axes_Std'] = [np.array(pca1_angles).std(axis=0)]
    #df['Angle_Between_Axes_CoV'] = [np.array(avg_angles).std(axis=0) / np.array(avg_angles).mean(axis=0)]
    #df['PCA_1_Angle_Between_Axes_SEM'] = [np.array(pca1_andgles_mean).std(axis=0)]
    
    df['PCA_2_Angle_Between_Axes_Mean'] = [np.array(pca2_angles).mean(axis=0)]
    df['PCA_2_Angle_Between_Axes_Std'] = [np.array(pca2_angles).std(axis=0)]
    #df['Angle_Between_Axes_CoV'] = [np.array(avg_angles).std(axis=0) / np.array(avg_angles).mean(axis=0)]
    #df['PCA_2_Angle_Between_Axes_SEM'] = [np.array(pca2_andgles_mean).std(axis=0)]
    
    #angles,angles_mean,angles_std,meanuv = get_angles_between_pcas([],[],[])
    df['PCA_Angular_Precision_Mean'] = [np.array([np.array(pca0_angles).mean(axis=0),\
                                                  np.array(pca1_angles).mean(axis=0),np.array(pca2_angles).mean(axis=0)]).mean()]
    df['PCA_Angular_Precision_Std'] = [np.array([np.array(pca0_angles).std(axis=0),\
                                                 np.array(pca1_angles).std(axis=0),np.array(pca2_angles).std(axis=0)]).mean()]
    df['PCA_Angular_Precision_SEM'] = [np.array([np.array(pca0_angles).mean(axis=0),\
                                                 np.array(pca1_angles).mean(axis=0),np.array(pca2_angles).mean(axis=0)]).std()]
    
    print('center pca done')
    ##############################################################################
    ###################### Barrels ###############################################
    
    sg1 = AmiraSpatialGraph(input_path_root+\
                                   'Surfaces/'+'MG49_'+hem+'_barrels_50.am',barrel_projections_present=True)
    sg2 = AmiraSpatialGraph(input_path_root+\
                                       'Surfaces/'+'MG50_'+hem+'_barrels_50.am',barrel_projections_present=True)
    s1_hull = Surface(input_path_root+'Surfaces/'+'MG49_'+hem+'_s1_hull.vtk')
    s2_hull = Surface(input_path_root+'Surfaces/'+'MG50_'+hem+'_s1_hull.vtk')
    dists,dists_mean,dists_std = get_centroid_and_dists([s1_hull.get_center_of_mass(),s2_hull.get_center_of_mass()])
    df['Barrel_Cortex_Center_Ellipsoid'] = [np.array([np.array(s1_hull.get_center_of_mass()),\
                                                              np.array(s2_hull.get_center_of_mass())]).std(axis=0)]
    df['Barrel_Cortex_Center_Ellipsoid_Norm'] = [np.linalg.norm(np.array([np.array(s1_hull.get_center_of_mass()),\
                                                              np.array(s2_hull.get_center_of_mass())]).std(axis=0))]
    df['Barrel_Cortex_Center_Precision_Mean'] = [dists_mean]
    df['Barrel_Cortex_Center_Precision_Std'] = [dists_std]
    df['Barrel_Cortex_Center_Error_Dists_SEM'] = [dists_std/np.sqrt(2)]
    
    # Get the barrel pia and wm errors
    matched_barrels = MatchBarrels(sg1.barrels,sg2.barrels,)
    error_df,pia_centers_stds,wm_centers_stds,barrel_centers_stds,column_centers_stds  = (matched_barrels.get_errors(exp_name='lhs',alignment_type='alignment_type'))
    
    df['Barrels_Pia_Precision_Mean'] = [ np.array(error_df['Pia_Errors'].values).mean(axis=0)/2 ]
    df['Barrels_Pia_Precision_std'] = [ np.array(error_df['Pia_Errors'].values).std(axis=0)/2 ]
    df['Barrels_WM_Precision_Mean'] = [ np.array(error_df['WM_Errors'].values).mean(axis=0)/2 ]
    df['Barrels_WM_Precision_Std'] = [ np.array(error_df['WM_Errors'].values).std(axis=0)/2 ]
    df['Barrels_Barrel_Center_Precision_Mean'] = [ np.array(error_df['Barrel_Center_Errors'].values).mean(axis=0)/2 ]
    df['Barrels_Barrel_Center_Precision_Std'] = [ np.array(error_df['Barrel_Center_Errors'].values).std(axis=0)/2 ]
    df['Barrels_Column_Center_Precision_Mean'] = [ np.array(error_df['Column_Center_Errors'].values).mean(axis=0)/2 ]
    df['Barrels_Column_Center_Precision_Std'] = [ np.array(error_df['Column_Center_Errors'].values).std(axis=0)/2 ]
    
    df['Barrels_Pia_Error_Ellipsoid_Mean'] = [np.array(pia_centers_stds).mean(axis=0)]
    df['Barrels_Pia_Error_Ellipsoid_Mean_Norm'] = [np.linalg.norm(np.array(pia_centers_stds).mean(axis=0))]
    df['Barrels_Pia_Error_Ellipsoid_Std'] = [np.array(pia_centers_stds).std(axis=0)]
    df['Barrels_Pia_Error_Ellipsoid_Std_Norm'] = [np.linalg.norm(np.array(pia_centers_stds).std(axis=0))]
    
    df['Barrels_WM_Error_Ellipsoid_Mean'] = [np.array(wm_centers_stds).mean(axis=0)]
    df['Barrels_WM_Error_Ellipsoid_Mean_Norm'] = [np.linalg.norm(np.array(wm_centers_stds).mean(axis=0))]
    df['Barrels_WM_Error_Ellipsoid_Std'] = [np.array(wm_centers_stds).std(axis=0)]
    df['Barrels_WM_Error_Ellipsoid_Std_Norm'] = [np.linalg.norm(np.array(wm_centers_stds).std(axis=0))]
    
    df['Barrels_Barrel_Centers_Error_Ellipsoid_Mean'] = [np.array(barrel_centers_stds).mean(axis=0)]
    df['Barrels_Barrel_Centers_Error_Ellipsoid_Mean_Norm'] = [np.linalg.norm(np.array(barrel_centers_stds).mean(axis=0))]
    df['Barrels_Barrel_Centers_Error_Ellipsoid_Std'] = [np.array(barrel_centers_stds).std(axis=0)]
    df['Barrels_Barrel_Centers_Error_Ellipsoid_Std_Norm'] = [np.linalg.norm(np.array(barrel_centers_stds).std(axis=0))]
    
    df['Barrels_Column_Error_Ellipsoid_Mean'] = [np.array(column_centers_stds).mean(axis=0)]
    df['Barrels_Column_Error_Ellipsoid_Mean_Norm'] = [np.linalg.norm(np.array(column_centers_stds).mean(axis=0))]
    df['Barrels_Column_Error_Ellipsoid_Std'] = [np.array(column_centers_stds).std(axis=0)]
    df['Barrels_Column_Error_Ellipsoid_Std_Norm'] = [np.linalg.norm(np.array(column_centers_stds).std(axis=0))]
    
    # get angular precision
    barrels_angles_means = []
    barrels_angles_stds = []
    ref_barrel_axes,barrel_axes = matched_barrels.get_matching_axes()
    for i in range(len(ref_barrel_axes)):
        angles,angles_mean,angles_std,mean_uv = get_mean_angle_between_vectors\
                                    ([ref_barrel_axes[i],barrel_axes[i]],unit_vectors_given=False)
        barrels_angles_means.append(angles_mean)
        barrels_angles_stds.append(angles_std)
    
    df['Barrels_Angular_Precision_Mean'] = [np.array(barrels_angles_means).mean(axis=0)]
    df['Barrels_Angular_Precision_Std'] = [np.array(barrels_angles_stds).mean(axis=0)]
    df['Barrels_Angular_Error_SEM'] = [np.array(barrels_angles_means).std(axis=0)]
    
    if debug:
        Landmarks(pts=[s1_hull.get_center_of_mass()]).write_landmarks(output_tmp+'barrels_centoid1.landmarksAscii')
        Landmarks(pts=[s2_hull.get_center_of_mass()]).write_landmarks(output_tmp+'barrels_centoid2.landmarksAscii')
    
    print('barrels done')
    
    df['Number_Of_Axis_Used'] = [axis_count]
    
    return df

In [ ]:
# Params: 
# For each surf type: center, pca, pia-wm error ellipsoids and orientation, volume overlap by bounding box, 
# rabies overlap with atlas areas, 
surf_res = 50
df_main = pd.DataFrame()
outer_alignment_type = 'Global_Alignment'
#for folder in ['Global_Alignment']:#['Global_Alignment','vS1','vM1','S1BF']:
for alignment_type in ['Using_Local_Surfaces','Using_Local_Rabies_Surfaces','Using_Local_Centers','Using_Surface','Using_M1_SBF','Using_Center']:#[]
    if alignment_type == 'Using_Center':
        input_path_root = output_path_alignment_global_alignment_centers
    if alignment_type == 'Using_Local_Centers':
        input_path_root = output_path_alignment_global_alignment_local_centers
    if alignment_type == 'Using_Surface':
        input_path_root = output_path_alignment_global_alignment_surfaces
    if alignment_type == 'Using_Local_Surfaces':
        input_path_root = output_path_alignment_global_alignment_local_surfaces
    if alignment_type == 'Using_Local_Rabies_Surfaces':
        input_path_root = output_path_alignment_global_alignment_local_rabies_surfaces
    if alignment_type == 'Using_M1_SBF':
        input_path_root = output_path_alignment_global_alignment_local_surfaces_m1_sbf

    
    # get centers, pca, 
    for surf_type in ['vS1','WB','vM1',]:
        #tmp_output = input_path_root+'/tmp/'
        #pathlib.Path(tmp_output).mkdir(exist_ok=True)
        for hem in ['lhs','rhs']:
            print(outer_alignment_type,alignment_type,surf_type,hem)
            df = get_alignment_stats(input_path_root,outer_alignment_type,alignment_type,surf_type,hem)
            df_main = df_main.append(df)
                    
    
#         
df_main.to_csv(output_path_alignment_analysis+outer_alignment_type+'_stats.csv')
       

In [ ]:
surf_res = 50
df_main = pd.DataFrame()
outer_alignment_type = 'vM1'
#for folder in ['Global_Alignment']:#['Global_Alignment','vS1','vM1','S1BF']:
for alignment_type in ['Using_Center','Using_Rabies_Surface','Using_Surface']:#[]
    if alignment_type == 'Using_Center':
        input_path_root = output_path_alignment_vm1_center
    if alignment_type == 'Using_Rabies_Surface':
        input_path_root = output_path_alignment_vm1_rabies_surface
    if alignment_type == 'Using_Surface':
        input_path_root = output_path_alignment_vm1_surface
    
    # get centers, pca, 
    for surf_type in ['vS1','WB','vM1',]:
        #tmp_output = input_path_root+'/tmp/'
        #pathlib.Path(tmp_output).mkdir(exist_ok=True)
        for hem in ['lhs','rhs']:
            print(outer_alignment_type,alignment_type,surf_type,hem)
            df = get_alignment_stats(input_path_root,outer_alignment_type,alignment_type,surf_type,hem)
            df_main = df_main.append(df)   
#         
df_main.to_csv(output_path_alignment_analysis+outer_alignment_type+'_stats.csv')
 

In [ ]:
surf_res = 50
df_main = pd.DataFrame()
outer_alignment_type = 'vS1'
#for folder in ['Global_Alignment']:#['Global_Alignment','vS1','vM1','S1BF']:
for alignment_type in ['Using_Center','Using_Rabies_Surface','Using_Surface']:#[]
    if alignment_type == 'Using_Center':
        input_path_root = output_path_alignment_vs1_center
    if alignment_type == 'Using_Rabies_Surface':
        input_path_root = output_path_alignment_vs1_rabies_surface
    if alignment_type == 'Using_Surface':
        input_path_root = output_path_alignment_vs1_surface
    
    # get centers, pca, 
    for surf_type in ['vS1','WB','vM1',]:
        #tmp_output = input_path_root+'/tmp/'
        #pathlib.Path(tmp_output).mkdir(exist_ok=True)
        for hem in ['lhs','rhs']:
            print(outer_alignment_type,alignment_type,surf_type,hem)
            df = get_alignment_stats(input_path_root,outer_alignment_type,alignment_type,surf_type,hem)
            df_main = df_main.append(df)   
#         
df_main.to_csv(output_path_alignment_analysis+outer_alignment_type+'_stats.csv')
 

In [ ]:
surf_res = 50
df_main = pd.DataFrame()
outer_alignment_type = 'S1BF'
#for folder in ['Global_Alignment']:#['Global_Alignment','vS1','vM1','S1BF']:
for alignment_type in ['Use_Barrels_Alone','Use_Projections_Alone','Use_Both']:#[]
    if alignment_type == 'Use_Barrels_Alone':
        input_path_root = output_path_alignment_s1bf_barrels_alone
    if alignment_type == 'Use_Projections_Alone':
        input_path_root = output_path_alignment_s1bf_projections_alone
    if alignment_type == 'Use_Both':
        input_path_root = output_path_alignment_s1bf_barrels_and_projections
    
    # get centers, pca, 
    for surf_type in ['vS1','WB','vM1',]:
        #tmp_output = input_path_root+'/tmp/'
        #pathlib.Path(tmp_output).mkdir(exist_ok=True)
        for hem in ['lhs','rhs']:
            print(outer_alignment_type,alignment_type,surf_type,hem)
            df = get_alignment_stats(input_path_root,outer_alignment_type,alignment_type,surf_type,hem)
            df_main = df_main.append(df)   
#         
df_main.to_csv(output_path_alignment_analysis+outer_alignment_type+'_stats.csv')
 

In [ ]:
# df_main = pd.DataFrame()
# df_main_Centers = pd.DataFrame(columns=['Alignment_Type','Surface_Type','Hemisphere','Mean','STD'])
#for aligned_using_surf_res in [50]:#surface_resolutions:
# for alignment_type in ['Using_Center','Using_Local_Centers','Using_Surface','Using_Local_Surfaces',]:#[]
#     if alignment_type == 'Using_Center':
#         input_path_root = output_path_alignment_global_alignment_centers
#     if alignment_type == 'Using_Local_Centers':
#         input_path_root = output_path_alignment_global_alignment_local_centers
#     if alignment_type == 'Using_Surface':
#         input_path_root = output_path_alignment_global_alignment_surfaces
#     if alignment_type == 'Using_Local_Surfaces':
#         input_path_root = output_path_alignment_global_alignment_local_surfaces
    
#     # Compute error in centers and surf fro wb,s1 and m1
    
#     # wb surf errors
#     #input_path_root = output_path_alignment_surf_res+'/Surface_Res_{}/{}/'.format(aligned_using_surf_res,alignment_type)
#     #for surf_res in surface_resolutions:
#     surf_res = 200
#     aligned_using_surf_res = 200
    
#     for surf_type in ['WB','vS1','vM1']:
#         # get aligned surfaces list
#         centroid,aligned_pias_list_lhs,aligned_wms_list_lhs,aligned_pias_list_rhs,aligned_wms_list_rhs = \
#             Surface().get_aligned_surfaces_list(input_path_root,200,surf_type=surf_type)

#         # get obb tree for surf lists
#         obb_pias_lhs = Surface().get_obb_tree_list(aligned_pias_list_lhs)
#         obb_pias_rhs = Surface().get_obb_tree_list(aligned_pias_list_rhs)
#         obb_wms_lhs = Surface().get_obb_tree_list(aligned_wms_list_lhs)
#         obb_wms_rhs = Surface().get_obb_tree_list(aligned_wms_list_rhs)

#         # get rays from surfaces center
#         #rays_sg = Surface().get_rays(centroid,theta_res=33,phi_res=32)
#         rays_sg,sphere = Surface().get_rays(centroid,theta_res=33,phi_res=32,ray_core_radius=1,tesselation=True,return_sphere=True)
#         rays_sg.write_spatial_graph(input_path_root+'/Average/Rays_{}.am'.format(surf_res))
#         Surface(polydata=sphere).write_surface_mesh(input_path_root+'/Average/Sphere_{}.vtk'.format(surf_res))

#         # get avg intersection pts for rays cast
#         #for hem in ['lhs','rhs']:
#         avg_pts_pia_lhs,avg_pts_wm_lhs,wm_pia_rays_sg_lhs,ellisoids_lhs,df_lhs = Surface().get_avg_pts\
#             (rays_sg,obb_pias_lhs,obb_wms_lhs,alignment_type=alignment_type,surf_type=surf_type,hem='LHS',\
#              aligned_surf_res=aligned_using_surf_res,surf_res=surf_res,create_error_ellisoids=False)
#         #ellisoids_lhs.write_surface_mesh(input_path_root+'/Average/Avg_Error_Ellipsoids_LHS_{}.vtk'.format(surf_res))

#         df_main = df_main.append(df_lhs)
#         avg_pts_pia_rhs,avg_pts_wm_rhs,wm_pia_rays_sg_rhs,ellisoids_rhs,df_rhs = Surface().get_avg_pts\
#             (rays_sg,obb_pias_rhs,obb_wms_rhs,alignment_type=alignment_type,surf_type=surf_type,hem='RHS'\
#              ,aligned_surf_res=aligned_using_surf_res,surf_res=surf_res,create_error_ellisoids=False)
#         #ellisoids_rhs.write_surface_mesh(input_path_root+'/Average/Avg_Error_Ellipsoids_RHS_{}.vtk'.format(surf_res))
#         df_main = df_main.append(df_rhs)
#         # write output avg rays
#         wm_pia_rays_sg_lhs.write_spatial_graph(input_path_root+'/Average/Avg_Rays_LHS_{}.am'.format(surf_res))
#         wm_pia_rays_sg_rhs.write_spatial_graph(input_path_root+'/Average/Avg_Rays_RHS_{}.am'.format(surf_res))
    
#         # now do centers
#         if surf_type == 'WB':
#             centroid_mean_lhs,centroid_std_lhs = Surface().get_surf_center_errors(aligned_pias_list_lhs,aligned_wms_list_lhs)
#             df_main_Centers = df_main_Centers.append(pd.DataFrame(columns=df_main_Centers.columns,data=\
#                                                                   [[alignment_type,'WB','LHS',centroid_mean_lhs,centroid_std_lhs]]))
#             centroid_mean_rhs,centroid_std_rhs = Surface().get_surf_center_errors(aligned_pias_list_rhs,aligned_wms_list_rhs)
#             df_main_Centers = df_main_Centers.append(pd.DataFrame(columns=df_main_Centers.columns,data=\
#                                                                   [[alignment_type,'WB','RHS',centroid_mean_rhs,centroid_std_rhs]]))
            
#         elif surf_type == 'vS1':
#             mean,std = get_br_centers(input_path_root,br='vS1',hem='lhs')
#             df_main_Centers = df_main_Centers.append(pd.DataFrame(columns=df_main_Centers.columns,data=\
#                                                                   [[alignment_type,'vS1','LHS',mean,std]]))
#             mean,std = get_br_centers(input_path_root,br='vS1',hem='rhs')
#             df_main_Centers = df_main_Centers.append(pd.DataFrame(columns=df_main_Centers.columns,data=\
#                                                                   [[alignment_type,'vS1','RHS',mean,std]]))
#         elif surf_type == 'vM1':
#             mean,std = get_br_centers(input_path_root,br='vM1',hem='lhs')
#             df_main_Centers = df_main_Centers.append(pd.DataFrame(columns=df_main_Centers.columns,data=\
#                                                                   [[alignment_type,'vM1','LHS',mean,std]]))
#             mean,std = get_br_centers(input_path_root,br='vM1',hem='rhs')
#             df_main_Centers = df_main_Centers.append(pd.DataFrame(columns=df_main_Centers.columns,data=\
#                                                                   [[alignment_type,'vM1','RHS',mean,std]]))
            
    

# df_main.to_csv(output_path_alignment_global_alignment+'/Stats.csv')
# df_main_Centers.to_csv(output_path_alignment_global_alignment+'/Stats_Centers.csv')


# Create Avg Ref Frame

## Create Avg Pia Surface

In [ ]:
input_path_root = output_path_alignment_global_alignment_local_rabies_surfaces
surf_res = 50

In [ ]:
centroid,centroid_lhs,centroid_rhs,aligned_pias_list_lhs,aligned_wms_list_lhs,aligned_pias_list_rhs,aligned_wms_list_rhs = \
            Surface().get_aligned_surfaces_list(input_path_root,surf_res,surf_type='WB',closed_surf=True)

# get obb tree for surf lists
obb_pias_lhs = Surface().get_obb_tree_list(aligned_pias_list_lhs)
obb_pias_rhs = Surface().get_obb_tree_list(aligned_pias_list_rhs)
obb_wms_lhs = Surface().get_obb_tree_list(aligned_wms_list_lhs)
obb_wms_rhs = Surface().get_obb_tree_list(aligned_wms_list_rhs)


# get avg intersection pts for rays cast
#for hem in ['lhs','rhs']:


In [ ]:
# get rays from surfaces center
#rays_sg = Surface().get_rays(centroid,theta_res=33,phi_res=32)

# rays_sg_lhs,sphere = Surface().get_rays(centroid_lhs,theta_res=120,phi_res=120,ray_core_radius=1,tesselation=True,return_sphere=True)
# rays_sg_rhs,sphere = Surface().get_rays(centroid_rhs,theta_res=120,phi_res=120,ray_core_radius=1,tesselation=True,return_sphere=True)

# rays_sg_lhs.write_spatial_graph(input_path_root+'/Average/Rays_lhs_{}_120_120.am'.format(surf_res))
# rays_sg_rhs.write_spatial_graph(input_path_root+'/Average/Rays_rhs_{}_120_120.am'.format(surf_res))

#Surface(polydata=sphere).write_surface_mesh(input_path_root+'/Average/Sphere_{}.vtk'.format(surf_res))


In [ ]:
rays_sg_lhs = AmiraSpatialGraph(input_path_root+'/Average/Rays_lhs_120_120.am',generic_graph=True)
rays_sg_rhs = AmiraSpatialGraph(input_path_root+'/Average/Rays_rhs_120_120.am',generic_graph=True)


In [ ]:
avg_pts_pia_lhs,avg_pts_wm_lhs,wm_pia_rays_sg_lhs,ellisoids_lhs,df_lhs = Surface().get_avg_pts\
                                                            (rays_sg_lhs,obb_pias_lhs,obb_wms_lhs,)
avg_pts_pia_rhs,avg_pts_wm_rhs,wm_pia_rays_sg_rhs,ellisoids_lhs,df_lhs = Surface().get_avg_pts\
                                                            (rays_sg_rhs,obb_pias_rhs,obb_wms_rhs,)
    
Landmarks(pts=avg_pts_pia_lhs).write_landmarks(input_path_root+'Average/pia_lhs.landmarks')
Landmarks(pts=avg_pts_pia_rhs).write_landmarks(input_path_root+'Average/pia_rhs.landmarks')
Landmarks(pts=avg_pts_wm_lhs).write_landmarks(input_path_root+'Average/wm_lhs.landmarks')
Landmarks(pts=avg_pts_wm_rhs).write_landmarks(input_path_root+'Average/wm_rhs.landmarks')

In [ ]:
avg_pts_pia_lhs = Landmarks(input_path_root+'Average/pia_lhs.landmarks').pts
avg_pts_pia_rhs = Landmarks(input_path_root+'Average/pia_rhs.landmarks').pts
avg_pts_wm_lhs = Landmarks(input_path_root+'Average/wm_lhs.landmarks').pts
avg_pts_wm_rhs = Landmarks(input_path_root+'Average/wm_rhs.landmarks').pts

lhs_hull_pia = Surface(pts=avg_pts_pia_lhs).create_delunay_surface_3d(output_filename=input_path_root+'Average/Avg_Pia_lhs_delunay.vtk',return_hull=True)
rhs_hull_pia = Surface(pts=avg_pts_pia_rhs).create_delunay_surface_3d(output_filename=input_path_root+'Average/Avg_Pia_rhs_delunay.vtk',return_hull=True)
lhs_hull_wm = Surface(pts=avg_pts_wm_lhs).create_delunay_surface_3d(output_filename=input_path_root+'Average/Avg_WM_lhs_delunay.vtk',return_hull=True)
lhs_hull_wm = Surface(pts=avg_pts_wm_rhs).create_delunay_surface_3d(output_filename=input_path_root+'Average/Avg_WM_rhs_delunay.vtk',return_hull=True)

## Generate Avg Axis Field

In [ ]:
# use mg48 axis field for now

## Create Avg vM1 and vS1 Ref Frames

### Generate Avg PCA Axes

In [ ]:
def get_mean_angle_between_vectors(uvs,unit_vectors_given=False):
    mean_uv = np.array(uvs).mean(axis=0)
    
#     print(mean_uv)
#     print(uvs[0])
#     print(uvs[1])
#     print(uvs[2])
    angles = []
    if unit_vectors_given:
        for i in range(len(uvs)):
            angles.append(Vectors().get_angle_between_unit_vectors(mean_uv,uvs[i]))
    else:
        for i in range(len(uvs)):
            angles.append(Vectors().get_angle_between_vectors(mean_uv,uvs[i]))
        #dists.append(distance.euclidean(centers[i],centroid))
    
    return angles,np.array(angles).mean(axis=0),np.array(angles).std(axis=0),mean_uv

In [ ]:
def get_mean_unit_vec_of_vectors(vecs,neg_vecs,unit_vectors_given=False):
    uvs = []
    extents = []
    uvs.append(Vectors().get_unit_vec(vecs[0][0],vecs[0][1]))
    uvs.append(Vectors().get_unit_vec(vecs[1][0],vecs[1][1]))
    uvs.append(Vectors().get_unit_vec(vecs[2][0],vecs[2][1]))
        
    # find angle between vectors to see if some uvs need to be flipped. i,e if angle is greater than 90
    angle1 = Vectors().get_angle_between_unit_vectors(uvs[0],uvs[1],ignore_opposite_direction=False)
    extents.append(distance.euclidean(vecs[0][0],vecs[0][1]))
    if angle1>90:
        uvs[1] = -uvs[1]
        extents.append(distance.euclidean(neg_vecs[1][0],neg_vecs[1][1]))
    else:
        extents.append(distance.euclidean(vecs[1][0],vecs[1][1]))
    angle2 = Vectors().get_angle_between_unit_vectors(uvs[0],uvs[2],ignore_opposite_direction=False)
    if angle2>90:
        uvs[2] = -uvs[2]
        extents.append(distance.euclidean(neg_vecs[2][0],neg_vecs[2][1]))
    else:
        extents.append(distance.euclidean(vecs[2][0],vecs[2][1]))
    mean_uv = np.array(uvs).mean(axis=0)
    mean_extent = np.array(extents).mean(axis=0)
    return mean_uv,mean_extent

In [ ]:
def get_closest_vector(ref_pca_comp,pca_coms_list):
    closest_axis = []
    min_angle = 10000
    closest_ind = 0
    for i in range(1,4):
        angle = Vectors().get_angle_between_vectors(ref_pca_comp,[pca_coms_list[0],pca_coms_list[i]],)
        #print('angle ',angle)
        if angle < min_angle:
            min_angle = angle
            closest_axis = [pca_coms_list[0],pca_coms_list[i]]
            closest_ind = i
    print('closest_ind is {}'.format(closest_ind))
    return closest_axis

In [ ]:
def get_angles_between_pcas(pcas_animal1,pcas_animal2,pcas_animal3,negpcas_animal1,negpcas_animal2,negpcas_animal3):
    pca0_angles = []
    pca1_angles= []
    pca2_angles= []
    
    # use the nearest pca from other animals to determine the angle
    # since pcas across animasl are not in the same order
    pca0 = [pcas_animal1[0],pcas_animal1[1]]
    pca0_neg = [negpcas_animal1[0],negpcas_animal1[1]]
    pca0_list = []
    pca0_list_neg = []
    pca0_list.append([pcas_animal1[0],pcas_animal1[1]])
    pca0_list_neg.append([negpcas_animal1[0],negpcas_animal1[1]])
    print('for pca 1')
    print('looking at animal 1')
    pca0_list.append(get_closest_vector(pca0,pcas_animal2))
    pca0_list_neg.append(get_closest_vector(pca0_neg,negpcas_animal2))
    print('looking at animal 2')
    pca0_list.append(get_closest_vector(pca0,pcas_animal3))
    pca0_list_neg.append(get_closest_vector(pca0_neg,negpcas_animal3))
    
    mean_uv_0,mean_extent_0=get_mean_unit_vec_of_vectors(pca0_list,pca0_list_neg,unit_vectors_given=False)
    
    print('for pca 2')
    pca1 = [pcas_animal1[0],pcas_animal1[2]]
    pca1_neg = [pcas_animal1[0],pcas_animal1[2]]
    pca1_list = []
    pca1_list_neg = []
    pca1_list.append([pcas_animal1[0],pcas_animal1[2]])
    pca1_list_neg.append([negpcas_animal1[0],negpcas_animal1[2]])
    print('looking at animal 1')
    pca1_list.append(get_closest_vector(pca1,pcas_animal2))
    pca1_list_neg.append(get_closest_vector(pca1_neg,negpcas_animal2))
    print('looking at animal 2')
    pca1_list.append(get_closest_vector(pca1,pcas_animal3))
    pca1_list_neg.append(get_closest_vector(pca1_neg,negpcas_animal3))
    
    mean_uv_1,mean_extent_1=get_mean_unit_vec_of_vectors(pca1_list,pca1_list_neg,unit_vectors_given=False)
    
    print('for pca 3')
    pca2 = [pcas_animal1[0],pcas_animal1[3]]
    pca2_neg = [pcas_animal1[0],pcas_animal1[3]]
    pca2_list = []
    pca2_list_neg = []
    print('looking at animal 1')
    pca2_list.append([pcas_animal1[0],pcas_animal1[3]])
    pca2_list_neg.append([negpcas_animal1[0],negpcas_animal1[3]])
    print('looking at animal 2')
    pca2_list.append(get_closest_vector(pca2,pcas_animal2))
    pca2_list_neg.append(get_closest_vector(pca2_neg,negpcas_animal2))
    pca2_list.append(get_closest_vector(pca2,pcas_animal3))
    pca2_list_neg.append(get_closest_vector(pca2_neg,negpcas_animal3))
    
    mean_uv_2,mean_extent_2=get_mean_unit_vec_of_vectors(pca2_list,pca2_list_neg,unit_vectors_given=False)
    
    
    
    return mean_uv_0,mean_uv_1,mean_uv_2,mean_extent_0,mean_extent_1,mean_extent_2

In [ ]:
def write_pca(avg_vecs,pos_extents,neg_extents,centroid,output_path):
    for i in range(3):
        avg_pca0_front = Vectors().create_pt_along_unit_vector_at_given_distance(pos_extents[i],centroid,avg_vecs[i])
        avg_pca0_back = Vectors().create_pt_along_unit_vector_at_given_distance(-neg_extents[i],centroid,avg_vecs[i])
        avg_pca0_front_sg = AmiraSpatialGraph(generic_graph=True)
        avg_pca0_front_sg.graph_data.add_edge(centroid,avg_pca0_front)
        avg_pca0_front_sg.write_spatial_graph(output_path+'Avg_{}_{}_PCA{}.am'.format(br,hem,i))
        avg_pca0_back_sg = AmiraSpatialGraph(generic_graph=True)
        avg_pca0_back_sg.graph_data.add_edge(centroid,avg_pca0_back)
        avg_pca0_back_sg.write_spatial_graph(output_path+'Avg_{}_{}_PCA_{}_Neg.am'.format(br,hem,i))

In [ ]:
def get_avg_pca(input_path_root,br,hem,centroid,output_path):
    # center pcas
    pca_animals = []
    pca_animals_neg = []
    pca_animals_extents = []
    for exp in ['MG48','MG49','MG50']:
        pca_animal = []
        pca_animal_neg = []
        pca_extents = []
        for i in range(3):
            #print(i)
            pca_comp_sg  = AmiraSpatialGraph(input_path_root+'{}_Ref_Surfaces'.format(br)+\
                                '/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_{}.am'.format(exp,hem,i),\
                              generic_graph=True)
            center = pca_comp_sg.graph_data.edge_list[0][0]
            pca_comp_sg.apply_transformation([1,0,0,0, 0,1,0,0, 0,0,1,0, centroid[0]-center[0],centroid[1]-center[1],centroid[2]-center[2],1])
            pca_comp_sg.write_spatial_graph(output_path+'{}_{}_{}_centered_pca_{}'.format(br,exp,hem,i))
            pca_comp_sg= AmiraSpatialGraph(output_path+'{}_{}_{}_centered_pca_{}'.format(br,exp,hem,i),generic_graph=True)
            pca_comp_neg_sg  = AmiraSpatialGraph(input_path_root+'{}_Ref_Surfaces'.format(br)+\
                                '/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_{}_Neg.am'.format(exp,hem,i),\
                              generic_graph=True)
            pca_comp_neg_sg.apply_transformation([1,0,0,0, 0,1,0,0, 0,0,1,0, centroid[0]-center[0],centroid[1]-center[1],centroid[2]-center[2],1])
            pca_comp_neg_sg.write_spatial_graph(output_path+'{}_{}_{}_centered_pca_neg_{}'.format(br,exp,hem,i))
            pca_comp_neg_sg= AmiraSpatialGraph(output_path+'{}_{}_{}_centered_pca_neg_{}'.format(br,exp,hem,i),generic_graph=True)
            
            pca_extents.append(distance.euclidean(pca_comp_sg.graph_data.edge_list[0][1],pca_comp_neg_sg.graph_data.edge_list[0][1]))
            
            # add this to the list
            if i == 0:
                # add both origin and first component
                pca_animal.append(centroid)
                pca_animal_neg.append(centroid)
            pca_animal.append(pca_comp_sg.graph_data.edge_list[0][1])
            pca_animal_neg.append(pca_comp_neg_sg.graph_data.edge_list[0][1])
        
        pca_animals.append(pca_animal)
        pca_animals_neg.append(pca_animal_neg)
        
        pca_animals_extents.append(pca_extents)  
    #print(pca_animals[0])
    
    # get avg axis
    mean_uvec_0,mean_uvec_1,mean_uvec_2,pos_extent_uvec0,pos_extent_uvec1,pos_extent_uvec2 = get_angles_between_pcas\
                (pca_animals[0],pca_animals[1],pca_animals[2],pca_animals_neg[0],pca_animals_neg[1],pca_animals_neg[2])
        
    # get avg extents
    total_avg_pca0_extent = (pca_animals_extents[0][0] + pca_animals_extents[1][0] + pca_animals_extents[2][0])/3
    total_avg_pca1_extent = (pca_animals_extents[0][1] + pca_animals_extents[1][1] + pca_animals_extents[2][1])/3
    total_avg_pca2_extent = (pca_animals_extents[0][2] + pca_animals_extents[1][2] + pca_animals_extents[2][2])/3
    
    neg_expent_uvec0 = total_avg_pca0_extent - pos_extent_uvec0
    neg_expent_uvec1 = total_avg_pca1_extent - pos_extent_uvec1
    neg_expent_uvec2 = total_avg_pca2_extent - pos_extent_uvec2
    
    #print(avg_pca0_extent)
    # get avg pca
    #Landmarks(pts=[mean_uvec_0[0],mean_vec_0[1]]).write_landmarks(output_path+'tmp')
    #avg_pca0_uv = Vectors().get_unit_vec(mean_vec_0[0],mean_vec_0[1])
    write_pca([mean_uvec_0,mean_uvec_1,mean_uvec_2],[pos_extent_uvec0,pos_extent_uvec1,pos_extent_uvec2],\
              [neg_expent_uvec0,neg_expent_uvec1,neg_expent_uvec2], centroid,output_path )
    
    

In [ ]:
input_path = output_path_alignment_global_alignment_local_rabies_surfaces

# Get avg pca
# Find centoid of vM1
for br in ['vM1','vS1']:
    if br == 'vM1':
        input_path_local_alignment = output_path_alignment_vm1_rabies_surface
    else:
        input_path_local_alignment = output_path_alignment_vs1_rabies_surface
        
    output_path = output_path_alignment_global_alignment_local_rabies_surfaces + 'Average/'+br+'/'
    pathlib.Path(output_path).mkdir(exist_ok=True)
    for hem in ['lhs','rhs']:
        centers = []
        for exp in ['MG48','MG49','MG50']:
            centers.append(Landmarks(input_path+\
                                         '/{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_Centroid.landmarksAscii'.format(br,exp,hem)).pts[0])
        Landmarks(pts=centers).write_landmarks(output_path+'aligned_centers_{}_{}.landmarksAscii'.format(br,hem))
        
        centroid = np.array(centers).mean(axis=0)
        Landmarks(pts=[centroid]).write_landmarks(output_path+'centroid_{}_{}.landmarksAscii'.format(br,hem))
        
        get_avg_pca(input_path_local_alignment,br,hem,centroid,output_path)
        
    
    

### Generate vM1 and vS1 Avg Surface

In [ ]:
# combine axis aligned axis fields so that i can be used to get the local orientation of cortex at pca end points 
# to determine the cutting planes
input_path = output_path_alignment_global_alignment_local_rabies_surfaces+ 'Axis_Fields/'

for hem in ['lhs','rhs']:
    combined_axis_field_sg = AmiraSpatialGraph(generic_graph=True)
    for exp in ['MG48','MG49','MG50']:
        sg = AmiraSpatialGraph(input_path+'{}_{}_wm_to_pia_axis_field_voxel_size_50.am'.format(exp,hem),generic_graph=True)
        combined_axis_field_sg.graph_data =  combined_axis_field_sg.combine_subgraphs([combined_axis_field_sg.graph_data,sg.graph_data])
    combined_axis_field_sg.write_spatial_graph(output_path_alignment_global_alignment_local_rabies_surfaces+'/Average/combined_axis_field_{}.am'.format(hem))

In [ ]:
input_path = output_path_alignment_global_alignment_local_rabies_surfaces+'/Average/'
#if exp == 'MG48_rhs' :
#    cutting_order = [True,False,True,False]
#else:
cutting_order = [True,False,False,True]
for br in ['vM1','vS1']:
    output_path = input_path + br+'/'
    pathlib.Path(output_path).mkdir(exist_ok=True)
    
    for hem in ['lhs','rhs']:

        pia_surf = Surface(input_path+'/Avg_Pia_{}_delunay.vtk'.format(hem))
        wm_surf = Surface(input_path+'/Avg_WM_{}_delunay.vtk'.format(hem))
        axis_field_sg = AmiraSpatialGraph(input_path+'/combined_axis_field_{}.am'.format(hem),generic_graph=True)
        axis_field = axis_field_sg.graph_data.edge_list
        
        pca0 = AmiraSpatialGraph(output_path+'Avg_{}_{}_PCA0.am'.format(br,hem),generic_graph=True)\
                .graph_data.edge_list[0]
        pca1 = AmiraSpatialGraph(output_path+'Avg_{}_{}_PCA1.am'.format(br,hem),generic_graph=True)\
                .graph_data.edge_list[0]
        pca2 = AmiraSpatialGraph(output_path+'Avg_{}_{}_PCA2.am'.format(br,hem),generic_graph=True)\
                .graph_data.edge_list[0]

        pca0_neg = AmiraSpatialGraph(output_path+'Avg_{}_{}_PCA_0_Neg.am'.format(br,hem),generic_graph=True)\
                .graph_data.edge_list[0]
        pca1_neg = AmiraSpatialGraph(output_path+'Avg_{}_{}_PCA_1_Neg.am'.format(br,hem),generic_graph=True)\
                .graph_data.edge_list[0]
        pca2_neg = AmiraSpatialGraph(output_path+'Avg_{}_{}_PCA_2_Neg.am'.format(br,hem),generic_graph=True)\
                .graph_data.edge_list[0]

        # landmarks are needed to determine the end pts though which the extreme planes pass. therefore use the pca end pts
        landmarks = [pca0[0],pca0[1],pca0_neg[1],pca1[1],pca1_neg[1],pca2[1],pca2_neg[1]]
        central_axis = [pca0[0],pca2[1]]
        
        plane_right,plane_left,plane_front,plane_back = create_vm1_surface\
                (pia_surf,wm_surf,[pca0,pca1,pca2],[pca0_neg,pca1_neg,pca2_neg],axis_field,landmarks,central_axis,cutting_order)
        
        vm1_final,vm1_hull,vm1_pia,vm1_wm = Surface().clip_surfaces_using_given_planes(pia_surf.surface,wm_surf.surface,\
                                                                           plane_back,plane_front,\
                                                                           plane_right,plane_left,landmarks=[],\
                                                                           central_axis=central_axis,\
                                                                           order_of_cutting=cutting_order,invert_cell_direction=True)
        
        Surface(polydata=vm1_final).write_surface_mesh(output_path+'Avg_{}_{}.vtk'.format(br,hem))
        Surface(polydata=vm1_hull).write_surface_mesh(output_path+'Avg_{}_{}_Hull.vtk'.format(br,hem))
        Surface(polydata=vm1_pia).write_surface_mesh(output_path+'Avg_{}_{}_Pia.vtk'.format(br,hem))
        Surface(polydata=vm1_wm).write_surface_mesh(output_path+'Avg_{}_{}_WM.vtk'.format(br,hem))
        
        # trim pcas to fit the surf
        trimmed_edge_0_sg,bla,ble = Surface().get_axis_field_within_instersecting_surface\
                        (vm1_final,[pca0],trim_edge=True,vec_extension_len=5000)
        trimmed_edge_0_sg.write_spatial_graph(output_path+'Avg_{}_{}_PCA0_Trimmed.am'.format(br,hem))
        
        trimmed_edge_1_sg,bla,ble = Surface().get_axis_field_within_instersecting_surface\
                        (vm1_final,[pca1],trim_edge=True,vec_extension_len=5000)
        trimmed_edge_1_sg.write_spatial_graph(output_path+'Avg_{}_{}_PCA1_Trimmed.am'.format(br,hem))
        
        trimmed_edge_2_sg,bla,ble = Surface().get_axis_field_within_instersecting_surface\
                        (vm1_final,[pca2],trim_edge=True,vec_extension_len=5000)
        trimmed_edge_2_sg.write_spatial_graph(output_path+'Avg_{}_{}_PCA2_Trimmed.am'.format(br,hem))
        
        trimmed_edge_0neg_sg,bla,ble = Surface().get_axis_field_within_instersecting_surface\
                        (vm1_final,[pca0_neg],trim_edge=True,vec_extension_len=5000)
        trimmed_edge_0neg_sg.write_spatial_graph(output_path+'Avg_{}_{}_PCA_0_Neg_Trimmed.am'.format(br,hem))
        
        trimmed_edge_1neg_sg,bla,ble = Surface().get_axis_field_within_instersecting_surface\
                        (vm1_final,[pca1_neg],trim_edge=True,vec_extension_len=5000)
        trimmed_edge_1neg_sg.write_spatial_graph(output_path+'Avg_{}_{}_PCA_1_Neg_Trimmed.am'.format(br,hem))
        
        trimmed_edge_1neg_sg,bla,ble = Surface().get_axis_field_within_instersecting_surface\
                        (vm1_final,[pca2_neg],trim_edge=True,vec_extension_len=5000)
        trimmed_edge_1neg_sg.write_spatial_graph(output_path+'Avg_{}_{}_PCA_2_Neg_Trimmed.am'.format(br,hem))
            

## Create Avg SBF Surface

In [ ]:
def get_cylinder(radius,edge):
    center = (np.array(edge[0])+np.array(edge[1]))/2
    height = Vectors().get_vec_length(edge)
    cylinderSource = vtk.vtkCylinderSource()
    cylinderSource.SetCenter(center)
    cylinderSource.SetRadius(radius)
    cylinderSource.SetHeight(height)
    cylinderSource.SetResolution(100)
    cylinderSource.Update()

    pts = []
    ref_pts = []
    pts.append(center)
    end_pt = [center[0],cylinderSource.GetOutput().GetBounds()[2],center[2] ]
    pts.append(end_pt)

    ref_pts.append(center)
    end_pt = Vectors().get_unit_vec(center,edge[1])*height/2 + center
    ref_pts.append(end_pt)
    #Landmarks(pts=pts).write_landmarks(output_path_neun_profiles+'pts')
    #Landmarks(pts=ref_pts).write_landmarks(output_path_neun_profiles+'ref_pts')

    landmarks = Landmarks(pts=pts)
    icp,txmat = landmarks.align_landmarks(ref_pts)

    cyl_surf = Surface(polydata=cylinderSource.GetOutput())
    cyl_surf.apply_icp_transform(icp)
    #print(txmat)

    #Surface(polydata=cylinderSource.GetOutput()).write_surface_mesh(output_path_neun_profiles+'_cylinder.vtk')

    return cyl_surf.surface 

In [ ]:
# use matching barrels and to find avg column axis and top and bottom barrels should be the mean of distance from the top
#input_path = output_path_alignment_s1bf_barrels_and_projections
input_path = output_path_alignment_global_alignment_local_rabies_surfaces
output_path = output_path_alignment_global_alignment_local_rabies_surfaces + 'Average/SBF/'
pathlib.Path(output_path).mkdir(exist_ok=True)
for hem in ['lhs','rhs']:
    all_barrel_columns_surfs = Surface()
    sg1 = AmiraSpatialGraph(input_path+'Surfaces/MG49_{}_barrels_50.am'.format(hem),barrel_projections_present=True,reverse_barrel_direction=True)
    sg2 = AmiraSpatialGraph(input_path+'Surfaces/MG50_{}_barrels_50.am'.format(hem),barrel_projections_present=True,reverse_barrel_direction=True)
    
    # get avg axis for corresponding barrels
    matching_barrels = MatchBarrels(sg1.barrels,sg2.barrels,)
    axes1,axes2 = matching_barrels.get_matching_axes()
    
    avg_axes = np.array([axes1,axes2]).mean(axis=0)
    sg = AmiraSpatialGraph(generic_graph=True)
    for i in range(len(avg_axes)):
        sg.graph_data.add_edge(avg_axes[i][0],avg_axes[i][1])
    sg.write_spatial_graph(output_path+'Avg_Axes_{}.am'.format(hem))
    
    # trim axes to the surface
    pia_wm = Surface()
    pia = Surface(output_path_alignment_global_alignment_local_rabies_surfaces + 'Average/Avg_Pia_{}_delunay.vtk'.format(hem))
    wm = Surface(output_path_alignment_global_alignment_local_rabies_surfaces + 'Average/Avg_WM_{}_delunay.vtk'.format(hem))
    pia_wm.append(pia.surface)
    pia_wm.append(wm.surface)
    
    trimmed_edge_sg,bla,ble,blup = Surface().get_axis_field_within_instersecting_surface_with_trimming\
                        (pia.surface,wm.surface,avg_axes,vec_extension_offset=0.5)
    trimmed_edge_sg.write_spatial_graph(output_path+'Avg_Axes_{}_Trimmed.am'.format(hem))
    for trimmed_edge in trimmed_edge_sg.graph_data.edge_list:
        cylinder = get_cylinder(250,trimmed_edge)
        surf = Surface(polydata=cylinder).create_delunay_surface_3d(return_hull=True)
        all_barrel_columns_surfs.append(surf)

    Surface(polydata=all_barrel_columns_surfs.surface).write_surface_mesh(output_path+'Avg_Column_Surfaces_{}.vtk'.format(hem))

    Surface(polydata=all_barrel_columns_surfs.surface).create_delunay_surface_3d(output_path+'Avg_Column_Surfaces_Hull_{}.vtk'.format(hem))


# Atlas

## Fix Atlas spatial graph if needed

In [ ]:
# following adds y offset staring from section 28 so that surface is convex. rostral part has olofactory bulb related issues in the atlas

offset = 30
y = 0
sg_main = AmiraSpatialGraph(input_path+'/atlas/sectiongraphs_txed/S001_transformed.am',read_header_only=True)
for i in range(161,0,-1):
    #sg = AmiraSpatialGraph(input_path+'/atlas/sectiongraphs_txed/S{:03d}_transformed.am'.format(i),read_header_only=True)
    sg1 = AmiraSpatialGraph(input_path+'/atlas/sectiongraphs_txed/S{:03d}_transformed.am'.format(i),generic_graph=True)
    if i == 23:
        y=y+offset+200
    elif i == 6:
        y=y+offset+650
    elif i > 23:
        y=0
    else:
        y=y+offset
    #print(sg1.graph_data.vertices[1])
    mat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,y,0,1]
    sg1.apply_transformation(mat)
    #print(sg1.graph_data.edge_list[1])
    #print('/n')
    sg1.write_spatial_graph(input_path+'/atlas/sectiongraphs_txed/tmp.am')
    sg1 = AmiraSpatialGraph(input_path+'/atlas/sectiongraphs_txed/tmp.am',generic_graph=True)
    #sg1.update_graph_data()
    #sg.graph_data = sg1.graph_data
    
    sg_main.graph_data = sg_main.combine_subgraphs([sg_main.graph_data,sg1.graph_data])
    
sg_main.write_spatial_graph_2(input_path+'/atlas/Atlas.am')

## Align to Average

In [ ]:
sg = AmiraSpatialGraph(input_path+'/atlas/Atlas.am',read_header_only=True)
sg1 = AmiraSpatialGraph(input_path+'/atlas/Atlas.am',generic_graph=True)
tr_mat = [-1, 0, 0, 0, 0, 0, 1, 0, 0, -1, 0, 0, 0, 0, 0, 1]
sg1.apply_transformation(tr_mat)
sg.graph_data = sg.combine_subgraphs([sg.graph_data,sg1.graph_data])
sg.write_spatial_graph_2(output_path_atlas+'/Atlas.am')

In [ ]:
# create delunay for mg48 and atasl pia
sg_atlas_pia = AmiraSpatialGraph(output_path_atlas+'Atlas_pia.am',generic_graph=True)
sg_atlas_pia_hull = Surface(pts=sg_atlas_pia.graph_data.edge_pt_coords).create_delunay_surface_3d(return_hull=True,\
                                                                                    output_filename=output_path_atlas+'Atlas_pia.vtk')


In [ ]:
surf_lhs_pts = Surface(output_path_atlas+'Avg_Pia_lhs_delunay.vtk').get_surface_pts()
surf_rhs_pts = Surface(output_path_atlas+'Avg_Pia_rhs_delunay.vtk').get_surface_pts()
l = Landmarks(pts=surf_lhs_pts)
l.append_landmarks(surf_rhs_pts)

avg_pia = Surface(pts=l.pts).create_delunay_surface_3d(return_hull=True,\
                                                output_filename=output_path_atlas+'Avg_Pia_hull.vtk')


In [ ]:
output_path = output_path_atlas+'/Aligned_To_Avg/'
pathlib.Path(output_path).mkdir(exist_ok=True)

aligner = Alignment(avg_pia,sg_atlas_pia_hull,mode='similarity')
aligner.get_icp_transform()
surf = Surface(polydata=sg_atlas_pia_hull)
surf.apply_icp_transform(aligner.get_icp_transform())
surf.write_surface_mesh(output_path+'atlas_pia.vtk')
#aligner.transform_surfaces_from_txmat(output_path_atlas,output_path,aligner.get_transformation_matrix())
#aligner.write_transformation_matrix(filename=output_path+'Txmat.txt')

## Align to MG48 Surface

In [ ]:
# make canonical MG48 and atlas spatial graphs
sg = AmiraSpatialGraph(input_path+'/atlas/Atlas.am',read_header_only=True)
sg1 = AmiraSpatialGraph(input_path+'/atlas/Atlas.am',generic_graph=True)
tr_mat = [-1, 0, 0, 0, 0, 0, 1, 0, 0, -1, 0, 0, 0, 0, 0, 1]
sg1.apply_transformation(tr_mat)
sg.graph_data = sg.combine_subgraphs([sg.graph_data,sg1.graph_data])
sg.write_spatial_graph_2(output_path_atlas+'/Atlas.am')
#Alignment().write_transformation_matrix(output_path_atlas+'Canonical_Transofrmation_Atlas.txt')



In [ ]:
sg = AmiraSpatialGraph(input_path+'/atlas/MG48_cortex_merged_final.am',read_header_only=True)
sg1 = AmiraSpatialGraph(input_path+'/atlas/MG48_cortex_merged_final.am',generic_graph=True)
tr_mat = [-1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1]
sg1.apply_transformation(tr_mat)
sg.graph_data = sg.combine_subgraphs([sg.graph_data,sg1.graph_data])
sg.write_spatial_graph_2(output_path_atlas+'/MG48_cortex_merged_final.am')
#Alignment().write_transformation(output_path_atlas+'Canonical_Transofrmation_Atlas.txt')

In [ ]:
# create delunay for mg48 and atasl pia
sg_atlas_pia = AmiraSpatialGraph(output_path_atlas+'atlas_pia.am',generic_graph=True)
sg_atlas_pia_hull = Surface(pts=sg_atlas_pia.graph_data.edge_pt_coords).create_delunay_surface_3d(return_hull=True,\
                                                                                    output_filename=output_path_atlas+'atlas_pia.vtk')


In [ ]:
sg_mg48_pia = AmiraSpatialGraph(output_path_atlas+'MG48_pia.am',generic_graph=True)
sg_mg48_pia_hull = Surface(pts=sg_mg48_pia.graph_data.edge_pt_coords).create_delunay_surface_3d(return_hull=True,\
                                                                                    output_filename=output_path_atlas+'mg48_pia.vtk')


In [ ]:
sg_mg48_pia_hull = Surface(output_path_atlas+'mg48_pia.vtk').surface
sg_atlas_pia_hull = Surface(output_path_atlas+'Atlas_pia.vtk').surface

output_path = output_path_atlas+'/Aligned_To_MG48_New/'
pathlib.Path(output_path).mkdir(exist_ok=True)

aligner = Alignment(sg_mg48_pia_hull,sg_atlas_pia_hull,mode='similarity')
surf = Surface(polydata=sg_atlas_pia_hull)
surf.apply_icp_transform(aligner.get_icp_transform())
surf.write_surface_mesh(output_path+'atlas_pia.vtk')
#aligner.transform_surfaces_from_txmat(output_path_atlas,output_path,aligner.get_transformation_matrix())
#aligner.write_transformation_matrix(filename=output_path+'Txmat.txt')


# sg = AmiraSpatialGraph(output_path_atlas+'/canonical/Atlas_final.am',read_header_only=True)
# sg1 = AmiraSpatialGraph(output_path_atlas+'/canonical/Atlas_final.am',generic_graph=True)
# tr_mat = aligner.get_transformation_matrix()
# sg1.apply_transformation(tr_mat)
# sg.graph_data = sg.combine_subgraphs([sg.graph_data,sg1.graph_data])
# sg.write_spatial_graph_2(output_path+'Atlas_final.am')

In [ ]:
# clip at the bottom
# mg48_open = Surface(output_path_atlas+'mg48_pia.vtk')
# mg48_open.clip_surface_at_given_y(0,y_limit=mg48_open.surface.GetBounds()[3]-100,get_top=True)
# #mg48.clip_surface_at_given_y(0,y_limit=10000,get_top=True)
# mg48_open.write_surface_mesh(output_path_atlas+'mg48_pia_open.vtk')

Atlas_Pia_open = Surface(output_path_atlas+'Atlas_pia.vtk')
Atlas_Pia_open.clip_surface_at_given_y(0,y_limit=Atlas_Pia_open.surface.GetBounds()[3]-100,get_top=True)
#mg48.clip_surface_at_given_y(0,y_limit=10000,get_top=True)
Atlas_Pia_open.write_surface_mesh(output_path_atlas+'atlas_pia_open.vtk')

In [ ]:
output_path = output_path_atlas+'/Aligned_To_MG48_bottom_open_final_New2/'
pathlib.Path(output_path).mkdir(exist_ok=True)

mg48_bottom_clipped = Surface(output_path_atlas+'mg48_pia_open.vtk')
atlas_bottom_clipped = Surface(output_path_atlas+'atlas_pia_open.vtk')

aligner = Alignment(mg48_bottom_clipped.surface,atlas_bottom_clipped.surface,mode='similarity')
surf = Surface(polydata=atlas_bottom_clipped.surface)
surf.apply_icp_transform(aligner.get_icp_transform())
surf.write_surface_mesh(output_path+'atlas_pia.vtk')
#aligner.transform_surfaces_from_txmat(output_path_atlas,output_path,aligner.get_transformation_matrix())
aligner.set_transformation_matrix(aligner.get_transformation_matrix())
aligner.write_transformation_matrix(filename=output_path+'Txmat_bottom_clipped.txt')




In [ ]:
sg = AmiraSpatialGraph(output_path_atlas+'Atlas.am',read_header_only=True)
sg1 = AmiraSpatialGraph(output_path_atlas+'Atlas.am',generic_graph=True)
#tr_mat = [-1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1]
sg1.apply_transformation(aligner.get_transformation_matrix())
sg.graph_data = sg.combine_subgraphs([sg.graph_data,sg1.graph_data])
sg.write_spatial_graph_2(output_path+'/Atlas.am')

## Create surfaces

In [ ]:
pia_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Pia.am',generic_graph=True)
m1_lhs_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/M1_lhs.am',generic_graph=True)
m1_rhs_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/M1_rhs.am',generic_graph=True)
m2_lhs_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/M2_lhs.am',generic_graph=True)
m2_rhs_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/M2_rhs.am',generic_graph=True)
sbf_lhs_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/S1BF_lhs.am',generic_graph=True)
sbf_rhs_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/S1BF_rhs.am',generic_graph=True)
s1_fl_lhs_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/S1FL_lhs.am',generic_graph=True)
s1_fl_rhs_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/S1FL_rhs.am',generic_graph=True)
s2_lhs_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/S2_lhs.am',generic_graph=True)
s2_rhs_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/S2_rhs.am',generic_graph=True)
s1ulp_lhs_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/S1ULp_lhs.am',generic_graph=True)
s1ulp_rhs_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/S1ULp_rhs.am',generic_graph=True)



In [ ]:
pia=Surface(pts=pia_sg.graph_data.edge_pt_coords).create_delunay_surface_3d(output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Pia.vtk',return_hull=True)

In [ ]:
#pia=Surface().get_surface_from_contours(pia_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Pia.vtk',return_hull=True)
m1_lhs = Surface().get_surface_from_contours(m1_lhs_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/M1_lhs_sg.vtk')
m1_rhs = Surface().get_surface_from_contours(m1_rhs_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/M1_rhs_sg.vtk')
m2_lhs = Surface().get_surface_from_contours(m2_lhs_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/M2_lhs_sg.vtk')
m2_rhs = Surface().get_surface_from_contours(m2_rhs_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/M2_rhs_sg.vtk')
sbf_lhs = Surface().get_surface_from_contours(sbf_lhs_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/SBF_lhs_sg.vtk')
sbf_rhs = Surface().get_surface_from_contours(sbf_rhs_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/SBF_rhs_sg.vtk')
s1_fl_lhs = Surface().get_surface_from_contours(s1_fl_lhs_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/S1_fl_lhs_sg.vtk')
s1_fl_rhs = Surface().get_surface_from_contours(s1_fl_rhs_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/S1_fl_rhs_sg.vtk')
s2_lhs = Surface().get_surface_from_contours(s2_lhs_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/S2_lhs_sg.vtk')
s2_rhs = Surface().get_surface_from_contours(s2_rhs_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/S2_rhs_sg.vtk')
s1ulp_lhs = Surface().get_surface_from_contours(s1ulp_lhs_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/S1ULp_lhs.vtk')
s1ulp_rhs = Surface().get_surface_from_contours(s1ulp_rhs_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/S1ULp_rhs.vtk')

In [ ]:
insular_rhs_aid_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_rhs_AID.am',generic_graph=True)
insular_rhs_aip_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_rhs_AIP.am',generic_graph=True)
insular_rhs_aiv_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_rhs_AIV.am',generic_graph=True)
insular_rhs_di_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_rhs_DI.am',generic_graph=True)
insular_rhs_gi_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_rhs_GI.am',generic_graph=True)

insular_lhs_aid_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_lhs_AID.am',generic_graph=True)
insular_lhs_aip_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_lhs_AIP.am',generic_graph=True)
insular_lhs_aiv_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_lhs_AIV.am',generic_graph=True)
insular_lhs_di_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_lhs_DI.am',generic_graph=True)
insular_lhs_gi_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_lhs_GI.am',generic_graph=True)

insular_rhs = Surface()
insular_rhs_aid = Surface().get_surface_from_contours(insular_rhs_aid_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_rhs_AID.vtk')
insular_rhs.append(insular_rhs_aid.surface)
insular_rhs_aip = Surface().get_surface_from_contours(insular_rhs_aip_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_rhs_AIP.vtk')
insular_rhs.append(insular_rhs_aip.surface)
insular_rhs_aiv = Surface().get_surface_from_contours(insular_rhs_aiv_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_rhs_AIV.vtk')
insular_rhs.append(insular_rhs_aiv.surface)
insular_rhs_di = Surface().get_surface_from_contours(insular_rhs_di_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_rhs_DI.vtk')
insular_rhs.append(insular_rhs_di.surface)
insular_rhs_gi = Surface().get_surface_from_contours(insular_rhs_gi_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_rhs_GI.vtk')
insular_rhs.append(insular_rhs_gi.surface)
insular_rhs.write_surface_mesh(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_rhs.vtk')

insular_lhs = Surface()
insular_lhs_aid = Surface().get_surface_from_contours(insular_lhs_aid_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_lhs_AID.vtk')
insular_lhs.append(insular_lhs_aid.surface)
insular_lhs_aip = Surface().get_surface_from_contours(insular_lhs_aip_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_lhs_AIP.vtk')
insular_lhs.append(insular_lhs_aip.surface)
insular_lhs_aiv = Surface().get_surface_from_contours(insular_lhs_aiv_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_lhs_AIV.vtk')
insular_lhs.append(insular_lhs_aiv.surface)
insular_lhs_di = Surface().get_surface_from_contours(insular_lhs_di_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_lhs_DI.vtk')
insular_lhs.append(insular_lhs_di.surface)
insular_lhs_gi = Surface().get_surface_from_contours(insular_lhs_gi_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_lhs_GI.vtk')
insular_lhs.append(insular_lhs_gi.surface)
insular_lhs.write_surface_mesh(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Insular_lhs.vtk')

In [ ]:
pia_sg = AmiraSpatialGraph(output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Pia.am',generic_graph=True)
pia = Surface().get_surface_from_contours(pia_sg,output_filename=output_path_atlas+'Aligned_To_MG48_bottom_open_final_New/Pia_surf_from_contours.vtk')


In [ ]:
len(insular_rhs_aid_sg.graph_data.edge_list)

In [ ]:
# make canonical
tr_mat = [-1, 0, 0, 0, 0, 0, 1, 0, 0, -1, 0, 0, 0, 0, 0, 1]

output_path = output_path_atlas+'/canonical/'
pathlib.Path(output_path).mkdir(exist_ok=True)

aligner = Alignment()
aligner.set_transformation_matrix(txmat=tr_mat)
aligner.transform_surfaces_from_txmat(output_path_atlas,output_path,tr_mat)
aligner.write_transformation_matrix(filename=output_path+'Canonical_Txmat.txt')

In [ ]:
sg = AmiraSpatialGraph(input_path+'/atlas/MG48_cortex_merged_final.am',read_header_only=True)
sg1 = AmiraSpatialGraph(input_path+'/atlas/MG48_cortex_merged_final.am',generic_graph=True)
tr_mat = [-1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1]
sg1.apply_transformation(tr_mat)
sg.graph_data = sg.combine_subgraphs([sg.graph_data,sg1.graph_data])
sg.write_spatial_graph_2(output_path_atlas+'/canonical/MG48_cortex_merged_final.am')

In [ ]:
sg = AmiraSpatialGraph(output_path_atlas+'/canonical/Atlas_final.am',read_header_only=True)
sg1 = AmiraSpatialGraph(output_path_atlas+'/canonical/Atlas_final.am',generic_graph=True)
tr_mat = aligner.get_transformation_matrix()
sg1.apply_transformation(tr_mat)
sg.graph_data = sg.combine_subgraphs([sg.graph_data,sg1.graph_data])
sg.write_spatial_graph_2(output_path+'Atlas_final.am')

In [ ]:
# prepare mg48
mg48_pia = Surface()
mg48_pia_lhs = Surface(output_path_surfaces+'MG48_lhs_pia_open_bottom_voxel_size_50.vtk')
mg48_pia_rhs = Surface(output_path_surfaces+'MG48_rhs_pia_open_bottom_voxel_size_50.vtk')
mg48_pia.append(mg48_pia_lhs.surface)
mg48_pia.append(mg48_pia_rhs.surface)

#mg48_del = mg48_pia.create_delunay_surface_3d(output_filename=output_path_atlas+'MG48.vtk',return_hull=True)


In [ ]:
# clip at the bottom
mg48_open = Surface(output_path_atlas+'MG48.vtk')
mg48_open.clip_surface_at_given_y(0,y_limit=mg48_open.surface.GetBounds()[3]-100,get_top=True)
#mg48.clip_surface_at_given_y(0,y_limit=10000,get_top=True)
mg48_open.write_surface_mesh(output_path_atlas+'MG48_open.vtk')

Atlas_Pia_open = Surface(output_path_atlas+'/canonical/Atlas_Pia.vtk')
Atlas_Pia_open.clip_surface_at_given_y(0,y_limit=Atlas_Pia_open.surface.GetBounds()[3]-100,get_top=True)
#mg48.clip_surface_at_given_y(0,y_limit=10000,get_top=True)
Atlas_Pia_open.write_surface_mesh(output_path_atlas+'/canonical/Atlas_Pia_open.vtk')

In [ ]:
output_path = output_path_atlas+'/Aligned_To_MG48/'
pathlib.Path(output_path).mkdir(exist_ok=True)

aligner = Alignment(mg48_open.surface,Atlas_Pia_open.surface,mode='similarity')

aligner.transform_surfaces_from_txmat(output_path_atlas+'/canonical/',output_path,aligner.get_transformation_matrix())
aligner.write_transformation_matrix(filename=output_path+'Txmat.txt')


sg = AmiraSpatialGraph(output_path_atlas+'/canonical/Atlas_final.am',read_header_only=True)
sg1 = AmiraSpatialGraph(output_path_atlas+'/canonical/Atlas_final.am',generic_graph=True)
tr_mat = aligner.get_transformation_matrix()
sg1.apply_transformation(tr_mat)
sg.graph_data = sg.combine_subgraphs([sg.graph_data,sg1.graph_data])
sg.write_spatial_graph_2(output_path+'Atlas_final.am')

## Correct atlas sg

In [ ]:
sg1 = AmiraSpatialGraph(input_path+'atlas_correction/atlas_M3.am',generic_graph=True)

In [ ]:
sg2 = AmiraSpatialGraph(input_path+'atlas_correction/atlas_M4.am',generic_graph=True)

In [ ]:
sg = AmiraSpatialGraph(generic_graph=True)
for edge in sg1.graph_data.edge_list:
    sg.graph_data.add_edge(edge[0],edge[len(edge)-1])
for edge in sg2.graph_data.edge_list:
    sg.graph_data.add_edge(edge[0],edge[len(edge)-1])

sg.write_spatial_graph(input_path+'atlas_correction/atlas_M2_fixed.am')

## Find Overlap between atlas and our regions

# End

# Align Surfaces

## Align Tangential To Coronal

In [ ]:
#LHS

In [ ]:
# Align tangential to coronal surfaces
# first align lhs to lhs using vm1 surface and vs1 rabies landmarks
#ref_surf_coronal_lhs = Surface(filename=output_path_surfaces+'MG48_lhs_pia_open_bottom_voxel_size_50.vtk')
ref_surf_coronal_lhs_vM1 = Surface(filename=output_path_vM1+'MG48_lhs_vM1_surf_res_50_voxel_size_500_cutoff_0.05.vtk')
vs1_rabies_cluster = Landmarks(output_path_landmarks+'Density_Clusters/MG48_lhs_vS1_rabies_landmarks_voxel_500_cutoff_0.05.LandmarkAscii')
ref_surf_coronal_lhs_vS1_rabies_surf = Surface(pts=vs1_rabies_cluster.pts)
ref_surf = Surface(polydata=ref_surf_coronal_lhs_vM1.surface)
ref_surf.append(ref_surf_coronal_lhs_vS1_rabies_surf.surface)

output_path = output_path_alignment + '/Tangential_To_Coronal/'
pathlib.Path(output_path).mkdir(exist_ok=True)
output_path =   output_path + '/LHS/'
pathlib.Path(output_path).mkdir(exist_ok=True)

for exp in ['MG48_lhs']:#exp_names:
    surf = Surface(filename=output_path_alignment + 'Semi_Coronal' + '/vM1_Ref_Surfaces/' + exp + '_vM1_surf_res_50_voxel_size_500_cutoff_0.05.vtk')
    #surf.append(Surface(filename=output_path_alignment + 'Semi_Coronal' + '/Surfaces/' + exp + '_WM_open_bottom_voxel_size_450.vtk').surface)
    vs1_rabies_landamrks = Landmarks(output_path_landmarks+'Density_Clusters/{}_vS1_rabies_landmarks_voxel_500_cutoff_0.05.LandmarkAscii'.format(exp))
    vs1_rabies_landamrks_surf = Surface(pts=vs1_rabies_landamrks.pts)
    surf.append(vs1_rabies_landamrks_surf.surface)
    
    if exp.find('_lhs')>0:
        aligner = Alignment(ref_surf.surface,surf.surface)
    
        txmat = aligner.get_transformation_matrix()
        icp = aligner.get_icp_transform()
        aligner.transform_folders(output_path_alignment + '/Semi_Coronal/',output_path,exp,icp=icp,txmat = txmat,\
                                txfilename = output_path_txmats+exp+'Tangential_Coronal_Transformation_Matrix.txt')
        aligner.transform_folders(output_path_alignment + '/Semi_Coronal/'+'/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat)
        #aligner.transform_folders(output_path_alignment + '/Semi_Coronal/'+'/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat)
    

In [ ]:
# MG50_lhs and rhs to coronal aligned mg49 lhs
ref_surf_tangential_lhs_pia = Surface(filename=output_path_alignment + '/Tangential_To_Coronal/LHS/Surfaces/'+'MG49_lhs_pia_open_bottom_voxel_size_50.vtk'.format(exp[0:4]))
ref_surf_tangential_lhs_wm = Surface(filename=output_path_alignment + '/Tangential_To_Coronal/LHS/Surfaces/'+'MG49_lhs_WM_open_bottom_voxel_size_50.vtk'.format(exp[0:4]))

ref_surf = Surface(polydata=ref_surf_tangential_lhs_pia.surface)
ref_surf.append(ref_surf_tangential_lhs_wm.surface)

for exp in ['MG49_rhs',  ]:
    if exp.find('_rhs')>0: 

        output_path = output_path_alignment + '/Tangential_To_Coronal/RHS_Mirrored/'
        pathlib.Path(output_path).mkdir(exist_ok=True)
        
        surf_tangential_lhs_pia = Surface(filename=output_path_alignment + '/RHS_Mirrored/Surfaces/'+'{}_pia_open_bottom_voxel_size_50.vtk'.format(exp))
        surf_tangential_lhs_wm = Surface(filename=output_path_alignment + '/RHS_Mirrored/Surfaces/'+'{}_WM_open_bottom_voxel_size_50.vtk'.format(exp))

    elif exp.find('MG50_lhs')==0:
        # MG 50 lhs
        output_path = output_path_alignment + '/Tangential_To_Coronal/LHS/'
        pathlib.Path(output_path).mkdir(exist_ok=True)
        surf_tangential_lhs_pia = Surface(filename=output_path_alignment + 'Semi_Coronal/Surfaces/'+'{}_pia_open_bottom_voxel_size_50.vtk'.format(exp))
        surf_tangential_lhs_wm = Surface(filename=output_path_alignment + 'Semi_Coronal/Surfaces/'+'{}_WM_open_bottom_voxel_size_50.vtk'.format(exp))

    surf = Surface(polydata=surf_tangential_lhs_pia.surface)
    surf.append(surf_tangential_lhs_wm.surface)

    print(ref_surf.surface.GetNumberOfCells(),surf.surface.GetNumberOfCells())
    aligner = Alignment(ref_surf.surface,surf.surface)

    txmat = aligner.get_transformation_matrix()
    icp = aligner.get_icp_transform()
    
    if exp.find('_rhs')>0:
        aligner.transform_folders(output_path_alignment + '/RHS_Mirrored/',output_path,exp,icp=icp,txmat = txmat,\
                            txfilename = output_path_txmats+exp+'Tangential_Coronal_Transformation_Matrix.txt')
        aligner.transform_folders(output_path_alignment + '/RHS_Mirrored/'+'/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat)
        aligner.transform_folders(output_path_alignment + '/RHS_Mirrored/'+'/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat)

    else:
        aligner.transform_folders(output_path_alignment + '/Semi_Coronal/',output_path,exp,icp=icp,txmat = txmat,\
                            txfilename = output_path_txmats+exp+'Tangential_Coronal_Transformation_Matrix.txt')
        aligner.transform_folders(output_path_alignment + '/Semi_Coronal/'+'/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat)
        aligner.transform_folders(output_path_alignment + '/Semi_Coronal/'+'/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat)

    

In [ ]:
# # Rhs mirrored and aligned with vm1 and vs1 surfaces
# for exp in exp_names_tangential:
#     if exp.find('_rhs')>0: 
#         ref_surf_tangential_lhs_vM1 = Surface(filename=output_path_alignment + '/Tangential_To_Coronal/LHS/vM1_Ref_Surfaces/'+'{}_lhs_vM1.vtk'.format(exp[0:4]))
#         ref_surf_tangential_lhs_vS1 = Surface(filename=output_path_alignment + '/Tangential_To_Coronal/LHS/vS1_Ref_Surfaces/'+'{}_lhs_vS1_hull.vtk'.format(exp[0:4]))

#         ref_surf = Surface(polydata=ref_surf_tangential_lhs_vM1.surface)
#         ref_surf.append(ref_surf_tangential_lhs_vS1.surface)

#         output_path = output_path_alignment + '/Tangential_To_Coronal/RHS_Mirrored/'
#         pathlib.Path(output_path).mkdir(exist_ok=True)
        
#         surf_tangential_lhs_vM1 = Surface(filename=output_path_alignment + '/RHS_Mirrored/vM1_Ref_Surfaces/'+'{}_vM1.vtk'.format(exp))
#         surf_tangential_lhs_vS1 = Surface(filename=output_path_alignment + '/RHS_Mirrored/vS1_Ref_Surfaces/'+'{}_vS1_hull.vtk'.format(exp))
        
#         surf = Surface(polydata=surf_tangential_lhs_vM1.surface)
#         surf.append(surf_tangential_lhs_vS1.surface)
        
#         print(ref_surf.surface.GetNumberOfCells(),surf.surface.GetNumberOfCells())
#         aligner = Alignment(ref_surf.surface,surf.surface)
    
#         txmat = aligner.get_transformation_matrix()
#         icp = aligner.get_icp_transform()
#         aligner.transform_folders(output_path_alignment + '/RHS_Mirrored/',output_path,exp,icp=icp,txmat = txmat,\
#                                 txfilename = output_path_txmats+exp+'Tangential_Coronal_Transformation_Matrix.txt')
#         aligner.transform_folders(output_path_alignment + '/RHS_Mirrored/'+'/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat)
#         aligner.transform_folders(output_path_alignment + '/RHS_Mirrored/'+'/Landmarks/',output_path+'/Landmarks/',exp,icp=icp,txmat = txmat)
    
        

In [ ]:
# Flip mirrored RHS and refine with coronal rhs
output_path = output_path_alignment + '/Tangential_To_Coronal/RHS/'
pathlib.Path(output_path).mkdir(exist_ok=True)
for exp in ['MG49_rhs']:#exp_names_tangential:
    if exp.find('_rhs')>0: 
        #for exp in exp_names_tangential:
        aligner = Alignment()    
        aligner.transform_rhs_into_lhs(output_path_alignment + '/Tangential_To_Coronal/RHS_Mirrored/' + '/',output_path)
        aligner.transform_rhs_into_lhs(output_path_alignment + '/Tangential_To_Coronal/RHS_Mirrored/' + '/'+'/Landmarks/',output_path+'/Landmarks/')


In [ ]:
# translate to match rhs 
output_path = output_path_alignment + '/Tangential_To_Coronal/RHS_Translated/'
pathlib.Path(output_path).mkdir(exist_ok=True)
#ref_rhs_surf = Surface(output_path_surfaces+'/MG48_rhs_pia_open_bottom_voxel_size_50.vtk')
#ref_tr_x = ref_rhs_surf.surface.GetBounds()[0]

for exp in ['MG49_rhs']:#exp_names_tangential:
    if exp.find('_rhs')>0: 
        
        lhs_surf = Surface(output_path_alignment + '/Tangential_To_Coronal/RHS_Mirrored/Surfaces'+'/{}_rhs_pia_open_bottom_voxel_size_50.vtk'.format(exp[0:4]))
        ref_tr_x = lhs_surf.surface.GetBounds()[1]

        rhs_surf = Surface(output_path_alignment + '/Tangential_To_Coronal/RHS/Surfaces'+'/{}_rhs_pia_open_bottom_voxel_size_50.vtk'.format(exp[0:4]))
        tr_x = rhs_surf.surface.GetBounds()[0]
        
        print(ref_tr_x-tr_x)
        txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, ref_tr_x-tr_x,0,0,1]
        aligner = Alignment()
        aligner.set_transformation_matrix(txmat)
        aligner.transform_folders(output_path_alignment + '/Tangential_To_Coronal/RHS/',output_path,exp,icp=None,txmat = txmat,\
                                txfilename = output_path_txmats+exp+'_Canonical_Transformation_Matrix.txt',\
                                 apply_polydata_transform=True)
        
        aligner.transform_folders(output_path_alignment + '/Tangential_To_Coronal/RHS/Landmarks/',output_path+'/Landmarks/',exp,icp=None,txmat = txmat,\
                                txfilename = output_path_txmats+exp+'_Canonical_Transformation_Matrix.txt',\
                                 apply_polydata_transform=True)

In [ ]:
# align to match coronal rhs 
output_path = output_path_alignment + '/Tangential_To_Coronal/RHS_Aligned/'
pathlib.Path(output_path).mkdir(exist_ok=True)

ref_surf_coronal_rhs_pia = Surface(filename=output_path_surfaces + '/MG48_rhs_pia_open_bottom_voxel_size_50.vtk')
ref_surf_coronal_rhs_wm = Surface(filename=output_path_alignment + '/MG48_rhs_WM_open_bottom_voxel_size_50.vtk')

ref_surf = Surface(polydata=ref_surf_coronal_rhs_pia.surface)
ref_surf.append(ref_surf_coronal_rhs_wm.surface)

for exp in ['MG48_rhs']:#exp_names:
    if exp.find('MG48_rhs')>=0:
        # just keep the MG48 rhs in the same folder
        
        aligner = Alignment()
        txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
        
        aligner.transform_folders(output_root ,output_path,exp,icp=None,
                                  txmat = txmat,apply_polydata_transform=True,\
                                txfilename = output_path_txmats+exp+'Tangential_Coronal_Transformation_Matrix.txt')
        aligner.transform_folders(output_root +'/Landmarks/',\
                                  output_path+'/Landmarks/',exp,icp=None,\
                                  txmat = txmat,apply_polydata_transform=True)
    elif exp.find('_rhs')>0: 
        pia = Surface( output_path_alignment + '/Tangential_To_Coronal/RHS_Translated/Surfaces/{}_pia_open_bottom_voxel_size_50.vtk'.format(exp))
        wm = Surface( output_path_alignment + '/Tangential_To_Coronal/RHS_Translated/Surfaces/{}_WM_open_bottom_voxel_size_50.vtk'.format(exp))
        
        surf = pia
        surf.append(wm.surface)
        
        print(ref_surf.surface.GetNumberOfCells(),surf.surface.GetNumberOfCells())
        aligner = Alignment(ref_surf.surface,surf.surface)
        txmat = aligner.get_transformation_matrix()
        icp = aligner.get_icp_transform()
        
        aligner.transform_folders(output_path_alignment +'/Tangential_To_Coronal/RHS_Translated/',output_path,exp,icp=icp,txmat = txmat,\
                                txfilename = output_path_txmats+exp+'Tangential_Coronal_Transformation_Matrix.txt')
        aligner.transform_folders(output_path_alignment + '/Tangential_To_Coronal/RHS_Translated/'+'/Landmarks/',\
                                  output_path+'/Landmarks/',exp,icp=icp,txmat = txmat)
        

In [ ]:
# align to match coronal lhs 
output_path = output_path_alignment + '/Tangential_To_Coronal/LHS_Aligned/'
pathlib.Path(output_path).mkdir(exist_ok=True)

ref_surf_coronal_rhs_pia = Surface(filename=output_path_surfaces + '/MG48_lhs_pia_open_bottom_voxel_size_50.vtk')
ref_surf_coronal_rhs_wm = Surface(filename=output_path_alignment + '/MG48_lhs_WM_open_bottom_voxel_size_50.vtk')

ref_surf = Surface(polydata=ref_surf_coronal_rhs_pia.surface)
ref_surf.append(ref_surf_coronal_rhs_wm.surface)

for exp in ['MG48_lhs']:#exp_names:
    if exp.find('MG48_lhs')>=0:
        # just keep the MG48 rhs in the same folder
        
        aligner = Alignment()
        txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]
        
        aligner.transform_folders(output_root ,output_path,exp,icp=None,
                                  txmat = txmat,apply_polydata_transform=True,\
                                txfilename = output_path_txmats+exp+'Tangential_Coronal_Transformation_Matrix.txt')
        aligner.transform_folders(output_root +'/Landmarks/',\
                                  output_path+'/Landmarks/',exp,icp=None,\
                                  txmat = txmat,apply_polydata_transform=True)
    elif exp.find('_lhs')>0: 
        pia = Surface( output_path_alignment + '/Tangential_To_Coronal/LHS/Surfaces/{}_pia_open_bottom_voxel_size_50.vtk'.format(exp))
        wm = Surface( output_path_alignment + '/Tangential_To_Coronal/LHS/Surfaces/{}_WM_open_bottom_voxel_size_50.vtk'.format(exp))
        
        surf = pia
        surf.append(wm.surface)
        
        print(ref_surf.surface.GetNumberOfCells(),surf.surface.GetNumberOfCells())
        aligner = Alignment(ref_surf.surface,surf.surface)
        txmat = aligner.get_transformation_matrix()
        icp = aligner.get_icp_transform()
        
        aligner.transform_folders(output_path_alignment +'/Tangential_To_Coronal/LHS/',output_path,exp,icp=icp,txmat = txmat,\
                                txfilename = output_path_txmats+exp+'Tangential_Coronal_Transformation_Matrix.txt')
        aligner.transform_folders(output_path_alignment + '/Tangential_To_Coronal/LHS/'+'/Landmarks/',\
                                  output_path+'/Landmarks/',exp,icp=icp,txmat = txmat)

## Create Coronal Avg


In [ ]:
def get_rays(center,theta_res=33,phi_res=32,):
    pia_sphere = vtk.vtkSphereSource()
    pia_sphere.SetCenter(center)
    pia_sphere.SetRadius(1)
    pia_sphere.SetThetaResolution(33)
    pia_sphere.SetPhiResolution(32)
    pia_sphere.SetStartPhi(0)
    pia_sphere.SetEndPhi(360)
    pia_sphere.SetStartTheta(0)
    pia_sphere.SetEndTheta(360)
    #pia_sphere.SetLatLongTessellation(True)
    pia_sphere.Update()

    pia_cell_centers = vtk.vtkCellCenters()
    pia_cell_centers.SetInputData(pia_sphere.GetOutput())
    pia_cell_centers.Update()
    pia_centroid_cell_centers = pia_cell_centers.GetOutput()
    ##print('pia_centroid_cell_centers {}'.format(pia_centroid_cell_centers))
    pia_sphere_normals = vtk.vtkPolyDataNormals()
    pia_sphere_normals.SetInputConnection(pia_sphere.GetOutputPort())
    pia_sphere_normals.ComputePointNormalsOff()
    pia_sphere_normals.ComputeCellNormalsOn()
    pia_sphere_normals.SplittingOff()
    pia_sphere_normals.FlipNormalsOff()
    pia_sphere_normals.AutoOrientNormalsOn()
    pia_sphere_normals.Update()
    pia_source_normals = pia_sphere_normals.GetOutput().GetCellData().GetNormals()
    
    rays_sg = AmiraSpatialGraph(generic_graph=True)
    phi_step = int((360-0)/theta_res)
    theta_step = int((360-0)/phi_res)
    idx = -1
    # for phi in range(0,360-phi_step,phi_step):
    # for theta in range(0,360-theta_step,theta_step):
    for idx in range(pia_centroid_cell_centers.GetNumberOfPoints()-1):
        idx = idx + 1
        point = pia_centroid_cell_centers.GetPoint(idx)
        ##print(point)
        normal = pia_source_normals.GetTuple(idx)

        # Calculate the 'target' of the ray based on 'RayCastLength'
        pointRayTarget = list(np.array(point) + 10000*np.array(normal))
        rays_sg.graph_data.add_edge(point,pointRayTarget)
    
    return rays_sg

In [ ]:
def get_obb_tree_list(surf_list):
    obb_tree_list = []
    for surf in surf_list:
        obb = vtk.vtkOBBTree()
        obb.SetDataSet(surf)
        obb.BuildLocator()
        obb_tree_list.append(obb)
    return obb_tree_list

In [ ]:
def get_avg_pts(rays,obb_tree_list):
    avg_pts = []
    for edge in rays_sg.graph_data.edge_list:
        intersected_pts = []
        for obb in obb_tree_list:
            points = vtk.vtkPoints()
            code = obb.IntersectWithLine(edge[1], edge[0], points, None)
            if points.GetNumberOfPoints() > 0:
                intersected_pts.append(points.GetPoint(0))
        if len(intersected_pts) > 1:
            avg_pts.append(np.array(intersected_pts).mean(axis=0))
            
        elif len(intersected_pts) > 0:
            avg_pts.append(intersected_pts[0])
    return avg_pts

In [ ]:
# Write Aligned Surfaces
for hem in ['LHS','RHS']:
    output_path = output_path_alignment+'Tangential_To_Coronal/Average/'
    pathlib.Path(output_path).mkdir(exist_ok=True)

    output_path = output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/'
    pathlib.Path(output_path).mkdir(exist_ok=True)

    aligned_pias = Surface()
    aligned_wms = Surface()

    surf_list_pias = []
    surf_list_wms = []

    if hem == 'LHS':
        exp_list = ['MG48_lhs','MG49_lhs','MG50_lhs']
    else:
        exp_list = ['MG48_rhs','MG49_rhs','MG50_rhs']

    for exp in exp_list: 
        print(exp)
        #if exp.find('MG48') == 0 :
        pia = Surface(output_path_alignment+'Tangential_To_Coronal/' + '{}_Aligned/Surfaces/'.format(hem)+ \
                      exp+'_pia_open_bottom_voxel_size_50.vtk')
        wm = Surface(output_path_alignment+'Tangential_To_Coronal/' + '{}_Aligned/Surfaces/'.format(hem)+ \
                      exp+'_WM_open_bottom_voxel_size_50.vtk')
        aligned_pias.append(pia.surface)
        aligned_wms.append(wm.surface)
        surf_list_pias.append(pia.surface)
        surf_list_wms.append(wm.surface)

    Surface(polydata=aligned_pias.surface).write_surface_mesh(output_path+'pias.vtk')
    Surface(polydata=aligned_pias.surface).write_surface_mesh(output_path+'wms.vtk')



In [ ]:
# Get rays for casting
all_surfs =  Surface()
all_surfs.append(Surface(output_path_alignment+'Tangential_To_Coronal/Average/LHS/'+'pias.vtk').surface)
all_surfs.append(Surface(output_path_alignment+'Tangential_To_Coronal/Average/RHS/'+'pias.vtk').surface)
centroid = all_surfs.get_center_of_mass()
Landmarks(pts=[centroid]).write_landmarks(output_path_alignment+'Tangential_To_Coronal/Average/'+'Rays_Origin.LandmarkAscii')

rays_sg = get_rays(centroid)
rays_sg.write_spatial_graph(output_path_alignment+'Tangential_To_Coronal/Average/'+'Rays.am')


In [ ]:
# cast rays and get avg pts
#surf_res = 50
rays_sg = AmiraSpatialGraph(output_path_alignment+'Tangential_To_Coronal/Average/Rays.am',generic_graph=True)
for  surf_res in surface_resolutions:
    for hem in ['LHS','RHS']:
        surf_list_pias = []
        surf_list_wms = []
        output_path = output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/'
        if hem == 'LHS':
            exp_list = ['MG48_lhs','MG49_lhs','MG50_lhs']
        else:
            exp_list = ['MG48_rhs','MG49_rhs','MG50_rhs']

        for exp in exp_list: 
            print(exp)
            #if exp.find('MG48') == 0 :
            pia = Surface(output_path_alignment+'Tangential_To_Coronal/' + '{}_Aligned/Surfaces/'.format(hem)+ \
                          exp+'_pia_open_bottom_voxel_size_{}.vtk'.format(surf_res))
            wm = Surface(output_path_alignment+'Tangential_To_Coronal/' + '{}_Aligned/Surfaces/'.format(hem)+ \
                          exp+'_WM_open_bottom_voxel_size_{}.vtk'.format(surf_res))

            surf_list_pias.append(pia.surface)
            surf_list_wms.append(wm.surface)

        obb_pias = get_obb_tree_list(surf_list_pias)
        obb_wms = get_obb_tree_list(surf_list_wms)

        avg_pts_pia = get_avg_pts(rays_sg,obb_pias)
        avg_pts_wm = get_avg_pts(rays_sg,obb_wms)

        Landmarks(pts = avg_pts_pia).write_landmarks(output_path+'Avg_Pia_Pts_{}.LandmarkAscii'.format(surf_res))
        Landmarks(pts = avg_pts_wm).write_landmarks(output_path+'Avg_WM_Pts_{}.LandmarkAscii'.format(surf_res))

In [ ]:
# Generate Surface
for surface_resolution in surface_resolutions:
    for hem in ['LHS','RHS']:
        output_path = output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/'
        pia_pts = Landmarks(filename=output_path + '/Avg_Pia_Pts_{}.LandmarkAscii'.format(surface_resolution))
        pia_surf = Surface(pts=pia_pts.pts)
        bounds = pia_surf.surface.GetBounds()
        #x_span =  bounds[1] - bounds[0]
        if hem == 'LHS':
            clip_at_x = bounds[1] - surface_resolution
        else:
            clip_at_x = bounds[0] + surface_resolution
            
        offset = 0#z_span - 5000
        pia_surf.create_surface_from_unorganized_points(pia_pts.pts,sample_resolution=surface_resolution,
                                                                        clip_offset=0,limit=clip_at_x,clipping_plane='x',\
                                                output_filename=output_path+'/Avg_Pia_{}.vtk'.format(surface_resolution))

        wm_pts = Landmarks(filename=output_path + '/Avg_WM_Pts_{}.LandmarkAscii'.format(surface_resolution))
        wm_surf = Surface(pts=wm_pts.pts)
        wm_surf.create_surface_from_unorganized_points(wm_pts.pts,sample_resolution=surface_resolution,
                                                                        clip_offset=0,limit=clip_at_x,clipping_plane='x',\
                                                output_filename=output_path+'/Avg_WM_{}.vtk'.format(surface_resolution))

    

In [ ]:
# Generate hull
for hem in ['LHS','RHS']:
    output_path = output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/'
    pia_pts = Landmarks(filename=output_path + '/Avg_Pia_Pts.LandmarkAscii')
    pia_surf = Surface(pts=pia_pts.pts)
    pia_surf.create_delunay_surface_3d(bounding_offset=50,output_filename=output_path+'/Avg_Pia_Hull.vtk')

    wm_pts = Landmarks(filename=output_path + '/Avg_WM_Pts.LandmarkAscii')
    wm_surf = Surface(pts=wm_pts.pts)
    pia_surf.create_delunay_surface_3d(output_filename=output_path+'/Avg_WM_Hull.vtk')

In [ ]:
# Generate Axis Fields using 450 res and extend to other res

for hem in ['LHS','RHS']:
    output_path = output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/'
    pia = Surface(output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/Avg_Pia_450.vtk')
    wm = Surface(output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/Avg_WM_450.vtk')
    #wm = wm.divide_surface(4,wm.surface)
    
    if hem=='LHS':
        flip_normals = False
    else:
        flip_normals = True
    vectors_wm_pia_flipped = wm.create_axis_field(pia,op_sg_name=None,\
                         flip_normals=flip_normals, return_axes = True,max_allowed_axis_length=4000)
#     vectors_wm_pia = Surface(polydata=wm).create_axis_field(pia,op_sg_name=None,\
#                          flip_normals=False, return_axes = True,max_allowed_axis_length=4000)
    axis_field_sg = AmiraSpatialGraph(generic_graph=True)
    for edge in vectors_wm_pia_flipped:
        axis_field_sg.graph_data.add_edge(edge[0],edge[1])
#     for edge in vectors_wm_pia:
#         axis_field_sg.graph_data.add_edge(edge[0],edge[1])
    axis_field_sg.write_spatial_graph(output_path+'Avg_Axis_Field.am')
    
    for surf_res in surface_resolutions:
        pia = Surface(output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/Avg_Pia_{}.vtk'.format(surf_res))
        wm = Surface(output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/Avg_WM_{}.vtk'.format(surf_res))
        selected_axis_field_sg,wm_pts,pia_pts,missing_axis_field = pia.get_axis_field_within_instersecting_surface_with_trimming\
        (pia.surface,wm.surface,axis_field_sg.graph_data.edge_list,vec_extension_offset=1)
        selected_axis_field_sg.write_spatial_graph(output_path+'Avg_Axis_Field_{}.am'.format(surf_res))

## Create Avg vM1 Using Aligned Rabies Landmarks


In [ ]:
# Read all rabies
output_path = output_path_alignment+'Tangential_To_Coronal/Average/vM1/'
pathlib.Path(output_path).mkdir(exist_ok=True)

for hem in ['LHS','RHS']:
    aligned_landmarks = Landmarks()
    for exp in exp_names:
        if hem == 'LHS':
            path = output_path_alignment+'Tangential_To_Coronal/{}/'.format(hem)+'Landmarks/Density_Clusters/'+\
                    exp + '_vM1_rabies_landmarks_voxel_500_cutoff_0.05.LandmarkAscii'
        else:
            path = output_path_alignment+'Tangential_To_Coronal/{}_Aligned/'.format(hem)+'Landmarks/Density_Clusters/'+\
                    exp + '_vM1_rabies_landmarks_voxel_500_cutoff_0.05.LandmarkAscii'
                
        if os.path.exists(path):
            
            rabies = Landmarks(path).pts
            
            aligned_landmarks.append_landmarks(rabies)
    aligned_landmarks.write_landmarks(output_path+hem+'_vM1_Rabies.landmarksAscii')

In [ ]:
#for voxel_size in [500]:#[50,100,200,300,400,500]:
#    for cutoff in [0.05]:#[0,0.05,0.1,0.15,0.2,0.3,0.4,0.5]:
       # for exp in ['MG49_lhs']:#exp_names:
#                 exp='MG50_rhs'
#                 voxel_size = 100
#                 cutoff = 0.3
#cutting_order = [True,True,False,True]

cutting_order = [True,True,True,False]





voxel_size = 500
cutoff = 0.05
surf_res = 50
for hem in ['RHS']:
    pia = Surface(output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/Avg_Pia_{}.vtk'.format(surf_res))
    wm = Surface(output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/Avg_WM_{}.vtk'.format(surf_res))
    #print(exp,voxel_size,cutoff)
    vm1_rabies_pts = Landmarks(output_path_alignment+'Tangential_To_Coronal/Average/vM1/'+hem+'_vM1_Rabies.landmarksAscii').pts
    vm1_axis_field = AmiraSpatialGraph(output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/Avg_Axis_Field_{}.am'.format(50),generic_graph=True)

    print(len(vm1_rabies_pts),len(vm1_axis_field.graph_data.edge_list))

    #if len(vm1_rabies_pts) > 20:
    pcas,neg_pcas,central_axis = get_pca(vm1_rabies_pts,vm1_axis_field.graph_data.edge_list,pia,wm,\
                                         output_path_alignment+'Tangential_To_Coronal/Average/vM1/'+hem)

    print(len(pcas),len(neg_pcas))
    plane_right,plane_left,plane_front,plane_back = create_vm1_surface(pia.surface,wm.surface,pcas,neg_pcas,\
                                                    vm1_axis_field.graph_data.edge_list,landmarks=vm1_rabies_pts,\
                                                    central_axis=central_axis,cutting_order=cutting_order)

    for surf_res in surface_resolutions:
        pia = Surface(output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/Avg_Pia_{}.vtk'.format(surf_res))
        wm = Surface(output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/Avg_WM_{}.vtk'.format(surf_res))
        
        vm1_final,vm1_hull,vm1_pia,vm1_wm = Surface().clip_surfaces_using_given_planes(pia.surface,wm.surface,\
                                                               plane_right,plane_left,\
                                                               plane_front,plane_back,landmarks=[],\
                                                               central_axis=central_axis,\
                                                               order_of_cutting=cutting_order)

        Surface(polydata=vm1_final).write_surface_mesh(output_path_alignment+'Tangential_To_Coronal/Average/vM1/'+hem+'_vM1_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.\
                                                       format(surf_res,voxel_size,cutoff))
        Surface(polydata=vm1_hull).write_surface_mesh(output_path_alignment+'Tangential_To_Coronal/Average/vM1/'+hem+'_vM1_hull_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.\
                                                      format(surf_res,voxel_size,cutoff))
        Surface(polydata=vm1_pia).write_surface_mesh(output_path_alignment+'Tangential_To_Coronal/Average/vM1/'+hem+'_vm1_pia_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.\
                                                     format(surf_res,voxel_size,cutoff))
        Surface(polydata=vm1_wm).write_surface_mesh(output_path_alignment+'Tangential_To_Coronal/Average/vM1/'+hem+'_vm1_wm_surf_res_{}_voxel_size_{}_cutoff_{}.vtk'.\
                                                    format(surf_res,voxel_size,cutoff))


In [ ]:
# Find vM1 axis field
for surf_res in surface_resolutions:
    for hem in ['LHS','RHS']:
        axis_field_sg = AmiraSpatialGraph(output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/Avg_Axis_Field_{}.am'.format(surf_res),\
                          generic_graph=True,)
    
        pia = Surface(output_path_alignment+'Tangential_To_Coronal/Average/vM1/'+hem+'_vm1_pia_surf_res_{}_voxel_size_500_cutoff_0.05.vtk'.format(surf_res))
        wm = Surface(output_path_alignment+'Tangential_To_Coronal/Average/vM1/'+hem+'_vm1_wm_surf_res_{}_voxel_size_500_cutoff_0.05.vtk'.format(surf_res))
        selected_axis_field_sg,wm_pts,pia_pts,missing_axis_field = pia.get_axis_field_within_instersecting_surface_with_trimming\
        (pia.surface,wm.surface,axis_field_sg.graph_data.edge_list,vec_extension_offset=1)
        selected_axis_field_sg.write_spatial_graph(output_path_alignment+'Tangential_To_Coronal/Average/vM1/'+hem+'Avg_Axis_Field_{}.am'.format(surf_res))

In [ ]:
# generate avg vm1 layer borders
# neun
output_path = output_path_alignment+'Tangential_To_Coronal/Average/vM1/Layer_Borders/'
pathlib.Path(output_path).mkdir(exist_ok=True)
for surf_res in surface_resolutions:
    for hem in ['LHS','RHS']:
        vectors_sg = AmiraSpatialGraph(output_path_alignment+'Tangential_To_Coronal/Average/vM1/'+hem+'Avg_Axis_Field_{}.am'.format(surf_res),\
                          generic_graph=True,)
       
        lb = pd.read_csv(output_path_neun_layer_borders+'vM1_Borders.csv')
        for layer in ['L1_Border_Per','L2_3_Border_Per','L5A_Border_Per','L5B_Border_Per',]:
            layer_depth_mean = lb[layer].mean() * 100
            layer_depth_upper = layer_depth_mean - (lb[layer].std() * 100)
            layer_depth_lower = layer_depth_mean + (lb[layer].std() * 100)
            #print(layer_depth)
            for layer_depth in [layer_depth_mean,layer_depth_upper,layer_depth_lower]:
                layer_border_pts = Vectors().get_layer_pts_for_given_depth(vectors_sg.graph_data.edge_list,layer_depth,\
                                                                                              realtive_depths=True,invert_axes=True,rel_depth_given=True)
                layer_surf = Surface(pts=layer_border_pts).create_delunay_surface_2d(return_hull=True)
                # write mean, upper or lower layer border surface
                if layer_depth == layer_depth_mean:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_path+'_vM1_'+hem+layer[:-4]+'_{}_mean.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_path+'_vM1_'+hem+layer[:-4]+'_{}_mean.vtk'.format(surf_res))
                elif layer_depth == layer_depth_upper:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_path+'_vM1_'+hem+layer[:-4]+'_{}_upper.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_path+'_vM1_'+hem+layer[:-4]+'_{}_upper.vtk'.format(surf_res))
                else:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_path+'_vM1_'+hem+layer[:-4]+'_{}_lower.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_path+'_vM1_'+hem+layer[:-4]+'_{}_lower.vtk'.format(surf_res))



In [ ]:
# generate avg vm1 layer borders
# rabies
output_path = output_path_alignment+'Tangential_To_Coronal/Average/vM1/Layer_Borders/'
pathlib.Path(output_path).mkdir(exist_ok=True)
for surf_res in surface_resolutions:
    for hem in ['LHS','RHS']:
        vectors_sg = AmiraSpatialGraph(output_path_alignment+'Tangential_To_Coronal/Average/vM1/'+hem+'Avg_Axis_Field_{}.am'.format(surf_res),\
                          generic_graph=True,)
       
        lb = pd.read_csv(output_path_rabies_layers+'vM1_Borders.csv')
        for layer in ['L5_Upper_Border_Per','L5_Mean_Border_Per','L5_Lower_Border_Per',]:
            layer_depth_mean = lb[layer].mean() * 100
            
            #print(layer_depth)
            for layer_depth in [layer_depth_mean,layer_depth_upper,layer_depth_lower]:
                layer_border_pts = Vectors().get_layer_pts_for_given_depth(vectors_sg.graph_data.edge_list,layer_depth,\
                                                                                              realtive_depths=True,invert_axes=True,rel_depth_given=True)
                layer_surf = Surface(pts=layer_border_pts).create_delunay_surface_2d(return_hull=True)
                # write mean, upper or lower layer border surface
                if layer_depth == layer_depth_mean:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_path+'_vM1_Rabies_'+hem+layer[:-4]+'_{}_mean.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_path+'_vM1_Rabies_'+hem+layer[:-4]+'_{}_mean.vtk'.format(surf_res))
                

## Create Avg vS1


In [ ]:
# read vs1
output_path = output_path_alignment+'Tangential_To_Coronal/Average/vS1/'
pathlib.Path(output_path).mkdir(exist_ok=True)
for hem in ['LHS','RHS']:
    for surf_res in surface_resolutions:
        #for exp in ['MG49_lhs']:#exp_names_tangential:
        if hem == 'LHS':
            exp = 'MG49_lhs'
        else:
            exp = 'MG49_rhs'
            
        sg = AmiraSpatialGraph(output_path_alignment+'Tangential_To_Coronal/{}_Aligned/Surfaces/'.format(hem)+exp+'_barrels_{}.am'.format(surf_res),barrel_projections_present=True\
                               ,reverse_barrel_direction=True)
        all_barrel_columns_surfs = Surface()
        vs1_axis_field = AmiraSpatialGraph(generic_graph=True)
        for row in sg.barrels.rows_list:
            for barrel in row.single_barrels_list:
                # Extract barrel radius
                #print(barrel.contours.edge_pt_coords)
                if len(barrel.wm_projection_barrel_centroid) > 0 and len(barrel.pia_projection_barrel_centroid):
                    column_axis = [barrel.wm_projection_barrel_centroid,barrel.pia_projection_barrel_centroid]
                    
                    pia = Surface(output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/Avg_Pia_{}.vtk'.format(surf_res))
                    wm = Surface(output_path_alignment+'Tangential_To_Coronal/Average/'+hem+'/Avg_WM_{}.vtk'.format(surf_res))
                    
                    
                    selected_axis_field_sg,wm_pts,pia_pts,missing_axis_field = pia.get_axis_field_within_instersecting_surface_with_trimming\
                    (pia.surface,wm.surface,[column_axis],)
                    #selected_axis_field_sg.write_spatial_graph(output_path_alignment+'Tangential_To_Coronal/Average/vM1/'+hem+'Avg_Axis_Field_{}.am'.format(surf_res))
                    if len(selected_axis_field_sg.graph_data.edge_list)>0:
                        trimmed_edge = selected_axis_field_sg.graph_data.edge_list[0]
                        vs1_axis_field.graph_data.add_edge(trimmed_edge[0],trimmed_edge[1])
                        cylinder = get_cylinder(250,trimmed_edge)
                        surf = Surface(polydata=cylinder).create_delunay_surface_3d(return_hull=True,\
                                                                                    output_filename=output_path_vS1+exp+'_'+barrel.barrel_name+'.vtk')
                        all_barrel_columns_surfs.append(surf)

        Surface(polydata=all_barrel_columns_surfs.surface).write_surface_mesh(output_path+hem+'_all_barrel_columns_{}.vtk'.format(surf_res))

        Surface(polydata=all_barrel_columns_surfs.surface).create_delunay_surface_3d(output_filename=output_path+hem+'_vS1_hull_{}.vtk'.format(surf_res))

        vs1_axis_field.write_spatial_graph(output_path+hem+'_vS1_axis_field_{}.am'.format(surf_res))

In [ ]:
# Get Layers Neun
output_path = output_path_alignment+'Tangential_To_Coronal/Average/vS1/Layer_Borders/'
pathlib.Path(output_path).mkdir(exist_ok=True)
for surf_res in surface_resolutions:
    for hem in ['LHS','RHS']:
        vectors_sg = AmiraSpatialGraph(output_path_alignment+'Tangential_To_Coronal/Average/vS1/'+hem+'_vS1_axis_field_{}.am'.format(surf_res),\
                          generic_graph=True,)
       
        lb = pd.read_csv(output_path_neun_layer_borders+'vS1_Borders.csv')
        for layer in ['L1_Border_Per','L2_3_Border_Per','L4_Border_Per','L5_Border_Per',]:
            layer_depth_mean = lb[layer].mean() * 100
            layer_depth_upper = layer_depth_mean - (lb[layer].std() * 100)
            layer_depth_lower = layer_depth_mean + (lb[layer].std() * 100)
            #print(layer_depth)
            for layer_depth in [layer_depth_mean,layer_depth_upper,layer_depth_lower]:
                layer_border_pts = Vectors().get_layer_pts_for_given_depth(vectors_sg.graph_data.edge_list,layer_depth,\
                                                                                              realtive_depths=True,invert_axes=True,rel_depth_given=True)
                layer_surf = Surface(pts=layer_border_pts).create_delunay_surface_2d(return_hull=True)
                # write mean, upper or lower layer border surface
                if layer_depth == layer_depth_mean:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_path+'_vS1_'+hem+layer[:-4]+'_{}_mean.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_path+'_vS1_'+hem+layer[:-4]+'_{}_mean.vtk'.format(surf_res))
                elif layer_depth == layer_depth_upper:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_path+'_vS1_'+hem+layer[:-4]+'_{}_upper.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_path+'_vS1_'+hem+layer[:-4]+'_{}_upper.vtk'.format(surf_res))
                else:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_path+'_vS1_'+hem+layer[:-4]+'_{}_lower.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_path+'_vS1_'+hem+layer[:-4]+'_{}_lower.vtk'.format(surf_res))



In [ ]:
# rabies layers
output_path = output_path_alignment+'Tangential_To_Coronal/Average/vS1/Layer_Borders/'
pathlib.Path(output_path).mkdir(exist_ok=True)
for surf_res in surface_resolutions:
    for hem in ['LHS','RHS']:
        vectors_sg = AmiraSpatialGraph(output_path_alignment+'Tangential_To_Coronal/Average/vS1/'+hem+'_vS1_axis_field_{}.am'.format(surf_res),\
                          generic_graph=True,)
       
        lb = pd.read_csv(output_path_rabies_layers+'vS1_Borders.csv')
        for layer in ['L5_Upper_Border_Per','L5_Mean_Border_Per','L5_Lower_Border_Per',]:
            layer_depth_mean = lb[layer].mean() * 100
            
            #print(layer_depth)
            for layer_depth in [layer_depth_mean,layer_depth_upper,layer_depth_lower]:
                layer_border_pts = Vectors().get_layer_pts_for_given_depth(vectors_sg.graph_data.edge_list,layer_depth,\
                                                                                              realtive_depths=True,invert_axes=True,rel_depth_given=True)
                layer_surf = Surface(pts=layer_border_pts).create_delunay_surface_2d(return_hull=True)
                # write mean, upper or lower layer border surface
                if layer_depth == layer_depth_mean:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_path+'_vM1_Rabies_'+hem+layer[:-4]+'_{}_mean.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_path+'_vM1_Rabies_'+hem+layer[:-4]+'_{}_mean.vtk'.format(surf_res))
        

In [ ]:
def get_center_stats(centers,alignment_type,center_type,df):
    
    centroid = np.array(centers).mean(axis=0)
    center_std = np.array(centers).std(axis=0)
    center_se = center_std / np.sqrt(len(centers))
    
    Landmarks(pts=centers).write_landmarks(output_path_alignment+alignment_type+'/Landmarks/'+\
                                           'Axis_Center_Precision_Pts.landmarksAscii')
    Landmarks(pts=[centroid]).write_landmarks(output_path_alignment+alignment_type+'/Landmarks/'+\
                                           'Axis_Center_Precision_Centroid.landmarksAscii')
    dists = []
    for center in centers:
        dists.append(distance.euclidean(center,centroid))
    dist_std = [np.array(dists).mean(axis=0)]
    dist_se = dist_std / np.sqrt(len(centers))
    
    df['{}_Center_STD'.format(center_type)] = [center_std]
    df['{}_Center_SE'.format(center_type)] = [center_se] 
    df['{}_Center_STD_3D_Distance'.format(center_type)] = [dist_std]
    df['{}_Center_SE_3D_Distance'.format(center_type)] = [dist_se]
    
    return df

In [ ]:
def get_axes_alignment_stats(central_axes,alignment_type,axis_name):
    # This function will compute errors between aligned central axes: at pia, wm and axis center
    
    df = pd.DataFrame()
    df['Alignment_Type'] = [alignment_type]
    df['Axis_Name'] = [axis_name]
    
    # axis center, 
    axis_centers = []
    for axis in central_axes:
        center = (np.array(axis[0]) + np.array(axis[1])) / 2
        axis_centers.append(center)
    
    df = get_center_stats(axis_centers,alignment_type,'Axis',df)
    
    # pia center
    pia_centers = []
    for axis in central_axes:
        pia_centers.append(axis[1])
    df = get_center_stats(pia_centers,alignment_type,'Pia',df)
    
    # wm center
    wm_centers = []
    for axis in central_axes:
        wm_centers.append(axis[0])
    df = get_center_stats(pia_centers,alignment_type,'WM',df)
    
    #volume center, 
    volume_centers = []
    if axis_name == 'vM1':
        for exp in exp_names_tangential:
            vm1_surf = Surface(output_path_alignment+alignment_type+'/vM1_Ref_Frame/'+exp+'_vM1.vtk')
            center = vm1_surf.get_center_of_mass()
            volume_centers.append(center)
    else:
        for exp in exp_names_tangential:
            vs1_surf = Surface(output_path_alignment+alignment_type+'/Layers/'+exp+'_reg_s1_hull.vtk')
            center = vs1_surf.get_center_of_mass()
            volume_centers.append(center)
    df = get_center_stats(volume_centers,alignment_type,'Volume',df)
    
    # Rabies Centers
    rabies_centers = [] 
    if axis_name == 'vM1':
        for exp in exp_names_tangential:
                l = Landmarks(output_path_alignment+alignment_type+'/Landmarks/'+exp+'_vM1_rabies_landmarks.LandmarkAscii')
                if len(l.pts) > 1:
                    center = np.array(l.pts).mean(axis=0)
                    rabies_centers.append(center)
    else:
        for exp in exp_names_tangential:
                l = Landmarks(output_path_alignment+alignment_type+'/Landmarks/'+exp+'_vS1_rabies_landmarks.LandmarkAscii')
                if len(l.pts) > 1:
                    center = np.array(l.pts).mean(axis=0)
                    rabies_centers.append(center)
    df = get_center_stats(rabies_centers,alignment_type,'Rabies_Cluster',df)
    
    #angles
    # Find avg unit vec then find angles between this and the other 
    unit_vecs = []
    for axis in central_axes:
        uv = Vectors().get_unit_vec(axis[0],axis[1])
        unit_vecs.append(uv)
    resultant_vec = np.array(unit_vecs).mean(axis=0)
    
    angles = []
    for uv in unit_vecs:
        angle = Vectors().get_angle_between_unit_vectors(uv,resultant_vec)
        angles.append(angle)
    df['Anglular_STD_in_degrees'] = [np.array(angles).mean(axis=0)]
    df['Anglular_SE_in_degrees'] = [(np.array(angles).mean(axis=0))/np.sqrt(len(angles))]
    
    #df.to_csv(output_path_stats+'7_Alignment_Stats_for_vM1_{}.csv'.format(alignment_type))
    return df

In [ ]:
def get_vm1_alignment_stats(alignment_type,df):
    vm1_central_axes = []
    for exp in exp_names_tangential:
        sg = AmiraSpatialGraph(output_path_alignment+alignment_type+'/Axis_Fields/'+exp+'_vM1_PCA_2.am',generic_graph=True)
        vm1_central_axes.append(sg.graph_data.edge_list[0])

    df = df.append(get_axes_alignment_stats(vm1_central_axes,alignment_type,axis_name='vM1'))
    return df

In [ ]:
def get_barrel_alignment_stats(alignment_type,df):
    for row_ind in range(6):
        for barrel_ind in range(7):
            barrel_axes = []
            axis_present = False
            for exp in exp_names_tangential:
                sg = AmiraSpatialGraph(output_path_alignment+alignment_type+'/Surfaces/'+exp+'_barrels.am',\
                                       barrel_projections_present=True)

                barrel = sg.barrels.rows_list[row_ind].single_barrels_list[barrel_ind]
                          
                if len(barrel.pia_projection_barrel_centroid)>0 :
                    barrel_axes.append([barrel.wm_projection_barrel_centroid,barrel.pia_projection_barrel_centroid])
            if len(barrel_axes) > 1:
                df = df.append(get_axes_alignment_stats(barrel_axes,alignment_type,axis_name=barrel.barrel_name))
    return df

In [ ]:
# Align using different methods and create avg ref frame
# align to global ref frame
ref_exp = 'MG49_lhs'

ref_surf = Surface(output_path_alignment+'RHS_Mirrored/Surfaces/'+ref_exp+'_pia_open_bottom_voxel_size_50.vtk')
ref_surf.append(Surface(output_path_alignment+'RHS_Mirrored/Surfaces/'+ref_exp+'_WM_open_bottom_voxel_size_50.vtk').surface)

center = np.array(ref_surf.get_center_of_mass())
txmat = [1,0,0,0,0,1,0,0,0,0,1,0,-center[0],-center[1],-center[2],1]

# translate the ref exp to the center
ref_sg = AmiraSpatialGraph(output_path_alignment+'RHS_Mirrored/Surfaces/'+ref_exp+'_barrels.am',\
                           barrel_projections_present=True)
#ref_sg.apply_transformation(txmat)
ref_surf = Surface(output_path_alignment+'RHS_Mirrored/Surfaces/'+ref_exp+'_pia_open_bottom_voxel_size_50.vtk')
ref_surf.append(Surface(output_path_alignment+'RHS_Mirrored/Surfaces/'+ref_exp+'_WM_open_bottom_voxel_size_50.vtk').surface)
#ref_surf.apply_transformation(txmat)

ref_surf_vM1 = Surface(filename=output_path_alignment+'RHS_Mirrored/vM1_Ref_Frame/' + ref_exp + '_vM1.vtk')
#ref_surf_vM1.apply_transformation(txmat)

ref_surf_vM1_vS1 = Surface(filename=output_path_alignment+'RHS_Mirrored/vM1_Ref_Frame/' + ref_exp + '_vM1.vtk')
ref_surf_vM1_vS1.append(Surface(filename=output_path_alignment+'RHS_Mirrored/Surfaces/' + ref_exp + '_reg_s1_hull.vtk').surface)
#ref_surf_vM1_vS1.apply_transformation(txmat)

ref_vM1_axis_field = AmiraSpatialGraph(output_path_alignment+'RHS_Mirrored/Axis_Fields/'+ref_exp+'_vM1_axis_field.am',generic_graph=True)

df_alignment = pd.DataFrame(columns = ['Alignment_Type','Axis_Name',\
               'Axis_Center_STD','Axis_Center_SE',\
               'Axis_Center_STD_3D_Distance','Axis_Center_SE_3D_Distance',\
               'Pia_Center_STD','Pia_Center_SE',\
               'Pia_Center_STD_3D_Distance','Pia_Center_SE_3D_Distance',\
               'WM_Center_STD','WM_Center_SE',\
               'WM_Center_STD_3D_Distance','WM_Center_SE_3D_Distance',\
               'Volume_Center_STD','Volume_Center_SE',\
               'Volume_Center_STD_3D_Distance','Volume_Center_SE_3D_Distance',\
               'Rabies_Cluster_Center_STD','Rabies_Cluster_Center_SE',\
               'Rabies_Cluster_Center_STD_3D_Distance','Rabies_Cluster_Center_SE_3D_Distance',
               'Anglular_STD_in_degrees','Anglular_SE_in_degrees'])

for alignment_type in ['Using_vS1_vM1_Surface_Points',]:
    output_path = output_path_alignment + alignment_type + '/'
    pathlib.Path(output_path).mkdir(exist_ok=True)
    for exp in exp_names_tangential:
        print(exp,alignment_type)
        if alignment_type == 'Using_vS1':
            # align vS1 barrel_centers
            #ref_sg = AmiraSpatialGraph(output_path_surfaces+ref_exp+'_barrels.am')
            sg =  AmiraSpatialGraph(output_path_surfaces+exp+'_barrels.am',barrel_projections_present=True)
            matching_barrels = MatchBarrels(ref_sg.barrels,sg.barrels,use_projections_alone=True)
            ref_s1_pts,s1_pts = matching_barrels.get_matching_barrel_centroids()
            landmarks = Landmarks(pts=s1_pts)
            icp,txmat = landmarks.align_landmarks(ref_s1_pts)
            aligner = Alignment()
            #print(txmat)
            aligner.transform_folders(output_root,output_path,exp,icp=icp,txmat = txmat,\
                                    txfilename = output_path_txmats+exp+'_Canonical_Transformation_Matrix.txt')
        elif alignment_type == 'Using_vM1':
            # align using surfaces
            #ref_surf = Surface(filename=output_path_vM1 + ref_exp + '_vM1.vtk')
            surf = Surface(filename=output_path_vM1 + exp + '_vM1.vtk')
            aligner = Alignment(ref_surf_vM1.surface,surf.surface)
            txmat = aligner.get_transformation_matrix()
            icp = aligner.get_icp_transform()
            aligner.transform_folders(output_root,output_path,exp,icp=icp,txmat = txmat,\
                                    txfilename = output_path_txmats+exp+'_Canonical_Transformation_Matrix.txt')

        elif alignment_type == 'Using_vM1_PCA':
            # align using surfaces
            #ref_surf = Surface(filename=output_path_vM1 + ref_exp + '_vM1.vtk')
            #surf = Surface(filename=output_path_vM1 + exp + '_vM1.vtk')
            #aligner = Alignment(ref_surf_vM1.surface,surf.surface)
            #txmat = aligner.get_transformation_matrix()
            #icp = aligner.get_icp_transform()
            aligner = Alignment()
            txmat = aligner.read_transformation_matrix(output_path_txmats+exp+'vM1_PCA_Transformation.txt')
            #vm1=Surface(output_path_vM1+exp+'_vM1.vtk').surface
            #tred_vm1 = apply_polydata_transformation(vm1,tx_mat)
            #Surface(polydata=tred_vm1).write_surface_mesh(output_path_alignment+alignment_type+exp+'_vM1.vtk')
            aligner.set_transformation_matrix(txmat)
            aligner.transform_folders(output_root,output_path,exp,icp=None,txmat = txmat,\
                                    txfilename = output_path_txmats+exp+'_Canonical_Transformation_Matrix.txt',\
                                     apply_polydata_transform=True)

        elif alignment_type == 'Using_vS1_vM1_Surface_Points':
            # align by entire pia wm surfaces
            # get barrel projection points
            # get equal number of vm1 pts
            sg =  AmiraSpatialGraph(output_path_alignment+'RHS_Mirrored/Surfaces/'+exp+'_barrels_50.am',barrel_projections_present=True)
            vM1_axis_field = AmiraSpatialGraph(output_path_alignment+'RHS_Mirrored/Axis_Fields/'+exp+'_vM1_axis_field.am',generic_graph=True)
            matching_barrels = MatchBarrels(ref_sg.barrels,sg.barrels,use_projections_alone=True)
            ref_s1_pts,s1_pts = matching_barrels.get_matching_barrel_centroids()

            selected_ref_axes = get_number_of_axes_from_field(ref_vM1_axis_field.graph_data.edge_list,len(ref_s1_pts))
            selected_axes = get_number_of_axes_from_field(vM1_axis_field.graph_data.edge_list,len(s1_pts))

            ref_selected_sg = AmiraSpatialGraph(generic_graph=True)
            selected_sg = AmiraSpatialGraph(generic_graph=True)

            print(len(selected_ref_axes))
            ref_pts = []
            pts = []
            for pt in ref_s1_pts:
                ref_pts.append(pt)
            for axis in selected_ref_axes:
                ref_pts.append(axis[0])
                ref_pts.append(axis[1])
                ref_selected_sg.graph_data.add_edge(axis[0],axis[1])
            for pt in s1_pts:
                pts.append(pt)
            for axis in selected_axes:
                pts.append(axis[0])
                pts.append(axis[1])
                selected_sg.graph_data.add_edge(axis[0],axis[1])
            print(len(ref_pts),len(pts))
            ref_selected_sg.write_spatial_graph(output_path_alignment+alignment_type+'/'+ref_exp+'ref_axes.am')
            selected_sg.write_spatial_graph(output_path_alignment+alignment_type+'/'+exp+'axes.am')

            landmarks = Landmarks(pts=pts)
            icp,txmat = landmarks.align_landmarks(ref_pts)
            aligner = Alignment()
            #print(txmat)
            aligner.transform_folders(output_path_alignment+'RHS_Mirrored/',output_path,exp,icp=icp,txmat = txmat,\
                                    txfilename = output_path_txmats+exp+'_Canonical_Transformation_Matrix.txt')
        else:
            # align by entire pia wm surfaces
            #ref_surf = Surface(filename=output_path_surfaces + ref_exp + '_pia.vtk')
            #ref_surf.append(Surface(filename=output_path_surfaces + ref_exp + '_WM.vtk').surface)
            surf = Surface(filename=output_path_surfaces + exp + '_pia_open_bottom_voxel_size_50.vtk')
            surf.append(Surface(filename=output_path_surfaces + exp + '_WM_open_bottom_voxel_size_50.vtk').surface)

            aligner = Alignment(ref_surf.surface,surf.surface)
            txmat = aligner.get_transformation_matrix()
            icp = aligner.get_icp_transform()
            aligner.transform_folders(output_root,output_path,exp,icp=icp,txmat = txmat,\
                                    txfilename = output_path_txmats+exp+'_Canonical_Transformation_Matrix.txt')

    df_alignment = get_vm1_alignment_stats(alignment_type,df_alignment)
    df_alignment = get_barrel_alignment_stats(alignment_type,df_alignment)
    
    df_alignment.to_csv(output_path_stats+'7_Alignment_Stats_{}.csv'.format(alignment_type))
    
    break

## Clip surfaces to create front and side views

In [ ]:
# VS1 Slice
def get_slices(exp,hem,br,view,cutting_thickness=500,surf_res=500):
    br_surf_barrels = None
    landmarks_neun = None
    output_path = output_path_alignment+'Tangential_To_Coronal/Sliced/'
    pathlib.Path(output_path).mkdir(exist_ok=True)
    # read canonical surfaces
    if exp != 'MG48':
        if hem=='lhs':
            pia = Surface(output_path_alignment+'/Tangential_To_Coronal/LHS/Surfaces/'+exp+\
                          '_lhs_pia_open_bottom_voxel_size_{}.vtk'.format(surf_res))
            wm = Surface(output_path_alignment+'/Tangential_To_Coronal/LHS/Surfaces/'+exp+\
                         '_lhs_WM_open_bottom_voxel_size_{}.vtk'.format(surf_res))
            landmarks = Landmarks(output_path_alignment+'/Tangential_To_Coronal/LHS/Landmarks/Density_Clusters/'+exp+\
                                  '_lhs_{}_rabies_landmarks_cutoff_0.1.LandmarkAscii'.format(br))
            landmarks_cortical_rabies = Landmarks(output_path_alignment+'/Tangential_To_Coronal/LHS/Landmarks/'\
                                                  +exp+'_'+hem+'_rabies_cortical_landmarks.LandmarkAscii')
            landmarks_neun = Landmarks(output_path_alignment+'/Tangential_To_Coronal/LHS/Landmarks/'+exp+\
                                  '_lhs_neun_landmarks_{}.LandmarkAscii'.format(br))
            br_surf = Surface(output_path_alignment+'/Tangential_To_Coronal/LHS/'+br+'_Ref_Surfaces/'+\
                              exp+'_'+hem+'_'+br+'_hull.vtk')
            
            if br ==  'vS1':
                br_surf_barrels = Surface(output_path_alignment+'/Tangential_To_Coronal/LHS/'+br+'_Ref_Surfaces/'+exp+\
                                          '_'+hem+'_all_barrel_columns.vtk')
            
        else:
            pia = Surface(output_path_alignment+'/Tangential_To_Coronal/RHS_Aligned/Surfaces/'+exp+\
                          '_rhs_pia_open_bottom_voxel_size_{}.vtk'.format(surf_res))
            wm = Surface(output_path_alignment+'/Tangential_To_Coronal/RHS_Aligned/Surfaces/'+exp+\
                         '_rhs_WM_open_bottom_voxel_size_{}.vtk'.format(surf_res))
            landmarks = Landmarks(output_path_alignment+'/Tangential_To_Coronal/RHS_Aligned/Landmarks/Density_Clusters/'+exp+\
                                  '_rhs_{}_rabies_landmarks_cutoff_0.1.LandmarkAscii'.format(br))
            landmarks_cortical_rabies = Landmarks(output_path_alignment+'/Tangential_To_Coronal/RHS_Aligned/Landmarks/'+exp+\
                                                  '_'+hem+'_rabies_cortical_landmarks.LandmarkAscii')
            landmarks_neun = Landmarks(output_path_alignment+'/Tangential_To_Coronal/RHS_Aligned/Landmarks/'+exp+\
                                  '_rhs_neun_landmarks_{}.LandmarkAscii'.format(br))
            br_surf = Surface(output_path_alignment+'/Tangential_To_Coronal/RHS_Aligned/'+br+'_Ref_Surfaces/'+exp+\
                              '_'+hem+'_'+'_hull.vtk')
            
            if br ==  'vS1':
                br_surf_barrels = Surface(output_path_alignment+'/Tangential_To_Coronal/RHS_Aligned/'+br+'_Ref_Surfaces/'+\
                                          exp+'_'+hem+'_all_barrel_columns.vtk')
            
        # create planes around lhs rabies landmarks
        landmarks_lhs = Landmarks(output_path_alignment+'/Tangential_To_Coronal/LHS/Landmarks/Density_Clusters/'+exp+\
                                      '_lhs_{}_rabies_landmarks_cutoff_0.1.LandmarkAscii'.format(br))
        landmarks_center = np.array(landmarks_lhs.pts).mean(axis=0)
    
    else:
        pia = Surface(output_path_surfaces+exp+'_'+hem+'_pia_open_bottom_voxel_size_{}.vtk'.format(surf_res))
        wm = Surface(output_path_surfaces+exp+'_'+hem+'_WM_open_bottom_voxel_size_{}.vtk'.format(surf_res))
        landmarks = Landmarks(output_path_landmarks+'/Density_Clusters/'+exp+\
                              '_'+hem+'_{}_rabies_landmarks_cutoff_0.1.LandmarkAscii'.format(br))
        landmarks_cortical_rabies = Landmarks(output_path_landmarks+exp+\
                                              '_'+hem+'_rabies_cortical_landmarks.LandmarkAscii')
        
        br_surf = Surface(output_path_alignment+'/Tangential_To_Coronal/RHS_Translated/'+br+'_Ref_Surfaces/'+exp+\
                          '_'+hem+'_'+'_hull.vtk')
            
        if br ==  'vS1':
            br_surf_barrels = Surface(output_path_alignment+'/Tangential_To_Coronal/RHS_Translated/'+br+'_Ref_Surfaces/'+\
                                      exp+'_'+hem+'_all_barrel_columns.vtk')
        else:
            landmarks_neun = Landmarks(output_path_landmarks+exp+\
                              '_'+hem+'_neun_landmarks_{}.LandmarkAscii'.format(br))
        # create planes around lhs rabies landmarks
        landmarks_lhs = Landmarks(output_path_landmarks+'/Density_Clusters/'+exp+\
                                      '_lhs_{}_rabies_landmarks_cutoff_0.1.LandmarkAscii'.format(br))
        landmarks_center = np.array(landmarks_lhs.pts).mean(axis=0)
    
    print(len(landmarks.pts))
    if view == 'coronal':
        pt1 = [landmarks_center[0],landmarks_center[1]+cutting_thickness,landmarks_center[2]]
        pt2 = [landmarks_center[0]+10,landmarks_center[1]+cutting_thickness,landmarks_center[2]]
        pt3 = [landmarks_center[0],landmarks_center[1]+cutting_thickness,landmarks_center[2]+10]
        plane_front,tri_front = Surface().get_cutting_plane(pt1,pt2,pt3)
        #Surface(polydata=tri_front).write_surface_mesh(output_root+'bla.vtk')
        #Surface(polydata=pia1).write_surface_mesh(output_root+'b1.vtk')
        pt1 = [landmarks_center[0],landmarks_center[1]-cutting_thickness,landmarks_center[2]]
        pt2 = [landmarks_center[0]+10,landmarks_center[1]-cutting_thickness,landmarks_center[2]]
        pt3 = [landmarks_center[0],landmarks_center[1]-cutting_thickness,landmarks_center[2]+10]
        plane_back,tri_back = Surface().get_cutting_plane(pt1,pt2,pt3)
    elif view == 'sagital':
        pt1 = [landmarks_center[0]-cutting_thickness,landmarks_center[1],landmarks_center[2]]
        pt2 = [landmarks_center[0]-cutting_thickness,landmarks_center[1]+10,landmarks_center[2]]
        pt3 = [landmarks_center[0]-cutting_thickness,landmarks_center[1],landmarks_center[2]+10]
        plane_front,tri_front = Surface().get_cutting_plane(pt1,pt2,pt3)
        #Surface(polydata=tri_front).write_surface_mesh(output_root+'bla.vtk')
        #Surface(polydata=pia1).write_surface_mesh(output_root+'b1.vtk')
        pt1 = [landmarks_center[0]+cutting_thickness,landmarks_center[1],landmarks_center[2]]
        pt2 = [landmarks_center[0]+cutting_thickness,landmarks_center[1]+10,landmarks_center[2]]
        pt3 = [landmarks_center[0]+cutting_thickness,landmarks_center[1],landmarks_center[2]+10]
        plane_back,tri_back = Surface().get_cutting_plane(pt1,pt2,pt3)
    else:
        # top view
        pt1 = [landmarks_center[0],landmarks_center[1],landmarks_center[2]-cutting_thickness]
        pt2 = [landmarks_center[0]+10,landmarks_center[1],landmarks_center[2]-cutting_thickness]
        pt3 = [landmarks_center[0],landmarks_center[1]+10,landmarks_center[2]-cutting_thickness]
        plane_front,tri_front = Surface().get_cutting_plane(pt1,pt2,pt3)
        #Surface(polydata=tri_front).write_surface_mesh(output_root+'bla.vtk')
        #Surface(polydata=pia1).write_surface_mesh(output_root+'b1.vtk')
        pt1 = [landmarks_center[0],landmarks_center[1],landmarks_center[2]+cutting_thickness]
        pt2 = [landmarks_center[0]+10,landmarks_center[1],landmarks_center[2]+cutting_thickness]
        pt3 = [landmarks_center[0],landmarks_center[1]+10,landmarks_center[2]+cutting_thickness]
        plane_back,tri_back = Surface().get_cutting_plane(pt1,pt2,pt3)
            
    output_path = output_path+view+'/'
    pathlib.Path(output_path).mkdir(exist_ok=True)
    
    # clip pia wm surfaces
    pia1,wm1 = Surface().clip_surfaces(pia.surface,wm.surface,plane_front,get_left=True)
    pia2,wm2 = Surface().clip_surfaces(pia1,wm1,plane_back,get_left=False)
    Surface(polydata=pia2).write_surface_mesh(output_path+exp+'_{}_{}_{}_clipped_pia.vtk'.format(hem,br,view))
    Surface(polydata=wm2).write_surface_mesh(output_path+exp+'_{}_{}_{}_clipped_wm.vtk'.format(hem,br,view))
    
    # clip vm1 or vs1 hull 
    br_surf1,br_surf2 = Surface().clip_surface(br_surf.surface,plane_front)
    br_surf_clipped1,br_surf_clipped2 = Surface().clip_surface(br_surf1,plane_back)
    Surface(polydata=br_surf_clipped2).write_surface_mesh(output_path+exp+'_{}_{}_{}_clipped_surfs.vtk'.format(hem,br,view))
    
    if br_surf_barrels is not None:
        br_surf1_barrels,br_surf2_barrels = Surface().clip_surface(br_surf_barrels.surface,plane_front)
        br_surf_clipped1_barrels,br_surf_clipped2_barrels = Surface().clip_surface(br_surf1_barrels,plane_back)
        Surface(polydata=br_surf_clipped2_barrels).write_surface_mesh(output_path+exp+'_{}_{}_{}_clipped_surfs_barrels.vtk'.format(hem,br,view))

    # clip landmarks
    clipped_landmarks = landmarks.get_landmarks_in_bounding_box(br_surf_clipped2.GetBounds())
    Landmarks(pts = clipped_landmarks).write_landmarks(output_path +exp+'_{}_{}_{}_clipped_landmarks.landmarksAscii'.format(hem,br,view))
    
    clipped_landmarks_cortical_rabies = landmarks_cortical_rabies.get_landmarks_in_bounding_box(br_surf_clipped2.GetBounds())
    Landmarks(pts = clipped_landmarks_cortical_rabies).write_landmarks(output_path +exp+'_{}_{}_{}_clipped_cortical_rabies.landmarksAscii'.format(hem,br,view))
    
    landmarks.write_landmarks(output_path +exp+'_{}_{}_{}_landmarks_rabies.landmarksAscii'.format(hem,br,view))
    landmarks_cortical_rabies.write_landmarks(output_path +exp+'_{}_{}_{}_landmarks_cortical_rabies.landmarksAscii'.format(hem,br,view))
    
    # wrie landmarks as it is.. both rabies and neun
    if landmarks_neun is not None:
        clipped_landmarks_neun = landmarks_neun.get_landmarks_in_bounding_box(br_surf_clipped2.GetBounds())
        Landmarks(pts = clipped_landmarks_neun).write_landmarks(output_path +exp+'_{}_{}_{}_clipped_landmarks_neun.landmarksAscii'.format(hem,br,view))

        landmarks_neun.write_landmarks(output_path +exp+'_{}_{}_{}_landmarks_neun.landmarksAscii'.format(hem,br,view))
    

In [ ]:
for view in ['coronal','sagital','top']:
    for exp in ['MG48','MG49','MG50']:
        for hem in ['lhs','rhs']:
            for br in ['vS1','vM1']:
                get_slices(exp,hem,br,view,cutting_thickness=50,surf_res=200)
    

## Create density images and iso contours

In [ ]:
df_vM1_rabies_cluster = pd.DataFrame(columns=['Exp_Name','Voxel_Size','Density_Lower_Threshold_Percentage','Max_Density',\
                                              'Actual_Lower_Density_Threshold'])

output_path = output_path_alignment+'Tangential_To_Coronal/'
pathlib.Path(output_path).mkdir(exist_ok=True)

voxel_size=int(500)
dens_cutoff_threshold=0.1
ouput_path = output_path_landmarks+'Density_Clusters/'
pathlib.Path(ouput_path).mkdir(exist_ok=True)

for voxel_size in [500]:#[100,200,300,400,500]:
    for dens_cutoff_threshold in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5]:
        for exp_name in exp_names:
            print(exp_name)
            if exp_name.find('lhs')>0:
                output_path_aligned_landmarks = output_path+'/LHS/Landmarks/'
            else:
                output_path_aligned_landmarks = output_path+'/RHS_Aligned/Landmarks/'
            
            l = Landmarks(output_path_aligned_landmarks+'/K-Means/'+'{}_rabies_kmeans_cluster_vM1.landmarksAscii'.format(exp_name))
            cluster_in_bounds,max_dens,dens_cutoff_threshold,threhold = \
                l.get_density_cluster(output_file=output_path_aligned_landmarks+'/Density_Clusters/'+'{}_vM1_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
                                  format(exp_name,str(voxel_size),str(dens_cutoff_threshold)),\
                                  voxel_size=voxel_size,dens_cutoff_threshold=dens_cutoff_threshold,\
                                  write_density_images=True)

            df_vM1_rabies_cluster = df_vM1_rabies_cluster.append(pd.DataFrame(columns=df_vM1_rabies_cluster.columns,\
                                                                 data=[[exp_name,float(voxel_size),\
                                                                        dens_cutoff_threshold*100,max_dens,threhold]]))
            

stats = Stats(df_vM1_rabies_cluster)
stats.add_mean_std_cov(output_filename=output_path_stats+'3_Rabies_Cluster_Stats.csv')

# Generate Avg vM1 Reference Frame

## Generate Avg Surf Pts

In [ ]:
# Generate avg pts
for alignment_type in ['Using_vS1_vM1_Surface_Points']:
    aligned_pias = Surface()
    aligned_wms = Surface()
    output_path = output_path_avg + alignment_type
    pathlib.Path(output_path).mkdir(exist_ok=True)
    aligned_pias_list = []
    aligned_wms_list = []
    for exp in exp_names_tangential:
        if exp.find('MG48') < 0:
            pia = Surface(output_path_alignment+alignment_type+'/Surfaces/'+exp+'_pia_open_bottom_voxel_size_50.vtk')
            aligned_pias.append(pia.surface)
            aligned_pias_list.append(pia.surface)
            wm = Surface(output_path_alignment+alignment_type+'/Surfaces/'+exp+'_WM_open_bottom_voxel_size_50.vtk')
            aligned_wms.append(wm.surface)
            aligned_wms_list.append(wm.surface)
    pia_wms = Surface()
    pia_wms.append(aligned_pias.surface)
    pia_wms.append(aligned_wms.surface)

    centroid = pia_wms.get_center_of_mass()
    ray_origin = [centroid[0],centroid[1],pia_wms.surface.GetBounds()[5]]

    [pia_pts] = aligned_pias.get_ray_cast_surface_pts(center=ray_origin,surf_list=aligned_pias_list,theta_res=60,phi_res=60,\
                                                     start_phi=0,end_phi=360,start_theta=0,end_theta=360,validation_nr_pts=1)

    Landmarks(pts=pia_pts).write_landmarks(output_path+'/avg_pia_pts.landmarksAscii')

    [wm_pts] = aligned_wms.get_ray_cast_surface_pts(center=ray_origin,surf_list=aligned_wms_list,theta_res=60,phi_res=60,\
                                                     start_phi=0,end_phi=360,start_theta=0,end_theta=360,validation_nr_pts=1)

    Landmarks(pts=wm_pts).write_landmarks(output_path+'/avg_wm_pts.landmarksAscii')

    


## Generate Avg Surfaces

In [ ]:
# generate surface from avg pts
for surface_resolution in surface_resolutions:
    for alignment_type in ['Using_vS1_vM1_Surface_Points']:
        pia_pts = Landmarks(filename=output_path_avg + alignment_type + '/avg_pia_pts.landmarksAscii')
        pia_surf = Surface(pts=pia_pts.pts)
        bounds = pia_surf.surface.GetBounds()
        z_span = bounds[5] - bounds[4]
        offset = z_span - 5000
        pia_surf.create_surface_from_unorganized_points(pia_pts.pts,sample_resolution=surface_resolution,
                                                                                       clip_offset=-offset,z_limit=None,\
                                                output_filename=output_path_avg+alignment_type+'/Avg_Pia_{}.vtk'.format(surface_resolution))

        wm_pts = Landmarks(filename=output_path_avg + alignment_type + '/avg_wm_pts.landmarksAscii')
        wm_surf = Surface(pts=wm_pts.pts)
        wm_surf.create_surface_from_unorganized_points(wm_pts.pts,sample_resolution=surface_resolution,
                                                                                       clip_offset=-offset,z_limit=None,\
                                                output_filename=output_path_avg+alignment_type+'/Avg_WM_{}.vtk'.format(surface_resolution))



## Generate Avg Surface Axis Field

In [ ]:
for alignment_type in ['Using_vS1_vM1_Surface_Points']:
    for surf_res in surface_resolutions:
        pia = Surface(output_path_avg+alignment_type+'/Avg_Pia_{}.vtk'.format(surf_res))
        wm = Surface(output_path_avg+alignment_type+'/Avg_WM_{}.vtk'.format(surf_res))
        wm.create_axis_field(pia,flip_normals=False,op_sg_name=output_path_avg+alignment_type+'/Avg_Axis_Field_{}.am'.format(surf_res))

In [ ]:
for alignment_type in ['Using_vS1_vM1_Surface_Points']:
    vec_extension_offset = 0.75
    axis_field_sg = AmiraSpatialGraph(output_path_avg+alignment_type+'/Avg_Axis_Field_{}.am'.format(surf_res),generic_graph=True)
    for surf_res in surface_resolutions:
        pia = Surface(output_path_avg+alignment_type+'/Avg_Pia_{}.vtk'.format(surf_res))
        wm = Surface(output_path_avg+alignment_type+'/Avg_WM_{}.vtk'.format(surf_res))

        selected_axis_field_sg,wm_pts,pia_pts,missing_axis_field = pia.get_axis_field_within_instersecting_surface_with_trimming\
        (pia.surface,wm.surface,axis_field_sg.graph_data.edge_list,vec_extension_offset=vec_extension_offset)
        selected_axis_field_sg.write_spatial_graph(output_path_avg+alignment_type+\
                                '/Avg_Axis_Field_{}.am'.format(surf_res))

## Generate Avg vM1 PCA

In [ ]:
# Generate avg vM1 surface using centroid and avg angles 
# find avg pcas

for alignment_type in ['Using_vS1_vM1_Surface_Points']:
    #print(alignment_type)
    ref_exp = 'MG49_lhs'
    ref_pcas = []
    for i in range(3):
        ref_pcas.append(AmiraSpatialGraph(output_path_alignment+alignment_type+'/Axis_Fields/'+ref_exp+'_vM1_PCA_{}.am'.format(i),\
                                          generic_graph=True))
    pca_centers = []
    centered_pcas = []
    pca_unit_vecs_all_exps = []
    pca_vec_lens_all_exps = []
    for exp in exp_names:
        #print(exp)
        if exp.find('MG48') < 0:
            pcas = []
            pca_angles = []
            pca_unit_vecs = []
            pca_lengths = []
            for i in range(3):
                if exp == 'MG49_rhs' and i==0:
                    pcas.append(AmiraSpatialGraph(output_path_alignment+alignment_type+'/Axis_Fields/'+exp+'_vM1_PCA_{}_Neg.am'.format(i),\
                                          generic_graph=True))
                else:
                    pcas.append(AmiraSpatialGraph(output_path_alignment+alignment_type+'/Axis_Fields/'+exp+'_vM1_PCA_{}.am'.format(i),\
                                          generic_graph=True))
                pca_angles.append( Vectors().get_angle_between_vectors(pcas[i].graph_data.edge_list[0],\
                                                       ref_pcas[i].graph_data.edge_list[0],\
                                                       ignore_opposite_direction=False))
                pca_unit_vecs.append(Vectors().get_unit_vec(pcas[i].graph_data.edge_list[0][0],\
                                                            pcas[i].graph_data.edge_list[0][1]))
                pca_lengths.append(Vectors().get_vec_length((pcas[i].graph_data.edge_list[0])))
            pca_unit_vecs_all_exps.append(pca_unit_vecs)
            pca_centers.append(pcas[0].graph_data.edge_list[0][0])
            pca_vec_lens_all_exps.append(pca_lengths)
    
    avg_pca_center = np.array(pca_centers).mean(axis=0)
    avg_pca_lens = np.array(pca_vec_lens_all_exps).mean(axis=0)
    max_pca_lens = np.array(pca_vec_lens_all_exps).max(axis=0)
    #avg_pca_lens = avg_pca_lens[0]+1000,avg_pca_lens[1]+1000,avg_pca_lens[2]+0
    # write centered pcas
    for exp in exp_names_tangential:
        for i in range(3):
            if exp == 'MG49_rhs' and i==0:
                #print(exp)
                pca_sg = AmiraSpatialGraph(output_path_alignment+alignment_type+'/Axis_Fields/'+exp+'_vM1_PCA_{}_Neg.am'.format(i),\
                                          generic_graph=True)
            else:
                pca_sg = AmiraSpatialGraph(output_path_alignment+alignment_type+'/Axis_Fields/'+exp+'_vM1_PCA_{}.am'.format(i),\
                                          generic_graph=True)
            translation = pca_sg.graph_data.edge_list[0][0] - avg_pca_center
            txmat = [1,0,0,0,0,1,0,0,0,0,1,0,-translation[0],-translation[1],-translation[2],1]
            pca_sg.apply_transformation(txmat)
            pca_sg.write_spatial_graph(output_path_avg+'/'+alignment_type+'/'+exp+'_centered_pca_{}'.format(i))

            pca_sg = AmiraSpatialGraph(output_path_alignment+alignment_type+'/Axis_Fields/'+exp+'_vM1_PCA_{}_Neg.am'.format(i),\
                                          generic_graph=True)
            translation = pca_sg.graph_data.edge_list[0][0] - avg_pca_center
            txmat = [1,0,0,0,0,1,0,0,0,0,1,0,-translation[0],-translation[1],-translation[2],1]
            pca_sg.apply_transformation(txmat)
            pca_sg.write_spatial_graph(output_path_avg+'/'+alignment_type+'/'+exp+'_centered_pca_{}_neg'.format(i))

    centered_pca_ref_sg = AmiraSpatialGraph(output_path_avg+'/'+alignment_type+'/'+ref_exp+'_centered_pca_{}'.format(0),\
                                            generic_graph=True)
    # write avg_pca 0
    
    avg_pca_unit_vec = np.array(pca_unit_vecs_all_exps)[:,0].mean(axis=0)
    avg_pca_vec = avg_pca_unit_vec*avg_pca_lens[0] + avg_pca_center
    avg_pca0_sg = AmiraSpatialGraph(generic_graph=True)
    avg_pca0_sg.graph_data.add_edge(avg_pca_center,avg_pca_vec)
    avg_pca0_sg.write_spatial_graph(output_path_avg+'/'+alignment_type+'/'+'Avg_Pca_{}'.format(0))
    
    # rotate the ref_pca 0 to avg pca 0 to get the other 2 avg pcas
    pts = []
    ref_pts = []
    pts.append(avg_pca_center)
    pts.append(centered_pca_ref_sg.graph_data.edge_list[0][1])
    ref_pts.append(avg_pca_center)
    ref_pts.append(avg_pca0_sg.graph_data.edge_list[0][1])
    landmarks = Landmarks(pts=pts)
    icp,txmat = landmarks.align_landmarks(ref_pts)
    
    #print(txmat)
    for i in range(3):
        sg = AmiraSpatialGraph(output_path_avg+'/'+alignment_type+'/'+ref_exp+'_centered_pca_{}'.format(i),generic_graph=True)
        sg.apply_transformation(transformation_matrix=txmat)
        sg.write_spatial_graph(output_path_avg+'/'+alignment_type+'/'+ref_exp+'_centered_pca_{}_tmp'.format(i),)
        sg = AmiraSpatialGraph(output_path_avg+'/'+alignment_type+'/'+ref_exp+'_centered_pca_{}_tmp'.format(i),generic_graph=True)
        
        # make it as big as the avg and max pca and write them
        avg_pca = (Vectors().get_unit_vec(sg.graph_data.edge_list[0][0],sg.graph_data.edge_list[0][1]))*avg_pca_lens[i]+avg_pca_center
        sg_avg = AmiraSpatialGraph(generic_graph=True)
        sg_avg.graph_data.add_edge(avg_pca_center,avg_pca)
        sg_avg.write_spatial_graph(output_path_avg+'/'+alignment_type+'/'+'Avg_vM1_PCA_{}.am'.format(i))
        avg_pca = (Vectors().get_unit_vec(sg.graph_data.edge_list[0][0],sg.graph_data.edge_list[0][1]))*-avg_pca_lens[i]+avg_pca_center
        sg_avg = AmiraSpatialGraph(generic_graph=True)
        sg_avg.graph_data.add_edge(avg_pca_center,avg_pca)
        sg_avg.write_spatial_graph(output_path_avg+'/'+alignment_type+'/'+'Avg_vM1_PCA_{}_Neg.am'.format(i))
        
        max_pca = (Vectors().get_unit_vec(sg.graph_data.edge_list[0][0],sg.graph_data.edge_list[0][1]))*max_pca_lens[i]+avg_pca_center
        sg_max = AmiraSpatialGraph(generic_graph=True)
        sg_max.graph_data.add_edge(avg_pca_center,max_pca)
        sg_max.write_spatial_graph(output_path_avg+'/'+alignment_type+'/'+'Max_vM1_PCA_{}.am'.format(i))
        max_pca = (Vectors().get_unit_vec(sg.graph_data.edge_list[0][0],sg.graph_data.edge_list[0][1]))*-max_pca_lens[i]+avg_pca_center
        sg_max = AmiraSpatialGraph(generic_graph=True)
        sg_max.graph_data.add_edge(avg_pca_center,max_pca)
        sg_max.write_spatial_graph(output_path_avg+'/'+alignment_type+'/'+'Max_vM1_PCA_{}_Neg.am'.format(i))

## Generate Avg vM1 Surface

In [ ]:
# Create avg vm1 surface
for alignment_type in ['Using_vS1_vM1_Surface_Points']:
    for surf_res in surface_resolutions:
        pia_surf = Surface(output_path_avg+'/'+alignment_type+'/Avg_Pia_{}.vtk'.format(surf_res)).surface
        wm_surf = Surface(output_path_avg+'/'+alignment_type+'/Avg_WM.vtk'.format(surf_res)).surface
        axis_field_sg = AmiraSpatialGraph(output_path_avg+'/'+alignment_type+'/Avg_Axis_Field_{}.am'.format(surf_res),generic_graph=True)
        axis_field = axis_field_sg.graph_data.edge_list
        pca0 = AmiraSpatialGraph(output_path_avg+'/'+alignment_type+'/'+'Avg_vM1_PCA_{}.am'.format(0),generic_graph=True)\
                .graph_data.edge_list[0]
        pca1 = AmiraSpatialGraph(output_path_avg+'/'+alignment_type+'/'+'Avg_vM1_PCA_{}.am'.format(1),generic_graph=True)\
                .graph_data.edge_list[0]
        pca2 = AmiraSpatialGraph(output_path_avg+'/'+alignment_type+'/'+'Avg_vM1_PCA_{}.am'.format(2),generic_graph=True)\
                .graph_data.edge_list[0]

        pca0_neg = AmiraSpatialGraph(output_path_avg+'/'+alignment_type+'/'+'Avg_vM1_PCA_{}_Neg.am'.format(0),generic_graph=True)\
                .graph_data.edge_list[0]
        pca1_neg = AmiraSpatialGraph(output_path_avg+'/'+alignment_type+'/'+'Avg_vM1_PCA_{}_Neg.am'.format(1),generic_graph=True)\
                .graph_data.edge_list[0]
        pca2_neg = AmiraSpatialGraph(output_path_avg+'/'+alignment_type+'/'+'Avg_vM1_PCA_{}_Neg.am'.format(2),generic_graph=True)\
                .graph_data.edge_list[0]

        vm1_final,vM1_hull,vm1_pia,vm1_wm = get_vm1(pia_surf,wm_surf,pcas=[pca0,pca1,pca2],neg_pcas=[pca0_neg,pca1_neg,pca2_neg],\
                                                         axis_field=axis_field,cutting_order=[True,True,False,False],\
                                                         avg_surf_path=output_path_avg+'/'+alignment_type+'/Avg')

        Landmarks(pts=[pca0[0]]).write_landmarks(output_path_avg+'/'+alignment_type+'/pca_center.landmarksAscii')
        Surface(polydata=vm1_final).write_surface_mesh(output_path_avg+'/'+alignment_type+'/Avg_vM1_{}.vtk'.format(surf_res))
        Surface(polydata=vm1_pia).write_surface_mesh(output_path_avg+'/'+alignment_type+'/Avg_vM1_Pia_{}.vtk'.format(surf_res))
        Surface(polydata=vm1_wm).write_surface_mesh(output_path_avg+'/'+alignment_type+'/Avg_vM1_WM_{}.vtk'.format(surf_res))
    
        

## Generate Avg vM1 Axis Field

In [ ]:
# Avg vM1 axis field
for surf_res in surface_resolutions:
    axis_field_sg = AmiraSpatialGraph(output_path_avg+'/'+alignment_type+'/Avg_Axis_Field_{}.am'.format(surf_res),generic_graph=True)
    vm1 = Surface(output_path_avg+'/'+alignment_type+'/Avg_vM1_{}.vtk'.format(surf_res))

    vm1_axis_field_sg,wm_pts,pia_pts = vm1.get_axis_field_within_instersecting_surface(vm1.surface,\
                                                                                       axis_field_sg.graph_data.edge_list)
    vm1_axis_field_sg.write_spatial_graph(output_path_avg+alignment_type+'/Avg_vM1_Axis_Field_{}.am'.format(surf_res))

# Generate layer border surfaces

In [ ]:
# Read the layer border csv
layer_borders_csv = pd.read_csv(output_path_neun_layer_borders+'vM1_Layer_Borders.csv')

central_axis_lengths = []
# get the central axis lengths and compute the per depths per layer border
for exp in exp_names:
    sg = AmiraSpatialGraph(output_path_axis_fileds+exp+'_vM1_PCA_2.am',generic_graph=True)
    sg_neg = AmiraSpatialGraph(output_path_axis_fileds+exp+'_vM1_PCA_2_Neg.am',generic_graph=True)
    central_axis_lengths.append(Vectors().get_vec_length([sg.graph_data.edge_list[0][1],sg_neg.graph_data.edge_list[0][1]]))

In [ ]:
# calcualte the per depths
layer_borders_csv['WM'] = (central_axis_lengths)
layer_borders_csv['L1_Per_Depth'] = layer_borders_csv['L1'] / layer_borders_csv['WM']*100
layer_borders_csv['L2/3_Per_Depth'] = layer_borders_csv['L2/3'] / layer_borders_csv['WM']*100
layer_borders_csv['L5A_Per_Depth'] = layer_borders_csv['L5A'] / layer_borders_csv['WM']*100
layer_borders_csv['L5B_Per_Depth'] = layer_borders_csv['L5B'] / layer_borders_csv['WM']*100
layer_borders_csv['L6A_Per_Depth'] = layer_borders_csv['L6A'] / layer_borders_csv['WM']*100

# add mean and std for the depths
bmean = layer_borders_csv.mean(axis=0)
bstd = layer_borders_csv.std(axis=0)

layer_borders_csv = layer_borders_csv.append(pd.DataFrame(columns=layer_borders_csv.columns,data = [bmean]))
layer_borders_csv = layer_borders_csv.append(pd.DataFrame(columns=layer_borders_csv.columns,data = [bstd]))

layer_borders_csv.to_csv(output_path_neun_layer_borders+'vM1_Layer_Borders_Per_Depths.csv')

In [ ]:
layer_borders_csv

In [ ]:
layer_borders_csv = pd.read_csv(output_path_neun_layer_borders+'vM1_Layer_Borders_Per_Depths.csv')
for alignment_type in ['Using_vS1_vM1_Surface_Points']:
    for surf_res in surface_resolutions:
        # vM1_Axis Field
        vectors_sg = AmiraSpatialGraph(output_path_avg+alignment_type+'/Avg_vM1_Axis_Field_{}.am'.format(surf_res),generic_graph=True)
        l1_pts = Vectors().get_layer_pts_for_given_per_depth\
                            (vectors_sg.graph_data.edge_list,abs(layer_borders_csv['L1_Per_Depth'].mean(axis=0)),\
                             invert_axes=True)
        l2_3_pts = Vectors().get_layer_pts_for_given_per_depth\
                            (vectors_sg.graph_data.edge_list,abs(layer_borders_csv['L2/3_Per_Depth'].mean(axis=0)),\
                             invert_axes=True)
        l5A_pts = Vectors().get_layer_pts_for_given_per_depth\
                            (vectors_sg.graph_data.edge_list,abs(layer_borders_csv['L5A_Per_Depth'].mean(axis=0)),\
                             invert_axes=True)
        l5B_pts = Vectors().get_layer_pts_for_given_per_depth\
                            (vectors_sg.graph_data.edge_list,abs(layer_borders_csv['L5B_Per_Depth'].mean(axis=0)),\
                             invert_axes=True)
        l6A_pts = Vectors().get_layer_pts_for_given_per_depth\
                            (vectors_sg.graph_data.edge_list,abs(layer_borders_csv['L6A_Per_Depth'].mean(axis=0)),\
                             invert_axes=True)
        
        Landmarks(pts=l1_pts).write_landmarks(output_path_neun_layer_borders+'L1_Border_pts_{}.landmarksAscii'.format(surf_res))
        Landmarks(pts=l2_3_pts).write_landmarks(output_path_neun_layer_borders+'L2_3_Border_pts_{}.landmarksAscii'.format(surf_res))
        Landmarks(pts=l5A_pts).write_landmarks(output_path_neun_layer_borders+'L5A_Border_pts_{}.landmarksAscii'.format(surf_res))
        Landmarks(pts=l5B_pts).write_landmarks(output_path_neun_layer_borders+'L5B_Border_pts_{}.landmarksAscii'.format(surf_res))
        Landmarks(pts=l6A_pts).write_landmarks(output_path_neun_layer_borders+'L6A_Border_pts_{}.landmarksAscii'.format(surf_res))

        Surface(pts=l1_pts).create_delunay_surface_2d(output_filename=output_path_neun_layer_borders+'L1_Border_{}.vtk'.format(surf_res))
        Surface(pts=l2_3_pts).create_delunay_surface_2d(output_filename=output_path_neun_layer_borders+'L2_3_Border_{}.vtk'.format(surf_res))
        Surface(pts=l5A_pts).create_delunay_surface_2d(output_filename=output_path_neun_layer_borders+'L5A_Border_{}.vtk'.format(surf_res))
        Surface(pts=l5B_pts).create_delunay_surface_2d(output_filename=output_path_neun_layer_borders+'L5B_Border_{}.vtk'.format(surf_res))
        Surface(pts=l6A_pts).create_delunay_surface_2d(output_filename=output_path_neun_layer_borders+'L6A_Border_{}.vtk'.format(surf_res))

# Generate Avg Rabies Layer Borders

In [ ]:
# read rabies depths
rabies_l5_df = pd.read_csv(output_path_stats+'5_vM1_L5_Stats.csv')
raw_df = rabies_l5_df.iloc[0:6]

avg_field_lens = []
per_upper_depth = []
# find the mean axis field len for each exp
for exp in exp_names:
    sg = AmiraSpatialGraph(output_path_axis_fileds+exp+'_vM1_axis_field.am',generic_graph=True)
    lengths = Vectors().get_vec_lengths(sg.graph_data.edge_list)
    avg_field_lens.append(np.array(lengths[0]).mean())
raw_df['Avg_Axis_Length'] = avg_field_lens
raw_df['L5_Upper_Border_Per'] = raw_df['L5_Depth_Upper_Mean'] / raw_df['Avg_Axis_Length'] * 100
raw_df['L5_Lower_Border_Per'] = raw_df['L5_Depth_Lower_Mean'] / raw_df['Avg_Axis_Length'] * 100

# means_df = pd.DataFrame(columns=rabies_l5_df.columns,data=[ [raw_df.mean(axis=0)]])
# std_df = pd.DataFrame(columns=rabies_l5_df.columns,data=[ [raw_df.std(axis=0)]])
# raw_df = raw_df.append(means_df)
# raw_df = raw_df.append(std_df)
raw_df.to_csv(output_path_neun_layer_borders+'vM1_Rabies_Borders.csv')

In [ ]:
# Generate rabies L5 upper and lower surfaces

rabies_l5_df = pd.read_csv(output_path_neun_layer_borders+'vM1_Rabies_Borders.csv')
upper_per_avg = rabies_l5_df['L5_Upper_Border_Per'].mean(axis=0)
lower_per_avg = rabies_l5_df['L5_Lower_Border_Per'].mean(axis=0)

for alignment_type in ['Using_vS1_vM1_Surface_Points']:
    for surf_res in surface_resolutions:
        # vM1_Axis Field
        vectors_sg = AmiraSpatialGraph(output_path_avg+alignment_type+'/Avg_vM1_Axis_Field_{}.am'.format(surf_res),generic_graph=True)
        l5_upper_pts = Vectors().get_layer_pts_for_given_per_depth\
                            (vectors_sg.graph_data.edge_list,upper_per_avg,\
                             invert_axes=True)
        l5_lower_pts = Vectors().get_layer_pts_for_given_per_depth\
                            (vectors_sg.graph_data.edge_list,lower_per_avg,\
                             invert_axes=True)

        Landmarks(pts=l5_upper_pts).write_landmarks(output_path_neun_layer_borders+'L5_Rabies_Borders_Upper{}.landmarksAscii'.format(surf_res))
        Landmarks(pts=l5_lower_pts).write_landmarks(output_path_neun_layer_borders+'L5_Rabies_Borders_Lower{}.landmarksAscii'.format(surf_res))
        
        Surface(pts=l5_upper_pts).create_delunay_surface_2d(output_filename=output_path_neun_layer_borders+'L5_Rabies_Borders_Upper{}.vtk'.format(surf_res))
        Surface(pts=l5_lower_pts).create_delunay_surface_2d(output_filename=output_path_neun_layer_borders+'L5_Rabies_Borders_Lower{}.vtk'.format(surf_res))
        

# Align avg sbf to global ref frame

In [ ]:
def get_avg_barrel_column_axis(barrel,pia,wm):
    
    D2_col_centers = []
    d2_uvs = []
    for exp in exp_names_tangential:
        sg = AmiraSpatialGraph(output_path_alignment+alignment_type+'/Surfaces/'+exp+'_barrels_50.am',barrel_projections_present=True)
        D2_col_centers.append(barrel.barrel_column_centroid)
        d2_axis = [barrel.wm_projection_barrel_centroid,barrel.pia_projection_barrel_centroid]
        d2_uv = Vectors().get_unit_vec(d2_axis[0],d2_axis[1])
        d2_uvs.append(d2_uv)
    avg_d2_axis_uv = np.array(d2_uvs).mean(axis=0)
    avg_d2_col_center = np.array(D2_col_centers).mean(axis=0)

    start_pt = 5000*avg_d2_axis_uv+avg_d2_col_center
    end_pt =-5000*avg_d2_axis_uv+avg_d2_col_center
    #Landmarks(pts=[avg_d2_col_center]).write_landmarks(output_path_avg+alignment_type+'/Avg_D2_Column_Center.landmarkAscii')

    pia_intersection_pt,dist= pia.get_vector_intersection_pt(start_pt,end_pt,extrapolation_len=0)
    wm_intersection_pt,dist= wm.get_vector_intersection_pt(end_pt,start_pt,extrapolation_len=0)

    # project avg d2 to pia wm
#     avg_d2_col_sg,wm_pts,pia_pts,missing_axes = pia.get_axis_field_within_instersecting_surface_with_trimming\
#                     (pia.surface,wm.surface,[[avg_d2_col_center,-1000*avg_d2_axis_uv+avg_d2_col_center]],vec_extension_offset=5000)

    #avg_d2_col_sg.write_spatial_graph(output_path_avg+alignment_type+'/Avg_D2_Axis.am')
    return [wm_intersection_pt,pia_intersection_pt]

In [ ]:
def get_barrel_projection(sg,pia,wm):
    #sg_new = AmiraSpatialGraph(generic_graph=True)
    projection_landmarks = []
    for barrel_row_ind in range(len(sg.barrels.rows_list)):
        for barrel_ind in range(len(sg.barrels.rows_list[barrel_row_ind].single_barrels_list)):
            #if barrel.barrel_name == 'E4':
            barrel = sg.barrels.rows_list[barrel_row_ind].single_barrels_list[barrel_ind]
            for i in range(len(barrel.pia_projection_barrel)):
                pia_intersection_pt,dist= pia.get_vector_intersection_pt(barrel.wm_projection_barrel[i],barrel.pia_projection_barrel[i],extrapolation_len=10000)
                wm_intersection_pt,dist= wm.get_vector_intersection_pt(barrel.pia_projection_barrel[i],barrel.wm_projection_barrel[i],extrapolation_len=10000)
#                 sg.barrels.rows_list[barrel_row_ind].single_barrels_list[barrel_ind].pia_projection_barrel[i] = pia_intersection_pt
#                 sg.barrels.rows_list[barrel_row_ind].single_barrels_list[barrel_ind].wm_projection_barrel[i] = wm_intersection_pt
                if len(pia_intersection_pt) > 0 and len(wm_intersection_pt)>0:
                    #sg_new.barrels.all_rows_graphdata.append(pia_intersection_pt)
                    #sg_new.barrels.all_rows_graphdata.append(wm_intersection_pt)
                    projection_landmarks.append(pia_intersection_pt)
                    projection_landmarks.append(wm_intersection_pt)
                    print(pia_intersection_pt,wm_intersection_pt)
    #sg_new.update_graph_data()
    return projection_landmarks

In [ ]:

for alignment_type in ['Using_vS1_vM1_Surface_Points']:
    pia = Surface(output_path_avg+alignment_type+'/Avg_Pia_50.vtk')
    wm = Surface(output_path_avg+alignment_type+'/Avg_WM_50.vtk')
    sbf = AmiraSpatialGraph('/nas1/Data_Mythreya/MotorCortexProject/V10/vM1_Ref_Frame/Original_Data/sbf/average_barrel_field.am',\
                                generic_graph=True,barrel_projections_present=True)
    
    ref_pts = []
    pts = []
    # get the avg C2,C3,D2,D3 axis from aligned sgs
    for barrel_row_ind in range(len(sg.barrels.rows_list)):
        for barrel_ind in range(len(sg.barrels.rows_list[barrel_row_ind].single_barrels_list)):
            #if barrel.barrel_name == 'C2' or barrel.barrel_name == 'C3' or barrel.barrel_name == 'D2' or barrel.barrel_name == 'D3':
            barrel = sg.barrels.rows_list[barrel_row_ind].single_barrels_list[barrel_ind]
            sbf_barrel = sbf.barrels.rows_list[barrel_row_ind].single_barrels_list[barrel_ind]
            if len(barrel.pia_projection_barrel_centroid)>0 and len(barrel.wm_projection_barrel_centroid) > 0 \
                and len(sbf_barrel.pia_projection_barrel_centroid)>0 and len(sbf_barrel.wm_projection_barrel_centroid) > 0 :
                #print(barrel.pia_projection_barrel_centroid,barrel.wm_projection_barrel_centroid)
                avg_column_axis = get_avg_barrel_column_axis(barrel,pia,wm)
                ref_pts.append(avg_column_axis[0])
                ref_pts.append(avg_column_axis[1])
                #print(sbf_barrel.pia_projection_barrel_centroid,sbf_barrel.wm_projection_barrel_centroid)
                pts.append(sbf_barrel.wm_projection_barrel_centroid)
                pts.append(sbf_barrel.pia_projection_barrel_centroid)
                
    #print(len(ref_pts),len(pts))
    # align old sbf to avg column axes
    icp,txmat = Landmarks(pts=pts).align_landmarks(ref_pts)
    sbf.apply_transformation(txmat)
    sbf.write_spatial_graph(output_path_avg+alignment_type+'/Avg_SBF.am')
    
    # project sbf to pia wm
#     sbf = AmiraSpatialGraph(output_path_avg+alignment_type+'/Avg_SBF.am',generic_graph=True,barrel_projections_present=True)
    
#     sbf_corrected = get_barrel_projection(sbf,pia,wm)
#     sbf_corrected.write_spatial_graph(output_path_avg+alignment_type+'/Avg_SBF.am')

In [ ]:
#TODO: project sbf to pia wm
sbf = AmiraSpatialGraph(output_path_avg+alignment_type+'/Avg_SBF.am',generic_graph=True,barrel_projections_present=True)
for alignment_type in ['Using_vS1_vM1_Surface_Points']:
    for surf_res in surface_resolutions:
        pia = Surface(output_path_avg+alignment_type+'/Avg_Pia_{}.vtk'.format(surf_res))
        wm = Surface(output_path_avg+alignment_type+'/Avg_WM_{}.vtk'.format(surf_res))

        projection_landmarks = get_barrel_projection(sbf,pia,wm)
#         Landmarks(pts=projection_landmarks).write_landmarks(output_path_avg+alignment_type+'/project_landmarks_')
        #sbf_corrected.write_spatial_graph(output_path_avg+alignment_type+'/Avg_SBF_{}.am'.format(surf_res))
        break

In [ ]:
# TODO: create delunay convex hull around avg sbf and find the axis field within it

# Avg vS1 axis field
# for alignment_type in alignment_types:
#     for surf_res in surface_resolutions:
#         axis_field_sg = AmiraSpatialGraph(output_path_avg+'/'+alignment_type+'/Avg_Axis_Field_{}.am'.format(surf_res),generic_graph=True)
#         sbf = AmiraSpatialGraph(generic_graph=True)
#         vs1_hull = Surface(pts=)
#         vm1 = Surface(output_path_avg+'/'+alignment_type+'/Avg_vM1_{}.vtk'.format(surf_res))

#         vm1_axis_field_sg,wm_pts,pia_pts = vm1.get_axis_field_within_instersecting_surface(vm1.surface,\
#                                                                                            axis_field_sg.graph_data.edge_list)
#         vm1_axis_field_sg.write_spatial_graph(output_path_avg+alignment_type+'/Avg_vM1_Axis_Field_{}.am'.format(surf_res))

# Find relation between vM1 and vS1

In [ ]:
# Find the 3D as well as surface distance and angle between barrel centers and vM1 center
barrel_soma_df = pd.read_csv('/nas1/Data_Mythreya/MotorCortexProject/V10/Registration_Local/Outputs_100/16_Barrel_Soma_Distance/Barrel_Soma_distances.csv')
pd.set_option('display.max_rows', barrel_soma_df.shape[0]+1)
pd.set_option('display.max_columns', barrel_soma_df.shape[1]+1)
#barrel_soma_df

In [ ]:
# Find the four cutting planes from the center of D2 that cover the vM1 surface

# Register avg to D2 column


# Transform to vM1 and vS1 local ref frame

In [ ]:
for alignment_type in ['Using_vS1_vM1_Surface_Points',]:
    pia = Surface(output_path_avg+alignment_type+'/Avg_Pia_{}.vtk'.format(50))
    sbf = AmiraSpatialGraph(output_path_avg+alignment_type+'/Avg_SBF_Without_Projections.am',generic_graph=True,\
                            barrel_projections_present=False,reverse_barrel_direction=True)
    vM1 = Surface(output_path_avg+alignment_type+'/Avg_vM1.vtk')
    vM1_center = vM1.get_center_of_mass()
    
    output_path = output_path_avg + alignment_type + '/In_SBF_Ref_Frame/'
    pathlib.Path(output_path).mkdir(exist_ok=True)
    output_path_lb = output_path_avg + alignment_type + '/In_SBF_Ref_Frame_Layer_Borders/'
    pathlib.Path(output_path_lb).mkdir(exist_ok=True)
    #for exp in exp_names_tangential:
    ref_pts = []
    pts = []
    
    sbf = AmiraSpatialGraph(output_path_avg+alignment_type+'/Avg_SBF_Without_Projections.am',generic_graph=True,\
                            barrel_projections_present=False,reverse_barrel_direction=True)
    d2_center_bottom = (np.array(sbf.barrels.d_row.two.top_barrel).mean(axis=0))
    d2_center_top = (np.array(sbf.barrels.d_row.two.bottom_barrel).mean(axis=0))
    d2_center = (d2_center_top + d2_center_bottom) / 2.0
    print(d2_center)
    pts.append(d2_center)
    ref_pts.append([0,0,0])
    
    pia_intersection_pt,dist= pia.get_vector_intersection_pt(d2_center,d2_center_top,extrapolation_len=10000)
    print(pia_intersection_pt)
    pts.append(pia_intersection_pt)
    ref_pts.append([0,0,-distance.euclidean(d2_center,pia_intersection_pt)])
    
    # Take the 3rd anchor point as vM1 center
#     print(vM1_center)
#     pts.append(vM1_center)
#     ref_pts.append([0,-distance.euclidean(d2_center,vM1_center),0])
    
    landmarks = Landmarks(pts=pts)
    icp,txmat = landmarks.align_landmarks(ref_pts)
        
    aligner = Alignment() 
    aligner.transform_folder(output_path_avg + alignment_type,output_path,txmat=txmat,icp=icp)
    aligner.transform_folder(output_path_neun_layer_borders,output_path_lb,txmat=txmat,icp=icp)
    
    # align to max y axis
    pia = Surface(output_path+'/Avg_Pia_50.vtk')
    vtkpts = pia.surface.GetPoints()
    vert_list = []
    for i in range(pia.surface.GetNumberOfPoints()):
        vert_list.append(vtkpts.GetPoint(i))
    maxy_pt = vert_list[np.array(vert_list)[:,1].argmin()]
    
    #print(maxy_pt)
    pts = []
    ref_pts = []
    pts.append([0,0,0])
    pts.append([maxy_pt[0],maxy_pt[1],0])
    ref_pts.append([0,0,0])
    dist = -distance.euclidean([0,0,0],[maxy_pt[0],maxy_pt[1],0])
    ref_pts.append([0,dist,0])
    landmarks = Landmarks(pts=pts)
    icp,txmat = landmarks.align_landmarks(ref_pts)
    
    #output_path = output_alignment_final+'Aligned_To_MaxY/'
    pathlib.Path((output_path)).mkdir(exist_ok=True)
    aligner = Alignment() 
    aligner.transform_folder(output_path,\
                              output_path,txmat=txmat,icp=icp)
    aligner.transform_folder(output_path_lb,output_path_lb,txmat=txmat,icp=icp)
    
    # apply scaling to match the scaling error in the registration experiments
    scaling_transform = [2.35/2.306,2.35/2.306,1]
    txmat = [scaling_transform[0],0,0,0, 0,scaling_transform[1],0,0, 0,0,scaling_transform[2],0, 0,0,0,1]
    output_path = output_path_avg + alignment_type + '/In_SBF_Ref_Frame_With_Scaling_Correction/'
    pathlib.Path(output_path).mkdir(exist_ok=True)
    output_path_lb_scaling_correction = output_path_avg + alignment_type + '/In_SBF_Ref_Frame_With_Scaling_Correction_Layer_Borders/'
    pathlib.Path(output_path_lb_scaling_correction).mkdir(exist_ok=True)
    aligner = Alignment() 
    aligner.transform_folder(output_path_avg + alignment_type + '/In_SBF_Ref_Frame/',output_path,txmat=txmat,\
                             icp=icp,apply_polydata_transform=True)
    aligner.transform_folder(output_path_lb,output_path_lb_scaling_correction,txmat=txmat,icp=icp)

In [ ]:
pia = Surface(output_path_avg+alignment_type+'/Avg_Pia_{}.vtk'.format(50))

In [ ]:
vtkpts = pia.surface.GetPoints()
vert_list = []
for i in range(pia.surface.GetNumberOfPoints()):
    vert_list.append(vtkpts.GetPoint(i))
#vert_list[:,1]
vert_list[np.array(vert_list)[:,1].argmin()]

# Avg Stats

In [ ]:
cols = ['Alignment_Type','Voxel_Size','Pia_Number_Of_Vertices','Pia_Number_Of_Triangles',\
          'WM_Number_Of_Vertices','WM_Number_Of_Triangles',\
          'Pia_Surface_Area','WM_Surface_Area','Pia_Volume','WM_Volume',\
          'Cortical_Thickness_Avg','Cortical_Thickness_Std',\
          'Cortical_Thickness_vM1_Avg','Cortical_Thickness_vM1_Std',\
          'Cortical_Thickness_vS1_Avg','Cortical_Thickness_vS1_Std',]
surf_stats_main = pd.DataFrame(columns=cols)
for alignment_type in alignment_types:
    for surf_res in surface_resolutions: 
        surf_stats_df = pd.DataFrame(columns=cols)
        #for exp_name in exp_names :
        pia = Surface(output_path_avg+alignment_type+'/Avg_Pia_{}.vtk'.format(surf_res))
        wm = Surface(output_path_avg+alignment_type+'/Avg_WM_{}.vtk'.format(surf_res))
        pia_closed = pia.create_delunay_surface_3d(return_hull=True,make_cube=False)
        wm_closed = wm.create_delunay_surface_3d(return_hull=True,make_cube=False)
        
        avg_axis_field_sg = AmiraSpatialGraph(output_path_avg+alignment_type+'/Avg_Axis_Field_{}.am'.format(surf_res),generic_graph=True)
        avg_axis_field_vM1_sg = AmiraSpatialGraph(output_path_avg+alignment_type+'/Avg_vM1_Axis_Field_{}.am'.format(surf_res),generic_graph=True)
        sbf = AmiraSpatialGraph(output_path_avg+alignment_type+'/Avg_SBF_Without_Projections.am',generic_graph=True)
        
        # Get vS1 axis field
        avg_axis_field_vS1_sg = AmiraSpatialGraph(generic_graph=True)
        # create hull around s1 and get axis field for s1
        for pt in sbf.barrels.all_rows_graphdata.edge_pt_coords:
            min_edge,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = \
            Vectors().get_nearest_axis_to_pt(pt,avg_axis_field_sg.graph_data.edge_list,axis_validation_distance=3000)
            avg_axis_field_vS1_sg.graph_data.add_edge(min_edge[0],min_edge[1])
        
        avg_axis_field_vS1_sg.write_spatial_graph(output_path_avg+alignment_type+'/Avg_vS1_Axis_Field_{}.am'.format(surf_res))
        
        vec_lens,bla1,bla2 = Vectors().get_vec_lengths(avg_axis_field_sg.graph_data.edge_list)
        vec_lens_vm1,bla,blee = Vectors().get_vec_lengths(avg_axis_field_vM1_sg.graph_data.edge_list)
        vec_lens_vs1,bla,blee = Vectors().get_vec_lengths(avg_axis_field_vS1_sg.graph_data.edge_list)
        
        surf_stats_df = surf_stats_df.append(pd.DataFrame(columns=surf_stats_df.columns,data = \
                                          [[alignment_type,float(surf_res),\
                                           float(pia.surface.GetNumberOfPoints()),float(pia.surface.GetNumberOfCells()),\
              float(wm.surface.GetNumberOfPoints()),float(wm.surface.GetNumberOfCells()),\
              pia.get_surface_properties(prop='area'),wm.get_surface_properties(prop='area'),\
              Surface(polydata=pia_closed).get_surface_properties(prop='volume'),Surface(polydata=wm_closed).get_surface_properties(prop='volume'),\
              np.array(vec_lens).mean(axis=0),np.array(vec_lens).std(axis=0),\
              np.array(vec_lens_vm1).mean(axis=0),np.array(vec_lens_vm1).std(axis=0),\
              np.array(vec_lens_vs1).mean(axis=0),np.array(vec_lens_vs1).std(axis=0)]]))
        
        #stats = Stats(surf_stats_df)
        #stats_returned = stats.add_mean_std_cov(tangential_exps=True, coronal_exps=True,)

        surf_stats_main = surf_stats_main.append(surf_stats_df)

surf_stats_main.to_csv(output_path_stats+'8_Avg_Pia_WM_Thickness_Stats.csv')

In [ ]:
surf_stats_main

In [ ]:
1900/1790, 2300/2030

# Align Tangential to Coronal

In [ ]:
# align everything to coronal pia wm to create coronal view
aligner = Alignment(coronal_pia_wm.surface,ref_pia_wm.surface)
txmat = aligner.get_transformation_matrix()
icp = aligner.get_icp_transform()

In [ ]:
aligner = Alignment()
output_canonical_rf = output_canonical+ '/Input_Ref_Frame/'
pathlib.Path(output_canonical_rf).mkdir(exist_ok=True)
Alignment().transform_folder(input_path_avg_rf,output_canonical_rf,\
                    icp=icp,txmat = txmat,)

Alignment().transform_folders(output_root,output_canonical,exp_name=None,\
                    icp=icp,txmat = txmat,)

In [ ]:
Alignment().transform_folders(output_alignment_using_neuron_axis_and_local_Avg_pia_soma+'Manual_Cell_Type_Assignment/',output_canonical+'Manual_Cell_Type_Assignment/',\
                    icp=icp,txmat = txmat,)

In [ ]:
pd.read_csv(output_path_stats+'1_Pia_WM_Surface_Stats.csv')

In [ ]:
pd.read_csv(output_path_stats+'2_Cortical_Thickness.csv')

In [ ]:
pd.read_csv(output_path_stats+'5_vS1_L5_Stats.csv')

In [ ]:
pd.read_csv(output_path_stats+'5_vM1_L5_Stats.csv')

In [ ]:
L5_pts = Landmarks(output_path_layers+'MG49_lhs_vM1_L5_avg_pts.landmarksAscii').pts
L5_center = Landmarks(output_path_landmarks+'MG49_lhs_L5_central_pt_using_L5_avg_pts.landmarksAscii').pts
surf = Surface(output_path_layers+'MG49_lhs_vM1_L5_avg_surf.vtk').surface
central_axis_sg = AmiraSpatialGraph(output_path_axis_fileds+'MG49_lhs_vM1_Central_Axis_L5.am',generic_graph=True)
center_ind = 0

# Final density clouds to visualize neurons

In [ ]:

#first copy the coronal aligned vm1 rabies landmarks to tmp folder and final avg surf ( coronal view) to the same folder
input_path = '/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Canonical_View/Input_Ref_Frame/'
rabies_landmarks = '/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Canonical_View/Input_Ref_Frame/combined_landmarks.landmarkAscii'
output_path_tmp = '/nas1/Data_Mythreya/MotorCortexProject/V10/vM1_Ref_Frame/Outputs/tmp/'
vectors_sg = AmiraSpatialGraph(filename=input_path+'Avg_Axis_Field_50.am',generic_graph=True)

l = Landmarks(rabies_landmarks)
for dens_cutoff_threshold in 0.5,0.4,0.3,0.2,0.15,0.1,0.05,0:
    cluster_in_bounds,max_dens,dens_cutoff_threshold,threhold = \
        l.get_rostral_most_cluster(output_file=output_path_tmp+'{}_vM1_rabies_landmarks.LandmarkAscii'.format(dens_cutoff_threshold),\
                              voxel_size=500,dens_cutoff_threshold=dens_cutoff_threshold,write_density_images=False)
        
    pts = Landmarks(filename=output_path_tmp+'{}_vM1_rabies_landmarks.LandmarkAscii'.format(dens_cutoff_threshold)).pts
    vm1_edge_list_sg,df_vm1 = get_intersecting_axes_and_layers(pts,vectors_sg,radius=250,\
                                                        make_cube=True,bounding_offset=0,\
                                             output_filename_axes=output_path_tmp+str(dens_cutoff_threshold)+'_vM1_axis_field.am',\
                                             output_path_layers=output_path_tmp+str(dens_cutoff_threshold)+'_vM1',)

In [ ]:
input_path = '/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Canonical_View/Input_Ref_Frame/'
rabies_landmarks = '/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Canonical_View/Input_Ref_Frame/combined_landmarks.landmarkAscii'
output_path_tmp = '/nas1/Data_Mythreya/MotorCortexProject/V10/vM1_Ref_Frame/Outputs/tmp/'
vectors_sg = AmiraSpatialGraph(filename=input_path+'Avg_Axis_Field_50.am',generic_graph=True)


cutting_order = [False,True,True,False]
#cutting_order = [True,False,False,True]
   
pia = Surface(filename=input_path+'Avg_Pia_50.vtk')
wm = Surface(filename=input_path+'Avg_WM_50.vtk')

for dens_cutoff_threshold in [0.5,0.4,0.3,0.2,0.15,0.1,0.05,0]:
    if dens_cutoff_threshold > 0.2 or dens_cutoff_threshold == 0.15:
        cutting_order = [True,False,False,True]
         
    else:
        cutting_order = [False,True,True,False]
    #cutting_order = [False,True,True,False]
    dens_cutoff_threshold = str(dens_cutoff_threshold)
    # get L5 center pt and central axis
    
    #axis_field,l5_surf,l5_pt,central_axis = get_central_axis(exp)
    vm1_axis_field = AmiraSpatialGraph(output_path_tmp+dens_cutoff_threshold+'_vM1_axis_field.am',generic_graph=True)
    avg_L5_m1_pts = Landmarks(filename=output_path_tmp+dens_cutoff_threshold+'_vM1_L5_avg_pts.landmarksAscii').pts
    avg_L5_surf = Surface(filename=output_path_tmp+dens_cutoff_threshold+'_vM1_L5_avg_surf.vtk')
    central_axis,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = Vectors().get_nearest_axis_to_pt\
                                        (np.array(avg_L5_m1_pts).mean(axis=0),vm1_axis_field.graph_data.edge_list)
    l5_pt,dist = avg_L5_surf.get_vector_intersection_pt(central_axis[0],central_axis[1],)
    
    # Flatten L5 and get flattened pts
    flattened_pts = flatten_pts(avg_L5_surf.surface,l5_pt,central_axis,dens_cutoff_threshold,write_output=False)
    
    # Get PCA for 2D L5 pts
    pcas,neg_pcas = get_pca_from_flattened_pts(flattened_pts,dens_cutoff_threshold,l5_pt,central_axis,pia.surface,wm.surface,write_output=False)

    
    # Get boundary axes to find angle for cutting planes
    #extreme_axis_x,extreme_axis_neg_x,extreme_axis_y,extreme_axis_neg_y = get_extreme_axes(pcas,neg_pcas,axis_field,\
    #                                                                                       flattened_pts,exp)
    vm1_final,vm1_hull,vm1_pia,vm1_wm = get_vm1(pia.surface,wm.surface,pcas=pcas,neg_pcas=neg_pcas,\
                                                     axis_field=vm1_axis_field.graph_data.edge_list,\
                                                     cutting_order=cutting_order)
    Surface(polydata=vm1_final).write_surface_mesh(output_path_tmp+dens_cutoff_threshold+'_vM1_surface.vtk')
    Surface(polydata=vm1_hull).write_surface_mesh(output_path_tmp+dens_cutoff_threshold+'_vM1_hull.vtk')
    Surface(polydata=vm1_pia).write_surface_mesh(output_path_tmp+dens_cutoff_threshold+'_vM1_pia.vtk')
    Surface(polydata=vm1_wm).write_surface_mesh(output_path_tmp+dens_cutoff_threshold+'_vM1_wm.vtk')
    
    

In [ ]:
vm1_final.GetNumberOfCells()

In [ ]:
avg_L5_m1_pts = Landmarks(filename=output_path_layers+exp+'_vM1_L5_avg_pts.landmarksAscii').pts
vm1_rabies_pts = Landmarks(output_path_landmarks+exp+'_vM1_rabies_landmarks.LandmarkAscii').pts
vm1_axis_field = AmiraSpatialGraph(output_path_axis_fileds+exp+'_vM1_axis_field.am',generic_graph=True)
pia = Surface(filename=output_path_surfaces+exp+'_pia_open_bottom_voxel_size_50.vtk')
wm = Surface(filename=output_path_surfaces+exp+'_WM_open_bottom_voxel_size_50.vtk')
avg_L5_surf = Surface(filename=output_path_layers+exp+'_vM1_L5_avg_surf.vtk')

In [ ]:
avg_L5_m1_pts = Landmarks(filename=output_path_layers+exp+'_vM1_L5_avg_pts.landmarksAscii').pts
    vm1_rabies_pts = Landmarks(output_path_landmarks+exp+'_vM1_rabies_landmarks.LandmarkAscii').pts
    vm1_axis_field = AmiraSpatialGraph(output_path_axis_fileds+exp+'_vM1_axis_field.am',generic_graph=True)
    pia = Surface(filename=output_path_surfaces+exp+'_pia_open_bottom_voxel_size_50.vtk')
    wm = Surface(filename=output_path_surfaces+exp+'_WM_open_bottom_voxel_size_50.vtk')
    avg_L5_surf = Surface(filename=output_path_layers+exp+'_vM1_L5_avg_surf.vtk')
    
    Landmarks(pts=[np.array(avg_L5_m1_pts).mean(axis=0)]).write_landmarks(output_path_landmarks+exp+'_vM1_L5_Centroid.landmarksAscii')
    Landmarks(pts=[np.array(vm1_rabies_pts).mean(axis=0)]).write_landmarks(output_path_landmarks+exp+'_vM1_Rabies_Centroid.landmarksAscii')
    
    centralaxis_l5,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = Vectors().get_nearest_axis_to_pt\
                                    (np.array(avg_L5_m1_pts).mean(axis=0),vm1_axis_field.graph_data.edge_list)
    centralaxis_rabies,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = Vectors().get_nearest_axis_to_pt\
                                    (np.array(vm1_rabies_pts).mean(axis=0),vm1_axis_field.graph_data.edge_list)
    # find the intersection of central axis with L5 surface
    L5_central_pt_using_L5_avg_pt,dist = avg_L5_surf.get_vector_intersection_pt(centralaxis_l5[0],centralaxis_l5[1],)
    L5_central_pt_using_Rabies_pt,dist = avg_L5_surf.get_vector_intersection_pt(centralaxis_rabies[0],centralaxis_rabies[1],)
    Landmarks(pts=[L5_central_pt_using_L5_avg_pt]).write_landmarks(output_path_landmarks+exp+\
                                                                  '_L5_central_pt_using_L5_avg_pts.landmarksAscii')
    Landmarks(pts=[L5_central_pt_using_Rabies_pt]).write_landmarks(output_path_landmarks+exp+\
                                                                  '_L5_central_pt_using_Rabies_pt.landmarksAscii')
    vM1_Central_Axis_L5_sg = AmiraSpatialGraph(generic_graph=True)
    vM1_Central_Axis_L5_sg.graph_data.add_edge(centralaxis_l5[0],centralaxis_l5[1])
    vM1_Central_Axis_L5_sg.write_spatial_graph(output_path_axis_fileds+exp+'_vM1_Central_Axis_L5.am')
    vM1_Central_Axis_Rabies_sg = AmiraSpatialGraph(generic_graph=True)
    vM1_Central_Axis_Rabies_sg.graph_data.add_edge(centralaxis_rabies[0],centralaxis_rabies[1])
    vM1_Central_Axis_Rabies_sg.write_spatial_graph(output_path_axis_fileds+exp+'_vM1_Central_Axis_Rabies.am')
    
    return vm1_axis_field.graph_data.edge_list,avg_L5_surf.surface,L5_central_pt_using_L5_avg_pt,[centralaxis_l5[0],centralaxis_l5[1]]

In [ ]:
centralaxis_l5,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = Vectors().get_nearest_axis_to_pt\
                                    (np.array(avg_L5_m1_pts).mean(axis=0),vm1_axis_field.graph_data.edge_list)
centralaxis_rabies,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = Vectors().get_nearest_axis_to_pt\
                                (np.array(vm1_rabies_pts).mean(axis=0),vm1_axis_field.graph_data.edge_list)

In [ ]:
sg = AmiraSpatialGraph('/nas1/Data_aman/00_Rabies/Pittsburgh/DATA/All/AN5_1to200_merged_labeled_ascii.am')

In [ ]:
create_surface('/nas1/Data_aman/00_Rabies/Pittsburgh/DATA/All/AN5_1to200_merged_labeled_ascii.am',\
                                                    coronal=True,rhs=True,nr_divisions=0,surf_resolution=surface_resolutions[i])

In [ ]:
sg = AmiraSpatialGraph('/nas1/Data_aman/00_Rabies/Pittsburgh/DATA/All/AN5_20201103_v4_left.am')

In [ ]:
sg_new = AmiraSpatialGraph()
sg_new.graph_data = sg.pia
sg_new.graph_data.set_label('9')

sg_new.write_spatial_graph('/nas1/Data_aman/00_Rabies/Pittsburgh/DATA/All/AN5_20201103_v4_left_relabelled.am')

In [ ]:
os.system('/home/mythreya/project_src/BarrelField3D/DataAnalysis3D/BF3DRecon {} {} {}'.\
              format('/nas1/Data_aman/00_Rabies/Pittsburgh/DATA/All/AN5_20201103_v4_left_relabelled.am',\
                     '/nas1/Data_aman/00_Rabies/Pittsburgh/DATA/All'+'AN5_20201103_v4_left_relabelled',str(100)))

In [ ]:
salary = 50
amt=1.5e7
for i in range(1,16):
    salary = 0#salary*1.05
    amt = salary + (amt)*1.15
    print('year {}: salary = {}: amt = {}'.format(i,salary,amt))

for i in range(1,21):
    amt = (amt)*1.25
    print('year {} amt = {}'.format(i,amt))


In [ ]:
5000000*15

In [ ]:
total = 11
for i in range(1,11):
    total = total*1.25
print(total)

In [ ]:
((((((55*1.2+60)*1.2+65)*1.2+70)*1.2+75)*1.2)+80)*1.2

In [ ]:
(50+55+60+65+70)*1.2

In [ ]:
for barrel in ['C2','C3','D2','D3']:
    path = '/nas1/Data_Mythreya/MotorCortexProject/Manual_NeuN_Landmarks/from_aman/MG49_lhs/'+barrel
    all_landmarks = []
    for file in sorted(glob.glob(path+'/S*_Landmarks.landmarkAscii')):
        print(file)
        if len(Landmarks(file).pts) > 1:
            for pt in (Landmarks(file).pts):
                all_landmarks.append(pt)
    Landmarks(pts=all_landmarks).write_landmarks(path+barrel+'_landmarks.landmarksAscii')

In [ ]:
glob.glob('/nas1/Data_Mythreya/MotorCortexProject/Manual_NeuN_Landmarks/from_aman/MG49_lhs/C2/*Landmarks.landmarkAscii')